# Data-informed parameter synthesis for population pDTMCs, notebook 2

This notebook sampling and visualisation of the polynomials as a result of parameter synthesis. It consists of these parts:
1. [Load Parameter synthesis results](#one)
3. [Sampling and visualisation](#two)

There are three things to setup in this notebook:
1. `agents_quantities` - set of agents number to be considered - set numbers for which you want to run analysis,
2. `size_q` in `visualise(dic_fun,size_q)` and `sample(dic_fun,size_q)` - number of samples in each parameter dimension, setting this number high easpecially in high dimensional may result in very long computation.

## IMPORTS, PATHS, and SETTINGS

if a package fails to load due to it is not installed please uncomment and use following code with respective package name

In [ ]:
#import sys
#!{sys.executable} -m pip install <package_name>

In [ ]:
import struct
print("You are running "+ str(struct.calcsize("P") * 8)+"bit Python, please verify that installed z3 is compatible")

In [ ]:
#imports evaluation
import time

In [ ]:
#imports for creation of models
import math
import sys

In [ ]:
#imports for running prism
import subprocess

In [ ]:
#imports for analysis
#imports
import glob, os, platform
import numpy as np
import re
from sympy import factor
#visualisation
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle
#multiply arrays
from numpy import prod
# define error
import scipy.stats as st

In [ ]:
# import enhaced print
import socket

In [ ]:
cwd = os.getcwd()

In [ ]:
os.getcwd()

In [ ]:
agents_quantities=[2,3,5,10]

<a name="one"></a>

## 1. LOADING OF PARAMETER SYNTHESIS RESULTS AND DATA

In [ ]:
os.chdir("./prism_results")

In [ ]:
os.getcwd()

Now we load list of propbabilities for respective BSCC and expected number of successes.

Here we provide first optimisation by factorising the polynomials received as an output from PRISM.

In [ ]:
def load_all_prism(path,factorize=True, rewards_only=False, f_only=False):
    """ Loads all results of parameter synthesis in *path* folder into two maps - f list of rational functions for each property, and rewards list of rational functions for each reward
    
    Parameters
    ----------
    path: string - file name regex
    factorize: if true it will factorise polynomial results 
    rewards_only: if true it compute only rewards
    f_only: if true it will compute only standard properties
    
    Returns:
        (f,reward), where
        f: dictionary N -> list of rational functions for each property
        rewards: dictionary N -> list of rational functions for each reward
    """

    here = False
    N = 0
    f = {}
    rewards = {}
    for file in glob.glob(path):
        #print(file)
        N = int(re.findall('\d+', file )[0])  
        file=open(file,"r")
        i=-1
        here=""
        f[N] = []
        rewards[N] = []
        for line in file:
            if line.startswith( 'Parametric model checking:' ):
                i=i+1 
            if line.startswith( 'Parametric model checking: R=?' ):
                here="r"
            if i>=0 and line.startswith( 'Result' ):
                line = line.split(":")[2]
                line = line.replace("{", "")
                line = line.replace("}", "")
                line = line.replace("p", "* p")
                line = line.replace("q", "* q")
                line = line.replace("**", "*")
                line = line.replace("* *", "*")
                line = line.replace("*  *", "*")
                line = line.replace("+ *", "+")
                line = line.replace("^", "**")
                line = line.replace(" ", "")
                if line.startswith( '*' ):
                    line=line[1:]
                if here=="r" and not f_only:
                    if factorize:
                        try:
                            rewards[N].append(str(factor(line[:-1])))
                        except:
                            print("Error while factorising rewards, used not factorised instead")
                            rewards[N].append(line[:-1])
                    else:
                        rewards[N]=line[:-1]
                elif not here=="r" and not rewards_only:
                    #print(f[N])
                    #print(line[:-1])
                    if factorize:
                        try:
                            f[N].append(str(factor(line[:-1])))
                        except:
                            print("Error while factorising polynome f[{}][{}], used not factorised instead".format(N,i))
                            f[N]=line[:-1]
                    else:
                        f[N].append(line[:-1])
    return(f,rewards)

def get_f(path,factorize):
    return load_all_prism(path,factorize,False,True)[0]

def get_rewards(path,factorize):
    return load_all_prism(path,factorize,True,False)[1]

Recalling N is number of agents, and f_N(i) is probability function of having i successes,

Load f_N, a map N-> array of f(i)s:


Factorised version:

In [ ]:
start_time = time.time()
f = get_f("./sem*[0-9].txt",True)
print("  It took", socket.gethostname(), time.time() - start_time, "seconds to run")

Not factorised (expanded version):

In [ ]:
start_time = time.time()
f_expanded = get_f("./sem*[0-9].txt",False)
print("  It took", socket.gethostname(), time.time() - start_time, "seconds to run")

Now load the expected number of sucesses in reward section:

In [ ]:
start_time = time.time()
rewards = get_rewards("./sem*[0-9].txt",True)
print("  It took", socket.gethostname(), time.time() - start_time, "seconds to run")

In [ ]:
start_time = time.time()
rewards_expanded = get_rewards("./sem*[0-9].txt",False)
print("  It took", socket.gethostname(), time.time() - start_time, "seconds to run")

These agents quantities are loaded

In [ ]:
print(sorted(f.keys()))

Load f for multiple param models.

This may take a while to factorise for f_multiparam[10], so I will load precomputed version for you

In [ ]:
#sys.setrecursionlimit(15000)
#start_time = time.time()
#f_multiparam = get_f("./multiparam_se*[0-9].txt",True)
#print("  It took", socket.gethostname(), time.time() - start_time, "seconds to run")
#sys.setrecursionlimit(1000)

In [ ]:
f_multiparam = get_f("./multiparam_se*_[0-9].txt",True)
f_multiparam[10]=[0,0,0,0,0,0,0,0,0,0,0]
f_multiparam[10][0] = '(p - 1)**10'
f_multiparam[10][1] = '10*p*(p - 1)**9*(q1 - 1)**9'
f_multiparam[10][2] = '-5*p*(p - 1)**8*(2*p*q1**9 + 2*p*q1**8*q2 - 18*p*q1**8 + 2*p*q1**7*q2**2 - 18*p*q1**7*q2 + 72*p*q1**7 + 2*p*q1**6*q2**3 - 18*p*q1**6*q2**2 + 72*p*q1**6*q2 - 168*p*q1**6 + 2*p*q1**5*q2**4 - 18*p*q1**5*q2**3 + 72*p*q1**5*q2**2 - 168*p*q1**5*q2 + 252*p*q1**5 + 2*p*q1**4*q2**5 - 18*p*q1**4*q2**4 + 72*p*q1**4*q2**3 - 168*p*q1**4*q2**2 + 252*p*q1**4*q2 - 252*p*q1**4 + 2*p*q1**3*q2**6 - 18*p*q1**3*q2**5 + 72*p*q1**3*q2**4 - 168*p*q1**3*q2**3 + 252*p*q1**3*q2**2 - 252*p*q1**3*q2 + 168*p*q1**3 + 2*p*q1**2*q2**7 - 18*p*q1**2*q2**6 + 72*p*q1**2*q2**5 - 168*p*q1**2*q2**4 + 252*p*q1**2*q2**3 - 252*p*q1**2*q2**2 + 168*p*q1**2*q2 - 72*p*q1**2 + 2*p*q1*q2**8 - 18*p*q1*q2**7 + 72*p*q1*q2**6 - 168*p*q1*q2**5 + 252*p*q1*q2**4 - 252*p*q1*q2**3 + 168*p*q1*q2**2 - 72*p*q1*q2 + 18*p*q1 - 9*p*q2**8 + 72*p*q2**7 - 252*p*q2**6 + 504*p*q2**5 - 630*p*q2**4 + 504*p*q2**3 - 252*p*q2**2 + 72*p*q2 - 9*p - 2*q1**9 - 2*q1**8*q2 + 18*q1**8 - 2*q1**7*q2**2 + 18*q1**7*q2 - 72*q1**7 - 2*q1**6*q2**3 + 18*q1**6*q2**2 - 72*q1**6*q2 + 168*q1**6 - 2*q1**5*q2**4 + 18*q1**5*q2**3 - 72*q1**5*q2**2 + 168*q1**5*q2 - 252*q1**5 - 2*q1**4*q2**5 + 18*q1**4*q2**4 - 72*q1**4*q2**3 + 168*q1**4*q2**2 - 252*q1**4*q2 + 252*q1**4 - 2*q1**3*q2**6 + 18*q1**3*q2**5 - 72*q1**3*q2**4 + 168*q1**3*q2**3 - 252*q1**3*q2**2 + 252*q1**3*q2 - 168*q1**3 - 2*q1**2*q2**7 + 18*q1**2*q2**6 - 72*q1**2*q2**5 + 168*q1**2*q2**4 - 252*q1**2*q2**3 + 252*q1**2*q2**2 - 168*q1**2*q2 + 72*q1**2 - 2*q1*q2**8 + 18*q1*q2**7 - 72*q1*q2**6 + 168*q1*q2**5 - 252*q1*q2**4 + 252*q1*q2**3 - 168*q1*q2**2 + 72*q1*q2 - 18*q1)'
f_multiparam[10][3] = '5*p*(p - 1)**7*(2*p**2*q1**8*q2 + 2*p**2*q1**7*q2**2 + 2*p**2*q1**7*q2*q3 - 18*p**2*q1**7*q2 + 2*p**2*q1**6*q2**3 + 2*p**2*q1**6*q2**2*q3 - 18*p**2*q1**6*q2**2 + 2*p**2*q1**6*q2*q3**2 - 18*p**2*q1**6*q2*q3 + 72*p**2*q1**6*q2 + 2*p**2*q1**5*q2**4 + 2*p**2*q1**5*q2**3*q3 - 18*p**2*q1**5*q2**3 + 2*p**2*q1**5*q2**2*q3**2 - 18*p**2*q1**5*q2**2*q3 + 72*p**2*q1**5*q2**2 + 2*p**2*q1**5*q2*q3**3 - 18*p**2*q1**5*q2*q3**2 + 72*p**2*q1**5*q2*q3 - 168*p**2*q1**5*q2 + 2*p**2*q1**4*q2**5 + 2*p**2*q1**4*q2**4*q3 - 18*p**2*q1**4*q2**4 + 2*p**2*q1**4*q2**3*q3**2 - 18*p**2*q1**4*q2**3*q3 + 72*p**2*q1**4*q2**3 + 2*p**2*q1**4*q2**2*q3**3 - 18*p**2*q1**4*q2**2*q3**2 + 72*p**2*q1**4*q2**2*q3 - 168*p**2*q1**4*q2**2 + 2*p**2*q1**4*q2*q3**4 - 18*p**2*q1**4*q2*q3**3 + 72*p**2*q1**4*q2*q3**2 - 168*p**2*q1**4*q2*q3 + 252*p**2*q1**4*q2 + 2*p**2*q1**3*q2**6 + 2*p**2*q1**3*q2**5*q3 - 18*p**2*q1**3*q2**5 + 2*p**2*q1**3*q2**4*q3**2 - 18*p**2*q1**3*q2**4*q3 + 72*p**2*q1**3*q2**4 + 2*p**2*q1**3*q2**3*q3**3 - 18*p**2*q1**3*q2**3*q3**2 + 72*p**2*q1**3*q2**3*q3 - 168*p**2*q1**3*q2**3 + 2*p**2*q1**3*q2**2*q3**4 - 18*p**2*q1**3*q2**2*q3**3 + 72*p**2*q1**3*q2**2*q3**2 - 168*p**2*q1**3*q2**2*q3 + 252*p**2*q1**3*q2**2 + 2*p**2*q1**3*q2*q3**5 - 18*p**2*q1**3*q2*q3**4 + 72*p**2*q1**3*q2*q3**3 - 168*p**2*q1**3*q2*q3**2 + 252*p**2*q1**3*q2*q3 - 252*p**2*q1**3*q2 + 2*p**2*q1**2*q2**7 + 2*p**2*q1**2*q2**6*q3 - 18*p**2*q1**2*q2**6 + 2*p**2*q1**2*q2**5*q3**2 - 18*p**2*q1**2*q2**5*q3 + 72*p**2*q1**2*q2**5 + 2*p**2*q1**2*q2**4*q3**3 - 18*p**2*q1**2*q2**4*q3**2 + 72*p**2*q1**2*q2**4*q3 - 168*p**2*q1**2*q2**4 + 2*p**2*q1**2*q2**3*q3**4 - 18*p**2*q1**2*q2**3*q3**3 + 72*p**2*q1**2*q2**3*q3**2 - 168*p**2*q1**2*q2**3*q3 + 252*p**2*q1**2*q2**3 + 2*p**2*q1**2*q2**2*q3**5 - 18*p**2*q1**2*q2**2*q3**4 + 72*p**2*q1**2*q2**2*q3**3 - 168*p**2*q1**2*q2**2*q3**2 + 252*p**2*q1**2*q2**2*q3 - 252*p**2*q1**2*q2**2 + 2*p**2*q1**2*q2*q3**6 - 18*p**2*q1**2*q2*q3**5 + 72*p**2*q1**2*q2*q3**4 - 168*p**2*q1**2*q2*q3**3 + 252*p**2*q1**2*q2*q3**2 - 252*p**2*q1**2*q2*q3 + 168*p**2*q1**2*q2 + 2*p**2*q1*q2**8 + 2*p**2*q1*q2**7*q3 - 18*p**2*q1*q2**7 + 2*p**2*q1*q2**6*q3**2 - 18*p**2*q1*q2**6*q3 + 72*p**2*q1*q2**6 + 2*p**2*q1*q2**5*q3**3 - 18*p**2*q1*q2**5*q3**2 + 72*p**2*q1*q2**5*q3 - 168*p**2*q1*q2**5 + 2*p**2*q1*q2**4*q3**4 - 18*p**2*q1*q2**4*q3**3 + 72*p**2*q1*q2**4*q3**2 - 168*p**2*q1*q2**4*q3 + 252*p**2*q1*q2**4 + 2*p**2*q1*q2**3*q3**5 - 18*p**2*q1*q2**3*q3**4 + 72*p**2*q1*q2**3*q3**3 - 168*p**2*q1*q2**3*q3**2 + 252*p**2*q1*q2**3*q3 - 252*p**2*q1*q2**3 + 2*p**2*q1*q2**2*q3**6 - 18*p**2*q1*q2**2*q3**5 + 72*p**2*q1*q2**2*q3**4 - 168*p**2*q1*q2**2*q3**3 + 252*p**2*q1*q2**2*q3**2 - 252*p**2*q1*q2**2*q3 + 168*p**2*q1*q2**2 + 2*p**2*q1*q2*q3**7 - 18*p**2*q1*q2*q3**6 + 72*p**2*q1*q2*q3**5 - 168*p**2*q1*q2*q3**4 + 252*p**2*q1*q2*q3**3 - 252*p**2*q1*q2*q3**2 + 168*p**2*q1*q2*q3 - 72*p**2*q1*q2 - 9*p**2*q2**8 - 9*p**2*q2**7*q3 + 72*p**2*q2**7 - 9*p**2*q2**6*q3**2 + 72*p**2*q2**6*q3 - 252*p**2*q2**6 - 9*p**2*q2**5*q3**3 + 72*p**2*q2**5*q3**2 - 252*p**2*q2**5*q3 + 504*p**2*q2**5 - 9*p**2*q2**4*q3**4 + 72*p**2*q2**4*q3**3 - 252*p**2*q2**4*q3**2 + 504*p**2*q2**4*q3 - 630*p**2*q2**4 - 9*p**2*q2**3*q3**5 + 72*p**2*q2**3*q3**4 - 252*p**2*q2**3*q3**3 + 504*p**2*q2**3*q3**2 - 630*p**2*q2**3*q3 + 504*p**2*q2**3 - 9*p**2*q2**2*q3**6 + 72*p**2*q2**2*q3**5 - 252*p**2*q2**2*q3**4 + 504*p**2*q2**2*q3**3 - 630*p**2*q2**2*q3**2 + 504*p**2*q2**2*q3 - 252*p**2*q2**2 - 9*p**2*q2*q3**7 + 72*p**2*q2*q3**6 - 252*p**2*q2*q3**5 + 504*p**2*q2*q3**4 - 630*p**2*q2*q3**3 + 504*p**2*q2*q3**2 - 252*p**2*q2*q3 + 72*p**2*q2 + 24*p**2*q3**7 - 168*p**2*q3**6 + 504*p**2*q3**5 - 840*p**2*q3**4 + 840*p**2*q3**3 - 504*p**2*q3**2 + 168*p**2*q3 - 24*p**2 - 4*p*q1**8*q2 - 4*p*q1**7*q2**2 - 4*p*q1**7*q2*q3 + 36*p*q1**7*q2 - 4*p*q1**6*q2**3 - 4*p*q1**6*q2**2*q3 + 36*p*q1**6*q2**2 - 4*p*q1**6*q2*q3**2 + 36*p*q1**6*q2*q3 - 144*p*q1**6*q2 - 4*p*q1**5*q2**4 - 4*p*q1**5*q2**3*q3 + 36*p*q1**5*q2**3 - 4*p*q1**5*q2**2*q3**2 + 36*p*q1**5*q2**2*q3 - 144*p*q1**5*q2**2 - 4*p*q1**5*q2*q3**3 + 36*p*q1**5*q2*q3**2 - 144*p*q1**5*q2*q3 + 336*p*q1**5*q2 - 4*p*q1**4*q2**5 - 4*p*q1**4*q2**4*q3 + 36*p*q1**4*q2**4 - 4*p*q1**4*q2**3*q3**2 + 36*p*q1**4*q2**3*q3 - 144*p*q1**4*q2**3 - 4*p*q1**4*q2**2*q3**3 + 36*p*q1**4*q2**2*q3**2 - 144*p*q1**4*q2**2*q3 + 336*p*q1**4*q2**2 - 4*p*q1**4*q2*q3**4 + 36*p*q1**4*q2*q3**3 - 144*p*q1**4*q2*q3**2 + 336*p*q1**4*q2*q3 - 504*p*q1**4*q2 - 4*p*q1**3*q2**6 - 4*p*q1**3*q2**5*q3 + 36*p*q1**3*q2**5 - 4*p*q1**3*q2**4*q3**2 + 36*p*q1**3*q2**4*q3 - 144*p*q1**3*q2**4 - 4*p*q1**3*q2**3*q3**3 + 36*p*q1**3*q2**3*q3**2 - 144*p*q1**3*q2**3*q3 + 336*p*q1**3*q2**3 - 4*p*q1**3*q2**2*q3**4 + 36*p*q1**3*q2**2*q3**3 - 144*p*q1**3*q2**2*q3**2 + 336*p*q1**3*q2**2*q3 - 504*p*q1**3*q2**2 - 4*p*q1**3*q2*q3**5 + 36*p*q1**3*q2*q3**4 - 144*p*q1**3*q2*q3**3 + 336*p*q1**3*q2*q3**2 - 504*p*q1**3*q2*q3 + 504*p*q1**3*q2 - 4*p*q1**2*q2**7 - 4*p*q1**2*q2**6*q3 + 36*p*q1**2*q2**6 - 4*p*q1**2*q2**5*q3**2 + 36*p*q1**2*q2**5*q3 - 144*p*q1**2*q2**5 - 4*p*q1**2*q2**4*q3**3 + 36*p*q1**2*q2**4*q3**2 - 144*p*q1**2*q2**4*q3 + 336*p*q1**2*q2**4 - 4*p*q1**2*q2**3*q3**4 + 36*p*q1**2*q2**3*q3**3 - 144*p*q1**2*q2**3*q3**2 + 336*p*q1**2*q2**3*q3 - 504*p*q1**2*q2**3 - 4*p*q1**2*q2**2*q3**5 + 36*p*q1**2*q2**2*q3**4 - 144*p*q1**2*q2**2*q3**3 + 336*p*q1**2*q2**2*q3**2 - 504*p*q1**2*q2**2*q3 + 504*p*q1**2*q2**2 - 4*p*q1**2*q2*q3**6 + 36*p*q1**2*q2*q3**5 - 144*p*q1**2*q2*q3**4 + 336*p*q1**2*q2*q3**3 - 504*p*q1**2*q2*q3**2 + 504*p*q1**2*q2*q3 - 336*p*q1**2*q2 - 4*p*q1*q2**8 - 4*p*q1*q2**7*q3 + 36*p*q1*q2**7 - 4*p*q1*q2**6*q3**2 + 36*p*q1*q2**6*q3 - 144*p*q1*q2**6 - 4*p*q1*q2**5*q3**3 + 36*p*q1*q2**5*q3**2 - 144*p*q1*q2**5*q3 + 336*p*q1*q2**5 - 4*p*q1*q2**4*q3**4 + 36*p*q1*q2**4*q3**3 - 144*p*q1*q2**4*q3**2 + 336*p*q1*q2**4*q3 - 504*p*q1*q2**4 - 4*p*q1*q2**3*q3**5 + 36*p*q1*q2**3*q3**4 - 144*p*q1*q2**3*q3**3 + 336*p*q1*q2**3*q3**2 - 504*p*q1*q2**3*q3 + 504*p*q1*q2**3 - 4*p*q1*q2**2*q3**6 + 36*p*q1*q2**2*q3**5 - 144*p*q1*q2**2*q3**4 + 336*p*q1*q2**2*q3**3 - 504*p*q1*q2**2*q3**2 + 504*p*q1*q2**2*q3 - 336*p*q1*q2**2 - 4*p*q1*q2*q3**7 + 36*p*q1*q2*q3**6 - 144*p*q1*q2*q3**5 + 336*p*q1*q2*q3**4 - 504*p*q1*q2*q3**3 + 504*p*q1*q2*q3**2 - 336*p*q1*q2*q3 + 144*p*q1*q2 + 9*p*q2**8 + 9*p*q2**7*q3 - 72*p*q2**7 + 9*p*q2**6*q3**2 - 72*p*q2**6*q3 + 252*p*q2**6 + 9*p*q2**5*q3**3 - 72*p*q2**5*q3**2 + 252*p*q2**5*q3 - 504*p*q2**5 + 9*p*q2**4*q3**4 - 72*p*q2**4*q3**3 + 252*p*q2**4*q3**2 - 504*p*q2**4*q3 + 630*p*q2**4 + 9*p*q2**3*q3**5 - 72*p*q2**3*q3**4 + 252*p*q2**3*q3**3 - 504*p*q2**3*q3**2 + 630*p*q2**3*q3 - 504*p*q2**3 + 9*p*q2**2*q3**6 - 72*p*q2**2*q3**5 + 252*p*q2**2*q3**4 - 504*p*q2**2*q3**3 + 630*p*q2**2*q3**2 - 504*p*q2**2*q3 + 252*p*q2**2 + 9*p*q2*q3**7 - 72*p*q2*q3**6 + 252*p*q2*q3**5 - 504*p*q2*q3**4 + 630*p*q2*q3**3 - 504*p*q2*q3**2 + 252*p*q2*q3 - 72*p*q2 + 2*q1**8*q2 + 2*q1**7*q2**2 + 2*q1**7*q2*q3 - 18*q1**7*q2 + 2*q1**6*q2**3 + 2*q1**6*q2**2*q3 - 18*q1**6*q2**2 + 2*q1**6*q2*q3**2 - 18*q1**6*q2*q3 + 72*q1**6*q2 + 2*q1**5*q2**4 + 2*q1**5*q2**3*q3 - 18*q1**5*q2**3 + 2*q1**5*q2**2*q3**2 - 18*q1**5*q2**2*q3 + 72*q1**5*q2**2 + 2*q1**5*q2*q3**3 - 18*q1**5*q2*q3**2 + 72*q1**5*q2*q3 - 168*q1**5*q2 + 2*q1**4*q2**5 + 2*q1**4*q2**4*q3 - 18*q1**4*q2**4 + 2*q1**4*q2**3*q3**2 - 18*q1**4*q2**3*q3 + 72*q1**4*q2**3 + 2*q1**4*q2**2*q3**3 - 18*q1**4*q2**2*q3**2 + 72*q1**4*q2**2*q3 - 168*q1**4*q2**2 + 2*q1**4*q2*q3**4 - 18*q1**4*q2*q3**3 + 72*q1**4*q2*q3**2 - 168*q1**4*q2*q3 + 252*q1**4*q2 + 2*q1**3*q2**6 + 2*q1**3*q2**5*q3 - 18*q1**3*q2**5 + 2*q1**3*q2**4*q3**2 - 18*q1**3*q2**4*q3 + 72*q1**3*q2**4 + 2*q1**3*q2**3*q3**3 - 18*q1**3*q2**3*q3**2 + 72*q1**3*q2**3*q3 - 168*q1**3*q2**3 + 2*q1**3*q2**2*q3**4 - 18*q1**3*q2**2*q3**3 + 72*q1**3*q2**2*q3**2 - 168*q1**3*q2**2*q3 + 252*q1**3*q2**2 + 2*q1**3*q2*q3**5 - 18*q1**3*q2*q3**4 + 72*q1**3*q2*q3**3 - 168*q1**3*q2*q3**2 + 252*q1**3*q2*q3 - 252*q1**3*q2 + 2*q1**2*q2**7 + 2*q1**2*q2**6*q3 - 18*q1**2*q2**6 + 2*q1**2*q2**5*q3**2 - 18*q1**2*q2**5*q3 + 72*q1**2*q2**5 + 2*q1**2*q2**4*q3**3 - 18*q1**2*q2**4*q3**2 + 72*q1**2*q2**4*q3 - 168*q1**2*q2**4 + 2*q1**2*q2**3*q3**4 - 18*q1**2*q2**3*q3**3 + 72*q1**2*q2**3*q3**2 - 168*q1**2*q2**3*q3 + 252*q1**2*q2**3 + 2*q1**2*q2**2*q3**5 - 18*q1**2*q2**2*q3**4 + 72*q1**2*q2**2*q3**3 - 168*q1**2*q2**2*q3**2 + 252*q1**2*q2**2*q3 - 252*q1**2*q2**2 + 2*q1**2*q2*q3**6 - 18*q1**2*q2*q3**5 + 72*q1**2*q2*q3**4 - 168*q1**2*q2*q3**3 + 252*q1**2*q2*q3**2 - 252*q1**2*q2*q3 + 168*q1**2*q2 + 2*q1*q2**8 + 2*q1*q2**7*q3 - 18*q1*q2**7 + 2*q1*q2**6*q3**2 - 18*q1*q2**6*q3 + 72*q1*q2**6 + 2*q1*q2**5*q3**3 - 18*q1*q2**5*q3**2 + 72*q1*q2**5*q3 - 168*q1*q2**5 + 2*q1*q2**4*q3**4 - 18*q1*q2**4*q3**3 + 72*q1*q2**4*q3**2 - 168*q1*q2**4*q3 + 252*q1*q2**4 + 2*q1*q2**3*q3**5 - 18*q1*q2**3*q3**4 + 72*q1*q2**3*q3**3 - 168*q1*q2**3*q3**2 + 252*q1*q2**3*q3 - 252*q1*q2**3 + 2*q1*q2**2*q3**6 - 18*q1*q2**2*q3**5 + 72*q1*q2**2*q3**4 - 168*q1*q2**2*q3**3 + 252*q1*q2**2*q3**2 - 252*q1*q2**2*q3 + 168*q1*q2**2 + 2*q1*q2*q3**7 - 18*q1*q2*q3**6 + 72*q1*q2*q3**5 - 168*q1*q2*q3**4 + 252*q1*q2*q3**3 - 252*q1*q2*q3**2 + 168*q1*q2*q3 - 72*q1*q2)'
f_multiparam[10][4] = '-5*p*(p - 1)**6*(2*p**3*q1**7*q2*q3 + 2*p**3*q1**6*q2**2*q3 + 2*p**3*q1**6*q2*q3**2 + 2*p**3*q1**6*q2*q3*q4 - 18*p**3*q1**6*q2*q3 + 2*p**3*q1**5*q2**3*q3 + 2*p**3*q1**5*q2**2*q3**2 + 2*p**3*q1**5*q2**2*q3*q4 - 18*p**3*q1**5*q2**2*q3 + 2*p**3*q1**5*q2*q3**3 + 2*p**3*q1**5*q2*q3**2*q4 - 18*p**3*q1**5*q2*q3**2 + 2*p**3*q1**5*q2*q3*q4**2 - 18*p**3*q1**5*q2*q3*q4 + 72*p**3*q1**5*q2*q3 + 2*p**3*q1**4*q2**4*q3 + 2*p**3*q1**4*q2**3*q3**2 + 2*p**3*q1**4*q2**3*q3*q4 - 18*p**3*q1**4*q2**3*q3 + 2*p**3*q1**4*q2**2*q3**3 + 2*p**3*q1**4*q2**2*q3**2*q4 - 18*p**3*q1**4*q2**2*q3**2 + 2*p**3*q1**4*q2**2*q3*q4**2 - 18*p**3*q1**4*q2**2*q3*q4 + 72*p**3*q1**4*q2**2*q3 + 2*p**3*q1**4*q2*q3**4 + 2*p**3*q1**4*q2*q3**3*q4 - 18*p**3*q1**4*q2*q3**3 + 2*p**3*q1**4*q2*q3**2*q4**2 - 18*p**3*q1**4*q2*q3**2*q4 + 72*p**3*q1**4*q2*q3**2 + 2*p**3*q1**4*q2*q3*q4**3 - 18*p**3*q1**4*q2*q3*q4**2 + 72*p**3*q1**4*q2*q3*q4 - 168*p**3*q1**4*q2*q3 + 2*p**3*q1**3*q2**5*q3 + 2*p**3*q1**3*q2**4*q3**2 + 2*p**3*q1**3*q2**4*q3*q4 - 18*p**3*q1**3*q2**4*q3 + 2*p**3*q1**3*q2**3*q3**3 + 2*p**3*q1**3*q2**3*q3**2*q4 - 18*p**3*q1**3*q2**3*q3**2 + 2*p**3*q1**3*q2**3*q3*q4**2 - 18*p**3*q1**3*q2**3*q3*q4 + 72*p**3*q1**3*q2**3*q3 + 2*p**3*q1**3*q2**2*q3**4 + 2*p**3*q1**3*q2**2*q3**3*q4 - 18*p**3*q1**3*q2**2*q3**3 + 2*p**3*q1**3*q2**2*q3**2*q4**2 - 18*p**3*q1**3*q2**2*q3**2*q4 + 72*p**3*q1**3*q2**2*q3**2 + 2*p**3*q1**3*q2**2*q3*q4**3 - 18*p**3*q1**3*q2**2*q3*q4**2 + 72*p**3*q1**3*q2**2*q3*q4 - 168*p**3*q1**3*q2**2*q3 + 2*p**3*q1**3*q2*q3**5 + 2*p**3*q1**3*q2*q3**4*q4 - 18*p**3*q1**3*q2*q3**4 + 2*p**3*q1**3*q2*q3**3*q4**2 - 18*p**3*q1**3*q2*q3**3*q4 + 72*p**3*q1**3*q2*q3**3 + 2*p**3*q1**3*q2*q3**2*q4**3 - 18*p**3*q1**3*q2*q3**2*q4**2 + 72*p**3*q1**3*q2*q3**2*q4 - 168*p**3*q1**3*q2*q3**2 + 2*p**3*q1**3*q2*q3*q4**4 - 18*p**3*q1**3*q2*q3*q4**3 + 72*p**3*q1**3*q2*q3*q4**2 - 168*p**3*q1**3*q2*q3*q4 + 252*p**3*q1**3*q2*q3 + 2*p**3*q1**2*q2**6*q3 + 2*p**3*q1**2*q2**5*q3**2 + 2*p**3*q1**2*q2**5*q3*q4 - 18*p**3*q1**2*q2**5*q3 + 2*p**3*q1**2*q2**4*q3**3 + 2*p**3*q1**2*q2**4*q3**2*q4 - 18*p**3*q1**2*q2**4*q3**2 + 2*p**3*q1**2*q2**4*q3*q4**2 - 18*p**3*q1**2*q2**4*q3*q4 + 72*p**3*q1**2*q2**4*q3 + 2*p**3*q1**2*q2**3*q3**4 + 2*p**3*q1**2*q2**3*q3**3*q4 - 18*p**3*q1**2*q2**3*q3**3 + 2*p**3*q1**2*q2**3*q3**2*q4**2 - 18*p**3*q1**2*q2**3*q3**2*q4 + 72*p**3*q1**2*q2**3*q3**2 + 2*p**3*q1**2*q2**3*q3*q4**3 - 18*p**3*q1**2*q2**3*q3*q4**2 + 72*p**3*q1**2*q2**3*q3*q4 - 168*p**3*q1**2*q2**3*q3 + 2*p**3*q1**2*q2**2*q3**5 + 2*p**3*q1**2*q2**2*q3**4*q4 - 18*p**3*q1**2*q2**2*q3**4 + 2*p**3*q1**2*q2**2*q3**3*q4**2 - 18*p**3*q1**2*q2**2*q3**3*q4 + 72*p**3*q1**2*q2**2*q3**3 + 2*p**3*q1**2*q2**2*q3**2*q4**3 - 18*p**3*q1**2*q2**2*q3**2*q4**2 + 72*p**3*q1**2*q2**2*q3**2*q4 - 168*p**3*q1**2*q2**2*q3**2 + 2*p**3*q1**2*q2**2*q3*q4**4 - 18*p**3*q1**2*q2**2*q3*q4**3 + 72*p**3*q1**2*q2**2*q3*q4**2 - 168*p**3*q1**2*q2**2*q3*q4 + 252*p**3*q1**2*q2**2*q3 + 2*p**3*q1**2*q2*q3**6 + 2*p**3*q1**2*q2*q3**5*q4 - 18*p**3*q1**2*q2*q3**5 + 2*p**3*q1**2*q2*q3**4*q4**2 - 18*p**3*q1**2*q2*q3**4*q4 + 72*p**3*q1**2*q2*q3**4 + 2*p**3*q1**2*q2*q3**3*q4**3 - 18*p**3*q1**2*q2*q3**3*q4**2 + 72*p**3*q1**2*q2*q3**3*q4 - 168*p**3*q1**2*q2*q3**3 + 2*p**3*q1**2*q2*q3**2*q4**4 - 18*p**3*q1**2*q2*q3**2*q4**3 + 72*p**3*q1**2*q2*q3**2*q4**2 - 168*p**3*q1**2*q2*q3**2*q4 + 252*p**3*q1**2*q2*q3**2 + 2*p**3*q1**2*q2*q3*q4**5 - 18*p**3*q1**2*q2*q3*q4**4 + 72*p**3*q1**2*q2*q3*q4**3 - 168*p**3*q1**2*q2*q3*q4**2 + 252*p**3*q1**2*q2*q3*q4 - 252*p**3*q1**2*q2*q3 + 2*p**3*q1*q2**7*q3 + 2*p**3*q1*q2**6*q3**2 + 2*p**3*q1*q2**6*q3*q4 - 18*p**3*q1*q2**6*q3 + 2*p**3*q1*q2**5*q3**3 + 2*p**3*q1*q2**5*q3**2*q4 - 18*p**3*q1*q2**5*q3**2 + 2*p**3*q1*q2**5*q3*q4**2 - 18*p**3*q1*q2**5*q3*q4 + 72*p**3*q1*q2**5*q3 + 2*p**3*q1*q2**4*q3**4 + 2*p**3*q1*q2**4*q3**3*q4 - 18*p**3*q1*q2**4*q3**3 + 2*p**3*q1*q2**4*q3**2*q4**2 - 18*p**3*q1*q2**4*q3**2*q4 + 72*p**3*q1*q2**4*q3**2 + 2*p**3*q1*q2**4*q3*q4**3 - 18*p**3*q1*q2**4*q3*q4**2 + 72*p**3*q1*q2**4*q3*q4 - 168*p**3*q1*q2**4*q3 + 2*p**3*q1*q2**3*q3**5 + 2*p**3*q1*q2**3*q3**4*q4 - 18*p**3*q1*q2**3*q3**4 + 2*p**3*q1*q2**3*q3**3*q4**2 - 18*p**3*q1*q2**3*q3**3*q4 + 72*p**3*q1*q2**3*q3**3 + 2*p**3*q1*q2**3*q3**2*q4**3 - 18*p**3*q1*q2**3*q3**2*q4**2 + 72*p**3*q1*q2**3*q3**2*q4 - 168*p**3*q1*q2**3*q3**2 + 2*p**3*q1*q2**3*q3*q4**4 - 18*p**3*q1*q2**3*q3*q4**3 + 72*p**3*q1*q2**3*q3*q4**2 - 168*p**3*q1*q2**3*q3*q4 + 252*p**3*q1*q2**3*q3 + 2*p**3*q1*q2**2*q3**6 + 2*p**3*q1*q2**2*q3**5*q4 - 18*p**3*q1*q2**2*q3**5 + 2*p**3*q1*q2**2*q3**4*q4**2 - 18*p**3*q1*q2**2*q3**4*q4 + 72*p**3*q1*q2**2*q3**4 + 2*p**3*q1*q2**2*q3**3*q4**3 - 18*p**3*q1*q2**2*q3**3*q4**2 + 72*p**3*q1*q2**2*q3**3*q4 - 168*p**3*q1*q2**2*q3**3 + 2*p**3*q1*q2**2*q3**2*q4**4 - 18*p**3*q1*q2**2*q3**2*q4**3 + 72*p**3*q1*q2**2*q3**2*q4**2 - 168*p**3*q1*q2**2*q3**2*q4 + 252*p**3*q1*q2**2*q3**2 + 2*p**3*q1*q2**2*q3*q4**5 - 18*p**3*q1*q2**2*q3*q4**4 + 72*p**3*q1*q2**2*q3*q4**3 - 168*p**3*q1*q2**2*q3*q4**2 + 252*p**3*q1*q2**2*q3*q4 - 252*p**3*q1*q2**2*q3 + 2*p**3*q1*q2*q3**7 + 2*p**3*q1*q2*q3**6*q4 - 18*p**3*q1*q2*q3**6 + 2*p**3*q1*q2*q3**5*q4**2 - 18*p**3*q1*q2*q3**5*q4 + 72*p**3*q1*q2*q3**5 + 2*p**3*q1*q2*q3**4*q4**3 - 18*p**3*q1*q2*q3**4*q4**2 + 72*p**3*q1*q2*q3**4*q4 - 168*p**3*q1*q2*q3**4 + 2*p**3*q1*q2*q3**3*q4**4 - 18*p**3*q1*q2*q3**3*q4**3 + 72*p**3*q1*q2*q3**3*q4**2 - 168*p**3*q1*q2*q3**3*q4 + 252*p**3*q1*q2*q3**3 + 2*p**3*q1*q2*q3**2*q4**5 - 18*p**3*q1*q2*q3**2*q4**4 + 72*p**3*q1*q2*q3**2*q4**3 - 168*p**3*q1*q2*q3**2*q4**2 + 252*p**3*q1*q2*q3**2*q4 - 252*p**3*q1*q2*q3**2 + 2*p**3*q1*q2*q3*q4**6 - 18*p**3*q1*q2*q3*q4**5 + 72*p**3*q1*q2*q3*q4**4 - 168*p**3*q1*q2*q3*q4**3 + 252*p**3*q1*q2*q3*q4**2 - 252*p**3*q1*q2*q3*q4 + 168*p**3*q1*q2*q3 - 9*p**3*q2**7*q3 - 9*p**3*q2**6*q3**2 - 9*p**3*q2**6*q3*q4 + 72*p**3*q2**6*q3 - 9*p**3*q2**5*q3**3 - 9*p**3*q2**5*q3**2*q4 + 72*p**3*q2**5*q3**2 - 9*p**3*q2**5*q3*q4**2 + 72*p**3*q2**5*q3*q4 - 252*p**3*q2**5*q3 - 9*p**3*q2**4*q3**4 - 9*p**3*q2**4*q3**3*q4 + 72*p**3*q2**4*q3**3 - 9*p**3*q2**4*q3**2*q4**2 + 72*p**3*q2**4*q3**2*q4 - 252*p**3*q2**4*q3**2 - 9*p**3*q2**4*q3*q4**3 + 72*p**3*q2**4*q3*q4**2 - 252*p**3*q2**4*q3*q4 + 504*p**3*q2**4*q3 - 9*p**3*q2**3*q3**5 - 9*p**3*q2**3*q3**4*q4 + 72*p**3*q2**3*q3**4 - 9*p**3*q2**3*q3**3*q4**2 + 72*p**3*q2**3*q3**3*q4 - 252*p**3*q2**3*q3**3 - 9*p**3*q2**3*q3**2*q4**3 + 72*p**3*q2**3*q3**2*q4**2 - 252*p**3*q2**3*q3**2*q4 + 504*p**3*q2**3*q3**2 - 9*p**3*q2**3*q3*q4**4 + 72*p**3*q2**3*q3*q4**3 - 252*p**3*q2**3*q3*q4**2 + 504*p**3*q2**3*q3*q4 - 630*p**3*q2**3*q3 - 9*p**3*q2**2*q3**6 - 9*p**3*q2**2*q3**5*q4 + 72*p**3*q2**2*q3**5 - 9*p**3*q2**2*q3**4*q4**2 + 72*p**3*q2**2*q3**4*q4 - 252*p**3*q2**2*q3**4 - 9*p**3*q2**2*q3**3*q4**3 + 72*p**3*q2**2*q3**3*q4**2 - 252*p**3*q2**2*q3**3*q4 + 504*p**3*q2**2*q3**3 - 9*p**3*q2**2*q3**2*q4**4 + 72*p**3*q2**2*q3**2*q4**3 - 252*p**3*q2**2*q3**2*q4**2 + 504*p**3*q2**2*q3**2*q4 - 630*p**3*q2**2*q3**2 - 9*p**3*q2**2*q3*q4**5 + 72*p**3*q2**2*q3*q4**4 - 252*p**3*q2**2*q3*q4**3 + 504*p**3*q2**2*q3*q4**2 - 630*p**3*q2**2*q3*q4 + 504*p**3*q2**2*q3 - 9*p**3*q2*q3**7 - 9*p**3*q2*q3**6*q4 + 72*p**3*q2*q3**6 - 9*p**3*q2*q3**5*q4**2 + 72*p**3*q2*q3**5*q4 - 252*p**3*q2*q3**5 - 9*p**3*q2*q3**4*q4**3 + 72*p**3*q2*q3**4*q4**2 - 252*p**3*q2*q3**4*q4 + 504*p**3*q2*q3**4 - 9*p**3*q2*q3**3*q4**4 + 72*p**3*q2*q3**3*q4**3 - 252*p**3*q2*q3**3*q4**2 + 504*p**3*q2*q3**3*q4 - 630*p**3*q2*q3**3 - 9*p**3*q2*q3**2*q4**5 + 72*p**3*q2*q3**2*q4**4 - 252*p**3*q2*q3**2*q4**3 + 504*p**3*q2*q3**2*q4**2 - 630*p**3*q2*q3**2*q4 + 504*p**3*q2*q3**2 - 9*p**3*q2*q3*q4**6 + 72*p**3*q2*q3*q4**5 - 252*p**3*q2*q3*q4**4 + 504*p**3*q2*q3*q4**3 - 630*p**3*q2*q3*q4**2 + 504*p**3*q2*q3*q4 - 252*p**3*q2*q3 + 24*p**3*q3**7 + 24*p**3*q3**6*q4 - 168*p**3*q3**6 + 24*p**3*q3**5*q4**2 - 168*p**3*q3**5*q4 + 504*p**3*q3**5 + 24*p**3*q3**4*q4**3 - 168*p**3*q3**4*q4**2 + 504*p**3*q3**4*q4 - 840*p**3*q3**4 + 24*p**3*q3**3*q4**4 - 168*p**3*q3**3*q4**3 + 504*p**3*q3**3*q4**2 - 840*p**3*q3**3*q4 + 840*p**3*q3**3 + 24*p**3*q3**2*q4**5 - 168*p**3*q3**2*q4**4 + 504*p**3*q3**2*q4**3 - 840*p**3*q3**2*q4**2 + 840*p**3*q3**2*q4 - 504*p**3*q3**2 + 24*p**3*q3*q4**6 - 168*p**3*q3*q4**5 + 504*p**3*q3*q4**4 - 840*p**3*q3*q4**3 + 840*p**3*q3*q4**2 - 504*p**3*q3*q4 + 168*p**3*q3 - 42*p**3*q4**6 + 252*p**3*q4**5 - 630*p**3*q4**4 + 840*p**3*q4**3 - 630*p**3*q4**2 + 252*p**3*q4 - 42*p**3 - 6*p**2*q1**7*q2*q3 - 6*p**2*q1**6*q2**2*q3 - 6*p**2*q1**6*q2*q3**2 - 6*p**2*q1**6*q2*q3*q4 + 54*p**2*q1**6*q2*q3 - 6*p**2*q1**5*q2**3*q3 - 6*p**2*q1**5*q2**2*q3**2 - 6*p**2*q1**5*q2**2*q3*q4 + 54*p**2*q1**5*q2**2*q3 - 6*p**2*q1**5*q2*q3**3 - 6*p**2*q1**5*q2*q3**2*q4 + 54*p**2*q1**5*q2*q3**2 - 6*p**2*q1**5*q2*q3*q4**2 + 54*p**2*q1**5*q2*q3*q4 - 216*p**2*q1**5*q2*q3 - 6*p**2*q1**4*q2**4*q3 - 6*p**2*q1**4*q2**3*q3**2 - 6*p**2*q1**4*q2**3*q3*q4 + 54*p**2*q1**4*q2**3*q3 - 6*p**2*q1**4*q2**2*q3**3 - 6*p**2*q1**4*q2**2*q3**2*q4 + 54*p**2*q1**4*q2**2*q3**2 - 6*p**2*q1**4*q2**2*q3*q4**2 + 54*p**2*q1**4*q2**2*q3*q4 - 216*p**2*q1**4*q2**2*q3 - 6*p**2*q1**4*q2*q3**4 - 6*p**2*q1**4*q2*q3**3*q4 + 54*p**2*q1**4*q2*q3**3 - 6*p**2*q1**4*q2*q3**2*q4**2 + 54*p**2*q1**4*q2*q3**2*q4 - 216*p**2*q1**4*q2*q3**2 - 6*p**2*q1**4*q2*q3*q4**3 + 54*p**2*q1**4*q2*q3*q4**2 - 216*p**2*q1**4*q2*q3*q4 + 504*p**2*q1**4*q2*q3 - 6*p**2*q1**3*q2**5*q3 - 6*p**2*q1**3*q2**4*q3**2 - 6*p**2*q1**3*q2**4*q3*q4 + 54*p**2*q1**3*q2**4*q3 - 6*p**2*q1**3*q2**3*q3**3 - 6*p**2*q1**3*q2**3*q3**2*q4 + 54*p**2*q1**3*q2**3*q3**2 - 6*p**2*q1**3*q2**3*q3*q4**2 + 54*p**2*q1**3*q2**3*q3*q4 - 216*p**2*q1**3*q2**3*q3 - 6*p**2*q1**3*q2**2*q3**4 - 6*p**2*q1**3*q2**2*q3**3*q4 + 54*p**2*q1**3*q2**2*q3**3 - 6*p**2*q1**3*q2**2*q3**2*q4**2 + 54*p**2*q1**3*q2**2*q3**2*q4 - 216*p**2*q1**3*q2**2*q3**2 - 6*p**2*q1**3*q2**2*q3*q4**3 + 54*p**2*q1**3*q2**2*q3*q4**2 - 216*p**2*q1**3*q2**2*q3*q4 + 504*p**2*q1**3*q2**2*q3 - 6*p**2*q1**3*q2*q3**5 - 6*p**2*q1**3*q2*q3**4*q4 + 54*p**2*q1**3*q2*q3**4 - 6*p**2*q1**3*q2*q3**3*q4**2 + 54*p**2*q1**3*q2*q3**3*q4 - 216*p**2*q1**3*q2*q3**3 - 6*p**2*q1**3*q2*q3**2*q4**3 + 54*p**2*q1**3*q2*q3**2*q4**2 - 216*p**2*q1**3*q2*q3**2*q4 + 504*p**2*q1**3*q2*q3**2 - 6*p**2*q1**3*q2*q3*q4**4 + 54*p**2*q1**3*q2*q3*q4**3 - 216*p**2*q1**3*q2*q3*q4**2 + 504*p**2*q1**3*q2*q3*q4 - 756*p**2*q1**3*q2*q3 - 6*p**2*q1**2*q2**6*q3 - 6*p**2*q1**2*q2**5*q3**2 - 6*p**2*q1**2*q2**5*q3*q4 + 54*p**2*q1**2*q2**5*q3 - 6*p**2*q1**2*q2**4*q3**3 - 6*p**2*q1**2*q2**4*q3**2*q4 + 54*p**2*q1**2*q2**4*q3**2 - 6*p**2*q1**2*q2**4*q3*q4**2 + 54*p**2*q1**2*q2**4*q3*q4 - 216*p**2*q1**2*q2**4*q3 - 6*p**2*q1**2*q2**3*q3**4 - 6*p**2*q1**2*q2**3*q3**3*q4 + 54*p**2*q1**2*q2**3*q3**3 - 6*p**2*q1**2*q2**3*q3**2*q4**2 + 54*p**2*q1**2*q2**3*q3**2*q4 - 216*p**2*q1**2*q2**3*q3**2 - 6*p**2*q1**2*q2**3*q3*q4**3 + 54*p**2*q1**2*q2**3*q3*q4**2 - 216*p**2*q1**2*q2**3*q3*q4 + 504*p**2*q1**2*q2**3*q3 - 6*p**2*q1**2*q2**2*q3**5 - 6*p**2*q1**2*q2**2*q3**4*q4 + 54*p**2*q1**2*q2**2*q3**4 - 6*p**2*q1**2*q2**2*q3**3*q4**2 + 54*p**2*q1**2*q2**2*q3**3*q4 - 216*p**2*q1**2*q2**2*q3**3 - 6*p**2*q1**2*q2**2*q3**2*q4**3 + 54*p**2*q1**2*q2**2*q3**2*q4**2 - 216*p**2*q1**2*q2**2*q3**2*q4 + 504*p**2*q1**2*q2**2*q3**2 - 6*p**2*q1**2*q2**2*q3*q4**4 + 54*p**2*q1**2*q2**2*q3*q4**3 - 216*p**2*q1**2*q2**2*q3*q4**2 + 504*p**2*q1**2*q2**2*q3*q4 - 756*p**2*q1**2*q2**2*q3 - 6*p**2*q1**2*q2*q3**6 - 6*p**2*q1**2*q2*q3**5*q4 + 54*p**2*q1**2*q2*q3**5 - 6*p**2*q1**2*q2*q3**4*q4**2 + 54*p**2*q1**2*q2*q3**4*q4 - 216*p**2*q1**2*q2*q3**4 - 6*p**2*q1**2*q2*q3**3*q4**3 + 54*p**2*q1**2*q2*q3**3*q4**2 - 216*p**2*q1**2*q2*q3**3*q4 + 504*p**2*q1**2*q2*q3**3 - 6*p**2*q1**2*q2*q3**2*q4**4 + 54*p**2*q1**2*q2*q3**2*q4**3 - 216*p**2*q1**2*q2*q3**2*q4**2 + 504*p**2*q1**2*q2*q3**2*q4 - 756*p**2*q1**2*q2*q3**2 - 6*p**2*q1**2*q2*q3*q4**5 + 54*p**2*q1**2*q2*q3*q4**4 - 216*p**2*q1**2*q2*q3*q4**3 + 504*p**2*q1**2*q2*q3*q4**2 - 756*p**2*q1**2*q2*q3*q4 + 756*p**2*q1**2*q2*q3 - 6*p**2*q1*q2**7*q3 - 6*p**2*q1*q2**6*q3**2 - 6*p**2*q1*q2**6*q3*q4 + 54*p**2*q1*q2**6*q3 - 6*p**2*q1*q2**5*q3**3 - 6*p**2*q1*q2**5*q3**2*q4 + 54*p**2*q1*q2**5*q3**2 - 6*p**2*q1*q2**5*q3*q4**2 + 54*p**2*q1*q2**5*q3*q4 - 216*p**2*q1*q2**5*q3 - 6*p**2*q1*q2**4*q3**4 - 6*p**2*q1*q2**4*q3**3*q4 + 54*p**2*q1*q2**4*q3**3 - 6*p**2*q1*q2**4*q3**2*q4**2 + 54*p**2*q1*q2**4*q3**2*q4 - 216*p**2*q1*q2**4*q3**2 - 6*p**2*q1*q2**4*q3*q4**3 + 54*p**2*q1*q2**4*q3*q4**2 - 216*p**2*q1*q2**4*q3*q4 + 504*p**2*q1*q2**4*q3 - 6*p**2*q1*q2**3*q3**5 - 6*p**2*q1*q2**3*q3**4*q4 + 54*p**2*q1*q2**3*q3**4 - 6*p**2*q1*q2**3*q3**3*q4**2 + 54*p**2*q1*q2**3*q3**3*q4 - 216*p**2*q1*q2**3*q3**3 - 6*p**2*q1*q2**3*q3**2*q4**3 + 54*p**2*q1*q2**3*q3**2*q4**2 - 216*p**2*q1*q2**3*q3**2*q4 + 504*p**2*q1*q2**3*q3**2 - 6*p**2*q1*q2**3*q3*q4**4 + 54*p**2*q1*q2**3*q3*q4**3 - 216*p**2*q1*q2**3*q3*q4**2 + 504*p**2*q1*q2**3*q3*q4 - 756*p**2*q1*q2**3*q3 - 6*p**2*q1*q2**2*q3**6 - 6*p**2*q1*q2**2*q3**5*q4 + 54*p**2*q1*q2**2*q3**5 - 6*p**2*q1*q2**2*q3**4*q4**2 + 54*p**2*q1*q2**2*q3**4*q4 - 216*p**2*q1*q2**2*q3**4 - 6*p**2*q1*q2**2*q3**3*q4**3 + 54*p**2*q1*q2**2*q3**3*q4**2 - 216*p**2*q1*q2**2*q3**3*q4 + 504*p**2*q1*q2**2*q3**3 - 6*p**2*q1*q2**2*q3**2*q4**4 + 54*p**2*q1*q2**2*q3**2*q4**3 - 216*p**2*q1*q2**2*q3**2*q4**2 + 504*p**2*q1*q2**2*q3**2*q4 - 756*p**2*q1*q2**2*q3**2 - 6*p**2*q1*q2**2*q3*q4**5 + 54*p**2*q1*q2**2*q3*q4**4 - 216*p**2*q1*q2**2*q3*q4**3 + 504*p**2*q1*q2**2*q3*q4**2 - 756*p**2*q1*q2**2*q3*q4 + 756*p**2*q1*q2**2*q3 - 6*p**2*q1*q2*q3**7 - 6*p**2*q1*q2*q3**6*q4 + 54*p**2*q1*q2*q3**6 - 6*p**2*q1*q2*q3**5*q4**2 + 54*p**2*q1*q2*q3**5*q4 - 216*p**2*q1*q2*q3**5 - 6*p**2*q1*q2*q3**4*q4**3 + 54*p**2*q1*q2*q3**4*q4**2 - 216*p**2*q1*q2*q3**4*q4 + 504*p**2*q1*q2*q3**4 - 6*p**2*q1*q2*q3**3*q4**4 + 54*p**2*q1*q2*q3**3*q4**3 - 216*p**2*q1*q2*q3**3*q4**2 + 504*p**2*q1*q2*q3**3*q4 - 756*p**2*q1*q2*q3**3 - 6*p**2*q1*q2*q3**2*q4**5 + 54*p**2*q1*q2*q3**2*q4**4 - 216*p**2*q1*q2*q3**2*q4**3 + 504*p**2*q1*q2*q3**2*q4**2 - 756*p**2*q1*q2*q3**2*q4 + 756*p**2*q1*q2*q3**2 - 6*p**2*q1*q2*q3*q4**6 + 54*p**2*q1*q2*q3*q4**5 - 216*p**2*q1*q2*q3*q4**4 + 504*p**2*q1*q2*q3*q4**3 - 756*p**2*q1*q2*q3*q4**2 + 756*p**2*q1*q2*q3*q4 - 504*p**2*q1*q2*q3 + 18*p**2*q2**7*q3 + 18*p**2*q2**6*q3**2 + 18*p**2*q2**6*q3*q4 - 144*p**2*q2**6*q3 + 18*p**2*q2**5*q3**3 + 18*p**2*q2**5*q3**2*q4 - 144*p**2*q2**5*q3**2 + 18*p**2*q2**5*q3*q4**2 - 144*p**2*q2**5*q3*q4 + 504*p**2*q2**5*q3 + 18*p**2*q2**4*q3**4 + 18*p**2*q2**4*q3**3*q4 - 144*p**2*q2**4*q3**3 + 18*p**2*q2**4*q3**2*q4**2 - 144*p**2*q2**4*q3**2*q4 + 504*p**2*q2**4*q3**2 + 18*p**2*q2**4*q3*q4**3 - 144*p**2*q2**4*q3*q4**2 + 504*p**2*q2**4*q3*q4 - 1008*p**2*q2**4*q3 + 18*p**2*q2**3*q3**5 + 18*p**2*q2**3*q3**4*q4 - 144*p**2*q2**3*q3**4 + 18*p**2*q2**3*q3**3*q4**2 - 144*p**2*q2**3*q3**3*q4 + 504*p**2*q2**3*q3**3 + 18*p**2*q2**3*q3**2*q4**3 - 144*p**2*q2**3*q3**2*q4**2 + 504*p**2*q2**3*q3**2*q4 - 1008*p**2*q2**3*q3**2 + 18*p**2*q2**3*q3*q4**4 - 144*p**2*q2**3*q3*q4**3 + 504*p**2*q2**3*q3*q4**2 - 1008*p**2*q2**3*q3*q4 + 1260*p**2*q2**3*q3 + 18*p**2*q2**2*q3**6 + 18*p**2*q2**2*q3**5*q4 - 144*p**2*q2**2*q3**5 + 18*p**2*q2**2*q3**4*q4**2 - 144*p**2*q2**2*q3**4*q4 + 504*p**2*q2**2*q3**4 + 18*p**2*q2**2*q3**3*q4**3 - 144*p**2*q2**2*q3**3*q4**2 + 504*p**2*q2**2*q3**3*q4 - 1008*p**2*q2**2*q3**3 + 18*p**2*q2**2*q3**2*q4**4 - 144*p**2*q2**2*q3**2*q4**3 + 504*p**2*q2**2*q3**2*q4**2 - 1008*p**2*q2**2*q3**2*q4 + 1260*p**2*q2**2*q3**2 + 18*p**2*q2**2*q3*q4**5 - 144*p**2*q2**2*q3*q4**4 + 504*p**2*q2**2*q3*q4**3 - 1008*p**2*q2**2*q3*q4**2 + 1260*p**2*q2**2*q3*q4 - 1008*p**2*q2**2*q3 + 18*p**2*q2*q3**7 + 18*p**2*q2*q3**6*q4 - 144*p**2*q2*q3**6 + 18*p**2*q2*q3**5*q4**2 - 144*p**2*q2*q3**5*q4 + 504*p**2*q2*q3**5 + 18*p**2*q2*q3**4*q4**3 - 144*p**2*q2*q3**4*q4**2 + 504*p**2*q2*q3**4*q4 - 1008*p**2*q2*q3**4 + 18*p**2*q2*q3**3*q4**4 - 144*p**2*q2*q3**3*q4**3 + 504*p**2*q2*q3**3*q4**2 - 1008*p**2*q2*q3**3*q4 + 1260*p**2*q2*q3**3 + 18*p**2*q2*q3**2*q4**5 - 144*p**2*q2*q3**2*q4**4 + 504*p**2*q2*q3**2*q4**3 - 1008*p**2*q2*q3**2*q4**2 + 1260*p**2*q2*q3**2*q4 - 1008*p**2*q2*q3**2 + 18*p**2*q2*q3*q4**6 - 144*p**2*q2*q3*q4**5 + 504*p**2*q2*q3*q4**4 - 1008*p**2*q2*q3*q4**3 + 1260*p**2*q2*q3*q4**2 - 1008*p**2*q2*q3*q4 + 504*p**2*q2*q3 - 24*p**2*q3**7 - 24*p**2*q3**6*q4 + 168*p**2*q3**6 - 24*p**2*q3**5*q4**2 + 168*p**2*q3**5*q4 - 504*p**2*q3**5 - 24*p**2*q3**4*q4**3 + 168*p**2*q3**4*q4**2 - 504*p**2*q3**4*q4 + 840*p**2*q3**4 - 24*p**2*q3**3*q4**4 + 168*p**2*q3**3*q4**3 - 504*p**2*q3**3*q4**2 + 840*p**2*q3**3*q4 - 840*p**2*q3**3 - 24*p**2*q3**2*q4**5 + 168*p**2*q3**2*q4**4 - 504*p**2*q3**2*q4**3 + 840*p**2*q3**2*q4**2 - 840*p**2*q3**2*q4 + 504*p**2*q3**2 - 24*p**2*q3*q4**6 + 168*p**2*q3*q4**5 - 504*p**2*q3*q4**4 + 840*p**2*q3*q4**3 - 840*p**2*q3*q4**2 + 504*p**2*q3*q4 - 168*p**2*q3 + 6*p*q1**7*q2*q3 + 6*p*q1**6*q2**2*q3 + 6*p*q1**6*q2*q3**2 + 6*p*q1**6*q2*q3*q4 - 54*p*q1**6*q2*q3 + 6*p*q1**5*q2**3*q3 + 6*p*q1**5*q2**2*q3**2 + 6*p*q1**5*q2**2*q3*q4 - 54*p*q1**5*q2**2*q3 + 6*p*q1**5*q2*q3**3 + 6*p*q1**5*q2*q3**2*q4 - 54*p*q1**5*q2*q3**2 + 6*p*q1**5*q2*q3*q4**2 - 54*p*q1**5*q2*q3*q4 + 216*p*q1**5*q2*q3 + 6*p*q1**4*q2**4*q3 + 6*p*q1**4*q2**3*q3**2 + 6*p*q1**4*q2**3*q3*q4 - 54*p*q1**4*q2**3*q3 + 6*p*q1**4*q2**2*q3**3 + 6*p*q1**4*q2**2*q3**2*q4 - 54*p*q1**4*q2**2*q3**2 + 6*p*q1**4*q2**2*q3*q4**2 - 54*p*q1**4*q2**2*q3*q4 + 216*p*q1**4*q2**2*q3 + 6*p*q1**4*q2*q3**4 + 6*p*q1**4*q2*q3**3*q4 - 54*p*q1**4*q2*q3**3 + 6*p*q1**4*q2*q3**2*q4**2 - 54*p*q1**4*q2*q3**2*q4 + 216*p*q1**4*q2*q3**2 + 6*p*q1**4*q2*q3*q4**3 - 54*p*q1**4*q2*q3*q4**2 + 216*p*q1**4*q2*q3*q4 - 504*p*q1**4*q2*q3 + 6*p*q1**3*q2**5*q3 + 6*p*q1**3*q2**4*q3**2 + 6*p*q1**3*q2**4*q3*q4 - 54*p*q1**3*q2**4*q3 + 6*p*q1**3*q2**3*q3**3 + 6*p*q1**3*q2**3*q3**2*q4 - 54*p*q1**3*q2**3*q3**2 + 6*p*q1**3*q2**3*q3*q4**2 - 54*p*q1**3*q2**3*q3*q4 + 216*p*q1**3*q2**3*q3 + 6*p*q1**3*q2**2*q3**4 + 6*p*q1**3*q2**2*q3**3*q4 - 54*p*q1**3*q2**2*q3**3 + 6*p*q1**3*q2**2*q3**2*q4**2 - 54*p*q1**3*q2**2*q3**2*q4 + 216*p*q1**3*q2**2*q3**2 + 6*p*q1**3*q2**2*q3*q4**3 - 54*p*q1**3*q2**2*q3*q4**2 + 216*p*q1**3*q2**2*q3*q4 - 504*p*q1**3*q2**2*q3 + 6*p*q1**3*q2*q3**5 + 6*p*q1**3*q2*q3**4*q4 - 54*p*q1**3*q2*q3**4 + 6*p*q1**3*q2*q3**3*q4**2 - 54*p*q1**3*q2*q3**3*q4 + 216*p*q1**3*q2*q3**3 + 6*p*q1**3*q2*q3**2*q4**3 - 54*p*q1**3*q2*q3**2*q4**2 + 216*p*q1**3*q2*q3**2*q4 - 504*p*q1**3*q2*q3**2 + 6*p*q1**3*q2*q3*q4**4 - 54*p*q1**3*q2*q3*q4**3 + 216*p*q1**3*q2*q3*q4**2 - 504*p*q1**3*q2*q3*q4 + 756*p*q1**3*q2*q3 + 6*p*q1**2*q2**6*q3 + 6*p*q1**2*q2**5*q3**2 + 6*p*q1**2*q2**5*q3*q4 - 54*p*q1**2*q2**5*q3 + 6*p*q1**2*q2**4*q3**3 + 6*p*q1**2*q2**4*q3**2*q4 - 54*p*q1**2*q2**4*q3**2 + 6*p*q1**2*q2**4*q3*q4**2 - 54*p*q1**2*q2**4*q3*q4 + 216*p*q1**2*q2**4*q3 + 6*p*q1**2*q2**3*q3**4 + 6*p*q1**2*q2**3*q3**3*q4 - 54*p*q1**2*q2**3*q3**3 + 6*p*q1**2*q2**3*q3**2*q4**2 - 54*p*q1**2*q2**3*q3**2*q4 + 216*p*q1**2*q2**3*q3**2 + 6*p*q1**2*q2**3*q3*q4**3 - 54*p*q1**2*q2**3*q3*q4**2 + 216*p*q1**2*q2**3*q3*q4 - 504*p*q1**2*q2**3*q3 + 6*p*q1**2*q2**2*q3**5 + 6*p*q1**2*q2**2*q3**4*q4 - 54*p*q1**2*q2**2*q3**4 + 6*p*q1**2*q2**2*q3**3*q4**2 - 54*p*q1**2*q2**2*q3**3*q4 + 216*p*q1**2*q2**2*q3**3 + 6*p*q1**2*q2**2*q3**2*q4**3 - 54*p*q1**2*q2**2*q3**2*q4**2 + 216*p*q1**2*q2**2*q3**2*q4 - 504*p*q1**2*q2**2*q3**2 + 6*p*q1**2*q2**2*q3*q4**4 - 54*p*q1**2*q2**2*q3*q4**3 + 216*p*q1**2*q2**2*q3*q4**2 - 504*p*q1**2*q2**2*q3*q4 + 756*p*q1**2*q2**2*q3 + 6*p*q1**2*q2*q3**6 + 6*p*q1**2*q2*q3**5*q4 - 54*p*q1**2*q2*q3**5 + 6*p*q1**2*q2*q3**4*q4**2 - 54*p*q1**2*q2*q3**4*q4 + 216*p*q1**2*q2*q3**4 + 6*p*q1**2*q2*q3**3*q4**3 - 54*p*q1**2*q2*q3**3*q4**2 + 216*p*q1**2*q2*q3**3*q4 - 504*p*q1**2*q2*q3**3 + 6*p*q1**2*q2*q3**2*q4**4 - 54*p*q1**2*q2*q3**2*q4**3 + 216*p*q1**2*q2*q3**2*q4**2 - 504*p*q1**2*q2*q3**2*q4 + 756*p*q1**2*q2*q3**2 + 6*p*q1**2*q2*q3*q4**5 - 54*p*q1**2*q2*q3*q4**4 + 216*p*q1**2*q2*q3*q4**3 - 504*p*q1**2*q2*q3*q4**2 + 756*p*q1**2*q2*q3*q4 - 756*p*q1**2*q2*q3 + 6*p*q1*q2**7*q3 + 6*p*q1*q2**6*q3**2 + 6*p*q1*q2**6*q3*q4 - 54*p*q1*q2**6*q3 + 6*p*q1*q2**5*q3**3 + 6*p*q1*q2**5*q3**2*q4 - 54*p*q1*q2**5*q3**2 + 6*p*q1*q2**5*q3*q4**2 - 54*p*q1*q2**5*q3*q4 + 216*p*q1*q2**5*q3 + 6*p*q1*q2**4*q3**4 + 6*p*q1*q2**4*q3**3*q4 - 54*p*q1*q2**4*q3**3 + 6*p*q1*q2**4*q3**2*q4**2 - 54*p*q1*q2**4*q3**2*q4 + 216*p*q1*q2**4*q3**2 + 6*p*q1*q2**4*q3*q4**3 - 54*p*q1*q2**4*q3*q4**2 + 216*p*q1*q2**4*q3*q4 - 504*p*q1*q2**4*q3 + 6*p*q1*q2**3*q3**5 + 6*p*q1*q2**3*q3**4*q4 - 54*p*q1*q2**3*q3**4 + 6*p*q1*q2**3*q3**3*q4**2 - 54*p*q1*q2**3*q3**3*q4 + 216*p*q1*q2**3*q3**3 + 6*p*q1*q2**3*q3**2*q4**3 - 54*p*q1*q2**3*q3**2*q4**2 + 216*p*q1*q2**3*q3**2*q4 - 504*p*q1*q2**3*q3**2 + 6*p*q1*q2**3*q3*q4**4 - 54*p*q1*q2**3*q3*q4**3 + 216*p*q1*q2**3*q3*q4**2 - 504*p*q1*q2**3*q3*q4 + 756*p*q1*q2**3*q3 + 6*p*q1*q2**2*q3**6 + 6*p*q1*q2**2*q3**5*q4 - 54*p*q1*q2**2*q3**5 + 6*p*q1*q2**2*q3**4*q4**2 - 54*p*q1*q2**2*q3**4*q4 + 216*p*q1*q2**2*q3**4 + 6*p*q1*q2**2*q3**3*q4**3 - 54*p*q1*q2**2*q3**3*q4**2 + 216*p*q1*q2**2*q3**3*q4 - 504*p*q1*q2**2*q3**3 + 6*p*q1*q2**2*q3**2*q4**4 - 54*p*q1*q2**2*q3**2*q4**3 + 216*p*q1*q2**2*q3**2*q4**2 - 504*p*q1*q2**2*q3**2*q4 + 756*p*q1*q2**2*q3**2 + 6*p*q1*q2**2*q3*q4**5 - 54*p*q1*q2**2*q3*q4**4 + 216*p*q1*q2**2*q3*q4**3 - 504*p*q1*q2**2*q3*q4**2 + 756*p*q1*q2**2*q3*q4 - 756*p*q1*q2**2*q3 + 6*p*q1*q2*q3**7 + 6*p*q1*q2*q3**6*q4 - 54*p*q1*q2*q3**6 + 6*p*q1*q2*q3**5*q4**2 - 54*p*q1*q2*q3**5*q4 + 216*p*q1*q2*q3**5 + 6*p*q1*q2*q3**4*q4**3 - 54*p*q1*q2*q3**4*q4**2 + 216*p*q1*q2*q3**4*q4 - 504*p*q1*q2*q3**4 + 6*p*q1*q2*q3**3*q4**4 - 54*p*q1*q2*q3**3*q4**3 + 216*p*q1*q2*q3**3*q4**2 - 504*p*q1*q2*q3**3*q4 + 756*p*q1*q2*q3**3 + 6*p*q1*q2*q3**2*q4**5 - 54*p*q1*q2*q3**2*q4**4 + 216*p*q1*q2*q3**2*q4**3 - 504*p*q1*q2*q3**2*q4**2 + 756*p*q1*q2*q3**2*q4 - 756*p*q1*q2*q3**2 + 6*p*q1*q2*q3*q4**6 - 54*p*q1*q2*q3*q4**5 + 216*p*q1*q2*q3*q4**4 - 504*p*q1*q2*q3*q4**3 + 756*p*q1*q2*q3*q4**2 - 756*p*q1*q2*q3*q4 + 504*p*q1*q2*q3 - 9*p*q2**7*q3 - 9*p*q2**6*q3**2 - 9*p*q2**6*q3*q4 + 72*p*q2**6*q3 - 9*p*q2**5*q3**3 - 9*p*q2**5*q3**2*q4 + 72*p*q2**5*q3**2 - 9*p*q2**5*q3*q4**2 + 72*p*q2**5*q3*q4 - 252*p*q2**5*q3 - 9*p*q2**4*q3**4 - 9*p*q2**4*q3**3*q4 + 72*p*q2**4*q3**3 - 9*p*q2**4*q3**2*q4**2 + 72*p*q2**4*q3**2*q4 - 252*p*q2**4*q3**2 - 9*p*q2**4*q3*q4**3 + 72*p*q2**4*q3*q4**2 - 252*p*q2**4*q3*q4 + 504*p*q2**4*q3 - 9*p*q2**3*q3**5 - 9*p*q2**3*q3**4*q4 + 72*p*q2**3*q3**4 - 9*p*q2**3*q3**3*q4**2 + 72*p*q2**3*q3**3*q4 - 252*p*q2**3*q3**3 - 9*p*q2**3*q3**2*q4**3 + 72*p*q2**3*q3**2*q4**2 - 252*p*q2**3*q3**2*q4 + 504*p*q2**3*q3**2 - 9*p*q2**3*q3*q4**4 + 72*p*q2**3*q3*q4**3 - 252*p*q2**3*q3*q4**2 + 504*p*q2**3*q3*q4 - 630*p*q2**3*q3 - 9*p*q2**2*q3**6 - 9*p*q2**2*q3**5*q4 + 72*p*q2**2*q3**5 - 9*p*q2**2*q3**4*q4**2 + 72*p*q2**2*q3**4*q4 - 252*p*q2**2*q3**4 - 9*p*q2**2*q3**3*q4**3 + 72*p*q2**2*q3**3*q4**2 - 252*p*q2**2*q3**3*q4 + 504*p*q2**2*q3**3 - 9*p*q2**2*q3**2*q4**4 + 72*p*q2**2*q3**2*q4**3 - 252*p*q2**2*q3**2*q4**2 + 504*p*q2**2*q3**2*q4 - 630*p*q2**2*q3**2 - 9*p*q2**2*q3*q4**5 + 72*p*q2**2*q3*q4**4 - 252*p*q2**2*q3*q4**3 + 504*p*q2**2*q3*q4**2 - 630*p*q2**2*q3*q4 + 504*p*q2**2*q3 - 9*p*q2*q3**7 - 9*p*q2*q3**6*q4 + 72*p*q2*q3**6 - 9*p*q2*q3**5*q4**2 + 72*p*q2*q3**5*q4 - 252*p*q2*q3**5 - 9*p*q2*q3**4*q4**3 + 72*p*q2*q3**4*q4**2 - 252*p*q2*q3**4*q4 + 504*p*q2*q3**4 - 9*p*q2*q3**3*q4**4 + 72*p*q2*q3**3*q4**3 - 252*p*q2*q3**3*q4**2 + 504*p*q2*q3**3*q4 - 630*p*q2*q3**3 - 9*p*q2*q3**2*q4**5 + 72*p*q2*q3**2*q4**4 - 252*p*q2*q3**2*q4**3 + 504*p*q2*q3**2*q4**2 - 630*p*q2*q3**2*q4 + 504*p*q2*q3**2 - 9*p*q2*q3*q4**6 + 72*p*q2*q3*q4**5 - 252*p*q2*q3*q4**4 + 504*p*q2*q3*q4**3 - 630*p*q2*q3*q4**2 + 504*p*q2*q3*q4 - 252*p*q2*q3 - 2*q1**7*q2*q3 - 2*q1**6*q2**2*q3 - 2*q1**6*q2*q3**2 - 2*q1**6*q2*q3*q4 + 18*q1**6*q2*q3 - 2*q1**5*q2**3*q3 - 2*q1**5*q2**2*q3**2 - 2*q1**5*q2**2*q3*q4 + 18*q1**5*q2**2*q3 - 2*q1**5*q2*q3**3 - 2*q1**5*q2*q3**2*q4 + 18*q1**5*q2*q3**2 - 2*q1**5*q2*q3*q4**2 + 18*q1**5*q2*q3*q4 - 72*q1**5*q2*q3 - 2*q1**4*q2**4*q3 - 2*q1**4*q2**3*q3**2 - 2*q1**4*q2**3*q3*q4 + 18*q1**4*q2**3*q3 - 2*q1**4*q2**2*q3**3 - 2*q1**4*q2**2*q3**2*q4 + 18*q1**4*q2**2*q3**2 - 2*q1**4*q2**2*q3*q4**2 + 18*q1**4*q2**2*q3*q4 - 72*q1**4*q2**2*q3 - 2*q1**4*q2*q3**4 - 2*q1**4*q2*q3**3*q4 + 18*q1**4*q2*q3**3 - 2*q1**4*q2*q3**2*q4**2 + 18*q1**4*q2*q3**2*q4 - 72*q1**4*q2*q3**2 - 2*q1**4*q2*q3*q4**3 + 18*q1**4*q2*q3*q4**2 - 72*q1**4*q2*q3*q4 + 168*q1**4*q2*q3 - 2*q1**3*q2**5*q3 - 2*q1**3*q2**4*q3**2 - 2*q1**3*q2**4*q3*q4 + 18*q1**3*q2**4*q3 - 2*q1**3*q2**3*q3**3 - 2*q1**3*q2**3*q3**2*q4 + 18*q1**3*q2**3*q3**2 - 2*q1**3*q2**3*q3*q4**2 + 18*q1**3*q2**3*q3*q4 - 72*q1**3*q2**3*q3 - 2*q1**3*q2**2*q3**4 - 2*q1**3*q2**2*q3**3*q4 + 18*q1**3*q2**2*q3**3 - 2*q1**3*q2**2*q3**2*q4**2 + 18*q1**3*q2**2*q3**2*q4 - 72*q1**3*q2**2*q3**2 - 2*q1**3*q2**2*q3*q4**3 + 18*q1**3*q2**2*q3*q4**2 - 72*q1**3*q2**2*q3*q4 + 168*q1**3*q2**2*q3 - 2*q1**3*q2*q3**5 - 2*q1**3*q2*q3**4*q4 + 18*q1**3*q2*q3**4 - 2*q1**3*q2*q3**3*q4**2 + 18*q1**3*q2*q3**3*q4 - 72*q1**3*q2*q3**3 - 2*q1**3*q2*q3**2*q4**3 + 18*q1**3*q2*q3**2*q4**2 - 72*q1**3*q2*q3**2*q4 + 168*q1**3*q2*q3**2 - 2*q1**3*q2*q3*q4**4 + 18*q1**3*q2*q3*q4**3 - 72*q1**3*q2*q3*q4**2 + 168*q1**3*q2*q3*q4 - 252*q1**3*q2*q3 - 2*q1**2*q2**6*q3 - 2*q1**2*q2**5*q3**2 - 2*q1**2*q2**5*q3*q4 + 18*q1**2*q2**5*q3 - 2*q1**2*q2**4*q3**3 - 2*q1**2*q2**4*q3**2*q4 + 18*q1**2*q2**4*q3**2 - 2*q1**2*q2**4*q3*q4**2 + 18*q1**2*q2**4*q3*q4 - 72*q1**2*q2**4*q3 - 2*q1**2*q2**3*q3**4 - 2*q1**2*q2**3*q3**3*q4 + 18*q1**2*q2**3*q3**3 - 2*q1**2*q2**3*q3**2*q4**2 + 18*q1**2*q2**3*q3**2*q4 - 72*q1**2*q2**3*q3**2 - 2*q1**2*q2**3*q3*q4**3 + 18*q1**2*q2**3*q3*q4**2 - 72*q1**2*q2**3*q3*q4 + 168*q1**2*q2**3*q3 - 2*q1**2*q2**2*q3**5 - 2*q1**2*q2**2*q3**4*q4 + 18*q1**2*q2**2*q3**4 - 2*q1**2*q2**2*q3**3*q4**2 + 18*q1**2*q2**2*q3**3*q4 - 72*q1**2*q2**2*q3**3 - 2*q1**2*q2**2*q3**2*q4**3 + 18*q1**2*q2**2*q3**2*q4**2 - 72*q1**2*q2**2*q3**2*q4 + 168*q1**2*q2**2*q3**2 - 2*q1**2*q2**2*q3*q4**4 + 18*q1**2*q2**2*q3*q4**3 - 72*q1**2*q2**2*q3*q4**2 + 168*q1**2*q2**2*q3*q4 - 252*q1**2*q2**2*q3 - 2*q1**2*q2*q3**6 - 2*q1**2*q2*q3**5*q4 + 18*q1**2*q2*q3**5 - 2*q1**2*q2*q3**4*q4**2 + 18*q1**2*q2*q3**4*q4 - 72*q1**2*q2*q3**4 - 2*q1**2*q2*q3**3*q4**3 + 18*q1**2*q2*q3**3*q4**2 - 72*q1**2*q2*q3**3*q4 + 168*q1**2*q2*q3**3 - 2*q1**2*q2*q3**2*q4**4 + 18*q1**2*q2*q3**2*q4**3 - 72*q1**2*q2*q3**2*q4**2 + 168*q1**2*q2*q3**2*q4 - 252*q1**2*q2*q3**2 - 2*q1**2*q2*q3*q4**5 + 18*q1**2*q2*q3*q4**4 - 72*q1**2*q2*q3*q4**3 + 168*q1**2*q2*q3*q4**2 - 252*q1**2*q2*q3*q4 + 252*q1**2*q2*q3 - 2*q1*q2**7*q3 - 2*q1*q2**6*q3**2 - 2*q1*q2**6*q3*q4 + 18*q1*q2**6*q3 - 2*q1*q2**5*q3**3 - 2*q1*q2**5*q3**2*q4 + 18*q1*q2**5*q3**2 - 2*q1*q2**5*q3*q4**2 + 18*q1*q2**5*q3*q4 - 72*q1*q2**5*q3 - 2*q1*q2**4*q3**4 - 2*q1*q2**4*q3**3*q4 + 18*q1*q2**4*q3**3 - 2*q1*q2**4*q3**2*q4**2 + 18*q1*q2**4*q3**2*q4 - 72*q1*q2**4*q3**2 - 2*q1*q2**4*q3*q4**3 + 18*q1*q2**4*q3*q4**2 - 72*q1*q2**4*q3*q4 + 168*q1*q2**4*q3 - 2*q1*q2**3*q3**5 - 2*q1*q2**3*q3**4*q4 + 18*q1*q2**3*q3**4 - 2*q1*q2**3*q3**3*q4**2 + 18*q1*q2**3*q3**3*q4 - 72*q1*q2**3*q3**3 - 2*q1*q2**3*q3**2*q4**3 + 18*q1*q2**3*q3**2*q4**2 - 72*q1*q2**3*q3**2*q4 + 168*q1*q2**3*q3**2 - 2*q1*q2**3*q3*q4**4 + 18*q1*q2**3*q3*q4**3 - 72*q1*q2**3*q3*q4**2 + 168*q1*q2**3*q3*q4 - 252*q1*q2**3*q3 - 2*q1*q2**2*q3**6 - 2*q1*q2**2*q3**5*q4 + 18*q1*q2**2*q3**5 - 2*q1*q2**2*q3**4*q4**2 + 18*q1*q2**2*q3**4*q4 - 72*q1*q2**2*q3**4 - 2*q1*q2**2*q3**3*q4**3 + 18*q1*q2**2*q3**3*q4**2 - 72*q1*q2**2*q3**3*q4 + 168*q1*q2**2*q3**3 - 2*q1*q2**2*q3**2*q4**4 + 18*q1*q2**2*q3**2*q4**3 - 72*q1*q2**2*q3**2*q4**2 + 168*q1*q2**2*q3**2*q4 - 252*q1*q2**2*q3**2 - 2*q1*q2**2*q3*q4**5 + 18*q1*q2**2*q3*q4**4 - 72*q1*q2**2*q3*q4**3 + 168*q1*q2**2*q3*q4**2 - 252*q1*q2**2*q3*q4 + 252*q1*q2**2*q3 - 2*q1*q2*q3**7 - 2*q1*q2*q3**6*q4 + 18*q1*q2*q3**6 - 2*q1*q2*q3**5*q4**2 + 18*q1*q2*q3**5*q4 - 72*q1*q2*q3**5 - 2*q1*q2*q3**4*q4**3 + 18*q1*q2*q3**4*q4**2 - 72*q1*q2*q3**4*q4 + 168*q1*q2*q3**4 - 2*q1*q2*q3**3*q4**4 + 18*q1*q2*q3**3*q4**3 - 72*q1*q2*q3**3*q4**2 + 168*q1*q2*q3**3*q4 - 252*q1*q2*q3**3 - 2*q1*q2*q3**2*q4**5 + 18*q1*q2*q3**2*q4**4 - 72*q1*q2*q3**2*q4**3 + 168*q1*q2*q3**2*q4**2 - 252*q1*q2*q3**2*q4 + 252*q1*q2*q3**2 - 2*q1*q2*q3*q4**6 + 18*q1*q2*q3*q4**5 - 72*q1*q2*q3*q4**4 + 168*q1*q2*q3*q4**3 - 252*q1*q2*q3*q4**2 + 252*q1*q2*q3*q4 - 168*q1*q2*q3)'
f_multiparam[10][5] = 'p*(p - 1)**5*(10*p**4*q1**6*q2*q3*q4 + 10*p**4*q1**5*q2**2*q3*q4 + 10*p**4*q1**5*q2*q3**2*q4 + 10*p**4*q1**5*q2*q3*q4**2 + 10*p**4*q1**5*q2*q3*q4*q5 - 90*p**4*q1**5*q2*q3*q4 + 10*p**4*q1**4*q2**3*q3*q4 + 10*p**4*q1**4*q2**2*q3**2*q4 + 10*p**4*q1**4*q2**2*q3*q4**2 + 10*p**4*q1**4*q2**2*q3*q4*q5 - 90*p**4*q1**4*q2**2*q3*q4 + 10*p**4*q1**4*q2*q3**3*q4 + 10*p**4*q1**4*q2*q3**2*q4**2 + 10*p**4*q1**4*q2*q3**2*q4*q5 - 90*p**4*q1**4*q2*q3**2*q4 + 10*p**4*q1**4*q2*q3*q4**3 + 10*p**4*q1**4*q2*q3*q4**2*q5 - 90*p**4*q1**4*q2*q3*q4**2 + 10*p**4*q1**4*q2*q3*q4*q5**2 - 90*p**4*q1**4*q2*q3*q4*q5 + 360*p**4*q1**4*q2*q3*q4 + 10*p**4*q1**3*q2**4*q3*q4 + 10*p**4*q1**3*q2**3*q3**2*q4 + 10*p**4*q1**3*q2**3*q3*q4**2 + 10*p**4*q1**3*q2**3*q3*q4*q5 - 90*p**4*q1**3*q2**3*q3*q4 + 10*p**4*q1**3*q2**2*q3**3*q4 + 10*p**4*q1**3*q2**2*q3**2*q4**2 + 10*p**4*q1**3*q2**2*q3**2*q4*q5 - 90*p**4*q1**3*q2**2*q3**2*q4 + 10*p**4*q1**3*q2**2*q3*q4**3 + 10*p**4*q1**3*q2**2*q3*q4**2*q5 - 90*p**4*q1**3*q2**2*q3*q4**2 + 10*p**4*q1**3*q2**2*q3*q4*q5**2 - 90*p**4*q1**3*q2**2*q3*q4*q5 + 360*p**4*q1**3*q2**2*q3*q4 + 10*p**4*q1**3*q2*q3**4*q4 + 10*p**4*q1**3*q2*q3**3*q4**2 + 10*p**4*q1**3*q2*q3**3*q4*q5 - 90*p**4*q1**3*q2*q3**3*q4 + 10*p**4*q1**3*q2*q3**2*q4**3 + 10*p**4*q1**3*q2*q3**2*q4**2*q5 - 90*p**4*q1**3*q2*q3**2*q4**2 + 10*p**4*q1**3*q2*q3**2*q4*q5**2 - 90*p**4*q1**3*q2*q3**2*q4*q5 + 360*p**4*q1**3*q2*q3**2*q4 + 10*p**4*q1**3*q2*q3*q4**4 + 10*p**4*q1**3*q2*q3*q4**3*q5 - 90*p**4*q1**3*q2*q3*q4**3 + 10*p**4*q1**3*q2*q3*q4**2*q5**2 - 90*p**4*q1**3*q2*q3*q4**2*q5 + 360*p**4*q1**3*q2*q3*q4**2 + 10*p**4*q1**3*q2*q3*q4*q5**3 - 90*p**4*q1**3*q2*q3*q4*q5**2 + 360*p**4*q1**3*q2*q3*q4*q5 - 840*p**4*q1**3*q2*q3*q4 + 10*p**4*q1**2*q2**5*q3*q4 + 10*p**4*q1**2*q2**4*q3**2*q4 + 10*p**4*q1**2*q2**4*q3*q4**2 + 10*p**4*q1**2*q2**4*q3*q4*q5 - 90*p**4*q1**2*q2**4*q3*q4 + 10*p**4*q1**2*q2**3*q3**3*q4 + 10*p**4*q1**2*q2**3*q3**2*q4**2 + 10*p**4*q1**2*q2**3*q3**2*q4*q5 - 90*p**4*q1**2*q2**3*q3**2*q4 + 10*p**4*q1**2*q2**3*q3*q4**3 + 10*p**4*q1**2*q2**3*q3*q4**2*q5 - 90*p**4*q1**2*q2**3*q3*q4**2 + 10*p**4*q1**2*q2**3*q3*q4*q5**2 - 90*p**4*q1**2*q2**3*q3*q4*q5 + 360*p**4*q1**2*q2**3*q3*q4 + 10*p**4*q1**2*q2**2*q3**4*q4 + 10*p**4*q1**2*q2**2*q3**3*q4**2 + 10*p**4*q1**2*q2**2*q3**3*q4*q5 - 90*p**4*q1**2*q2**2*q3**3*q4 + 10*p**4*q1**2*q2**2*q3**2*q4**3 + 10*p**4*q1**2*q2**2*q3**2*q4**2*q5 - 90*p**4*q1**2*q2**2*q3**2*q4**2 + 10*p**4*q1**2*q2**2*q3**2*q4*q5**2 - 90*p**4*q1**2*q2**2*q3**2*q4*q5 + 360*p**4*q1**2*q2**2*q3**2*q4 + 10*p**4*q1**2*q2**2*q3*q4**4 + 10*p**4*q1**2*q2**2*q3*q4**3*q5 - 90*p**4*q1**2*q2**2*q3*q4**3 + 10*p**4*q1**2*q2**2*q3*q4**2*q5**2 - 90*p**4*q1**2*q2**2*q3*q4**2*q5 + 360*p**4*q1**2*q2**2*q3*q4**2 + 10*p**4*q1**2*q2**2*q3*q4*q5**3 - 90*p**4*q1**2*q2**2*q3*q4*q5**2 + 360*p**4*q1**2*q2**2*q3*q4*q5 - 840*p**4*q1**2*q2**2*q3*q4 + 10*p**4*q1**2*q2*q3**5*q4 + 10*p**4*q1**2*q2*q3**4*q4**2 + 10*p**4*q1**2*q2*q3**4*q4*q5 - 90*p**4*q1**2*q2*q3**4*q4 + 10*p**4*q1**2*q2*q3**3*q4**3 + 10*p**4*q1**2*q2*q3**3*q4**2*q5 - 90*p**4*q1**2*q2*q3**3*q4**2 + 10*p**4*q1**2*q2*q3**3*q4*q5**2 - 90*p**4*q1**2*q2*q3**3*q4*q5 + 360*p**4*q1**2*q2*q3**3*q4 + 10*p**4*q1**2*q2*q3**2*q4**4 + 10*p**4*q1**2*q2*q3**2*q4**3*q5 - 90*p**4*q1**2*q2*q3**2*q4**3 + 10*p**4*q1**2*q2*q3**2*q4**2*q5**2 - 90*p**4*q1**2*q2*q3**2*q4**2*q5 + 360*p**4*q1**2*q2*q3**2*q4**2 + 10*p**4*q1**2*q2*q3**2*q4*q5**3 - 90*p**4*q1**2*q2*q3**2*q4*q5**2 + 360*p**4*q1**2*q2*q3**2*q4*q5 - 840*p**4*q1**2*q2*q3**2*q4 + 10*p**4*q1**2*q2*q3*q4**5 + 10*p**4*q1**2*q2*q3*q4**4*q5 - 90*p**4*q1**2*q2*q3*q4**4 + 10*p**4*q1**2*q2*q3*q4**3*q5**2 - 90*p**4*q1**2*q2*q3*q4**3*q5 + 360*p**4*q1**2*q2*q3*q4**3 + 10*p**4*q1**2*q2*q3*q4**2*q5**3 - 90*p**4*q1**2*q2*q3*q4**2*q5**2 + 360*p**4*q1**2*q2*q3*q4**2*q5 - 840*p**4*q1**2*q2*q3*q4**2 + 10*p**4*q1**2*q2*q3*q4*q5**4 - 90*p**4*q1**2*q2*q3*q4*q5**3 + 360*p**4*q1**2*q2*q3*q4*q5**2 - 840*p**4*q1**2*q2*q3*q4*q5 + 1260*p**4*q1**2*q2*q3*q4 + 10*p**4*q1*q2**6*q3*q4 + 10*p**4*q1*q2**5*q3**2*q4 + 10*p**4*q1*q2**5*q3*q4**2 + 10*p**4*q1*q2**5*q3*q4*q5 - 90*p**4*q1*q2**5*q3*q4 + 10*p**4*q1*q2**4*q3**3*q4 + 10*p**4*q1*q2**4*q3**2*q4**2 + 10*p**4*q1*q2**4*q3**2*q4*q5 - 90*p**4*q1*q2**4*q3**2*q4 + 10*p**4*q1*q2**4*q3*q4**3 + 10*p**4*q1*q2**4*q3*q4**2*q5 - 90*p**4*q1*q2**4*q3*q4**2 + 10*p**4*q1*q2**4*q3*q4*q5**2 - 90*p**4*q1*q2**4*q3*q4*q5 + 360*p**4*q1*q2**4*q3*q4 + 10*p**4*q1*q2**3*q3**4*q4 + 10*p**4*q1*q2**3*q3**3*q4**2 + 10*p**4*q1*q2**3*q3**3*q4*q5 - 90*p**4*q1*q2**3*q3**3*q4 + 10*p**4*q1*q2**3*q3**2*q4**3 + 10*p**4*q1*q2**3*q3**2*q4**2*q5 - 90*p**4*q1*q2**3*q3**2*q4**2 + 10*p**4*q1*q2**3*q3**2*q4*q5**2 - 90*p**4*q1*q2**3*q3**2*q4*q5 + 360*p**4*q1*q2**3*q3**2*q4 + 10*p**4*q1*q2**3*q3*q4**4 + 10*p**4*q1*q2**3*q3*q4**3*q5 - 90*p**4*q1*q2**3*q3*q4**3 + 10*p**4*q1*q2**3*q3*q4**2*q5**2 - 90*p**4*q1*q2**3*q3*q4**2*q5 + 360*p**4*q1*q2**3*q3*q4**2 + 10*p**4*q1*q2**3*q3*q4*q5**3 - 90*p**4*q1*q2**3*q3*q4*q5**2 + 360*p**4*q1*q2**3*q3*q4*q5 - 840*p**4*q1*q2**3*q3*q4 + 10*p**4*q1*q2**2*q3**5*q4 + 10*p**4*q1*q2**2*q3**4*q4**2 + 10*p**4*q1*q2**2*q3**4*q4*q5 - 90*p**4*q1*q2**2*q3**4*q4 + 10*p**4*q1*q2**2*q3**3*q4**3 + 10*p**4*q1*q2**2*q3**3*q4**2*q5 - 90*p**4*q1*q2**2*q3**3*q4**2 + 10*p**4*q1*q2**2*q3**3*q4*q5**2 - 90*p**4*q1*q2**2*q3**3*q4*q5 + 360*p**4*q1*q2**2*q3**3*q4 + 10*p**4*q1*q2**2*q3**2*q4**4 + 10*p**4*q1*q2**2*q3**2*q4**3*q5 - 90*p**4*q1*q2**2*q3**2*q4**3 + 10*p**4*q1*q2**2*q3**2*q4**2*q5**2 - 90*p**4*q1*q2**2*q3**2*q4**2*q5 + 360*p**4*q1*q2**2*q3**2*q4**2 + 10*p**4*q1*q2**2*q3**2*q4*q5**3 - 90*p**4*q1*q2**2*q3**2*q4*q5**2 + 360*p**4*q1*q2**2*q3**2*q4*q5 - 840*p**4*q1*q2**2*q3**2*q4 + 10*p**4*q1*q2**2*q3*q4**5 + 10*p**4*q1*q2**2*q3*q4**4*q5 - 90*p**4*q1*q2**2*q3*q4**4 + 10*p**4*q1*q2**2*q3*q4**3*q5**2 - 90*p**4*q1*q2**2*q3*q4**3*q5 + 360*p**4*q1*q2**2*q3*q4**3 + 10*p**4*q1*q2**2*q3*q4**2*q5**3 - 90*p**4*q1*q2**2*q3*q4**2*q5**2 + 360*p**4*q1*q2**2*q3*q4**2*q5 - 840*p**4*q1*q2**2*q3*q4**2 + 10*p**4*q1*q2**2*q3*q4*q5**4 - 90*p**4*q1*q2**2*q3*q4*q5**3 + 360*p**4*q1*q2**2*q3*q4*q5**2 - 840*p**4*q1*q2**2*q3*q4*q5 + 1260*p**4*q1*q2**2*q3*q4 + 10*p**4*q1*q2*q3**6*q4 + 10*p**4*q1*q2*q3**5*q4**2 + 10*p**4*q1*q2*q3**5*q4*q5 - 90*p**4*q1*q2*q3**5*q4 + 10*p**4*q1*q2*q3**4*q4**3 + 10*p**4*q1*q2*q3**4*q4**2*q5 - 90*p**4*q1*q2*q3**4*q4**2 + 10*p**4*q1*q2*q3**4*q4*q5**2 - 90*p**4*q1*q2*q3**4*q4*q5 + 360*p**4*q1*q2*q3**4*q4 + 10*p**4*q1*q2*q3**3*q4**4 + 10*p**4*q1*q2*q3**3*q4**3*q5 - 90*p**4*q1*q2*q3**3*q4**3 + 10*p**4*q1*q2*q3**3*q4**2*q5**2 - 90*p**4*q1*q2*q3**3*q4**2*q5 + 360*p**4*q1*q2*q3**3*q4**2 + 10*p**4*q1*q2*q3**3*q4*q5**3 - 90*p**4*q1*q2*q3**3*q4*q5**2 + 360*p**4*q1*q2*q3**3*q4*q5 - 840*p**4*q1*q2*q3**3*q4 + 10*p**4*q1*q2*q3**2*q4**5 + 10*p**4*q1*q2*q3**2*q4**4*q5 - 90*p**4*q1*q2*q3**2*q4**4 + 10*p**4*q1*q2*q3**2*q4**3*q5**2 - 90*p**4*q1*q2*q3**2*q4**3*q5 + 360*p**4*q1*q2*q3**2*q4**3 + 10*p**4*q1*q2*q3**2*q4**2*q5**3 - 90*p**4*q1*q2*q3**2*q4**2*q5**2 + 360*p**4*q1*q2*q3**2*q4**2*q5 - 840*p**4*q1*q2*q3**2*q4**2 + 10*p**4*q1*q2*q3**2*q4*q5**4 - 90*p**4*q1*q2*q3**2*q4*q5**3 + 360*p**4*q1*q2*q3**2*q4*q5**2 - 840*p**4*q1*q2*q3**2*q4*q5 + 1260*p**4*q1*q2*q3**2*q4 + 10*p**4*q1*q2*q3*q4**6 + 10*p**4*q1*q2*q3*q4**5*q5 - 90*p**4*q1*q2*q3*q4**5 + 10*p**4*q1*q2*q3*q4**4*q5**2 - 90*p**4*q1*q2*q3*q4**4*q5 + 360*p**4*q1*q2*q3*q4**4 + 10*p**4*q1*q2*q3*q4**3*q5**3 - 90*p**4*q1*q2*q3*q4**3*q5**2 + 360*p**4*q1*q2*q3*q4**3*q5 - 840*p**4*q1*q2*q3*q4**3 + 10*p**4*q1*q2*q3*q4**2*q5**4 - 90*p**4*q1*q2*q3*q4**2*q5**3 + 360*p**4*q1*q2*q3*q4**2*q5**2 - 840*p**4*q1*q2*q3*q4**2*q5 + 1260*p**4*q1*q2*q3*q4**2 + 10*p**4*q1*q2*q3*q4*q5**5 - 90*p**4*q1*q2*q3*q4*q5**4 + 360*p**4*q1*q2*q3*q4*q5**3 - 840*p**4*q1*q2*q3*q4*q5**2 + 1260*p**4*q1*q2*q3*q4*q5 - 1260*p**4*q1*q2*q3*q4 - 45*p**4*q2**6*q3*q4 - 45*p**4*q2**5*q3**2*q4 - 45*p**4*q2**5*q3*q4**2 - 45*p**4*q2**5*q3*q4*q5 + 360*p**4*q2**5*q3*q4 - 45*p**4*q2**4*q3**3*q4 - 45*p**4*q2**4*q3**2*q4**2 - 45*p**4*q2**4*q3**2*q4*q5 + 360*p**4*q2**4*q3**2*q4 - 45*p**4*q2**4*q3*q4**3 - 45*p**4*q2**4*q3*q4**2*q5 + 360*p**4*q2**4*q3*q4**2 - 45*p**4*q2**4*q3*q4*q5**2 + 360*p**4*q2**4*q3*q4*q5 - 1260*p**4*q2**4*q3*q4 - 45*p**4*q2**3*q3**4*q4 - 45*p**4*q2**3*q3**3*q4**2 - 45*p**4*q2**3*q3**3*q4*q5 + 360*p**4*q2**3*q3**3*q4 - 45*p**4*q2**3*q3**2*q4**3 - 45*p**4*q2**3*q3**2*q4**2*q5 + 360*p**4*q2**3*q3**2*q4**2 - 45*p**4*q2**3*q3**2*q4*q5**2 + 360*p**4*q2**3*q3**2*q4*q5 - 1260*p**4*q2**3*q3**2*q4 - 45*p**4*q2**3*q3*q4**4 - 45*p**4*q2**3*q3*q4**3*q5 + 360*p**4*q2**3*q3*q4**3 - 45*p**4*q2**3*q3*q4**2*q5**2 + 360*p**4*q2**3*q3*q4**2*q5 - 1260*p**4*q2**3*q3*q4**2 - 45*p**4*q2**3*q3*q4*q5**3 + 360*p**4*q2**3*q3*q4*q5**2 - 1260*p**4*q2**3*q3*q4*q5 + 2520*p**4*q2**3*q3*q4 - 45*p**4*q2**2*q3**5*q4 - 45*p**4*q2**2*q3**4*q4**2 - 45*p**4*q2**2*q3**4*q4*q5 + 360*p**4*q2**2*q3**4*q4 - 45*p**4*q2**2*q3**3*q4**3 - 45*p**4*q2**2*q3**3*q4**2*q5 + 360*p**4*q2**2*q3**3*q4**2 - 45*p**4*q2**2*q3**3*q4*q5**2 + 360*p**4*q2**2*q3**3*q4*q5 - 1260*p**4*q2**2*q3**3*q4 - 45*p**4*q2**2*q3**2*q4**4 - 45*p**4*q2**2*q3**2*q4**3*q5 + 360*p**4*q2**2*q3**2*q4**3 - 45*p**4*q2**2*q3**2*q4**2*q5**2 + 360*p**4*q2**2*q3**2*q4**2*q5 - 1260*p**4*q2**2*q3**2*q4**2 - 45*p**4*q2**2*q3**2*q4*q5**3 + 360*p**4*q2**2*q3**2*q4*q5**2 - 1260*p**4*q2**2*q3**2*q4*q5 + 2520*p**4*q2**2*q3**2*q4 - 45*p**4*q2**2*q3*q4**5 - 45*p**4*q2**2*q3*q4**4*q5 + 360*p**4*q2**2*q3*q4**4 - 45*p**4*q2**2*q3*q4**3*q5**2 + 360*p**4*q2**2*q3*q4**3*q5 - 1260*p**4*q2**2*q3*q4**3 - 45*p**4*q2**2*q3*q4**2*q5**3 + 360*p**4*q2**2*q3*q4**2*q5**2 - 1260*p**4*q2**2*q3*q4**2*q5 + 2520*p**4*q2**2*q3*q4**2 - 45*p**4*q2**2*q3*q4*q5**4 + 360*p**4*q2**2*q3*q4*q5**3 - 1260*p**4*q2**2*q3*q4*q5**2 + 2520*p**4*q2**2*q3*q4*q5 - 3150*p**4*q2**2*q3*q4 - 45*p**4*q2*q3**6*q4 - 45*p**4*q2*q3**5*q4**2 - 45*p**4*q2*q3**5*q4*q5 + 360*p**4*q2*q3**5*q4 - 45*p**4*q2*q3**4*q4**3 - 45*p**4*q2*q3**4*q4**2*q5 + 360*p**4*q2*q3**4*q4**2 - 45*p**4*q2*q3**4*q4*q5**2 + 360*p**4*q2*q3**4*q4*q5 - 1260*p**4*q2*q3**4*q4 - 45*p**4*q2*q3**3*q4**4 - 45*p**4*q2*q3**3*q4**3*q5 + 360*p**4*q2*q3**3*q4**3 - 45*p**4*q2*q3**3*q4**2*q5**2 + 360*p**4*q2*q3**3*q4**2*q5 - 1260*p**4*q2*q3**3*q4**2 - 45*p**4*q2*q3**3*q4*q5**3 + 360*p**4*q2*q3**3*q4*q5**2 - 1260*p**4*q2*q3**3*q4*q5 + 2520*p**4*q2*q3**3*q4 - 45*p**4*q2*q3**2*q4**5 - 45*p**4*q2*q3**2*q4**4*q5 + 360*p**4*q2*q3**2*q4**4 - 45*p**4*q2*q3**2*q4**3*q5**2 + 360*p**4*q2*q3**2*q4**3*q5 - 1260*p**4*q2*q3**2*q4**3 - 45*p**4*q2*q3**2*q4**2*q5**3 + 360*p**4*q2*q3**2*q4**2*q5**2 - 1260*p**4*q2*q3**2*q4**2*q5 + 2520*p**4*q2*q3**2*q4**2 - 45*p**4*q2*q3**2*q4*q5**4 + 360*p**4*q2*q3**2*q4*q5**3 - 1260*p**4*q2*q3**2*q4*q5**2 + 2520*p**4*q2*q3**2*q4*q5 - 3150*p**4*q2*q3**2*q4 - 45*p**4*q2*q3*q4**6 - 45*p**4*q2*q3*q4**5*q5 + 360*p**4*q2*q3*q4**5 - 45*p**4*q2*q3*q4**4*q5**2 + 360*p**4*q2*q3*q4**4*q5 - 1260*p**4*q2*q3*q4**4 - 45*p**4*q2*q3*q4**3*q5**3 + 360*p**4*q2*q3*q4**3*q5**2 - 1260*p**4*q2*q3*q4**3*q5 + 2520*p**4*q2*q3*q4**3 - 45*p**4*q2*q3*q4**2*q5**4 + 360*p**4*q2*q3*q4**2*q5**3 - 1260*p**4*q2*q3*q4**2*q5**2 + 2520*p**4*q2*q3*q4**2*q5 - 3150*p**4*q2*q3*q4**2 - 45*p**4*q2*q3*q4*q5**5 + 360*p**4*q2*q3*q4*q5**4 - 1260*p**4*q2*q3*q4*q5**3 + 2520*p**4*q2*q3*q4*q5**2 - 3150*p**4*q2*q3*q4*q5 + 2520*p**4*q2*q3*q4 + 120*p**4*q3**6*q4 + 120*p**4*q3**5*q4**2 + 120*p**4*q3**5*q4*q5 - 840*p**4*q3**5*q4 + 120*p**4*q3**4*q4**3 + 120*p**4*q3**4*q4**2*q5 - 840*p**4*q3**4*q4**2 + 120*p**4*q3**4*q4*q5**2 - 840*p**4*q3**4*q4*q5 + 2520*p**4*q3**4*q4 + 120*p**4*q3**3*q4**4 + 120*p**4*q3**3*q4**3*q5 - 840*p**4*q3**3*q4**3 + 120*p**4*q3**3*q4**2*q5**2 - 840*p**4*q3**3*q4**2*q5 + 2520*p**4*q3**3*q4**2 + 120*p**4*q3**3*q4*q5**3 - 840*p**4*q3**3*q4*q5**2 + 2520*p**4*q3**3*q4*q5 - 4200*p**4*q3**3*q4 + 120*p**4*q3**2*q4**5 + 120*p**4*q3**2*q4**4*q5 - 840*p**4*q3**2*q4**4 + 120*p**4*q3**2*q4**3*q5**2 - 840*p**4*q3**2*q4**3*q5 + 2520*p**4*q3**2*q4**3 + 120*p**4*q3**2*q4**2*q5**3 - 840*p**4*q3**2*q4**2*q5**2 + 2520*p**4*q3**2*q4**2*q5 - 4200*p**4*q3**2*q4**2 + 120*p**4*q3**2*q4*q5**4 - 840*p**4*q3**2*q4*q5**3 + 2520*p**4*q3**2*q4*q5**2 - 4200*p**4*q3**2*q4*q5 + 4200*p**4*q3**2*q4 + 120*p**4*q3*q4**6 + 120*p**4*q3*q4**5*q5 - 840*p**4*q3*q4**5 + 120*p**4*q3*q4**4*q5**2 - 840*p**4*q3*q4**4*q5 + 2520*p**4*q3*q4**4 + 120*p**4*q3*q4**3*q5**3 - 840*p**4*q3*q4**3*q5**2 + 2520*p**4*q3*q4**3*q5 - 4200*p**4*q3*q4**3 + 120*p**4*q3*q4**2*q5**4 - 840*p**4*q3*q4**2*q5**3 + 2520*p**4*q3*q4**2*q5**2 - 4200*p**4*q3*q4**2*q5 + 4200*p**4*q3*q4**2 + 120*p**4*q3*q4*q5**5 - 840*p**4*q3*q4*q5**4 + 2520*p**4*q3*q4*q5**3 - 4200*p**4*q3*q4*q5**2 + 4200*p**4*q3*q4*q5 - 2520*p**4*q3*q4 - 210*p**4*q4**6 - 210*p**4*q4**5*q5 + 1260*p**4*q4**5 - 210*p**4*q4**4*q5**2 + 1260*p**4*q4**4*q5 - 3150*p**4*q4**4 - 210*p**4*q4**3*q5**3 + 1260*p**4*q4**3*q5**2 - 3150*p**4*q4**3*q5 + 4200*p**4*q4**3 - 210*p**4*q4**2*q5**4 + 1260*p**4*q4**2*q5**3 - 3150*p**4*q4**2*q5**2 + 4200*p**4*q4**2*q5 - 3150*p**4*q4**2 - 210*p**4*q4*q5**5 + 1260*p**4*q4*q5**4 - 3150*p**4*q4*q5**3 + 4200*p**4*q4*q5**2 - 3150*p**4*q4*q5 + 1260*p**4*q4 + 252*p**4*q5**5 - 1260*p**4*q5**4 + 2520*p**4*q5**3 - 2520*p**4*q5**2 + 1260*p**4*q5 - 252*p**4 - 40*p**3*q1**6*q2*q3*q4 - 40*p**3*q1**5*q2**2*q3*q4 - 40*p**3*q1**5*q2*q3**2*q4 - 40*p**3*q1**5*q2*q3*q4**2 - 40*p**3*q1**5*q2*q3*q4*q5 + 360*p**3*q1**5*q2*q3*q4 - 40*p**3*q1**4*q2**3*q3*q4 - 40*p**3*q1**4*q2**2*q3**2*q4 - 40*p**3*q1**4*q2**2*q3*q4**2 - 40*p**3*q1**4*q2**2*q3*q4*q5 + 360*p**3*q1**4*q2**2*q3*q4 - 40*p**3*q1**4*q2*q3**3*q4 - 40*p**3*q1**4*q2*q3**2*q4**2 - 40*p**3*q1**4*q2*q3**2*q4*q5 + 360*p**3*q1**4*q2*q3**2*q4 - 40*p**3*q1**4*q2*q3*q4**3 - 40*p**3*q1**4*q2*q3*q4**2*q5 + 360*p**3*q1**4*q2*q3*q4**2 - 40*p**3*q1**4*q2*q3*q4*q5**2 + 360*p**3*q1**4*q2*q3*q4*q5 - 1440*p**3*q1**4*q2*q3*q4 - 40*p**3*q1**3*q2**4*q3*q4 - 40*p**3*q1**3*q2**3*q3**2*q4 - 40*p**3*q1**3*q2**3*q3*q4**2 - 40*p**3*q1**3*q2**3*q3*q4*q5 + 360*p**3*q1**3*q2**3*q3*q4 - 40*p**3*q1**3*q2**2*q3**3*q4 - 40*p**3*q1**3*q2**2*q3**2*q4**2 - 40*p**3*q1**3*q2**2*q3**2*q4*q5 + 360*p**3*q1**3*q2**2*q3**2*q4 - 40*p**3*q1**3*q2**2*q3*q4**3 - 40*p**3*q1**3*q2**2*q3*q4**2*q5 + 360*p**3*q1**3*q2**2*q3*q4**2 - 40*p**3*q1**3*q2**2*q3*q4*q5**2 + 360*p**3*q1**3*q2**2*q3*q4*q5 - 1440*p**3*q1**3*q2**2*q3*q4 - 40*p**3*q1**3*q2*q3**4*q4 - 40*p**3*q1**3*q2*q3**3*q4**2 - 40*p**3*q1**3*q2*q3**3*q4*q5 + 360*p**3*q1**3*q2*q3**3*q4 - 40*p**3*q1**3*q2*q3**2*q4**3 - 40*p**3*q1**3*q2*q3**2*q4**2*q5 + 360*p**3*q1**3*q2*q3**2*q4**2 - 40*p**3*q1**3*q2*q3**2*q4*q5**2 + 360*p**3*q1**3*q2*q3**2*q4*q5 - 1440*p**3*q1**3*q2*q3**2*q4 - 40*p**3*q1**3*q2*q3*q4**4 - 40*p**3*q1**3*q2*q3*q4**3*q5 + 360*p**3*q1**3*q2*q3*q4**3 - 40*p**3*q1**3*q2*q3*q4**2*q5**2 + 360*p**3*q1**3*q2*q3*q4**2*q5 - 1440*p**3*q1**3*q2*q3*q4**2 - 40*p**3*q1**3*q2*q3*q4*q5**3 + 360*p**3*q1**3*q2*q3*q4*q5**2 - 1440*p**3*q1**3*q2*q3*q4*q5 + 3360*p**3*q1**3*q2*q3*q4 - 40*p**3*q1**2*q2**5*q3*q4 - 40*p**3*q1**2*q2**4*q3**2*q4 - 40*p**3*q1**2*q2**4*q3*q4**2 - 40*p**3*q1**2*q2**4*q3*q4*q5 + 360*p**3*q1**2*q2**4*q3*q4 - 40*p**3*q1**2*q2**3*q3**3*q4 - 40*p**3*q1**2*q2**3*q3**2*q4**2 - 40*p**3*q1**2*q2**3*q3**2*q4*q5 + 360*p**3*q1**2*q2**3*q3**2*q4 - 40*p**3*q1**2*q2**3*q3*q4**3 - 40*p**3*q1**2*q2**3*q3*q4**2*q5 + 360*p**3*q1**2*q2**3*q3*q4**2 - 40*p**3*q1**2*q2**3*q3*q4*q5**2 + 360*p**3*q1**2*q2**3*q3*q4*q5 - 1440*p**3*q1**2*q2**3*q3*q4 - 40*p**3*q1**2*q2**2*q3**4*q4 - 40*p**3*q1**2*q2**2*q3**3*q4**2 - 40*p**3*q1**2*q2**2*q3**3*q4*q5 + 360*p**3*q1**2*q2**2*q3**3*q4 - 40*p**3*q1**2*q2**2*q3**2*q4**3 - 40*p**3*q1**2*q2**2*q3**2*q4**2*q5 + 360*p**3*q1**2*q2**2*q3**2*q4**2 - 40*p**3*q1**2*q2**2*q3**2*q4*q5**2 + 360*p**3*q1**2*q2**2*q3**2*q4*q5 - 1440*p**3*q1**2*q2**2*q3**2*q4 - 40*p**3*q1**2*q2**2*q3*q4**4 - 40*p**3*q1**2*q2**2*q3*q4**3*q5 + 360*p**3*q1**2*q2**2*q3*q4**3 - 40*p**3*q1**2*q2**2*q3*q4**2*q5**2 + 360*p**3*q1**2*q2**2*q3*q4**2*q5 - 1440*p**3*q1**2*q2**2*q3*q4**2 - 40*p**3*q1**2*q2**2*q3*q4*q5**3 + 360*p**3*q1**2*q2**2*q3*q4*q5**2 - 1440*p**3*q1**2*q2**2*q3*q4*q5 + 3360*p**3*q1**2*q2**2*q3*q4 - 40*p**3*q1**2*q2*q3**5*q4 - 40*p**3*q1**2*q2*q3**4*q4**2 - 40*p**3*q1**2*q2*q3**4*q4*q5 + 360*p**3*q1**2*q2*q3**4*q4 - 40*p**3*q1**2*q2*q3**3*q4**3 - 40*p**3*q1**2*q2*q3**3*q4**2*q5 + 360*p**3*q1**2*q2*q3**3*q4**2 - 40*p**3*q1**2*q2*q3**3*q4*q5**2 + 360*p**3*q1**2*q2*q3**3*q4*q5 - 1440*p**3*q1**2*q2*q3**3*q4 - 40*p**3*q1**2*q2*q3**2*q4**4 - 40*p**3*q1**2*q2*q3**2*q4**3*q5 + 360*p**3*q1**2*q2*q3**2*q4**3 - 40*p**3*q1**2*q2*q3**2*q4**2*q5**2 + 360*p**3*q1**2*q2*q3**2*q4**2*q5 - 1440*p**3*q1**2*q2*q3**2*q4**2 - 40*p**3*q1**2*q2*q3**2*q4*q5**3 + 360*p**3*q1**2*q2*q3**2*q4*q5**2 - 1440*p**3*q1**2*q2*q3**2*q4*q5 + 3360*p**3*q1**2*q2*q3**2*q4 - 40*p**3*q1**2*q2*q3*q4**5 - 40*p**3*q1**2*q2*q3*q4**4*q5 + 360*p**3*q1**2*q2*q3*q4**4 - 40*p**3*q1**2*q2*q3*q4**3*q5**2 + 360*p**3*q1**2*q2*q3*q4**3*q5 - 1440*p**3*q1**2*q2*q3*q4**3 - 40*p**3*q1**2*q2*q3*q4**2*q5**3 + 360*p**3*q1**2*q2*q3*q4**2*q5**2 - 1440*p**3*q1**2*q2*q3*q4**2*q5 + 3360*p**3*q1**2*q2*q3*q4**2 - 40*p**3*q1**2*q2*q3*q4*q5**4 + 360*p**3*q1**2*q2*q3*q4*q5**3 - 1440*p**3*q1**2*q2*q3*q4*q5**2 + 3360*p**3*q1**2*q2*q3*q4*q5 - 5040*p**3*q1**2*q2*q3*q4 - 40*p**3*q1*q2**6*q3*q4 - 40*p**3*q1*q2**5*q3**2*q4 - 40*p**3*q1*q2**5*q3*q4**2 - 40*p**3*q1*q2**5*q3*q4*q5 + 360*p**3*q1*q2**5*q3*q4 - 40*p**3*q1*q2**4*q3**3*q4 - 40*p**3*q1*q2**4*q3**2*q4**2 - 40*p**3*q1*q2**4*q3**2*q4*q5 + 360*p**3*q1*q2**4*q3**2*q4 - 40*p**3*q1*q2**4*q3*q4**3 - 40*p**3*q1*q2**4*q3*q4**2*q5 + 360*p**3*q1*q2**4*q3*q4**2 - 40*p**3*q1*q2**4*q3*q4*q5**2 + 360*p**3*q1*q2**4*q3*q4*q5 - 1440*p**3*q1*q2**4*q3*q4 - 40*p**3*q1*q2**3*q3**4*q4 - 40*p**3*q1*q2**3*q3**3*q4**2 - 40*p**3*q1*q2**3*q3**3*q4*q5 + 360*p**3*q1*q2**3*q3**3*q4 - 40*p**3*q1*q2**3*q3**2*q4**3 - 40*p**3*q1*q2**3*q3**2*q4**2*q5 + 360*p**3*q1*q2**3*q3**2*q4**2 - 40*p**3*q1*q2**3*q3**2*q4*q5**2 + 360*p**3*q1*q2**3*q3**2*q4*q5 - 1440*p**3*q1*q2**3*q3**2*q4 - 40*p**3*q1*q2**3*q3*q4**4 - 40*p**3*q1*q2**3*q3*q4**3*q5 + 360*p**3*q1*q2**3*q3*q4**3 - 40*p**3*q1*q2**3*q3*q4**2*q5**2 + 360*p**3*q1*q2**3*q3*q4**2*q5 - 1440*p**3*q1*q2**3*q3*q4**2 - 40*p**3*q1*q2**3*q3*q4*q5**3 + 360*p**3*q1*q2**3*q3*q4*q5**2 - 1440*p**3*q1*q2**3*q3*q4*q5 + 3360*p**3*q1*q2**3*q3*q4 - 40*p**3*q1*q2**2*q3**5*q4 - 40*p**3*q1*q2**2*q3**4*q4**2 - 40*p**3*q1*q2**2*q3**4*q4*q5 + 360*p**3*q1*q2**2*q3**4*q4 - 40*p**3*q1*q2**2*q3**3*q4**3 - 40*p**3*q1*q2**2*q3**3*q4**2*q5 + 360*p**3*q1*q2**2*q3**3*q4**2 - 40*p**3*q1*q2**2*q3**3*q4*q5**2 + 360*p**3*q1*q2**2*q3**3*q4*q5 - 1440*p**3*q1*q2**2*q3**3*q4 - 40*p**3*q1*q2**2*q3**2*q4**4 - 40*p**3*q1*q2**2*q3**2*q4**3*q5 + 360*p**3*q1*q2**2*q3**2*q4**3 - 40*p**3*q1*q2**2*q3**2*q4**2*q5**2 + 360*p**3*q1*q2**2*q3**2*q4**2*q5 - 1440*p**3*q1*q2**2*q3**2*q4**2 - 40*p**3*q1*q2**2*q3**2*q4*q5**3 + 360*p**3*q1*q2**2*q3**2*q4*q5**2 - 1440*p**3*q1*q2**2*q3**2*q4*q5 + 3360*p**3*q1*q2**2*q3**2*q4 - 40*p**3*q1*q2**2*q3*q4**5 - 40*p**3*q1*q2**2*q3*q4**4*q5 + 360*p**3*q1*q2**2*q3*q4**4 - 40*p**3*q1*q2**2*q3*q4**3*q5**2 + 360*p**3*q1*q2**2*q3*q4**3*q5 - 1440*p**3*q1*q2**2*q3*q4**3 - 40*p**3*q1*q2**2*q3*q4**2*q5**3 + 360*p**3*q1*q2**2*q3*q4**2*q5**2 - 1440*p**3*q1*q2**2*q3*q4**2*q5 + 3360*p**3*q1*q2**2*q3*q4**2 - 40*p**3*q1*q2**2*q3*q4*q5**4 + 360*p**3*q1*q2**2*q3*q4*q5**3 - 1440*p**3*q1*q2**2*q3*q4*q5**2 + 3360*p**3*q1*q2**2*q3*q4*q5 - 5040*p**3*q1*q2**2*q3*q4 - 40*p**3*q1*q2*q3**6*q4 - 40*p**3*q1*q2*q3**5*q4**2 - 40*p**3*q1*q2*q3**5*q4*q5 + 360*p**3*q1*q2*q3**5*q4 - 40*p**3*q1*q2*q3**4*q4**3 - 40*p**3*q1*q2*q3**4*q4**2*q5 + 360*p**3*q1*q2*q3**4*q4**2 - 40*p**3*q1*q2*q3**4*q4*q5**2 + 360*p**3*q1*q2*q3**4*q4*q5 - 1440*p**3*q1*q2*q3**4*q4 - 40*p**3*q1*q2*q3**3*q4**4 - 40*p**3*q1*q2*q3**3*q4**3*q5 + 360*p**3*q1*q2*q3**3*q4**3 - 40*p**3*q1*q2*q3**3*q4**2*q5**2 + 360*p**3*q1*q2*q3**3*q4**2*q5 - 1440*p**3*q1*q2*q3**3*q4**2 - 40*p**3*q1*q2*q3**3*q4*q5**3 + 360*p**3*q1*q2*q3**3*q4*q5**2 - 1440*p**3*q1*q2*q3**3*q4*q5 + 3360*p**3*q1*q2*q3**3*q4 - 40*p**3*q1*q2*q3**2*q4**5 - 40*p**3*q1*q2*q3**2*q4**4*q5 + 360*p**3*q1*q2*q3**2*q4**4 - 40*p**3*q1*q2*q3**2*q4**3*q5**2 + 360*p**3*q1*q2*q3**2*q4**3*q5 - 1440*p**3*q1*q2*q3**2*q4**3 - 40*p**3*q1*q2*q3**2*q4**2*q5**3 + 360*p**3*q1*q2*q3**2*q4**2*q5**2 - 1440*p**3*q1*q2*q3**2*q4**2*q5 + 3360*p**3*q1*q2*q3**2*q4**2 - 40*p**3*q1*q2*q3**2*q4*q5**4 + 360*p**3*q1*q2*q3**2*q4*q5**3 - 1440*p**3*q1*q2*q3**2*q4*q5**2 + 3360*p**3*q1*q2*q3**2*q4*q5 - 5040*p**3*q1*q2*q3**2*q4 - 40*p**3*q1*q2*q3*q4**6 - 40*p**3*q1*q2*q3*q4**5*q5 + 360*p**3*q1*q2*q3*q4**5 - 40*p**3*q1*q2*q3*q4**4*q5**2 + 360*p**3*q1*q2*q3*q4**4*q5 - 1440*p**3*q1*q2*q3*q4**4 - 40*p**3*q1*q2*q3*q4**3*q5**3 + 360*p**3*q1*q2*q3*q4**3*q5**2 - 1440*p**3*q1*q2*q3*q4**3*q5 + 3360*p**3*q1*q2*q3*q4**3 - 40*p**3*q1*q2*q3*q4**2*q5**4 + 360*p**3*q1*q2*q3*q4**2*q5**3 - 1440*p**3*q1*q2*q3*q4**2*q5**2 + 3360*p**3*q1*q2*q3*q4**2*q5 - 5040*p**3*q1*q2*q3*q4**2 - 40*p**3*q1*q2*q3*q4*q5**5 + 360*p**3*q1*q2*q3*q4*q5**4 - 1440*p**3*q1*q2*q3*q4*q5**3 + 3360*p**3*q1*q2*q3*q4*q5**2 - 5040*p**3*q1*q2*q3*q4*q5 + 5040*p**3*q1*q2*q3*q4 + 135*p**3*q2**6*q3*q4 + 135*p**3*q2**5*q3**2*q4 + 135*p**3*q2**5*q3*q4**2 + 135*p**3*q2**5*q3*q4*q5 - 1080*p**3*q2**5*q3*q4 + 135*p**3*q2**4*q3**3*q4 + 135*p**3*q2**4*q3**2*q4**2 + 135*p**3*q2**4*q3**2*q4*q5 - 1080*p**3*q2**4*q3**2*q4 + 135*p**3*q2**4*q3*q4**3 + 135*p**3*q2**4*q3*q4**2*q5 - 1080*p**3*q2**4*q3*q4**2 + 135*p**3*q2**4*q3*q4*q5**2 - 1080*p**3*q2**4*q3*q4*q5 + 3780*p**3*q2**4*q3*q4 + 135*p**3*q2**3*q3**4*q4 + 135*p**3*q2**3*q3**3*q4**2 + 135*p**3*q2**3*q3**3*q4*q5 - 1080*p**3*q2**3*q3**3*q4 + 135*p**3*q2**3*q3**2*q4**3 + 135*p**3*q2**3*q3**2*q4**2*q5 - 1080*p**3*q2**3*q3**2*q4**2 + 135*p**3*q2**3*q3**2*q4*q5**2 - 1080*p**3*q2**3*q3**2*q4*q5 + 3780*p**3*q2**3*q3**2*q4 + 135*p**3*q2**3*q3*q4**4 + 135*p**3*q2**3*q3*q4**3*q5 - 1080*p**3*q2**3*q3*q4**3 + 135*p**3*q2**3*q3*q4**2*q5**2 - 1080*p**3*q2**3*q3*q4**2*q5 + 3780*p**3*q2**3*q3*q4**2 + 135*p**3*q2**3*q3*q4*q5**3 - 1080*p**3*q2**3*q3*q4*q5**2 + 3780*p**3*q2**3*q3*q4*q5 - 7560*p**3*q2**3*q3*q4 + 135*p**3*q2**2*q3**5*q4 + 135*p**3*q2**2*q3**4*q4**2 + 135*p**3*q2**2*q3**4*q4*q5 - 1080*p**3*q2**2*q3**4*q4 + 135*p**3*q2**2*q3**3*q4**3 + 135*p**3*q2**2*q3**3*q4**2*q5 - 1080*p**3*q2**2*q3**3*q4**2 + 135*p**3*q2**2*q3**3*q4*q5**2 - 1080*p**3*q2**2*q3**3*q4*q5 + 3780*p**3*q2**2*q3**3*q4 + 135*p**3*q2**2*q3**2*q4**4 + 135*p**3*q2**2*q3**2*q4**3*q5 - 1080*p**3*q2**2*q3**2*q4**3 + 135*p**3*q2**2*q3**2*q4**2*q5**2 - 1080*p**3*q2**2*q3**2*q4**2*q5 + 3780*p**3*q2**2*q3**2*q4**2 + 135*p**3*q2**2*q3**2*q4*q5**3 - 1080*p**3*q2**2*q3**2*q4*q5**2 + 3780*p**3*q2**2*q3**2*q4*q5 - 7560*p**3*q2**2*q3**2*q4 + 135*p**3*q2**2*q3*q4**5 + 135*p**3*q2**2*q3*q4**4*q5 - 1080*p**3*q2**2*q3*q4**4 + 135*p**3*q2**2*q3*q4**3*q5**2 - 1080*p**3*q2**2*q3*q4**3*q5 + 3780*p**3*q2**2*q3*q4**3 + 135*p**3*q2**2*q3*q4**2*q5**3 - 1080*p**3*q2**2*q3*q4**2*q5**2 + 3780*p**3*q2**2*q3*q4**2*q5 - 7560*p**3*q2**2*q3*q4**2 + 135*p**3*q2**2*q3*q4*q5**4 - 1080*p**3*q2**2*q3*q4*q5**3 + 3780*p**3*q2**2*q3*q4*q5**2 - 7560*p**3*q2**2*q3*q4*q5 + 9450*p**3*q2**2*q3*q4 + 135*p**3*q2*q3**6*q4 + 135*p**3*q2*q3**5*q4**2 + 135*p**3*q2*q3**5*q4*q5 - 1080*p**3*q2*q3**5*q4 + 135*p**3*q2*q3**4*q4**3 + 135*p**3*q2*q3**4*q4**2*q5 - 1080*p**3*q2*q3**4*q4**2 + 135*p**3*q2*q3**4*q4*q5**2 - 1080*p**3*q2*q3**4*q4*q5 + 3780*p**3*q2*q3**4*q4 + 135*p**3*q2*q3**3*q4**4 + 135*p**3*q2*q3**3*q4**3*q5 - 1080*p**3*q2*q3**3*q4**3 + 135*p**3*q2*q3**3*q4**2*q5**2 - 1080*p**3*q2*q3**3*q4**2*q5 + 3780*p**3*q2*q3**3*q4**2 + 135*p**3*q2*q3**3*q4*q5**3 - 1080*p**3*q2*q3**3*q4*q5**2 + 3780*p**3*q2*q3**3*q4*q5 - 7560*p**3*q2*q3**3*q4 + 135*p**3*q2*q3**2*q4**5 + 135*p**3*q2*q3**2*q4**4*q5 - 1080*p**3*q2*q3**2*q4**4 + 135*p**3*q2*q3**2*q4**3*q5**2 - 1080*p**3*q2*q3**2*q4**3*q5 + 3780*p**3*q2*q3**2*q4**3 + 135*p**3*q2*q3**2*q4**2*q5**3 - 1080*p**3*q2*q3**2*q4**2*q5**2 + 3780*p**3*q2*q3**2*q4**2*q5 - 7560*p**3*q2*q3**2*q4**2 + 135*p**3*q2*q3**2*q4*q5**4 - 1080*p**3*q2*q3**2*q4*q5**3 + 3780*p**3*q2*q3**2*q4*q5**2 - 7560*p**3*q2*q3**2*q4*q5 + 9450*p**3*q2*q3**2*q4 + 135*p**3*q2*q3*q4**6 + 135*p**3*q2*q3*q4**5*q5 - 1080*p**3*q2*q3*q4**5 + 135*p**3*q2*q3*q4**4*q5**2 - 1080*p**3*q2*q3*q4**4*q5 + 3780*p**3*q2*q3*q4**4 + 135*p**3*q2*q3*q4**3*q5**3 - 1080*p**3*q2*q3*q4**3*q5**2 + 3780*p**3*q2*q3*q4**3*q5 - 7560*p**3*q2*q3*q4**3 + 135*p**3*q2*q3*q4**2*q5**4 - 1080*p**3*q2*q3*q4**2*q5**3 + 3780*p**3*q2*q3*q4**2*q5**2 - 7560*p**3*q2*q3*q4**2*q5 + 9450*p**3*q2*q3*q4**2 + 135*p**3*q2*q3*q4*q5**5 - 1080*p**3*q2*q3*q4*q5**4 + 3780*p**3*q2*q3*q4*q5**3 - 7560*p**3*q2*q3*q4*q5**2 + 9450*p**3*q2*q3*q4*q5 - 7560*p**3*q2*q3*q4 - 240*p**3*q3**6*q4 - 240*p**3*q3**5*q4**2 - 240*p**3*q3**5*q4*q5 + 1680*p**3*q3**5*q4 - 240*p**3*q3**4*q4**3 - 240*p**3*q3**4*q4**2*q5 + 1680*p**3*q3**4*q4**2 - 240*p**3*q3**4*q4*q5**2 + 1680*p**3*q3**4*q4*q5 - 5040*p**3*q3**4*q4 - 240*p**3*q3**3*q4**4 - 240*p**3*q3**3*q4**3*q5 + 1680*p**3*q3**3*q4**3 - 240*p**3*q3**3*q4**2*q5**2 + 1680*p**3*q3**3*q4**2*q5 - 5040*p**3*q3**3*q4**2 - 240*p**3*q3**3*q4*q5**3 + 1680*p**3*q3**3*q4*q5**2 - 5040*p**3*q3**3*q4*q5 + 8400*p**3*q3**3*q4 - 240*p**3*q3**2*q4**5 - 240*p**3*q3**2*q4**4*q5 + 1680*p**3*q3**2*q4**4 - 240*p**3*q3**2*q4**3*q5**2 + 1680*p**3*q3**2*q4**3*q5 - 5040*p**3*q3**2*q4**3 - 240*p**3*q3**2*q4**2*q5**3 + 1680*p**3*q3**2*q4**2*q5**2 - 5040*p**3*q3**2*q4**2*q5 + 8400*p**3*q3**2*q4**2 - 240*p**3*q3**2*q4*q5**4 + 1680*p**3*q3**2*q4*q5**3 - 5040*p**3*q3**2*q4*q5**2 + 8400*p**3*q3**2*q4*q5 - 8400*p**3*q3**2*q4 - 240*p**3*q3*q4**6 - 240*p**3*q3*q4**5*q5 + 1680*p**3*q3*q4**5 - 240*p**3*q3*q4**4*q5**2 + 1680*p**3*q3*q4**4*q5 - 5040*p**3*q3*q4**4 - 240*p**3*q3*q4**3*q5**3 + 1680*p**3*q3*q4**3*q5**2 - 5040*p**3*q3*q4**3*q5 + 8400*p**3*q3*q4**3 - 240*p**3*q3*q4**2*q5**4 + 1680*p**3*q3*q4**2*q5**3 - 5040*p**3*q3*q4**2*q5**2 + 8400*p**3*q3*q4**2*q5 - 8400*p**3*q3*q4**2 - 240*p**3*q3*q4*q5**5 + 1680*p**3*q3*q4*q5**4 - 5040*p**3*q3*q4*q5**3 + 8400*p**3*q3*q4*q5**2 - 8400*p**3*q3*q4*q5 + 5040*p**3*q3*q4 + 210*p**3*q4**6 + 210*p**3*q4**5*q5 - 1260*p**3*q4**5 + 210*p**3*q4**4*q5**2 - 1260*p**3*q4**4*q5 + 3150*p**3*q4**4 + 210*p**3*q4**3*q5**3 - 1260*p**3*q4**3*q5**2 + 3150*p**3*q4**3*q5 - 4200*p**3*q4**3 + 210*p**3*q4**2*q5**4 - 1260*p**3*q4**2*q5**3 + 3150*p**3*q4**2*q5**2 - 4200*p**3*q4**2*q5 + 3150*p**3*q4**2 + 210*p**3*q4*q5**5 - 1260*p**3*q4*q5**4 + 3150*p**3*q4*q5**3 - 4200*p**3*q4*q5**2 + 3150*p**3*q4*q5 - 1260*p**3*q4 + 60*p**2*q1**6*q2*q3*q4 + 60*p**2*q1**5*q2**2*q3*q4 + 60*p**2*q1**5*q2*q3**2*q4 + 60*p**2*q1**5*q2*q3*q4**2 + 60*p**2*q1**5*q2*q3*q4*q5 - 540*p**2*q1**5*q2*q3*q4 + 60*p**2*q1**4*q2**3*q3*q4 + 60*p**2*q1**4*q2**2*q3**2*q4 + 60*p**2*q1**4*q2**2*q3*q4**2 + 60*p**2*q1**4*q2**2*q3*q4*q5 - 540*p**2*q1**4*q2**2*q3*q4 + 60*p**2*q1**4*q2*q3**3*q4 + 60*p**2*q1**4*q2*q3**2*q4**2 + 60*p**2*q1**4*q2*q3**2*q4*q5 - 540*p**2*q1**4*q2*q3**2*q4 + 60*p**2*q1**4*q2*q3*q4**3 + 60*p**2*q1**4*q2*q3*q4**2*q5 - 540*p**2*q1**4*q2*q3*q4**2 + 60*p**2*q1**4*q2*q3*q4*q5**2 - 540*p**2*q1**4*q2*q3*q4*q5 + 2160*p**2*q1**4*q2*q3*q4 + 60*p**2*q1**3*q2**4*q3*q4 + 60*p**2*q1**3*q2**3*q3**2*q4 + 60*p**2*q1**3*q2**3*q3*q4**2 + 60*p**2*q1**3*q2**3*q3*q4*q5 - 540*p**2*q1**3*q2**3*q3*q4 + 60*p**2*q1**3*q2**2*q3**3*q4 + 60*p**2*q1**3*q2**2*q3**2*q4**2 + 60*p**2*q1**3*q2**2*q3**2*q4*q5 - 540*p**2*q1**3*q2**2*q3**2*q4 + 60*p**2*q1**3*q2**2*q3*q4**3 + 60*p**2*q1**3*q2**2*q3*q4**2*q5 - 540*p**2*q1**3*q2**2*q3*q4**2 + 60*p**2*q1**3*q2**2*q3*q4*q5**2 - 540*p**2*q1**3*q2**2*q3*q4*q5 + 2160*p**2*q1**3*q2**2*q3*q4 + 60*p**2*q1**3*q2*q3**4*q4 + 60*p**2*q1**3*q2*q3**3*q4**2 + 60*p**2*q1**3*q2*q3**3*q4*q5 - 540*p**2*q1**3*q2*q3**3*q4 + 60*p**2*q1**3*q2*q3**2*q4**3 + 60*p**2*q1**3*q2*q3**2*q4**2*q5 - 540*p**2*q1**3*q2*q3**2*q4**2 + 60*p**2*q1**3*q2*q3**2*q4*q5**2 - 540*p**2*q1**3*q2*q3**2*q4*q5 + 2160*p**2*q1**3*q2*q3**2*q4 + 60*p**2*q1**3*q2*q3*q4**4 + 60*p**2*q1**3*q2*q3*q4**3*q5 - 540*p**2*q1**3*q2*q3*q4**3 + 60*p**2*q1**3*q2*q3*q4**2*q5**2 - 540*p**2*q1**3*q2*q3*q4**2*q5 + 2160*p**2*q1**3*q2*q3*q4**2 + 60*p**2*q1**3*q2*q3*q4*q5**3 - 540*p**2*q1**3*q2*q3*q4*q5**2 + 2160*p**2*q1**3*q2*q3*q4*q5 - 5040*p**2*q1**3*q2*q3*q4 + 60*p**2*q1**2*q2**5*q3*q4 + 60*p**2*q1**2*q2**4*q3**2*q4 + 60*p**2*q1**2*q2**4*q3*q4**2 + 60*p**2*q1**2*q2**4*q3*q4*q5 - 540*p**2*q1**2*q2**4*q3*q4 + 60*p**2*q1**2*q2**3*q3**3*q4 + 60*p**2*q1**2*q2**3*q3**2*q4**2 + 60*p**2*q1**2*q2**3*q3**2*q4*q5 - 540*p**2*q1**2*q2**3*q3**2*q4 + 60*p**2*q1**2*q2**3*q3*q4**3 + 60*p**2*q1**2*q2**3*q3*q4**2*q5 - 540*p**2*q1**2*q2**3*q3*q4**2 + 60*p**2*q1**2*q2**3*q3*q4*q5**2 - 540*p**2*q1**2*q2**3*q3*q4*q5 + 2160*p**2*q1**2*q2**3*q3*q4 + 60*p**2*q1**2*q2**2*q3**4*q4 + 60*p**2*q1**2*q2**2*q3**3*q4**2 + 60*p**2*q1**2*q2**2*q3**3*q4*q5 - 540*p**2*q1**2*q2**2*q3**3*q4 + 60*p**2*q1**2*q2**2*q3**2*q4**3 + 60*p**2*q1**2*q2**2*q3**2*q4**2*q5 - 540*p**2*q1**2*q2**2*q3**2*q4**2 + 60*p**2*q1**2*q2**2*q3**2*q4*q5**2 - 540*p**2*q1**2*q2**2*q3**2*q4*q5 + 2160*p**2*q1**2*q2**2*q3**2*q4 + 60*p**2*q1**2*q2**2*q3*q4**4 + 60*p**2*q1**2*q2**2*q3*q4**3*q5 - 540*p**2*q1**2*q2**2*q3*q4**3 + 60*p**2*q1**2*q2**2*q3*q4**2*q5**2 - 540*p**2*q1**2*q2**2*q3*q4**2*q5 + 2160*p**2*q1**2*q2**2*q3*q4**2 + 60*p**2*q1**2*q2**2*q3*q4*q5**3 - 540*p**2*q1**2*q2**2*q3*q4*q5**2 + 2160*p**2*q1**2*q2**2*q3*q4*q5 - 5040*p**2*q1**2*q2**2*q3*q4 + 60*p**2*q1**2*q2*q3**5*q4 + 60*p**2*q1**2*q2*q3**4*q4**2 + 60*p**2*q1**2*q2*q3**4*q4*q5 - 540*p**2*q1**2*q2*q3**4*q4 + 60*p**2*q1**2*q2*q3**3*q4**3 + 60*p**2*q1**2*q2*q3**3*q4**2*q5 - 540*p**2*q1**2*q2*q3**3*q4**2 + 60*p**2*q1**2*q2*q3**3*q4*q5**2 - 540*p**2*q1**2*q2*q3**3*q4*q5 + 2160*p**2*q1**2*q2*q3**3*q4 + 60*p**2*q1**2*q2*q3**2*q4**4 + 60*p**2*q1**2*q2*q3**2*q4**3*q5 - 540*p**2*q1**2*q2*q3**2*q4**3 + 60*p**2*q1**2*q2*q3**2*q4**2*q5**2 - 540*p**2*q1**2*q2*q3**2*q4**2*q5 + 2160*p**2*q1**2*q2*q3**2*q4**2 + 60*p**2*q1**2*q2*q3**2*q4*q5**3 - 540*p**2*q1**2*q2*q3**2*q4*q5**2 + 2160*p**2*q1**2*q2*q3**2*q4*q5 - 5040*p**2*q1**2*q2*q3**2*q4 + 60*p**2*q1**2*q2*q3*q4**5 + 60*p**2*q1**2*q2*q3*q4**4*q5 - 540*p**2*q1**2*q2*q3*q4**4 + 60*p**2*q1**2*q2*q3*q4**3*q5**2 - 540*p**2*q1**2*q2*q3*q4**3*q5 + 2160*p**2*q1**2*q2*q3*q4**3 + 60*p**2*q1**2*q2*q3*q4**2*q5**3 - 540*p**2*q1**2*q2*q3*q4**2*q5**2 + 2160*p**2*q1**2*q2*q3*q4**2*q5 - 5040*p**2*q1**2*q2*q3*q4**2 + 60*p**2*q1**2*q2*q3*q4*q5**4 - 540*p**2*q1**2*q2*q3*q4*q5**3 + 2160*p**2*q1**2*q2*q3*q4*q5**2 - 5040*p**2*q1**2*q2*q3*q4*q5 + 7560*p**2*q1**2*q2*q3*q4 + 60*p**2*q1*q2**6*q3*q4 + 60*p**2*q1*q2**5*q3**2*q4 + 60*p**2*q1*q2**5*q3*q4**2 + 60*p**2*q1*q2**5*q3*q4*q5 - 540*p**2*q1*q2**5*q3*q4 + 60*p**2*q1*q2**4*q3**3*q4 + 60*p**2*q1*q2**4*q3**2*q4**2 + 60*p**2*q1*q2**4*q3**2*q4*q5 - 540*p**2*q1*q2**4*q3**2*q4 + 60*p**2*q1*q2**4*q3*q4**3 + 60*p**2*q1*q2**4*q3*q4**2*q5 - 540*p**2*q1*q2**4*q3*q4**2 + 60*p**2*q1*q2**4*q3*q4*q5**2 - 540*p**2*q1*q2**4*q3*q4*q5 + 2160*p**2*q1*q2**4*q3*q4 + 60*p**2*q1*q2**3*q3**4*q4 + 60*p**2*q1*q2**3*q3**3*q4**2 + 60*p**2*q1*q2**3*q3**3*q4*q5 - 540*p**2*q1*q2**3*q3**3*q4 + 60*p**2*q1*q2**3*q3**2*q4**3 + 60*p**2*q1*q2**3*q3**2*q4**2*q5 - 540*p**2*q1*q2**3*q3**2*q4**2 + 60*p**2*q1*q2**3*q3**2*q4*q5**2 - 540*p**2*q1*q2**3*q3**2*q4*q5 + 2160*p**2*q1*q2**3*q3**2*q4 + 60*p**2*q1*q2**3*q3*q4**4 + 60*p**2*q1*q2**3*q3*q4**3*q5 - 540*p**2*q1*q2**3*q3*q4**3 + 60*p**2*q1*q2**3*q3*q4**2*q5**2 - 540*p**2*q1*q2**3*q3*q4**2*q5 + 2160*p**2*q1*q2**3*q3*q4**2 + 60*p**2*q1*q2**3*q3*q4*q5**3 - 540*p**2*q1*q2**3*q3*q4*q5**2 + 2160*p**2*q1*q2**3*q3*q4*q5 - 5040*p**2*q1*q2**3*q3*q4 + 60*p**2*q1*q2**2*q3**5*q4 + 60*p**2*q1*q2**2*q3**4*q4**2 + 60*p**2*q1*q2**2*q3**4*q4*q5 - 540*p**2*q1*q2**2*q3**4*q4 + 60*p**2*q1*q2**2*q3**3*q4**3 + 60*p**2*q1*q2**2*q3**3*q4**2*q5 - 540*p**2*q1*q2**2*q3**3*q4**2 + 60*p**2*q1*q2**2*q3**3*q4*q5**2 - 540*p**2*q1*q2**2*q3**3*q4*q5 + 2160*p**2*q1*q2**2*q3**3*q4 + 60*p**2*q1*q2**2*q3**2*q4**4 + 60*p**2*q1*q2**2*q3**2*q4**3*q5 - 540*p**2*q1*q2**2*q3**2*q4**3 + 60*p**2*q1*q2**2*q3**2*q4**2*q5**2 - 540*p**2*q1*q2**2*q3**2*q4**2*q5 + 2160*p**2*q1*q2**2*q3**2*q4**2 + 60*p**2*q1*q2**2*q3**2*q4*q5**3 - 540*p**2*q1*q2**2*q3**2*q4*q5**2 + 2160*p**2*q1*q2**2*q3**2*q4*q5 - 5040*p**2*q1*q2**2*q3**2*q4 + 60*p**2*q1*q2**2*q3*q4**5 + 60*p**2*q1*q2**2*q3*q4**4*q5 - 540*p**2*q1*q2**2*q3*q4**4 + 60*p**2*q1*q2**2*q3*q4**3*q5**2 - 540*p**2*q1*q2**2*q3*q4**3*q5 + 2160*p**2*q1*q2**2*q3*q4**3 + 60*p**2*q1*q2**2*q3*q4**2*q5**3 - 540*p**2*q1*q2**2*q3*q4**2*q5**2 + 2160*p**2*q1*q2**2*q3*q4**2*q5 - 5040*p**2*q1*q2**2*q3*q4**2 + 60*p**2*q1*q2**2*q3*q4*q5**4 - 540*p**2*q1*q2**2*q3*q4*q5**3 + 2160*p**2*q1*q2**2*q3*q4*q5**2 - 5040*p**2*q1*q2**2*q3*q4*q5 + 7560*p**2*q1*q2**2*q3*q4 + 60*p**2*q1*q2*q3**6*q4 + 60*p**2*q1*q2*q3**5*q4**2 + 60*p**2*q1*q2*q3**5*q4*q5 - 540*p**2*q1*q2*q3**5*q4 + 60*p**2*q1*q2*q3**4*q4**3 + 60*p**2*q1*q2*q3**4*q4**2*q5 - 540*p**2*q1*q2*q3**4*q4**2 + 60*p**2*q1*q2*q3**4*q4*q5**2 - 540*p**2*q1*q2*q3**4*q4*q5 + 2160*p**2*q1*q2*q3**4*q4 + 60*p**2*q1*q2*q3**3*q4**4 + 60*p**2*q1*q2*q3**3*q4**3*q5 - 540*p**2*q1*q2*q3**3*q4**3 + 60*p**2*q1*q2*q3**3*q4**2*q5**2 - 540*p**2*q1*q2*q3**3*q4**2*q5 + 2160*p**2*q1*q2*q3**3*q4**2 + 60*p**2*q1*q2*q3**3*q4*q5**3 - 540*p**2*q1*q2*q3**3*q4*q5**2 + 2160*p**2*q1*q2*q3**3*q4*q5 - 5040*p**2*q1*q2*q3**3*q4 + 60*p**2*q1*q2*q3**2*q4**5 + 60*p**2*q1*q2*q3**2*q4**4*q5 - 540*p**2*q1*q2*q3**2*q4**4 + 60*p**2*q1*q2*q3**2*q4**3*q5**2 - 540*p**2*q1*q2*q3**2*q4**3*q5 + 2160*p**2*q1*q2*q3**2*q4**3 + 60*p**2*q1*q2*q3**2*q4**2*q5**3 - 540*p**2*q1*q2*q3**2*q4**2*q5**2 + 2160*p**2*q1*q2*q3**2*q4**2*q5 - 5040*p**2*q1*q2*q3**2*q4**2 + 60*p**2*q1*q2*q3**2*q4*q5**4 - 540*p**2*q1*q2*q3**2*q4*q5**3 + 2160*p**2*q1*q2*q3**2*q4*q5**2 - 5040*p**2*q1*q2*q3**2*q4*q5 + 7560*p**2*q1*q2*q3**2*q4 + 60*p**2*q1*q2*q3*q4**6 + 60*p**2*q1*q2*q3*q4**5*q5 - 540*p**2*q1*q2*q3*q4**5 + 60*p**2*q1*q2*q3*q4**4*q5**2 - 540*p**2*q1*q2*q3*q4**4*q5 + 2160*p**2*q1*q2*q3*q4**4 + 60*p**2*q1*q2*q3*q4**3*q5**3 - 540*p**2*q1*q2*q3*q4**3*q5**2 + 2160*p**2*q1*q2*q3*q4**3*q5 - 5040*p**2*q1*q2*q3*q4**3 + 60*p**2*q1*q2*q3*q4**2*q5**4 - 540*p**2*q1*q2*q3*q4**2*q5**3 + 2160*p**2*q1*q2*q3*q4**2*q5**2 - 5040*p**2*q1*q2*q3*q4**2*q5 + 7560*p**2*q1*q2*q3*q4**2 + 60*p**2*q1*q2*q3*q4*q5**5 - 540*p**2*q1*q2*q3*q4*q5**4 + 2160*p**2*q1*q2*q3*q4*q5**3 - 5040*p**2*q1*q2*q3*q4*q5**2 + 7560*p**2*q1*q2*q3*q4*q5 - 7560*p**2*q1*q2*q3*q4 - 135*p**2*q2**6*q3*q4 - 135*p**2*q2**5*q3**2*q4 - 135*p**2*q2**5*q3*q4**2 - 135*p**2*q2**5*q3*q4*q5 + 1080*p**2*q2**5*q3*q4 - 135*p**2*q2**4*q3**3*q4 - 135*p**2*q2**4*q3**2*q4**2 - 135*p**2*q2**4*q3**2*q4*q5 + 1080*p**2*q2**4*q3**2*q4 - 135*p**2*q2**4*q3*q4**3 - 135*p**2*q2**4*q3*q4**2*q5 + 1080*p**2*q2**4*q3*q4**2 - 135*p**2*q2**4*q3*q4*q5**2 + 1080*p**2*q2**4*q3*q4*q5 - 3780*p**2*q2**4*q3*q4 - 135*p**2*q2**3*q3**4*q4 - 135*p**2*q2**3*q3**3*q4**2 - 135*p**2*q2**3*q3**3*q4*q5 + 1080*p**2*q2**3*q3**3*q4 - 135*p**2*q2**3*q3**2*q4**3 - 135*p**2*q2**3*q3**2*q4**2*q5 + 1080*p**2*q2**3*q3**2*q4**2 - 135*p**2*q2**3*q3**2*q4*q5**2 + 1080*p**2*q2**3*q3**2*q4*q5 - 3780*p**2*q2**3*q3**2*q4 - 135*p**2*q2**3*q3*q4**4 - 135*p**2*q2**3*q3*q4**3*q5 + 1080*p**2*q2**3*q3*q4**3 - 135*p**2*q2**3*q3*q4**2*q5**2 + 1080*p**2*q2**3*q3*q4**2*q5 - 3780*p**2*q2**3*q3*q4**2 - 135*p**2*q2**3*q3*q4*q5**3 + 1080*p**2*q2**3*q3*q4*q5**2 - 3780*p**2*q2**3*q3*q4*q5 + 7560*p**2*q2**3*q3*q4 - 135*p**2*q2**2*q3**5*q4 - 135*p**2*q2**2*q3**4*q4**2 - 135*p**2*q2**2*q3**4*q4*q5 + 1080*p**2*q2**2*q3**4*q4 - 135*p**2*q2**2*q3**3*q4**3 - 135*p**2*q2**2*q3**3*q4**2*q5 + 1080*p**2*q2**2*q3**3*q4**2 - 135*p**2*q2**2*q3**3*q4*q5**2 + 1080*p**2*q2**2*q3**3*q4*q5 - 3780*p**2*q2**2*q3**3*q4 - 135*p**2*q2**2*q3**2*q4**4 - 135*p**2*q2**2*q3**2*q4**3*q5 + 1080*p**2*q2**2*q3**2*q4**3 - 135*p**2*q2**2*q3**2*q4**2*q5**2 + 1080*p**2*q2**2*q3**2*q4**2*q5 - 3780*p**2*q2**2*q3**2*q4**2 - 135*p**2*q2**2*q3**2*q4*q5**3 + 1080*p**2*q2**2*q3**2*q4*q5**2 - 3780*p**2*q2**2*q3**2*q4*q5 + 7560*p**2*q2**2*q3**2*q4 - 135*p**2*q2**2*q3*q4**5 - 135*p**2*q2**2*q3*q4**4*q5 + 1080*p**2*q2**2*q3*q4**4 - 135*p**2*q2**2*q3*q4**3*q5**2 + 1080*p**2*q2**2*q3*q4**3*q5 - 3780*p**2*q2**2*q3*q4**3 - 135*p**2*q2**2*q3*q4**2*q5**3 + 1080*p**2*q2**2*q3*q4**2*q5**2 - 3780*p**2*q2**2*q3*q4**2*q5 + 7560*p**2*q2**2*q3*q4**2 - 135*p**2*q2**2*q3*q4*q5**4 + 1080*p**2*q2**2*q3*q4*q5**3 - 3780*p**2*q2**2*q3*q4*q5**2 + 7560*p**2*q2**2*q3*q4*q5 - 9450*p**2*q2**2*q3*q4 - 135*p**2*q2*q3**6*q4 - 135*p**2*q2*q3**5*q4**2 - 135*p**2*q2*q3**5*q4*q5 + 1080*p**2*q2*q3**5*q4 - 135*p**2*q2*q3**4*q4**3 - 135*p**2*q2*q3**4*q4**2*q5 + 1080*p**2*q2*q3**4*q4**2 - 135*p**2*q2*q3**4*q4*q5**2 + 1080*p**2*q2*q3**4*q4*q5 - 3780*p**2*q2*q3**4*q4 - 135*p**2*q2*q3**3*q4**4 - 135*p**2*q2*q3**3*q4**3*q5 + 1080*p**2*q2*q3**3*q4**3 - 135*p**2*q2*q3**3*q4**2*q5**2 + 1080*p**2*q2*q3**3*q4**2*q5 - 3780*p**2*q2*q3**3*q4**2 - 135*p**2*q2*q3**3*q4*q5**3 + 1080*p**2*q2*q3**3*q4*q5**2 - 3780*p**2*q2*q3**3*q4*q5 + 7560*p**2*q2*q3**3*q4 - 135*p**2*q2*q3**2*q4**5 - 135*p**2*q2*q3**2*q4**4*q5 + 1080*p**2*q2*q3**2*q4**4 - 135*p**2*q2*q3**2*q4**3*q5**2 + 1080*p**2*q2*q3**2*q4**3*q5 - 3780*p**2*q2*q3**2*q4**3 - 135*p**2*q2*q3**2*q4**2*q5**3 + 1080*p**2*q2*q3**2*q4**2*q5**2 - 3780*p**2*q2*q3**2*q4**2*q5 + 7560*p**2*q2*q3**2*q4**2 - 135*p**2*q2*q3**2*q4*q5**4 + 1080*p**2*q2*q3**2*q4*q5**3 - 3780*p**2*q2*q3**2*q4*q5**2 + 7560*p**2*q2*q3**2*q4*q5 - 9450*p**2*q2*q3**2*q4 - 135*p**2*q2*q3*q4**6 - 135*p**2*q2*q3*q4**5*q5 + 1080*p**2*q2*q3*q4**5 - 135*p**2*q2*q3*q4**4*q5**2 + 1080*p**2*q2*q3*q4**4*q5 - 3780*p**2*q2*q3*q4**4 - 135*p**2*q2*q3*q4**3*q5**3 + 1080*p**2*q2*q3*q4**3*q5**2 - 3780*p**2*q2*q3*q4**3*q5 + 7560*p**2*q2*q3*q4**3 - 135*p**2*q2*q3*q4**2*q5**4 + 1080*p**2*q2*q3*q4**2*q5**3 - 3780*p**2*q2*q3*q4**2*q5**2 + 7560*p**2*q2*q3*q4**2*q5 - 9450*p**2*q2*q3*q4**2 - 135*p**2*q2*q3*q4*q5**5 + 1080*p**2*q2*q3*q4*q5**4 - 3780*p**2*q2*q3*q4*q5**3 + 7560*p**2*q2*q3*q4*q5**2 - 9450*p**2*q2*q3*q4*q5 + 7560*p**2*q2*q3*q4 + 120*p**2*q3**6*q4 + 120*p**2*q3**5*q4**2 + 120*p**2*q3**5*q4*q5 - 840*p**2*q3**5*q4 + 120*p**2*q3**4*q4**3 + 120*p**2*q3**4*q4**2*q5 - 840*p**2*q3**4*q4**2 + 120*p**2*q3**4*q4*q5**2 - 840*p**2*q3**4*q4*q5 + 2520*p**2*q3**4*q4 + 120*p**2*q3**3*q4**4 + 120*p**2*q3**3*q4**3*q5 - 840*p**2*q3**3*q4**3 + 120*p**2*q3**3*q4**2*q5**2 - 840*p**2*q3**3*q4**2*q5 + 2520*p**2*q3**3*q4**2 + 120*p**2*q3**3*q4*q5**3 - 840*p**2*q3**3*q4*q5**2 + 2520*p**2*q3**3*q4*q5 - 4200*p**2*q3**3*q4 + 120*p**2*q3**2*q4**5 + 120*p**2*q3**2*q4**4*q5 - 840*p**2*q3**2*q4**4 + 120*p**2*q3**2*q4**3*q5**2 - 840*p**2*q3**2*q4**3*q5 + 2520*p**2*q3**2*q4**3 + 120*p**2*q3**2*q4**2*q5**3 - 840*p**2*q3**2*q4**2*q5**2 + 2520*p**2*q3**2*q4**2*q5 - 4200*p**2*q3**2*q4**2 + 120*p**2*q3**2*q4*q5**4 - 840*p**2*q3**2*q4*q5**3 + 2520*p**2*q3**2*q4*q5**2 - 4200*p**2*q3**2*q4*q5 + 4200*p**2*q3**2*q4 + 120*p**2*q3*q4**6 + 120*p**2*q3*q4**5*q5 - 840*p**2*q3*q4**5 + 120*p**2*q3*q4**4*q5**2 - 840*p**2*q3*q4**4*q5 + 2520*p**2*q3*q4**4 + 120*p**2*q3*q4**3*q5**3 - 840*p**2*q3*q4**3*q5**2 + 2520*p**2*q3*q4**3*q5 - 4200*p**2*q3*q4**3 + 120*p**2*q3*q4**2*q5**4 - 840*p**2*q3*q4**2*q5**3 + 2520*p**2*q3*q4**2*q5**2 - 4200*p**2*q3*q4**2*q5 + 4200*p**2*q3*q4**2 + 120*p**2*q3*q4*q5**5 - 840*p**2*q3*q4*q5**4 + 2520*p**2*q3*q4*q5**3 - 4200*p**2*q3*q4*q5**2 + 4200*p**2*q3*q4*q5 - 2520*p**2*q3*q4 - 40*p*q1**6*q2*q3*q4 - 40*p*q1**5*q2**2*q3*q4 - 40*p*q1**5*q2*q3**2*q4 - 40*p*q1**5*q2*q3*q4**2 - 40*p*q1**5*q2*q3*q4*q5 + 360*p*q1**5*q2*q3*q4 - 40*p*q1**4*q2**3*q3*q4 - 40*p*q1**4*q2**2*q3**2*q4 - 40*p*q1**4*q2**2*q3*q4**2 - 40*p*q1**4*q2**2*q3*q4*q5 + 360*p*q1**4*q2**2*q3*q4 - 40*p*q1**4*q2*q3**3*q4 - 40*p*q1**4*q2*q3**2*q4**2 - 40*p*q1**4*q2*q3**2*q4*q5 + 360*p*q1**4*q2*q3**2*q4 - 40*p*q1**4*q2*q3*q4**3 - 40*p*q1**4*q2*q3*q4**2*q5 + 360*p*q1**4*q2*q3*q4**2 - 40*p*q1**4*q2*q3*q4*q5**2 + 360*p*q1**4*q2*q3*q4*q5 - 1440*p*q1**4*q2*q3*q4 - 40*p*q1**3*q2**4*q3*q4 - 40*p*q1**3*q2**3*q3**2*q4 - 40*p*q1**3*q2**3*q3*q4**2 - 40*p*q1**3*q2**3*q3*q4*q5 + 360*p*q1**3*q2**3*q3*q4 - 40*p*q1**3*q2**2*q3**3*q4 - 40*p*q1**3*q2**2*q3**2*q4**2 - 40*p*q1**3*q2**2*q3**2*q4*q5 + 360*p*q1**3*q2**2*q3**2*q4 - 40*p*q1**3*q2**2*q3*q4**3 - 40*p*q1**3*q2**2*q3*q4**2*q5 + 360*p*q1**3*q2**2*q3*q4**2 - 40*p*q1**3*q2**2*q3*q4*q5**2 + 360*p*q1**3*q2**2*q3*q4*q5 - 1440*p*q1**3*q2**2*q3*q4 - 40*p*q1**3*q2*q3**4*q4 - 40*p*q1**3*q2*q3**3*q4**2 - 40*p*q1**3*q2*q3**3*q4*q5 + 360*p*q1**3*q2*q3**3*q4 - 40*p*q1**3*q2*q3**2*q4**3 - 40*p*q1**3*q2*q3**2*q4**2*q5 + 360*p*q1**3*q2*q3**2*q4**2 - 40*p*q1**3*q2*q3**2*q4*q5**2 + 360*p*q1**3*q2*q3**2*q4*q5 - 1440*p*q1**3*q2*q3**2*q4 - 40*p*q1**3*q2*q3*q4**4 - 40*p*q1**3*q2*q3*q4**3*q5 + 360*p*q1**3*q2*q3*q4**3 - 40*p*q1**3*q2*q3*q4**2*q5**2 + 360*p*q1**3*q2*q3*q4**2*q5 - 1440*p*q1**3*q2*q3*q4**2 - 40*p*q1**3*q2*q3*q4*q5**3 + 360*p*q1**3*q2*q3*q4*q5**2 - 1440*p*q1**3*q2*q3*q4*q5 + 3360*p*q1**3*q2*q3*q4 - 40*p*q1**2*q2**5*q3*q4 - 40*p*q1**2*q2**4*q3**2*q4 - 40*p*q1**2*q2**4*q3*q4**2 - 40*p*q1**2*q2**4*q3*q4*q5 + 360*p*q1**2*q2**4*q3*q4 - 40*p*q1**2*q2**3*q3**3*q4 - 40*p*q1**2*q2**3*q3**2*q4**2 - 40*p*q1**2*q2**3*q3**2*q4*q5 + 360*p*q1**2*q2**3*q3**2*q4 - 40*p*q1**2*q2**3*q3*q4**3 - 40*p*q1**2*q2**3*q3*q4**2*q5 + 360*p*q1**2*q2**3*q3*q4**2 - 40*p*q1**2*q2**3*q3*q4*q5**2 + 360*p*q1**2*q2**3*q3*q4*q5 - 1440*p*q1**2*q2**3*q3*q4 - 40*p*q1**2*q2**2*q3**4*q4 - 40*p*q1**2*q2**2*q3**3*q4**2 - 40*p*q1**2*q2**2*q3**3*q4*q5 + 360*p*q1**2*q2**2*q3**3*q4 - 40*p*q1**2*q2**2*q3**2*q4**3 - 40*p*q1**2*q2**2*q3**2*q4**2*q5 + 360*p*q1**2*q2**2*q3**2*q4**2 - 40*p*q1**2*q2**2*q3**2*q4*q5**2 + 360*p*q1**2*q2**2*q3**2*q4*q5 - 1440*p*q1**2*q2**2*q3**2*q4 - 40*p*q1**2*q2**2*q3*q4**4 - 40*p*q1**2*q2**2*q3*q4**3*q5 + 360*p*q1**2*q2**2*q3*q4**3 - 40*p*q1**2*q2**2*q3*q4**2*q5**2 + 360*p*q1**2*q2**2*q3*q4**2*q5 - 1440*p*q1**2*q2**2*q3*q4**2 - 40*p*q1**2*q2**2*q3*q4*q5**3 + 360*p*q1**2*q2**2*q3*q4*q5**2 - 1440*p*q1**2*q2**2*q3*q4*q5 + 3360*p*q1**2*q2**2*q3*q4 - 40*p*q1**2*q2*q3**5*q4 - 40*p*q1**2*q2*q3**4*q4**2 - 40*p*q1**2*q2*q3**4*q4*q5 + 360*p*q1**2*q2*q3**4*q4 - 40*p*q1**2*q2*q3**3*q4**3 - 40*p*q1**2*q2*q3**3*q4**2*q5 + 360*p*q1**2*q2*q3**3*q4**2 - 40*p*q1**2*q2*q3**3*q4*q5**2 + 360*p*q1**2*q2*q3**3*q4*q5 - 1440*p*q1**2*q2*q3**3*q4 - 40*p*q1**2*q2*q3**2*q4**4 - 40*p*q1**2*q2*q3**2*q4**3*q5 + 360*p*q1**2*q2*q3**2*q4**3 - 40*p*q1**2*q2*q3**2*q4**2*q5**2 + 360*p*q1**2*q2*q3**2*q4**2*q5 - 1440*p*q1**2*q2*q3**2*q4**2 - 40*p*q1**2*q2*q3**2*q4*q5**3 + 360*p*q1**2*q2*q3**2*q4*q5**2 - 1440*p*q1**2*q2*q3**2*q4*q5 + 3360*p*q1**2*q2*q3**2*q4 - 40*p*q1**2*q2*q3*q4**5 - 40*p*q1**2*q2*q3*q4**4*q5 + 360*p*q1**2*q2*q3*q4**4 - 40*p*q1**2*q2*q3*q4**3*q5**2 + 360*p*q1**2*q2*q3*q4**3*q5 - 1440*p*q1**2*q2*q3*q4**3 - 40*p*q1**2*q2*q3*q4**2*q5**3 + 360*p*q1**2*q2*q3*q4**2*q5**2 - 1440*p*q1**2*q2*q3*q4**2*q5 + 3360*p*q1**2*q2*q3*q4**2 - 40*p*q1**2*q2*q3*q4*q5**4 + 360*p*q1**2*q2*q3*q4*q5**3 - 1440*p*q1**2*q2*q3*q4*q5**2 + 3360*p*q1**2*q2*q3*q4*q5 - 5040*p*q1**2*q2*q3*q4 - 40*p*q1*q2**6*q3*q4 - 40*p*q1*q2**5*q3**2*q4 - 40*p*q1*q2**5*q3*q4**2 - 40*p*q1*q2**5*q3*q4*q5 + 360*p*q1*q2**5*q3*q4 - 40*p*q1*q2**4*q3**3*q4 - 40*p*q1*q2**4*q3**2*q4**2 - 40*p*q1*q2**4*q3**2*q4*q5 + 360*p*q1*q2**4*q3**2*q4 - 40*p*q1*q2**4*q3*q4**3 - 40*p*q1*q2**4*q3*q4**2*q5 + 360*p*q1*q2**4*q3*q4**2 - 40*p*q1*q2**4*q3*q4*q5**2 + 360*p*q1*q2**4*q3*q4*q5 - 1440*p*q1*q2**4*q3*q4 - 40*p*q1*q2**3*q3**4*q4 - 40*p*q1*q2**3*q3**3*q4**2 - 40*p*q1*q2**3*q3**3*q4*q5 + 360*p*q1*q2**3*q3**3*q4 - 40*p*q1*q2**3*q3**2*q4**3 - 40*p*q1*q2**3*q3**2*q4**2*q5 + 360*p*q1*q2**3*q3**2*q4**2 - 40*p*q1*q2**3*q3**2*q4*q5**2 + 360*p*q1*q2**3*q3**2*q4*q5 - 1440*p*q1*q2**3*q3**2*q4 - 40*p*q1*q2**3*q3*q4**4 - 40*p*q1*q2**3*q3*q4**3*q5 + 360*p*q1*q2**3*q3*q4**3 - 40*p*q1*q2**3*q3*q4**2*q5**2 + 360*p*q1*q2**3*q3*q4**2*q5 - 1440*p*q1*q2**3*q3*q4**2 - 40*p*q1*q2**3*q3*q4*q5**3 + 360*p*q1*q2**3*q3*q4*q5**2 - 1440*p*q1*q2**3*q3*q4*q5 + 3360*p*q1*q2**3*q3*q4 - 40*p*q1*q2**2*q3**5*q4 - 40*p*q1*q2**2*q3**4*q4**2 - 40*p*q1*q2**2*q3**4*q4*q5 + 360*p*q1*q2**2*q3**4*q4 - 40*p*q1*q2**2*q3**3*q4**3 - 40*p*q1*q2**2*q3**3*q4**2*q5 + 360*p*q1*q2**2*q3**3*q4**2 - 40*p*q1*q2**2*q3**3*q4*q5**2 + 360*p*q1*q2**2*q3**3*q4*q5 - 1440*p*q1*q2**2*q3**3*q4 - 40*p*q1*q2**2*q3**2*q4**4 - 40*p*q1*q2**2*q3**2*q4**3*q5 + 360*p*q1*q2**2*q3**2*q4**3 - 40*p*q1*q2**2*q3**2*q4**2*q5**2 + 360*p*q1*q2**2*q3**2*q4**2*q5 - 1440*p*q1*q2**2*q3**2*q4**2 - 40*p*q1*q2**2*q3**2*q4*q5**3 + 360*p*q1*q2**2*q3**2*q4*q5**2 - 1440*p*q1*q2**2*q3**2*q4*q5 + 3360*p*q1*q2**2*q3**2*q4 - 40*p*q1*q2**2*q3*q4**5 - 40*p*q1*q2**2*q3*q4**4*q5 + 360*p*q1*q2**2*q3*q4**4 - 40*p*q1*q2**2*q3*q4**3*q5**2 + 360*p*q1*q2**2*q3*q4**3*q5 - 1440*p*q1*q2**2*q3*q4**3 - 40*p*q1*q2**2*q3*q4**2*q5**3 + 360*p*q1*q2**2*q3*q4**2*q5**2 - 1440*p*q1*q2**2*q3*q4**2*q5 + 3360*p*q1*q2**2*q3*q4**2 - 40*p*q1*q2**2*q3*q4*q5**4 + 360*p*q1*q2**2*q3*q4*q5**3 - 1440*p*q1*q2**2*q3*q4*q5**2 + 3360*p*q1*q2**2*q3*q4*q5 - 5040*p*q1*q2**2*q3*q4 - 40*p*q1*q2*q3**6*q4 - 40*p*q1*q2*q3**5*q4**2 - 40*p*q1*q2*q3**5*q4*q5 + 360*p*q1*q2*q3**5*q4 - 40*p*q1*q2*q3**4*q4**3 - 40*p*q1*q2*q3**4*q4**2*q5 + 360*p*q1*q2*q3**4*q4**2 - 40*p*q1*q2*q3**4*q4*q5**2 + 360*p*q1*q2*q3**4*q4*q5 - 1440*p*q1*q2*q3**4*q4 - 40*p*q1*q2*q3**3*q4**4 - 40*p*q1*q2*q3**3*q4**3*q5 + 360*p*q1*q2*q3**3*q4**3 - 40*p*q1*q2*q3**3*q4**2*q5**2 + 360*p*q1*q2*q3**3*q4**2*q5 - 1440*p*q1*q2*q3**3*q4**2 - 40*p*q1*q2*q3**3*q4*q5**3 + 360*p*q1*q2*q3**3*q4*q5**2 - 1440*p*q1*q2*q3**3*q4*q5 + 3360*p*q1*q2*q3**3*q4 - 40*p*q1*q2*q3**2*q4**5 - 40*p*q1*q2*q3**2*q4**4*q5 + 360*p*q1*q2*q3**2*q4**4 - 40*p*q1*q2*q3**2*q4**3*q5**2 + 360*p*q1*q2*q3**2*q4**3*q5 - 1440*p*q1*q2*q3**2*q4**3 - 40*p*q1*q2*q3**2*q4**2*q5**3 + 360*p*q1*q2*q3**2*q4**2*q5**2 - 1440*p*q1*q2*q3**2*q4**2*q5 + 3360*p*q1*q2*q3**2*q4**2 - 40*p*q1*q2*q3**2*q4*q5**4 + 360*p*q1*q2*q3**2*q4*q5**3 - 1440*p*q1*q2*q3**2*q4*q5**2 + 3360*p*q1*q2*q3**2*q4*q5 - 5040*p*q1*q2*q3**2*q4 - 40*p*q1*q2*q3*q4**6 - 40*p*q1*q2*q3*q4**5*q5 + 360*p*q1*q2*q3*q4**5 - 40*p*q1*q2*q3*q4**4*q5**2 + 360*p*q1*q2*q3*q4**4*q5 - 1440*p*q1*q2*q3*q4**4 - 40*p*q1*q2*q3*q4**3*q5**3 + 360*p*q1*q2*q3*q4**3*q5**2 - 1440*p*q1*q2*q3*q4**3*q5 + 3360*p*q1*q2*q3*q4**3 - 40*p*q1*q2*q3*q4**2*q5**4 + 360*p*q1*q2*q3*q4**2*q5**3 - 1440*p*q1*q2*q3*q4**2*q5**2 + 3360*p*q1*q2*q3*q4**2*q5 - 5040*p*q1*q2*q3*q4**2 - 40*p*q1*q2*q3*q4*q5**5 + 360*p*q1*q2*q3*q4*q5**4 - 1440*p*q1*q2*q3*q4*q5**3 + 3360*p*q1*q2*q3*q4*q5**2 - 5040*p*q1*q2*q3*q4*q5 + 5040*p*q1*q2*q3*q4 + 45*p*q2**6*q3*q4 + 45*p*q2**5*q3**2*q4 + 45*p*q2**5*q3*q4**2 + 45*p*q2**5*q3*q4*q5 - 360*p*q2**5*q3*q4 + 45*p*q2**4*q3**3*q4 + 45*p*q2**4*q3**2*q4**2 + 45*p*q2**4*q3**2*q4*q5 - 360*p*q2**4*q3**2*q4 + 45*p*q2**4*q3*q4**3 + 45*p*q2**4*q3*q4**2*q5 - 360*p*q2**4*q3*q4**2 + 45*p*q2**4*q3*q4*q5**2 - 360*p*q2**4*q3*q4*q5 + 1260*p*q2**4*q3*q4 + 45*p*q2**3*q3**4*q4 + 45*p*q2**3*q3**3*q4**2 + 45*p*q2**3*q3**3*q4*q5 - 360*p*q2**3*q3**3*q4 + 45*p*q2**3*q3**2*q4**3 + 45*p*q2**3*q3**2*q4**2*q5 - 360*p*q2**3*q3**2*q4**2 + 45*p*q2**3*q3**2*q4*q5**2 - 360*p*q2**3*q3**2*q4*q5 + 1260*p*q2**3*q3**2*q4 + 45*p*q2**3*q3*q4**4 + 45*p*q2**3*q3*q4**3*q5 - 360*p*q2**3*q3*q4**3 + 45*p*q2**3*q3*q4**2*q5**2 - 360*p*q2**3*q3*q4**2*q5 + 1260*p*q2**3*q3*q4**2 + 45*p*q2**3*q3*q4*q5**3 - 360*p*q2**3*q3*q4*q5**2 + 1260*p*q2**3*q3*q4*q5 - 2520*p*q2**3*q3*q4 + 45*p*q2**2*q3**5*q4 + 45*p*q2**2*q3**4*q4**2 + 45*p*q2**2*q3**4*q4*q5 - 360*p*q2**2*q3**4*q4 + 45*p*q2**2*q3**3*q4**3 + 45*p*q2**2*q3**3*q4**2*q5 - 360*p*q2**2*q3**3*q4**2 + 45*p*q2**2*q3**3*q4*q5**2 - 360*p*q2**2*q3**3*q4*q5 + 1260*p*q2**2*q3**3*q4 + 45*p*q2**2*q3**2*q4**4 + 45*p*q2**2*q3**2*q4**3*q5 - 360*p*q2**2*q3**2*q4**3 + 45*p*q2**2*q3**2*q4**2*q5**2 - 360*p*q2**2*q3**2*q4**2*q5 + 1260*p*q2**2*q3**2*q4**2 + 45*p*q2**2*q3**2*q4*q5**3 - 360*p*q2**2*q3**2*q4*q5**2 + 1260*p*q2**2*q3**2*q4*q5 - 2520*p*q2**2*q3**2*q4 + 45*p*q2**2*q3*q4**5 + 45*p*q2**2*q3*q4**4*q5 - 360*p*q2**2*q3*q4**4 + 45*p*q2**2*q3*q4**3*q5**2 - 360*p*q2**2*q3*q4**3*q5 + 1260*p*q2**2*q3*q4**3 + 45*p*q2**2*q3*q4**2*q5**3 - 360*p*q2**2*q3*q4**2*q5**2 + 1260*p*q2**2*q3*q4**2*q5 - 2520*p*q2**2*q3*q4**2 + 45*p*q2**2*q3*q4*q5**4 - 360*p*q2**2*q3*q4*q5**3 + 1260*p*q2**2*q3*q4*q5**2 - 2520*p*q2**2*q3*q4*q5 + 3150*p*q2**2*q3*q4 + 45*p*q2*q3**6*q4 + 45*p*q2*q3**5*q4**2 + 45*p*q2*q3**5*q4*q5 - 360*p*q2*q3**5*q4 + 45*p*q2*q3**4*q4**3 + 45*p*q2*q3**4*q4**2*q5 - 360*p*q2*q3**4*q4**2 + 45*p*q2*q3**4*q4*q5**2 - 360*p*q2*q3**4*q4*q5 + 1260*p*q2*q3**4*q4 + 45*p*q2*q3**3*q4**4 + 45*p*q2*q3**3*q4**3*q5 - 360*p*q2*q3**3*q4**3 + 45*p*q2*q3**3*q4**2*q5**2 - 360*p*q2*q3**3*q4**2*q5 + 1260*p*q2*q3**3*q4**2 + 45*p*q2*q3**3*q4*q5**3 - 360*p*q2*q3**3*q4*q5**2 + 1260*p*q2*q3**3*q4*q5 - 2520*p*q2*q3**3*q4 + 45*p*q2*q3**2*q4**5 + 45*p*q2*q3**2*q4**4*q5 - 360*p*q2*q3**2*q4**4 + 45*p*q2*q3**2*q4**3*q5**2 - 360*p*q2*q3**2*q4**3*q5 + 1260*p*q2*q3**2*q4**3 + 45*p*q2*q3**2*q4**2*q5**3 - 360*p*q2*q3**2*q4**2*q5**2 + 1260*p*q2*q3**2*q4**2*q5 - 2520*p*q2*q3**2*q4**2 + 45*p*q2*q3**2*q4*q5**4 - 360*p*q2*q3**2*q4*q5**3 + 1260*p*q2*q3**2*q4*q5**2 - 2520*p*q2*q3**2*q4*q5 + 3150*p*q2*q3**2*q4 + 45*p*q2*q3*q4**6 + 45*p*q2*q3*q4**5*q5 - 360*p*q2*q3*q4**5 + 45*p*q2*q3*q4**4*q5**2 - 360*p*q2*q3*q4**4*q5 + 1260*p*q2*q3*q4**4 + 45*p*q2*q3*q4**3*q5**3 - 360*p*q2*q3*q4**3*q5**2 + 1260*p*q2*q3*q4**3*q5 - 2520*p*q2*q3*q4**3 + 45*p*q2*q3*q4**2*q5**4 - 360*p*q2*q3*q4**2*q5**3 + 1260*p*q2*q3*q4**2*q5**2 - 2520*p*q2*q3*q4**2*q5 + 3150*p*q2*q3*q4**2 + 45*p*q2*q3*q4*q5**5 - 360*p*q2*q3*q4*q5**4 + 1260*p*q2*q3*q4*q5**3 - 2520*p*q2*q3*q4*q5**2 + 3150*p*q2*q3*q4*q5 - 2520*p*q2*q3*q4 + 10*q1**6*q2*q3*q4 + 10*q1**5*q2**2*q3*q4 + 10*q1**5*q2*q3**2*q4 + 10*q1**5*q2*q3*q4**2 + 10*q1**5*q2*q3*q4*q5 - 90*q1**5*q2*q3*q4 + 10*q1**4*q2**3*q3*q4 + 10*q1**4*q2**2*q3**2*q4 + 10*q1**4*q2**2*q3*q4**2 + 10*q1**4*q2**2*q3*q4*q5 - 90*q1**4*q2**2*q3*q4 + 10*q1**4*q2*q3**3*q4 + 10*q1**4*q2*q3**2*q4**2 + 10*q1**4*q2*q3**2*q4*q5 - 90*q1**4*q2*q3**2*q4 + 10*q1**4*q2*q3*q4**3 + 10*q1**4*q2*q3*q4**2*q5 - 90*q1**4*q2*q3*q4**2 + 10*q1**4*q2*q3*q4*q5**2 - 90*q1**4*q2*q3*q4*q5 + 360*q1**4*q2*q3*q4 + 10*q1**3*q2**4*q3*q4 + 10*q1**3*q2**3*q3**2*q4 + 10*q1**3*q2**3*q3*q4**2 + 10*q1**3*q2**3*q3*q4*q5 - 90*q1**3*q2**3*q3*q4 + 10*q1**3*q2**2*q3**3*q4 + 10*q1**3*q2**2*q3**2*q4**2 + 10*q1**3*q2**2*q3**2*q4*q5 - 90*q1**3*q2**2*q3**2*q4 + 10*q1**3*q2**2*q3*q4**3 + 10*q1**3*q2**2*q3*q4**2*q5 - 90*q1**3*q2**2*q3*q4**2 + 10*q1**3*q2**2*q3*q4*q5**2 - 90*q1**3*q2**2*q3*q4*q5 + 360*q1**3*q2**2*q3*q4 + 10*q1**3*q2*q3**4*q4 + 10*q1**3*q2*q3**3*q4**2 + 10*q1**3*q2*q3**3*q4*q5 - 90*q1**3*q2*q3**3*q4 + 10*q1**3*q2*q3**2*q4**3 + 10*q1**3*q2*q3**2*q4**2*q5 - 90*q1**3*q2*q3**2*q4**2 + 10*q1**3*q2*q3**2*q4*q5**2 - 90*q1**3*q2*q3**2*q4*q5 + 360*q1**3*q2*q3**2*q4 + 10*q1**3*q2*q3*q4**4 + 10*q1**3*q2*q3*q4**3*q5 - 90*q1**3*q2*q3*q4**3 + 10*q1**3*q2*q3*q4**2*q5**2 - 90*q1**3*q2*q3*q4**2*q5 + 360*q1**3*q2*q3*q4**2 + 10*q1**3*q2*q3*q4*q5**3 - 90*q1**3*q2*q3*q4*q5**2 + 360*q1**3*q2*q3*q4*q5 - 840*q1**3*q2*q3*q4 + 10*q1**2*q2**5*q3*q4 + 10*q1**2*q2**4*q3**2*q4 + 10*q1**2*q2**4*q3*q4**2 + 10*q1**2*q2**4*q3*q4*q5 - 90*q1**2*q2**4*q3*q4 + 10*q1**2*q2**3*q3**3*q4 + 10*q1**2*q2**3*q3**2*q4**2 + 10*q1**2*q2**3*q3**2*q4*q5 - 90*q1**2*q2**3*q3**2*q4 + 10*q1**2*q2**3*q3*q4**3 + 10*q1**2*q2**3*q3*q4**2*q5 - 90*q1**2*q2**3*q3*q4**2 + 10*q1**2*q2**3*q3*q4*q5**2 - 90*q1**2*q2**3*q3*q4*q5 + 360*q1**2*q2**3*q3*q4 + 10*q1**2*q2**2*q3**4*q4 + 10*q1**2*q2**2*q3**3*q4**2 + 10*q1**2*q2**2*q3**3*q4*q5 - 90*q1**2*q2**2*q3**3*q4 + 10*q1**2*q2**2*q3**2*q4**3 + 10*q1**2*q2**2*q3**2*q4**2*q5 - 90*q1**2*q2**2*q3**2*q4**2 + 10*q1**2*q2**2*q3**2*q4*q5**2 - 90*q1**2*q2**2*q3**2*q4*q5 + 360*q1**2*q2**2*q3**2*q4 + 10*q1**2*q2**2*q3*q4**4 + 10*q1**2*q2**2*q3*q4**3*q5 - 90*q1**2*q2**2*q3*q4**3 + 10*q1**2*q2**2*q3*q4**2*q5**2 - 90*q1**2*q2**2*q3*q4**2*q5 + 360*q1**2*q2**2*q3*q4**2 + 10*q1**2*q2**2*q3*q4*q5**3 - 90*q1**2*q2**2*q3*q4*q5**2 + 360*q1**2*q2**2*q3*q4*q5 - 840*q1**2*q2**2*q3*q4 + 10*q1**2*q2*q3**5*q4 + 10*q1**2*q2*q3**4*q4**2 + 10*q1**2*q2*q3**4*q4*q5 - 90*q1**2*q2*q3**4*q4 + 10*q1**2*q2*q3**3*q4**3 + 10*q1**2*q2*q3**3*q4**2*q5 - 90*q1**2*q2*q3**3*q4**2 + 10*q1**2*q2*q3**3*q4*q5**2 - 90*q1**2*q2*q3**3*q4*q5 + 360*q1**2*q2*q3**3*q4 + 10*q1**2*q2*q3**2*q4**4 + 10*q1**2*q2*q3**2*q4**3*q5 - 90*q1**2*q2*q3**2*q4**3 + 10*q1**2*q2*q3**2*q4**2*q5**2 - 90*q1**2*q2*q3**2*q4**2*q5 + 360*q1**2*q2*q3**2*q4**2 + 10*q1**2*q2*q3**2*q4*q5**3 - 90*q1**2*q2*q3**2*q4*q5**2 + 360*q1**2*q2*q3**2*q4*q5 - 840*q1**2*q2*q3**2*q4 + 10*q1**2*q2*q3*q4**5 + 10*q1**2*q2*q3*q4**4*q5 - 90*q1**2*q2*q3*q4**4 + 10*q1**2*q2*q3*q4**3*q5**2 - 90*q1**2*q2*q3*q4**3*q5 + 360*q1**2*q2*q3*q4**3 + 10*q1**2*q2*q3*q4**2*q5**3 - 90*q1**2*q2*q3*q4**2*q5**2 + 360*q1**2*q2*q3*q4**2*q5 - 840*q1**2*q2*q3*q4**2 + 10*q1**2*q2*q3*q4*q5**4 - 90*q1**2*q2*q3*q4*q5**3 + 360*q1**2*q2*q3*q4*q5**2 - 840*q1**2*q2*q3*q4*q5 + 1260*q1**2*q2*q3*q4 + 10*q1*q2**6*q3*q4 + 10*q1*q2**5*q3**2*q4 + 10*q1*q2**5*q3*q4**2 + 10*q1*q2**5*q3*q4*q5 - 90*q1*q2**5*q3*q4 + 10*q1*q2**4*q3**3*q4 + 10*q1*q2**4*q3**2*q4**2 + 10*q1*q2**4*q3**2*q4*q5 - 90*q1*q2**4*q3**2*q4 + 10*q1*q2**4*q3*q4**3 + 10*q1*q2**4*q3*q4**2*q5 - 90*q1*q2**4*q3*q4**2 + 10*q1*q2**4*q3*q4*q5**2 - 90*q1*q2**4*q3*q4*q5 + 360*q1*q2**4*q3*q4 + 10*q1*q2**3*q3**4*q4 + 10*q1*q2**3*q3**3*q4**2 + 10*q1*q2**3*q3**3*q4*q5 - 90*q1*q2**3*q3**3*q4 + 10*q1*q2**3*q3**2*q4**3 + 10*q1*q2**3*q3**2*q4**2*q5 - 90*q1*q2**3*q3**2*q4**2 + 10*q1*q2**3*q3**2*q4*q5**2 - 90*q1*q2**3*q3**2*q4*q5 + 360*q1*q2**3*q3**2*q4 + 10*q1*q2**3*q3*q4**4 + 10*q1*q2**3*q3*q4**3*q5 - 90*q1*q2**3*q3*q4**3 + 10*q1*q2**3*q3*q4**2*q5**2 - 90*q1*q2**3*q3*q4**2*q5 + 360*q1*q2**3*q3*q4**2 + 10*q1*q2**3*q3*q4*q5**3 - 90*q1*q2**3*q3*q4*q5**2 + 360*q1*q2**3*q3*q4*q5 - 840*q1*q2**3*q3*q4 + 10*q1*q2**2*q3**5*q4 + 10*q1*q2**2*q3**4*q4**2 + 10*q1*q2**2*q3**4*q4*q5 - 90*q1*q2**2*q3**4*q4 + 10*q1*q2**2*q3**3*q4**3 + 10*q1*q2**2*q3**3*q4**2*q5 - 90*q1*q2**2*q3**3*q4**2 + 10*q1*q2**2*q3**3*q4*q5**2 - 90*q1*q2**2*q3**3*q4*q5 + 360*q1*q2**2*q3**3*q4 + 10*q1*q2**2*q3**2*q4**4 + 10*q1*q2**2*q3**2*q4**3*q5 - 90*q1*q2**2*q3**2*q4**3 + 10*q1*q2**2*q3**2*q4**2*q5**2 - 90*q1*q2**2*q3**2*q4**2*q5 + 360*q1*q2**2*q3**2*q4**2 + 10*q1*q2**2*q3**2*q4*q5**3 - 90*q1*q2**2*q3**2*q4*q5**2 + 360*q1*q2**2*q3**2*q4*q5 - 840*q1*q2**2*q3**2*q4 + 10*q1*q2**2*q3*q4**5 + 10*q1*q2**2*q3*q4**4*q5 - 90*q1*q2**2*q3*q4**4 + 10*q1*q2**2*q3*q4**3*q5**2 - 90*q1*q2**2*q3*q4**3*q5 + 360*q1*q2**2*q3*q4**3 + 10*q1*q2**2*q3*q4**2*q5**3 - 90*q1*q2**2*q3*q4**2*q5**2 + 360*q1*q2**2*q3*q4**2*q5 - 840*q1*q2**2*q3*q4**2 + 10*q1*q2**2*q3*q4*q5**4 - 90*q1*q2**2*q3*q4*q5**3 + 360*q1*q2**2*q3*q4*q5**2 - 840*q1*q2**2*q3*q4*q5 + 1260*q1*q2**2*q3*q4 + 10*q1*q2*q3**6*q4 + 10*q1*q2*q3**5*q4**2 + 10*q1*q2*q3**5*q4*q5 - 90*q1*q2*q3**5*q4 + 10*q1*q2*q3**4*q4**3 + 10*q1*q2*q3**4*q4**2*q5 - 90*q1*q2*q3**4*q4**2 + 10*q1*q2*q3**4*q4*q5**2 - 90*q1*q2*q3**4*q4*q5 + 360*q1*q2*q3**4*q4 + 10*q1*q2*q3**3*q4**4 + 10*q1*q2*q3**3*q4**3*q5 - 90*q1*q2*q3**3*q4**3 + 10*q1*q2*q3**3*q4**2*q5**2 - 90*q1*q2*q3**3*q4**2*q5 + 360*q1*q2*q3**3*q4**2 + 10*q1*q2*q3**3*q4*q5**3 - 90*q1*q2*q3**3*q4*q5**2 + 360*q1*q2*q3**3*q4*q5 - 840*q1*q2*q3**3*q4 + 10*q1*q2*q3**2*q4**5 + 10*q1*q2*q3**2*q4**4*q5 - 90*q1*q2*q3**2*q4**4 + 10*q1*q2*q3**2*q4**3*q5**2 - 90*q1*q2*q3**2*q4**3*q5 + 360*q1*q2*q3**2*q4**3 + 10*q1*q2*q3**2*q4**2*q5**3 - 90*q1*q2*q3**2*q4**2*q5**2 + 360*q1*q2*q3**2*q4**2*q5 - 840*q1*q2*q3**2*q4**2 + 10*q1*q2*q3**2*q4*q5**4 - 90*q1*q2*q3**2*q4*q5**3 + 360*q1*q2*q3**2*q4*q5**2 - 840*q1*q2*q3**2*q4*q5 + 1260*q1*q2*q3**2*q4 + 10*q1*q2*q3*q4**6 + 10*q1*q2*q3*q4**5*q5 - 90*q1*q2*q3*q4**5 + 10*q1*q2*q3*q4**4*q5**2 - 90*q1*q2*q3*q4**4*q5 + 360*q1*q2*q3*q4**4 + 10*q1*q2*q3*q4**3*q5**3 - 90*q1*q2*q3*q4**3*q5**2 + 360*q1*q2*q3*q4**3*q5 - 840*q1*q2*q3*q4**3 + 10*q1*q2*q3*q4**2*q5**4 - 90*q1*q2*q3*q4**2*q5**3 + 360*q1*q2*q3*q4**2*q5**2 - 840*q1*q2*q3*q4**2*q5 + 1260*q1*q2*q3*q4**2 + 10*q1*q2*q3*q4*q5**5 - 90*q1*q2*q3*q4*q5**4 + 360*q1*q2*q3*q4*q5**3 - 840*q1*q2*q3*q4*q5**2 + 1260*q1*q2*q3*q4*q5 - 1260*q1*q2*q3*q4)'
f_multiparam[10][6] = '-p*(p - 1)**4*(10*p**5*q1**5*q2*q3*q4*q5 + 10*p**5*q1**4*q2**2*q3*q4*q5 + 10*p**5*q1**4*q2*q3**2*q4*q5 + 10*p**5*q1**4*q2*q3*q4**2*q5 + 10*p**5*q1**4*q2*q3*q4*q5**2 + 10*p**5*q1**4*q2*q3*q4*q5*q6 - 90*p**5*q1**4*q2*q3*q4*q5 + 10*p**5*q1**3*q2**3*q3*q4*q5 + 10*p**5*q1**3*q2**2*q3**2*q4*q5 + 10*p**5*q1**3*q2**2*q3*q4**2*q5 + 10*p**5*q1**3*q2**2*q3*q4*q5**2 + 10*p**5*q1**3*q2**2*q3*q4*q5*q6 - 90*p**5*q1**3*q2**2*q3*q4*q5 + 10*p**5*q1**3*q2*q3**3*q4*q5 + 10*p**5*q1**3*q2*q3**2*q4**2*q5 + 10*p**5*q1**3*q2*q3**2*q4*q5**2 + 10*p**5*q1**3*q2*q3**2*q4*q5*q6 - 90*p**5*q1**3*q2*q3**2*q4*q5 + 10*p**5*q1**3*q2*q3*q4**3*q5 + 10*p**5*q1**3*q2*q3*q4**2*q5**2 + 10*p**5*q1**3*q2*q3*q4**2*q5*q6 - 90*p**5*q1**3*q2*q3*q4**2*q5 + 10*p**5*q1**3*q2*q3*q4*q5**3 + 10*p**5*q1**3*q2*q3*q4*q5**2*q6 - 90*p**5*q1**3*q2*q3*q4*q5**2 + 10*p**5*q1**3*q2*q3*q4*q5*q6**2 - 90*p**5*q1**3*q2*q3*q4*q5*q6 + 360*p**5*q1**3*q2*q3*q4*q5 + 10*p**5*q1**2*q2**4*q3*q4*q5 + 10*p**5*q1**2*q2**3*q3**2*q4*q5 + 10*p**5*q1**2*q2**3*q3*q4**2*q5 + 10*p**5*q1**2*q2**3*q3*q4*q5**2 + 10*p**5*q1**2*q2**3*q3*q4*q5*q6 - 90*p**5*q1**2*q2**3*q3*q4*q5 + 10*p**5*q1**2*q2**2*q3**3*q4*q5 + 10*p**5*q1**2*q2**2*q3**2*q4**2*q5 + 10*p**5*q1**2*q2**2*q3**2*q4*q5**2 + 10*p**5*q1**2*q2**2*q3**2*q4*q5*q6 - 90*p**5*q1**2*q2**2*q3**2*q4*q5 + 10*p**5*q1**2*q2**2*q3*q4**3*q5 + 10*p**5*q1**2*q2**2*q3*q4**2*q5**2 + 10*p**5*q1**2*q2**2*q3*q4**2*q5*q6 - 90*p**5*q1**2*q2**2*q3*q4**2*q5 + 10*p**5*q1**2*q2**2*q3*q4*q5**3 + 10*p**5*q1**2*q2**2*q3*q4*q5**2*q6 - 90*p**5*q1**2*q2**2*q3*q4*q5**2 + 10*p**5*q1**2*q2**2*q3*q4*q5*q6**2 - 90*p**5*q1**2*q2**2*q3*q4*q5*q6 + 360*p**5*q1**2*q2**2*q3*q4*q5 + 10*p**5*q1**2*q2*q3**4*q4*q5 + 10*p**5*q1**2*q2*q3**3*q4**2*q5 + 10*p**5*q1**2*q2*q3**3*q4*q5**2 + 10*p**5*q1**2*q2*q3**3*q4*q5*q6 - 90*p**5*q1**2*q2*q3**3*q4*q5 + 10*p**5*q1**2*q2*q3**2*q4**3*q5 + 10*p**5*q1**2*q2*q3**2*q4**2*q5**2 + 10*p**5*q1**2*q2*q3**2*q4**2*q5*q6 - 90*p**5*q1**2*q2*q3**2*q4**2*q5 + 10*p**5*q1**2*q2*q3**2*q4*q5**3 + 10*p**5*q1**2*q2*q3**2*q4*q5**2*q6 - 90*p**5*q1**2*q2*q3**2*q4*q5**2 + 10*p**5*q1**2*q2*q3**2*q4*q5*q6**2 - 90*p**5*q1**2*q2*q3**2*q4*q5*q6 + 360*p**5*q1**2*q2*q3**2*q4*q5 + 10*p**5*q1**2*q2*q3*q4**4*q5 + 10*p**5*q1**2*q2*q3*q4**3*q5**2 + 10*p**5*q1**2*q2*q3*q4**3*q5*q6 - 90*p**5*q1**2*q2*q3*q4**3*q5 + 10*p**5*q1**2*q2*q3*q4**2*q5**3 + 10*p**5*q1**2*q2*q3*q4**2*q5**2*q6 - 90*p**5*q1**2*q2*q3*q4**2*q5**2 + 10*p**5*q1**2*q2*q3*q4**2*q5*q6**2 - 90*p**5*q1**2*q2*q3*q4**2*q5*q6 + 360*p**5*q1**2*q2*q3*q4**2*q5 + 10*p**5*q1**2*q2*q3*q4*q5**4 + 10*p**5*q1**2*q2*q3*q4*q5**3*q6 - 90*p**5*q1**2*q2*q3*q4*q5**3 + 10*p**5*q1**2*q2*q3*q4*q5**2*q6**2 - 90*p**5*q1**2*q2*q3*q4*q5**2*q6 + 360*p**5*q1**2*q2*q3*q4*q5**2 + 10*p**5*q1**2*q2*q3*q4*q5*q6**3 - 90*p**5*q1**2*q2*q3*q4*q5*q6**2 + 360*p**5*q1**2*q2*q3*q4*q5*q6 - 840*p**5*q1**2*q2*q3*q4*q5 + 10*p**5*q1*q2**5*q3*q4*q5 + 10*p**5*q1*q2**4*q3**2*q4*q5 + 10*p**5*q1*q2**4*q3*q4**2*q5 + 10*p**5*q1*q2**4*q3*q4*q5**2 + 10*p**5*q1*q2**4*q3*q4*q5*q6 - 90*p**5*q1*q2**4*q3*q4*q5 + 10*p**5*q1*q2**3*q3**3*q4*q5 + 10*p**5*q1*q2**3*q3**2*q4**2*q5 + 10*p**5*q1*q2**3*q3**2*q4*q5**2 + 10*p**5*q1*q2**3*q3**2*q4*q5*q6 - 90*p**5*q1*q2**3*q3**2*q4*q5 + 10*p**5*q1*q2**3*q3*q4**3*q5 + 10*p**5*q1*q2**3*q3*q4**2*q5**2 + 10*p**5*q1*q2**3*q3*q4**2*q5*q6 - 90*p**5*q1*q2**3*q3*q4**2*q5 + 10*p**5*q1*q2**3*q3*q4*q5**3 + 10*p**5*q1*q2**3*q3*q4*q5**2*q6 - 90*p**5*q1*q2**3*q3*q4*q5**2 + 10*p**5*q1*q2**3*q3*q4*q5*q6**2 - 90*p**5*q1*q2**3*q3*q4*q5*q6 + 360*p**5*q1*q2**3*q3*q4*q5 + 10*p**5*q1*q2**2*q3**4*q4*q5 + 10*p**5*q1*q2**2*q3**3*q4**2*q5 + 10*p**5*q1*q2**2*q3**3*q4*q5**2 + 10*p**5*q1*q2**2*q3**3*q4*q5*q6 - 90*p**5*q1*q2**2*q3**3*q4*q5 + 10*p**5*q1*q2**2*q3**2*q4**3*q5 + 10*p**5*q1*q2**2*q3**2*q4**2*q5**2 + 10*p**5*q1*q2**2*q3**2*q4**2*q5*q6 - 90*p**5*q1*q2**2*q3**2*q4**2*q5 + 10*p**5*q1*q2**2*q3**2*q4*q5**3 + 10*p**5*q1*q2**2*q3**2*q4*q5**2*q6 - 90*p**5*q1*q2**2*q3**2*q4*q5**2 + 10*p**5*q1*q2**2*q3**2*q4*q5*q6**2 - 90*p**5*q1*q2**2*q3**2*q4*q5*q6 + 360*p**5*q1*q2**2*q3**2*q4*q5 + 10*p**5*q1*q2**2*q3*q4**4*q5 + 10*p**5*q1*q2**2*q3*q4**3*q5**2 + 10*p**5*q1*q2**2*q3*q4**3*q5*q6 - 90*p**5*q1*q2**2*q3*q4**3*q5 + 10*p**5*q1*q2**2*q3*q4**2*q5**3 + 10*p**5*q1*q2**2*q3*q4**2*q5**2*q6 - 90*p**5*q1*q2**2*q3*q4**2*q5**2 + 10*p**5*q1*q2**2*q3*q4**2*q5*q6**2 - 90*p**5*q1*q2**2*q3*q4**2*q5*q6 + 360*p**5*q1*q2**2*q3*q4**2*q5 + 10*p**5*q1*q2**2*q3*q4*q5**4 + 10*p**5*q1*q2**2*q3*q4*q5**3*q6 - 90*p**5*q1*q2**2*q3*q4*q5**3 + 10*p**5*q1*q2**2*q3*q4*q5**2*q6**2 - 90*p**5*q1*q2**2*q3*q4*q5**2*q6 + 360*p**5*q1*q2**2*q3*q4*q5**2 + 10*p**5*q1*q2**2*q3*q4*q5*q6**3 - 90*p**5*q1*q2**2*q3*q4*q5*q6**2 + 360*p**5*q1*q2**2*q3*q4*q5*q6 - 840*p**5*q1*q2**2*q3*q4*q5 + 10*p**5*q1*q2*q3**5*q4*q5 + 10*p**5*q1*q2*q3**4*q4**2*q5 + 10*p**5*q1*q2*q3**4*q4*q5**2 + 10*p**5*q1*q2*q3**4*q4*q5*q6 - 90*p**5*q1*q2*q3**4*q4*q5 + 10*p**5*q1*q2*q3**3*q4**3*q5 + 10*p**5*q1*q2*q3**3*q4**2*q5**2 + 10*p**5*q1*q2*q3**3*q4**2*q5*q6 - 90*p**5*q1*q2*q3**3*q4**2*q5 + 10*p**5*q1*q2*q3**3*q4*q5**3 + 10*p**5*q1*q2*q3**3*q4*q5**2*q6 - 90*p**5*q1*q2*q3**3*q4*q5**2 + 10*p**5*q1*q2*q3**3*q4*q5*q6**2 - 90*p**5*q1*q2*q3**3*q4*q5*q6 + 360*p**5*q1*q2*q3**3*q4*q5 + 10*p**5*q1*q2*q3**2*q4**4*q5 + 10*p**5*q1*q2*q3**2*q4**3*q5**2 + 10*p**5*q1*q2*q3**2*q4**3*q5*q6 - 90*p**5*q1*q2*q3**2*q4**3*q5 + 10*p**5*q1*q2*q3**2*q4**2*q5**3 + 10*p**5*q1*q2*q3**2*q4**2*q5**2*q6 - 90*p**5*q1*q2*q3**2*q4**2*q5**2 + 10*p**5*q1*q2*q3**2*q4**2*q5*q6**2 - 90*p**5*q1*q2*q3**2*q4**2*q5*q6 + 360*p**5*q1*q2*q3**2*q4**2*q5 + 10*p**5*q1*q2*q3**2*q4*q5**4 + 10*p**5*q1*q2*q3**2*q4*q5**3*q6 - 90*p**5*q1*q2*q3**2*q4*q5**3 + 10*p**5*q1*q2*q3**2*q4*q5**2*q6**2 - 90*p**5*q1*q2*q3**2*q4*q5**2*q6 + 360*p**5*q1*q2*q3**2*q4*q5**2 + 10*p**5*q1*q2*q3**2*q4*q5*q6**3 - 90*p**5*q1*q2*q3**2*q4*q5*q6**2 + 360*p**5*q1*q2*q3**2*q4*q5*q6 - 840*p**5*q1*q2*q3**2*q4*q5 + 10*p**5*q1*q2*q3*q4**5*q5 + 10*p**5*q1*q2*q3*q4**4*q5**2 + 10*p**5*q1*q2*q3*q4**4*q5*q6 - 90*p**5*q1*q2*q3*q4**4*q5 + 10*p**5*q1*q2*q3*q4**3*q5**3 + 10*p**5*q1*q2*q3*q4**3*q5**2*q6 - 90*p**5*q1*q2*q3*q4**3*q5**2 + 10*p**5*q1*q2*q3*q4**3*q5*q6**2 - 90*p**5*q1*q2*q3*q4**3*q5*q6 + 360*p**5*q1*q2*q3*q4**3*q5 + 10*p**5*q1*q2*q3*q4**2*q5**4 + 10*p**5*q1*q2*q3*q4**2*q5**3*q6 - 90*p**5*q1*q2*q3*q4**2*q5**3 + 10*p**5*q1*q2*q3*q4**2*q5**2*q6**2 - 90*p**5*q1*q2*q3*q4**2*q5**2*q6 + 360*p**5*q1*q2*q3*q4**2*q5**2 + 10*p**5*q1*q2*q3*q4**2*q5*q6**3 - 90*p**5*q1*q2*q3*q4**2*q5*q6**2 + 360*p**5*q1*q2*q3*q4**2*q5*q6 - 840*p**5*q1*q2*q3*q4**2*q5 + 10*p**5*q1*q2*q3*q4*q5**5 + 10*p**5*q1*q2*q3*q4*q5**4*q6 - 90*p**5*q1*q2*q3*q4*q5**4 + 10*p**5*q1*q2*q3*q4*q5**3*q6**2 - 90*p**5*q1*q2*q3*q4*q5**3*q6 + 360*p**5*q1*q2*q3*q4*q5**3 + 10*p**5*q1*q2*q3*q4*q5**2*q6**3 - 90*p**5*q1*q2*q3*q4*q5**2*q6**2 + 360*p**5*q1*q2*q3*q4*q5**2*q6 - 840*p**5*q1*q2*q3*q4*q5**2 + 10*p**5*q1*q2*q3*q4*q5*q6**4 - 90*p**5*q1*q2*q3*q4*q5*q6**3 + 360*p**5*q1*q2*q3*q4*q5*q6**2 - 840*p**5*q1*q2*q3*q4*q5*q6 + 1260*p**5*q1*q2*q3*q4*q5 - 45*p**5*q2**5*q3*q4*q5 - 45*p**5*q2**4*q3**2*q4*q5 - 45*p**5*q2**4*q3*q4**2*q5 - 45*p**5*q2**4*q3*q4*q5**2 - 45*p**5*q2**4*q3*q4*q5*q6 + 360*p**5*q2**4*q3*q4*q5 - 45*p**5*q2**3*q3**3*q4*q5 - 45*p**5*q2**3*q3**2*q4**2*q5 - 45*p**5*q2**3*q3**2*q4*q5**2 - 45*p**5*q2**3*q3**2*q4*q5*q6 + 360*p**5*q2**3*q3**2*q4*q5 - 45*p**5*q2**3*q3*q4**3*q5 - 45*p**5*q2**3*q3*q4**2*q5**2 - 45*p**5*q2**3*q3*q4**2*q5*q6 + 360*p**5*q2**3*q3*q4**2*q5 - 45*p**5*q2**3*q3*q4*q5**3 - 45*p**5*q2**3*q3*q4*q5**2*q6 + 360*p**5*q2**3*q3*q4*q5**2 - 45*p**5*q2**3*q3*q4*q5*q6**2 + 360*p**5*q2**3*q3*q4*q5*q6 - 1260*p**5*q2**3*q3*q4*q5 - 45*p**5*q2**2*q3**4*q4*q5 - 45*p**5*q2**2*q3**3*q4**2*q5 - 45*p**5*q2**2*q3**3*q4*q5**2 - 45*p**5*q2**2*q3**3*q4*q5*q6 + 360*p**5*q2**2*q3**3*q4*q5 - 45*p**5*q2**2*q3**2*q4**3*q5 - 45*p**5*q2**2*q3**2*q4**2*q5**2 - 45*p**5*q2**2*q3**2*q4**2*q5*q6 + 360*p**5*q2**2*q3**2*q4**2*q5 - 45*p**5*q2**2*q3**2*q4*q5**3 - 45*p**5*q2**2*q3**2*q4*q5**2*q6 + 360*p**5*q2**2*q3**2*q4*q5**2 - 45*p**5*q2**2*q3**2*q4*q5*q6**2 + 360*p**5*q2**2*q3**2*q4*q5*q6 - 1260*p**5*q2**2*q3**2*q4*q5 - 45*p**5*q2**2*q3*q4**4*q5 - 45*p**5*q2**2*q3*q4**3*q5**2 - 45*p**5*q2**2*q3*q4**3*q5*q6 + 360*p**5*q2**2*q3*q4**3*q5 - 45*p**5*q2**2*q3*q4**2*q5**3 - 45*p**5*q2**2*q3*q4**2*q5**2*q6 + 360*p**5*q2**2*q3*q4**2*q5**2 - 45*p**5*q2**2*q3*q4**2*q5*q6**2 + 360*p**5*q2**2*q3*q4**2*q5*q6 - 1260*p**5*q2**2*q3*q4**2*q5 - 45*p**5*q2**2*q3*q4*q5**4 - 45*p**5*q2**2*q3*q4*q5**3*q6 + 360*p**5*q2**2*q3*q4*q5**3 - 45*p**5*q2**2*q3*q4*q5**2*q6**2 + 360*p**5*q2**2*q3*q4*q5**2*q6 - 1260*p**5*q2**2*q3*q4*q5**2 - 45*p**5*q2**2*q3*q4*q5*q6**3 + 360*p**5*q2**2*q3*q4*q5*q6**2 - 1260*p**5*q2**2*q3*q4*q5*q6 + 2520*p**5*q2**2*q3*q4*q5 - 45*p**5*q2*q3**5*q4*q5 - 45*p**5*q2*q3**4*q4**2*q5 - 45*p**5*q2*q3**4*q4*q5**2 - 45*p**5*q2*q3**4*q4*q5*q6 + 360*p**5*q2*q3**4*q4*q5 - 45*p**5*q2*q3**3*q4**3*q5 - 45*p**5*q2*q3**3*q4**2*q5**2 - 45*p**5*q2*q3**3*q4**2*q5*q6 + 360*p**5*q2*q3**3*q4**2*q5 - 45*p**5*q2*q3**3*q4*q5**3 - 45*p**5*q2*q3**3*q4*q5**2*q6 + 360*p**5*q2*q3**3*q4*q5**2 - 45*p**5*q2*q3**3*q4*q5*q6**2 + 360*p**5*q2*q3**3*q4*q5*q6 - 1260*p**5*q2*q3**3*q4*q5 - 45*p**5*q2*q3**2*q4**4*q5 - 45*p**5*q2*q3**2*q4**3*q5**2 - 45*p**5*q2*q3**2*q4**3*q5*q6 + 360*p**5*q2*q3**2*q4**3*q5 - 45*p**5*q2*q3**2*q4**2*q5**3 - 45*p**5*q2*q3**2*q4**2*q5**2*q6 + 360*p**5*q2*q3**2*q4**2*q5**2 - 45*p**5*q2*q3**2*q4**2*q5*q6**2 + 360*p**5*q2*q3**2*q4**2*q5*q6 - 1260*p**5*q2*q3**2*q4**2*q5 - 45*p**5*q2*q3**2*q4*q5**4 - 45*p**5*q2*q3**2*q4*q5**3*q6 + 360*p**5*q2*q3**2*q4*q5**3 - 45*p**5*q2*q3**2*q4*q5**2*q6**2 + 360*p**5*q2*q3**2*q4*q5**2*q6 - 1260*p**5*q2*q3**2*q4*q5**2 - 45*p**5*q2*q3**2*q4*q5*q6**3 + 360*p**5*q2*q3**2*q4*q5*q6**2 - 1260*p**5*q2*q3**2*q4*q5*q6 + 2520*p**5*q2*q3**2*q4*q5 - 45*p**5*q2*q3*q4**5*q5 - 45*p**5*q2*q3*q4**4*q5**2 - 45*p**5*q2*q3*q4**4*q5*q6 + 360*p**5*q2*q3*q4**4*q5 - 45*p**5*q2*q3*q4**3*q5**3 - 45*p**5*q2*q3*q4**3*q5**2*q6 + 360*p**5*q2*q3*q4**3*q5**2 - 45*p**5*q2*q3*q4**3*q5*q6**2 + 360*p**5*q2*q3*q4**3*q5*q6 - 1260*p**5*q2*q3*q4**3*q5 - 45*p**5*q2*q3*q4**2*q5**4 - 45*p**5*q2*q3*q4**2*q5**3*q6 + 360*p**5*q2*q3*q4**2*q5**3 - 45*p**5*q2*q3*q4**2*q5**2*q6**2 + 360*p**5*q2*q3*q4**2*q5**2*q6 - 1260*p**5*q2*q3*q4**2*q5**2 - 45*p**5*q2*q3*q4**2*q5*q6**3 + 360*p**5*q2*q3*q4**2*q5*q6**2 - 1260*p**5*q2*q3*q4**2*q5*q6 + 2520*p**5*q2*q3*q4**2*q5 - 45*p**5*q2*q3*q4*q5**5 - 45*p**5*q2*q3*q4*q5**4*q6 + 360*p**5*q2*q3*q4*q5**4 - 45*p**5*q2*q3*q4*q5**3*q6**2 + 360*p**5*q2*q3*q4*q5**3*q6 - 1260*p**5*q2*q3*q4*q5**3 - 45*p**5*q2*q3*q4*q5**2*q6**3 + 360*p**5*q2*q3*q4*q5**2*q6**2 - 1260*p**5*q2*q3*q4*q5**2*q6 + 2520*p**5*q2*q3*q4*q5**2 - 45*p**5*q2*q3*q4*q5*q6**4 + 360*p**5*q2*q3*q4*q5*q6**3 - 1260*p**5*q2*q3*q4*q5*q6**2 + 2520*p**5*q2*q3*q4*q5*q6 - 3150*p**5*q2*q3*q4*q5 + 120*p**5*q3**5*q4*q5 + 120*p**5*q3**4*q4**2*q5 + 120*p**5*q3**4*q4*q5**2 + 120*p**5*q3**4*q4*q5*q6 - 840*p**5*q3**4*q4*q5 + 120*p**5*q3**3*q4**3*q5 + 120*p**5*q3**3*q4**2*q5**2 + 120*p**5*q3**3*q4**2*q5*q6 - 840*p**5*q3**3*q4**2*q5 + 120*p**5*q3**3*q4*q5**3 + 120*p**5*q3**3*q4*q5**2*q6 - 840*p**5*q3**3*q4*q5**2 + 120*p**5*q3**3*q4*q5*q6**2 - 840*p**5*q3**3*q4*q5*q6 + 2520*p**5*q3**3*q4*q5 + 120*p**5*q3**2*q4**4*q5 + 120*p**5*q3**2*q4**3*q5**2 + 120*p**5*q3**2*q4**3*q5*q6 - 840*p**5*q3**2*q4**3*q5 + 120*p**5*q3**2*q4**2*q5**3 + 120*p**5*q3**2*q4**2*q5**2*q6 - 840*p**5*q3**2*q4**2*q5**2 + 120*p**5*q3**2*q4**2*q5*q6**2 - 840*p**5*q3**2*q4**2*q5*q6 + 2520*p**5*q3**2*q4**2*q5 + 120*p**5*q3**2*q4*q5**4 + 120*p**5*q3**2*q4*q5**3*q6 - 840*p**5*q3**2*q4*q5**3 + 120*p**5*q3**2*q4*q5**2*q6**2 - 840*p**5*q3**2*q4*q5**2*q6 + 2520*p**5*q3**2*q4*q5**2 + 120*p**5*q3**2*q4*q5*q6**3 - 840*p**5*q3**2*q4*q5*q6**2 + 2520*p**5*q3**2*q4*q5*q6 - 4200*p**5*q3**2*q4*q5 + 120*p**5*q3*q4**5*q5 + 120*p**5*q3*q4**4*q5**2 + 120*p**5*q3*q4**4*q5*q6 - 840*p**5*q3*q4**4*q5 + 120*p**5*q3*q4**3*q5**3 + 120*p**5*q3*q4**3*q5**2*q6 - 840*p**5*q3*q4**3*q5**2 + 120*p**5*q3*q4**3*q5*q6**2 - 840*p**5*q3*q4**3*q5*q6 + 2520*p**5*q3*q4**3*q5 + 120*p**5*q3*q4**2*q5**4 + 120*p**5*q3*q4**2*q5**3*q6 - 840*p**5*q3*q4**2*q5**3 + 120*p**5*q3*q4**2*q5**2*q6**2 - 840*p**5*q3*q4**2*q5**2*q6 + 2520*p**5*q3*q4**2*q5**2 + 120*p**5*q3*q4**2*q5*q6**3 - 840*p**5*q3*q4**2*q5*q6**2 + 2520*p**5*q3*q4**2*q5*q6 - 4200*p**5*q3*q4**2*q5 + 120*p**5*q3*q4*q5**5 + 120*p**5*q3*q4*q5**4*q6 - 840*p**5*q3*q4*q5**4 + 120*p**5*q3*q4*q5**3*q6**2 - 840*p**5*q3*q4*q5**3*q6 + 2520*p**5*q3*q4*q5**3 + 120*p**5*q3*q4*q5**2*q6**3 - 840*p**5*q3*q4*q5**2*q6**2 + 2520*p**5*q3*q4*q5**2*q6 - 4200*p**5*q3*q4*q5**2 + 120*p**5*q3*q4*q5*q6**4 - 840*p**5*q3*q4*q5*q6**3 + 2520*p**5*q3*q4*q5*q6**2 - 4200*p**5*q3*q4*q5*q6 + 4200*p**5*q3*q4*q5 - 210*p**5*q4**5*q5 - 210*p**5*q4**4*q5**2 - 210*p**5*q4**4*q5*q6 + 1260*p**5*q4**4*q5 - 210*p**5*q4**3*q5**3 - 210*p**5*q4**3*q5**2*q6 + 1260*p**5*q4**3*q5**2 - 210*p**5*q4**3*q5*q6**2 + 1260*p**5*q4**3*q5*q6 - 3150*p**5*q4**3*q5 - 210*p**5*q4**2*q5**4 - 210*p**5*q4**2*q5**3*q6 + 1260*p**5*q4**2*q5**3 - 210*p**5*q4**2*q5**2*q6**2 + 1260*p**5*q4**2*q5**2*q6 - 3150*p**5*q4**2*q5**2 - 210*p**5*q4**2*q5*q6**3 + 1260*p**5*q4**2*q5*q6**2 - 3150*p**5*q4**2*q5*q6 + 4200*p**5*q4**2*q5 - 210*p**5*q4*q5**5 - 210*p**5*q4*q5**4*q6 + 1260*p**5*q4*q5**4 - 210*p**5*q4*q5**3*q6**2 + 1260*p**5*q4*q5**3*q6 - 3150*p**5*q4*q5**3 - 210*p**5*q4*q5**2*q6**3 + 1260*p**5*q4*q5**2*q6**2 - 3150*p**5*q4*q5**2*q6 + 4200*p**5*q4*q5**2 - 210*p**5*q4*q5*q6**4 + 1260*p**5*q4*q5*q6**3 - 3150*p**5*q4*q5*q6**2 + 4200*p**5*q4*q5*q6 - 3150*p**5*q4*q5 + 252*p**5*q5**5 + 252*p**5*q5**4*q6 - 1260*p**5*q5**4 + 252*p**5*q5**3*q6**2 - 1260*p**5*q5**3*q6 + 2520*p**5*q5**3 + 252*p**5*q5**2*q6**3 - 1260*p**5*q5**2*q6**2 + 2520*p**5*q5**2*q6 - 2520*p**5*q5**2 + 252*p**5*q5*q6**4 - 1260*p**5*q5*q6**3 + 2520*p**5*q5*q6**2 - 2520*p**5*q5*q6 + 1260*p**5*q5 - 210*p**5*q6**4 + 840*p**5*q6**3 - 1260*p**5*q6**2 + 840*p**5*q6 - 210*p**5 - 50*p**4*q1**5*q2*q3*q4*q5 - 50*p**4*q1**4*q2**2*q3*q4*q5 - 50*p**4*q1**4*q2*q3**2*q4*q5 - 50*p**4*q1**4*q2*q3*q4**2*q5 - 50*p**4*q1**4*q2*q3*q4*q5**2 - 50*p**4*q1**4*q2*q3*q4*q5*q6 + 450*p**4*q1**4*q2*q3*q4*q5 - 50*p**4*q1**3*q2**3*q3*q4*q5 - 50*p**4*q1**3*q2**2*q3**2*q4*q5 - 50*p**4*q1**3*q2**2*q3*q4**2*q5 - 50*p**4*q1**3*q2**2*q3*q4*q5**2 - 50*p**4*q1**3*q2**2*q3*q4*q5*q6 + 450*p**4*q1**3*q2**2*q3*q4*q5 - 50*p**4*q1**3*q2*q3**3*q4*q5 - 50*p**4*q1**3*q2*q3**2*q4**2*q5 - 50*p**4*q1**3*q2*q3**2*q4*q5**2 - 50*p**4*q1**3*q2*q3**2*q4*q5*q6 + 450*p**4*q1**3*q2*q3**2*q4*q5 - 50*p**4*q1**3*q2*q3*q4**3*q5 - 50*p**4*q1**3*q2*q3*q4**2*q5**2 - 50*p**4*q1**3*q2*q3*q4**2*q5*q6 + 450*p**4*q1**3*q2*q3*q4**2*q5 - 50*p**4*q1**3*q2*q3*q4*q5**3 - 50*p**4*q1**3*q2*q3*q4*q5**2*q6 + 450*p**4*q1**3*q2*q3*q4*q5**2 - 50*p**4*q1**3*q2*q3*q4*q5*q6**2 + 450*p**4*q1**3*q2*q3*q4*q5*q6 - 1800*p**4*q1**3*q2*q3*q4*q5 - 50*p**4*q1**2*q2**4*q3*q4*q5 - 50*p**4*q1**2*q2**3*q3**2*q4*q5 - 50*p**4*q1**2*q2**3*q3*q4**2*q5 - 50*p**4*q1**2*q2**3*q3*q4*q5**2 - 50*p**4*q1**2*q2**3*q3*q4*q5*q6 + 450*p**4*q1**2*q2**3*q3*q4*q5 - 50*p**4*q1**2*q2**2*q3**3*q4*q5 - 50*p**4*q1**2*q2**2*q3**2*q4**2*q5 - 50*p**4*q1**2*q2**2*q3**2*q4*q5**2 - 50*p**4*q1**2*q2**2*q3**2*q4*q5*q6 + 450*p**4*q1**2*q2**2*q3**2*q4*q5 - 50*p**4*q1**2*q2**2*q3*q4**3*q5 - 50*p**4*q1**2*q2**2*q3*q4**2*q5**2 - 50*p**4*q1**2*q2**2*q3*q4**2*q5*q6 + 450*p**4*q1**2*q2**2*q3*q4**2*q5 - 50*p**4*q1**2*q2**2*q3*q4*q5**3 - 50*p**4*q1**2*q2**2*q3*q4*q5**2*q6 + 450*p**4*q1**2*q2**2*q3*q4*q5**2 - 50*p**4*q1**2*q2**2*q3*q4*q5*q6**2 + 450*p**4*q1**2*q2**2*q3*q4*q5*q6 - 1800*p**4*q1**2*q2**2*q3*q4*q5 - 50*p**4*q1**2*q2*q3**4*q4*q5 - 50*p**4*q1**2*q2*q3**3*q4**2*q5 - 50*p**4*q1**2*q2*q3**3*q4*q5**2 - 50*p**4*q1**2*q2*q3**3*q4*q5*q6 + 450*p**4*q1**2*q2*q3**3*q4*q5 - 50*p**4*q1**2*q2*q3**2*q4**3*q5 - 50*p**4*q1**2*q2*q3**2*q4**2*q5**2 - 50*p**4*q1**2*q2*q3**2*q4**2*q5*q6 + 450*p**4*q1**2*q2*q3**2*q4**2*q5 - 50*p**4*q1**2*q2*q3**2*q4*q5**3 - 50*p**4*q1**2*q2*q3**2*q4*q5**2*q6 + 450*p**4*q1**2*q2*q3**2*q4*q5**2 - 50*p**4*q1**2*q2*q3**2*q4*q5*q6**2 + 450*p**4*q1**2*q2*q3**2*q4*q5*q6 - 1800*p**4*q1**2*q2*q3**2*q4*q5 - 50*p**4*q1**2*q2*q3*q4**4*q5 - 50*p**4*q1**2*q2*q3*q4**3*q5**2 - 50*p**4*q1**2*q2*q3*q4**3*q5*q6 + 450*p**4*q1**2*q2*q3*q4**3*q5 - 50*p**4*q1**2*q2*q3*q4**2*q5**3 - 50*p**4*q1**2*q2*q3*q4**2*q5**2*q6 + 450*p**4*q1**2*q2*q3*q4**2*q5**2 - 50*p**4*q1**2*q2*q3*q4**2*q5*q6**2 + 450*p**4*q1**2*q2*q3*q4**2*q5*q6 - 1800*p**4*q1**2*q2*q3*q4**2*q5 - 50*p**4*q1**2*q2*q3*q4*q5**4 - 50*p**4*q1**2*q2*q3*q4*q5**3*q6 + 450*p**4*q1**2*q2*q3*q4*q5**3 - 50*p**4*q1**2*q2*q3*q4*q5**2*q6**2 + 450*p**4*q1**2*q2*q3*q4*q5**2*q6 - 1800*p**4*q1**2*q2*q3*q4*q5**2 - 50*p**4*q1**2*q2*q3*q4*q5*q6**3 + 450*p**4*q1**2*q2*q3*q4*q5*q6**2 - 1800*p**4*q1**2*q2*q3*q4*q5*q6 + 4200*p**4*q1**2*q2*q3*q4*q5 - 50*p**4*q1*q2**5*q3*q4*q5 - 50*p**4*q1*q2**4*q3**2*q4*q5 - 50*p**4*q1*q2**4*q3*q4**2*q5 - 50*p**4*q1*q2**4*q3*q4*q5**2 - 50*p**4*q1*q2**4*q3*q4*q5*q6 + 450*p**4*q1*q2**4*q3*q4*q5 - 50*p**4*q1*q2**3*q3**3*q4*q5 - 50*p**4*q1*q2**3*q3**2*q4**2*q5 - 50*p**4*q1*q2**3*q3**2*q4*q5**2 - 50*p**4*q1*q2**3*q3**2*q4*q5*q6 + 450*p**4*q1*q2**3*q3**2*q4*q5 - 50*p**4*q1*q2**3*q3*q4**3*q5 - 50*p**4*q1*q2**3*q3*q4**2*q5**2 - 50*p**4*q1*q2**3*q3*q4**2*q5*q6 + 450*p**4*q1*q2**3*q3*q4**2*q5 - 50*p**4*q1*q2**3*q3*q4*q5**3 - 50*p**4*q1*q2**3*q3*q4*q5**2*q6 + 450*p**4*q1*q2**3*q3*q4*q5**2 - 50*p**4*q1*q2**3*q3*q4*q5*q6**2 + 450*p**4*q1*q2**3*q3*q4*q5*q6 - 1800*p**4*q1*q2**3*q3*q4*q5 - 50*p**4*q1*q2**2*q3**4*q4*q5 - 50*p**4*q1*q2**2*q3**3*q4**2*q5 - 50*p**4*q1*q2**2*q3**3*q4*q5**2 - 50*p**4*q1*q2**2*q3**3*q4*q5*q6 + 450*p**4*q1*q2**2*q3**3*q4*q5 - 50*p**4*q1*q2**2*q3**2*q4**3*q5 - 50*p**4*q1*q2**2*q3**2*q4**2*q5**2 - 50*p**4*q1*q2**2*q3**2*q4**2*q5*q6 + 450*p**4*q1*q2**2*q3**2*q4**2*q5 - 50*p**4*q1*q2**2*q3**2*q4*q5**3 - 50*p**4*q1*q2**2*q3**2*q4*q5**2*q6 + 450*p**4*q1*q2**2*q3**2*q4*q5**2 - 50*p**4*q1*q2**2*q3**2*q4*q5*q6**2 + 450*p**4*q1*q2**2*q3**2*q4*q5*q6 - 1800*p**4*q1*q2**2*q3**2*q4*q5 - 50*p**4*q1*q2**2*q3*q4**4*q5 - 50*p**4*q1*q2**2*q3*q4**3*q5**2 - 50*p**4*q1*q2**2*q3*q4**3*q5*q6 + 450*p**4*q1*q2**2*q3*q4**3*q5 - 50*p**4*q1*q2**2*q3*q4**2*q5**3 - 50*p**4*q1*q2**2*q3*q4**2*q5**2*q6 + 450*p**4*q1*q2**2*q3*q4**2*q5**2 - 50*p**4*q1*q2**2*q3*q4**2*q5*q6**2 + 450*p**4*q1*q2**2*q3*q4**2*q5*q6 - 1800*p**4*q1*q2**2*q3*q4**2*q5 - 50*p**4*q1*q2**2*q3*q4*q5**4 - 50*p**4*q1*q2**2*q3*q4*q5**3*q6 + 450*p**4*q1*q2**2*q3*q4*q5**3 - 50*p**4*q1*q2**2*q3*q4*q5**2*q6**2 + 450*p**4*q1*q2**2*q3*q4*q5**2*q6 - 1800*p**4*q1*q2**2*q3*q4*q5**2 - 50*p**4*q1*q2**2*q3*q4*q5*q6**3 + 450*p**4*q1*q2**2*q3*q4*q5*q6**2 - 1800*p**4*q1*q2**2*q3*q4*q5*q6 + 4200*p**4*q1*q2**2*q3*q4*q5 - 50*p**4*q1*q2*q3**5*q4*q5 - 50*p**4*q1*q2*q3**4*q4**2*q5 - 50*p**4*q1*q2*q3**4*q4*q5**2 - 50*p**4*q1*q2*q3**4*q4*q5*q6 + 450*p**4*q1*q2*q3**4*q4*q5 - 50*p**4*q1*q2*q3**3*q4**3*q5 - 50*p**4*q1*q2*q3**3*q4**2*q5**2 - 50*p**4*q1*q2*q3**3*q4**2*q5*q6 + 450*p**4*q1*q2*q3**3*q4**2*q5 - 50*p**4*q1*q2*q3**3*q4*q5**3 - 50*p**4*q1*q2*q3**3*q4*q5**2*q6 + 450*p**4*q1*q2*q3**3*q4*q5**2 - 50*p**4*q1*q2*q3**3*q4*q5*q6**2 + 450*p**4*q1*q2*q3**3*q4*q5*q6 - 1800*p**4*q1*q2*q3**3*q4*q5 - 50*p**4*q1*q2*q3**2*q4**4*q5 - 50*p**4*q1*q2*q3**2*q4**3*q5**2 - 50*p**4*q1*q2*q3**2*q4**3*q5*q6 + 450*p**4*q1*q2*q3**2*q4**3*q5 - 50*p**4*q1*q2*q3**2*q4**2*q5**3 - 50*p**4*q1*q2*q3**2*q4**2*q5**2*q6 + 450*p**4*q1*q2*q3**2*q4**2*q5**2 - 50*p**4*q1*q2*q3**2*q4**2*q5*q6**2 + 450*p**4*q1*q2*q3**2*q4**2*q5*q6 - 1800*p**4*q1*q2*q3**2*q4**2*q5 - 50*p**4*q1*q2*q3**2*q4*q5**4 - 50*p**4*q1*q2*q3**2*q4*q5**3*q6 + 450*p**4*q1*q2*q3**2*q4*q5**3 - 50*p**4*q1*q2*q3**2*q4*q5**2*q6**2 + 450*p**4*q1*q2*q3**2*q4*q5**2*q6 - 1800*p**4*q1*q2*q3**2*q4*q5**2 - 50*p**4*q1*q2*q3**2*q4*q5*q6**3 + 450*p**4*q1*q2*q3**2*q4*q5*q6**2 - 1800*p**4*q1*q2*q3**2*q4*q5*q6 + 4200*p**4*q1*q2*q3**2*q4*q5 - 50*p**4*q1*q2*q3*q4**5*q5 - 50*p**4*q1*q2*q3*q4**4*q5**2 - 50*p**4*q1*q2*q3*q4**4*q5*q6 + 450*p**4*q1*q2*q3*q4**4*q5 - 50*p**4*q1*q2*q3*q4**3*q5**3 - 50*p**4*q1*q2*q3*q4**3*q5**2*q6 + 450*p**4*q1*q2*q3*q4**3*q5**2 - 50*p**4*q1*q2*q3*q4**3*q5*q6**2 + 450*p**4*q1*q2*q3*q4**3*q5*q6 - 1800*p**4*q1*q2*q3*q4**3*q5 - 50*p**4*q1*q2*q3*q4**2*q5**4 - 50*p**4*q1*q2*q3*q4**2*q5**3*q6 + 450*p**4*q1*q2*q3*q4**2*q5**3 - 50*p**4*q1*q2*q3*q4**2*q5**2*q6**2 + 450*p**4*q1*q2*q3*q4**2*q5**2*q6 - 1800*p**4*q1*q2*q3*q4**2*q5**2 - 50*p**4*q1*q2*q3*q4**2*q5*q6**3 + 450*p**4*q1*q2*q3*q4**2*q5*q6**2 - 1800*p**4*q1*q2*q3*q4**2*q5*q6 + 4200*p**4*q1*q2*q3*q4**2*q5 - 50*p**4*q1*q2*q3*q4*q5**5 - 50*p**4*q1*q2*q3*q4*q5**4*q6 + 450*p**4*q1*q2*q3*q4*q5**4 - 50*p**4*q1*q2*q3*q4*q5**3*q6**2 + 450*p**4*q1*q2*q3*q4*q5**3*q6 - 1800*p**4*q1*q2*q3*q4*q5**3 - 50*p**4*q1*q2*q3*q4*q5**2*q6**3 + 450*p**4*q1*q2*q3*q4*q5**2*q6**2 - 1800*p**4*q1*q2*q3*q4*q5**2*q6 + 4200*p**4*q1*q2*q3*q4*q5**2 - 50*p**4*q1*q2*q3*q4*q5*q6**4 + 450*p**4*q1*q2*q3*q4*q5*q6**3 - 1800*p**4*q1*q2*q3*q4*q5*q6**2 + 4200*p**4*q1*q2*q3*q4*q5*q6 - 6300*p**4*q1*q2*q3*q4*q5 + 180*p**4*q2**5*q3*q4*q5 + 180*p**4*q2**4*q3**2*q4*q5 + 180*p**4*q2**4*q3*q4**2*q5 + 180*p**4*q2**4*q3*q4*q5**2 + 180*p**4*q2**4*q3*q4*q5*q6 - 1440*p**4*q2**4*q3*q4*q5 + 180*p**4*q2**3*q3**3*q4*q5 + 180*p**4*q2**3*q3**2*q4**2*q5 + 180*p**4*q2**3*q3**2*q4*q5**2 + 180*p**4*q2**3*q3**2*q4*q5*q6 - 1440*p**4*q2**3*q3**2*q4*q5 + 180*p**4*q2**3*q3*q4**3*q5 + 180*p**4*q2**3*q3*q4**2*q5**2 + 180*p**4*q2**3*q3*q4**2*q5*q6 - 1440*p**4*q2**3*q3*q4**2*q5 + 180*p**4*q2**3*q3*q4*q5**3 + 180*p**4*q2**3*q3*q4*q5**2*q6 - 1440*p**4*q2**3*q3*q4*q5**2 + 180*p**4*q2**3*q3*q4*q5*q6**2 - 1440*p**4*q2**3*q3*q4*q5*q6 + 5040*p**4*q2**3*q3*q4*q5 + 180*p**4*q2**2*q3**4*q4*q5 + 180*p**4*q2**2*q3**3*q4**2*q5 + 180*p**4*q2**2*q3**3*q4*q5**2 + 180*p**4*q2**2*q3**3*q4*q5*q6 - 1440*p**4*q2**2*q3**3*q4*q5 + 180*p**4*q2**2*q3**2*q4**3*q5 + 180*p**4*q2**2*q3**2*q4**2*q5**2 + 180*p**4*q2**2*q3**2*q4**2*q5*q6 - 1440*p**4*q2**2*q3**2*q4**2*q5 + 180*p**4*q2**2*q3**2*q4*q5**3 + 180*p**4*q2**2*q3**2*q4*q5**2*q6 - 1440*p**4*q2**2*q3**2*q4*q5**2 + 180*p**4*q2**2*q3**2*q4*q5*q6**2 - 1440*p**4*q2**2*q3**2*q4*q5*q6 + 5040*p**4*q2**2*q3**2*q4*q5 + 180*p**4*q2**2*q3*q4**4*q5 + 180*p**4*q2**2*q3*q4**3*q5**2 + 180*p**4*q2**2*q3*q4**3*q5*q6 - 1440*p**4*q2**2*q3*q4**3*q5 + 180*p**4*q2**2*q3*q4**2*q5**3 + 180*p**4*q2**2*q3*q4**2*q5**2*q6 - 1440*p**4*q2**2*q3*q4**2*q5**2 + 180*p**4*q2**2*q3*q4**2*q5*q6**2 - 1440*p**4*q2**2*q3*q4**2*q5*q6 + 5040*p**4*q2**2*q3*q4**2*q5 + 180*p**4*q2**2*q3*q4*q5**4 + 180*p**4*q2**2*q3*q4*q5**3*q6 - 1440*p**4*q2**2*q3*q4*q5**3 + 180*p**4*q2**2*q3*q4*q5**2*q6**2 - 1440*p**4*q2**2*q3*q4*q5**2*q6 + 5040*p**4*q2**2*q3*q4*q5**2 + 180*p**4*q2**2*q3*q4*q5*q6**3 - 1440*p**4*q2**2*q3*q4*q5*q6**2 + 5040*p**4*q2**2*q3*q4*q5*q6 - 10080*p**4*q2**2*q3*q4*q5 + 180*p**4*q2*q3**5*q4*q5 + 180*p**4*q2*q3**4*q4**2*q5 + 180*p**4*q2*q3**4*q4*q5**2 + 180*p**4*q2*q3**4*q4*q5*q6 - 1440*p**4*q2*q3**4*q4*q5 + 180*p**4*q2*q3**3*q4**3*q5 + 180*p**4*q2*q3**3*q4**2*q5**2 + 180*p**4*q2*q3**3*q4**2*q5*q6 - 1440*p**4*q2*q3**3*q4**2*q5 + 180*p**4*q2*q3**3*q4*q5**3 + 180*p**4*q2*q3**3*q4*q5**2*q6 - 1440*p**4*q2*q3**3*q4*q5**2 + 180*p**4*q2*q3**3*q4*q5*q6**2 - 1440*p**4*q2*q3**3*q4*q5*q6 + 5040*p**4*q2*q3**3*q4*q5 + 180*p**4*q2*q3**2*q4**4*q5 + 180*p**4*q2*q3**2*q4**3*q5**2 + 180*p**4*q2*q3**2*q4**3*q5*q6 - 1440*p**4*q2*q3**2*q4**3*q5 + 180*p**4*q2*q3**2*q4**2*q5**3 + 180*p**4*q2*q3**2*q4**2*q5**2*q6 - 1440*p**4*q2*q3**2*q4**2*q5**2 + 180*p**4*q2*q3**2*q4**2*q5*q6**2 - 1440*p**4*q2*q3**2*q4**2*q5*q6 + 5040*p**4*q2*q3**2*q4**2*q5 + 180*p**4*q2*q3**2*q4*q5**4 + 180*p**4*q2*q3**2*q4*q5**3*q6 - 1440*p**4*q2*q3**2*q4*q5**3 + 180*p**4*q2*q3**2*q4*q5**2*q6**2 - 1440*p**4*q2*q3**2*q4*q5**2*q6 + 5040*p**4*q2*q3**2*q4*q5**2 + 180*p**4*q2*q3**2*q4*q5*q6**3 - 1440*p**4*q2*q3**2*q4*q5*q6**2 + 5040*p**4*q2*q3**2*q4*q5*q6 - 10080*p**4*q2*q3**2*q4*q5 + 180*p**4*q2*q3*q4**5*q5 + 180*p**4*q2*q3*q4**4*q5**2 + 180*p**4*q2*q3*q4**4*q5*q6 - 1440*p**4*q2*q3*q4**4*q5 + 180*p**4*q2*q3*q4**3*q5**3 + 180*p**4*q2*q3*q4**3*q5**2*q6 - 1440*p**4*q2*q3*q4**3*q5**2 + 180*p**4*q2*q3*q4**3*q5*q6**2 - 1440*p**4*q2*q3*q4**3*q5*q6 + 5040*p**4*q2*q3*q4**3*q5 + 180*p**4*q2*q3*q4**2*q5**4 + 180*p**4*q2*q3*q4**2*q5**3*q6 - 1440*p**4*q2*q3*q4**2*q5**3 + 180*p**4*q2*q3*q4**2*q5**2*q6**2 - 1440*p**4*q2*q3*q4**2*q5**2*q6 + 5040*p**4*q2*q3*q4**2*q5**2 + 180*p**4*q2*q3*q4**2*q5*q6**3 - 1440*p**4*q2*q3*q4**2*q5*q6**2 + 5040*p**4*q2*q3*q4**2*q5*q6 - 10080*p**4*q2*q3*q4**2*q5 + 180*p**4*q2*q3*q4*q5**5 + 180*p**4*q2*q3*q4*q5**4*q6 - 1440*p**4*q2*q3*q4*q5**4 + 180*p**4*q2*q3*q4*q5**3*q6**2 - 1440*p**4*q2*q3*q4*q5**3*q6 + 5040*p**4*q2*q3*q4*q5**3 + 180*p**4*q2*q3*q4*q5**2*q6**3 - 1440*p**4*q2*q3*q4*q5**2*q6**2 + 5040*p**4*q2*q3*q4*q5**2*q6 - 10080*p**4*q2*q3*q4*q5**2 + 180*p**4*q2*q3*q4*q5*q6**4 - 1440*p**4*q2*q3*q4*q5*q6**3 + 5040*p**4*q2*q3*q4*q5*q6**2 - 10080*p**4*q2*q3*q4*q5*q6 + 12600*p**4*q2*q3*q4*q5 - 360*p**4*q3**5*q4*q5 - 360*p**4*q3**4*q4**2*q5 - 360*p**4*q3**4*q4*q5**2 - 360*p**4*q3**4*q4*q5*q6 + 2520*p**4*q3**4*q4*q5 - 360*p**4*q3**3*q4**3*q5 - 360*p**4*q3**3*q4**2*q5**2 - 360*p**4*q3**3*q4**2*q5*q6 + 2520*p**4*q3**3*q4**2*q5 - 360*p**4*q3**3*q4*q5**3 - 360*p**4*q3**3*q4*q5**2*q6 + 2520*p**4*q3**3*q4*q5**2 - 360*p**4*q3**3*q4*q5*q6**2 + 2520*p**4*q3**3*q4*q5*q6 - 7560*p**4*q3**3*q4*q5 - 360*p**4*q3**2*q4**4*q5 - 360*p**4*q3**2*q4**3*q5**2 - 360*p**4*q3**2*q4**3*q5*q6 + 2520*p**4*q3**2*q4**3*q5 - 360*p**4*q3**2*q4**2*q5**3 - 360*p**4*q3**2*q4**2*q5**2*q6 + 2520*p**4*q3**2*q4**2*q5**2 - 360*p**4*q3**2*q4**2*q5*q6**2 + 2520*p**4*q3**2*q4**2*q5*q6 - 7560*p**4*q3**2*q4**2*q5 - 360*p**4*q3**2*q4*q5**4 - 360*p**4*q3**2*q4*q5**3*q6 + 2520*p**4*q3**2*q4*q5**3 - 360*p**4*q3**2*q4*q5**2*q6**2 + 2520*p**4*q3**2*q4*q5**2*q6 - 7560*p**4*q3**2*q4*q5**2 - 360*p**4*q3**2*q4*q5*q6**3 + 2520*p**4*q3**2*q4*q5*q6**2 - 7560*p**4*q3**2*q4*q5*q6 + 12600*p**4*q3**2*q4*q5 - 360*p**4*q3*q4**5*q5 - 360*p**4*q3*q4**4*q5**2 - 360*p**4*q3*q4**4*q5*q6 + 2520*p**4*q3*q4**4*q5 - 360*p**4*q3*q4**3*q5**3 - 360*p**4*q3*q4**3*q5**2*q6 + 2520*p**4*q3*q4**3*q5**2 - 360*p**4*q3*q4**3*q5*q6**2 + 2520*p**4*q3*q4**3*q5*q6 - 7560*p**4*q3*q4**3*q5 - 360*p**4*q3*q4**2*q5**4 - 360*p**4*q3*q4**2*q5**3*q6 + 2520*p**4*q3*q4**2*q5**3 - 360*p**4*q3*q4**2*q5**2*q6**2 + 2520*p**4*q3*q4**2*q5**2*q6 - 7560*p**4*q3*q4**2*q5**2 - 360*p**4*q3*q4**2*q5*q6**3 + 2520*p**4*q3*q4**2*q5*q6**2 - 7560*p**4*q3*q4**2*q5*q6 + 12600*p**4*q3*q4**2*q5 - 360*p**4*q3*q4*q5**5 - 360*p**4*q3*q4*q5**4*q6 + 2520*p**4*q3*q4*q5**4 - 360*p**4*q3*q4*q5**3*q6**2 + 2520*p**4*q3*q4*q5**3*q6 - 7560*p**4*q3*q4*q5**3 - 360*p**4*q3*q4*q5**2*q6**3 + 2520*p**4*q3*q4*q5**2*q6**2 - 7560*p**4*q3*q4*q5**2*q6 + 12600*p**4*q3*q4*q5**2 - 360*p**4*q3*q4*q5*q6**4 + 2520*p**4*q3*q4*q5*q6**3 - 7560*p**4*q3*q4*q5*q6**2 + 12600*p**4*q3*q4*q5*q6 - 12600*p**4*q3*q4*q5 + 420*p**4*q4**5*q5 + 420*p**4*q4**4*q5**2 + 420*p**4*q4**4*q5*q6 - 2520*p**4*q4**4*q5 + 420*p**4*q4**3*q5**3 + 420*p**4*q4**3*q5**2*q6 - 2520*p**4*q4**3*q5**2 + 420*p**4*q4**3*q5*q6**2 - 2520*p**4*q4**3*q5*q6 + 6300*p**4*q4**3*q5 + 420*p**4*q4**2*q5**4 + 420*p**4*q4**2*q5**3*q6 - 2520*p**4*q4**2*q5**3 + 420*p**4*q4**2*q5**2*q6**2 - 2520*p**4*q4**2*q5**2*q6 + 6300*p**4*q4**2*q5**2 + 420*p**4*q4**2*q5*q6**3 - 2520*p**4*q4**2*q5*q6**2 + 6300*p**4*q4**2*q5*q6 - 8400*p**4*q4**2*q5 + 420*p**4*q4*q5**5 + 420*p**4*q4*q5**4*q6 - 2520*p**4*q4*q5**4 + 420*p**4*q4*q5**3*q6**2 - 2520*p**4*q4*q5**3*q6 + 6300*p**4*q4*q5**3 + 420*p**4*q4*q5**2*q6**3 - 2520*p**4*q4*q5**2*q6**2 + 6300*p**4*q4*q5**2*q6 - 8400*p**4*q4*q5**2 + 420*p**4*q4*q5*q6**4 - 2520*p**4*q4*q5*q6**3 + 6300*p**4*q4*q5*q6**2 - 8400*p**4*q4*q5*q6 + 6300*p**4*q4*q5 - 252*p**4*q5**5 - 252*p**4*q5**4*q6 + 1260*p**4*q5**4 - 252*p**4*q5**3*q6**2 + 1260*p**4*q5**3*q6 - 2520*p**4*q5**3 - 252*p**4*q5**2*q6**3 + 1260*p**4*q5**2*q6**2 - 2520*p**4*q5**2*q6 + 2520*p**4*q5**2 - 252*p**4*q5*q6**4 + 1260*p**4*q5*q6**3 - 2520*p**4*q5*q6**2 + 2520*p**4*q5*q6 - 1260*p**4*q5 + 100*p**3*q1**5*q2*q3*q4*q5 + 100*p**3*q1**4*q2**2*q3*q4*q5 + 100*p**3*q1**4*q2*q3**2*q4*q5 + 100*p**3*q1**4*q2*q3*q4**2*q5 + 100*p**3*q1**4*q2*q3*q4*q5**2 + 100*p**3*q1**4*q2*q3*q4*q5*q6 - 900*p**3*q1**4*q2*q3*q4*q5 + 100*p**3*q1**3*q2**3*q3*q4*q5 + 100*p**3*q1**3*q2**2*q3**2*q4*q5 + 100*p**3*q1**3*q2**2*q3*q4**2*q5 + 100*p**3*q1**3*q2**2*q3*q4*q5**2 + 100*p**3*q1**3*q2**2*q3*q4*q5*q6 - 900*p**3*q1**3*q2**2*q3*q4*q5 + 100*p**3*q1**3*q2*q3**3*q4*q5 + 100*p**3*q1**3*q2*q3**2*q4**2*q5 + 100*p**3*q1**3*q2*q3**2*q4*q5**2 + 100*p**3*q1**3*q2*q3**2*q4*q5*q6 - 900*p**3*q1**3*q2*q3**2*q4*q5 + 100*p**3*q1**3*q2*q3*q4**3*q5 + 100*p**3*q1**3*q2*q3*q4**2*q5**2 + 100*p**3*q1**3*q2*q3*q4**2*q5*q6 - 900*p**3*q1**3*q2*q3*q4**2*q5 + 100*p**3*q1**3*q2*q3*q4*q5**3 + 100*p**3*q1**3*q2*q3*q4*q5**2*q6 - 900*p**3*q1**3*q2*q3*q4*q5**2 + 100*p**3*q1**3*q2*q3*q4*q5*q6**2 - 900*p**3*q1**3*q2*q3*q4*q5*q6 + 3600*p**3*q1**3*q2*q3*q4*q5 + 100*p**3*q1**2*q2**4*q3*q4*q5 + 100*p**3*q1**2*q2**3*q3**2*q4*q5 + 100*p**3*q1**2*q2**3*q3*q4**2*q5 + 100*p**3*q1**2*q2**3*q3*q4*q5**2 + 100*p**3*q1**2*q2**3*q3*q4*q5*q6 - 900*p**3*q1**2*q2**3*q3*q4*q5 + 100*p**3*q1**2*q2**2*q3**3*q4*q5 + 100*p**3*q1**2*q2**2*q3**2*q4**2*q5 + 100*p**3*q1**2*q2**2*q3**2*q4*q5**2 + 100*p**3*q1**2*q2**2*q3**2*q4*q5*q6 - 900*p**3*q1**2*q2**2*q3**2*q4*q5 + 100*p**3*q1**2*q2**2*q3*q4**3*q5 + 100*p**3*q1**2*q2**2*q3*q4**2*q5**2 + 100*p**3*q1**2*q2**2*q3*q4**2*q5*q6 - 900*p**3*q1**2*q2**2*q3*q4**2*q5 + 100*p**3*q1**2*q2**2*q3*q4*q5**3 + 100*p**3*q1**2*q2**2*q3*q4*q5**2*q6 - 900*p**3*q1**2*q2**2*q3*q4*q5**2 + 100*p**3*q1**2*q2**2*q3*q4*q5*q6**2 - 900*p**3*q1**2*q2**2*q3*q4*q5*q6 + 3600*p**3*q1**2*q2**2*q3*q4*q5 + 100*p**3*q1**2*q2*q3**4*q4*q5 + 100*p**3*q1**2*q2*q3**3*q4**2*q5 + 100*p**3*q1**2*q2*q3**3*q4*q5**2 + 100*p**3*q1**2*q2*q3**3*q4*q5*q6 - 900*p**3*q1**2*q2*q3**3*q4*q5 + 100*p**3*q1**2*q2*q3**2*q4**3*q5 + 100*p**3*q1**2*q2*q3**2*q4**2*q5**2 + 100*p**3*q1**2*q2*q3**2*q4**2*q5*q6 - 900*p**3*q1**2*q2*q3**2*q4**2*q5 + 100*p**3*q1**2*q2*q3**2*q4*q5**3 + 100*p**3*q1**2*q2*q3**2*q4*q5**2*q6 - 900*p**3*q1**2*q2*q3**2*q4*q5**2 + 100*p**3*q1**2*q2*q3**2*q4*q5*q6**2 - 900*p**3*q1**2*q2*q3**2*q4*q5*q6 + 3600*p**3*q1**2*q2*q3**2*q4*q5 + 100*p**3*q1**2*q2*q3*q4**4*q5 + 100*p**3*q1**2*q2*q3*q4**3*q5**2 + 100*p**3*q1**2*q2*q3*q4**3*q5*q6 - 900*p**3*q1**2*q2*q3*q4**3*q5 + 100*p**3*q1**2*q2*q3*q4**2*q5**3 + 100*p**3*q1**2*q2*q3*q4**2*q5**2*q6 - 900*p**3*q1**2*q2*q3*q4**2*q5**2 + 100*p**3*q1**2*q2*q3*q4**2*q5*q6**2 - 900*p**3*q1**2*q2*q3*q4**2*q5*q6 + 3600*p**3*q1**2*q2*q3*q4**2*q5 + 100*p**3*q1**2*q2*q3*q4*q5**4 + 100*p**3*q1**2*q2*q3*q4*q5**3*q6 - 900*p**3*q1**2*q2*q3*q4*q5**3 + 100*p**3*q1**2*q2*q3*q4*q5**2*q6**2 - 900*p**3*q1**2*q2*q3*q4*q5**2*q6 + 3600*p**3*q1**2*q2*q3*q4*q5**2 + 100*p**3*q1**2*q2*q3*q4*q5*q6**3 - 900*p**3*q1**2*q2*q3*q4*q5*q6**2 + 3600*p**3*q1**2*q2*q3*q4*q5*q6 - 8400*p**3*q1**2*q2*q3*q4*q5 + 100*p**3*q1*q2**5*q3*q4*q5 + 100*p**3*q1*q2**4*q3**2*q4*q5 + 100*p**3*q1*q2**4*q3*q4**2*q5 + 100*p**3*q1*q2**4*q3*q4*q5**2 + 100*p**3*q1*q2**4*q3*q4*q5*q6 - 900*p**3*q1*q2**4*q3*q4*q5 + 100*p**3*q1*q2**3*q3**3*q4*q5 + 100*p**3*q1*q2**3*q3**2*q4**2*q5 + 100*p**3*q1*q2**3*q3**2*q4*q5**2 + 100*p**3*q1*q2**3*q3**2*q4*q5*q6 - 900*p**3*q1*q2**3*q3**2*q4*q5 + 100*p**3*q1*q2**3*q3*q4**3*q5 + 100*p**3*q1*q2**3*q3*q4**2*q5**2 + 100*p**3*q1*q2**3*q3*q4**2*q5*q6 - 900*p**3*q1*q2**3*q3*q4**2*q5 + 100*p**3*q1*q2**3*q3*q4*q5**3 + 100*p**3*q1*q2**3*q3*q4*q5**2*q6 - 900*p**3*q1*q2**3*q3*q4*q5**2 + 100*p**3*q1*q2**3*q3*q4*q5*q6**2 - 900*p**3*q1*q2**3*q3*q4*q5*q6 + 3600*p**3*q1*q2**3*q3*q4*q5 + 100*p**3*q1*q2**2*q3**4*q4*q5 + 100*p**3*q1*q2**2*q3**3*q4**2*q5 + 100*p**3*q1*q2**2*q3**3*q4*q5**2 + 100*p**3*q1*q2**2*q3**3*q4*q5*q6 - 900*p**3*q1*q2**2*q3**3*q4*q5 + 100*p**3*q1*q2**2*q3**2*q4**3*q5 + 100*p**3*q1*q2**2*q3**2*q4**2*q5**2 + 100*p**3*q1*q2**2*q3**2*q4**2*q5*q6 - 900*p**3*q1*q2**2*q3**2*q4**2*q5 + 100*p**3*q1*q2**2*q3**2*q4*q5**3 + 100*p**3*q1*q2**2*q3**2*q4*q5**2*q6 - 900*p**3*q1*q2**2*q3**2*q4*q5**2 + 100*p**3*q1*q2**2*q3**2*q4*q5*q6**2 - 900*p**3*q1*q2**2*q3**2*q4*q5*q6 + 3600*p**3*q1*q2**2*q3**2*q4*q5 + 100*p**3*q1*q2**2*q3*q4**4*q5 + 100*p**3*q1*q2**2*q3*q4**3*q5**2 + 100*p**3*q1*q2**2*q3*q4**3*q5*q6 - 900*p**3*q1*q2**2*q3*q4**3*q5 + 100*p**3*q1*q2**2*q3*q4**2*q5**3 + 100*p**3*q1*q2**2*q3*q4**2*q5**2*q6 - 900*p**3*q1*q2**2*q3*q4**2*q5**2 + 100*p**3*q1*q2**2*q3*q4**2*q5*q6**2 - 900*p**3*q1*q2**2*q3*q4**2*q5*q6 + 3600*p**3*q1*q2**2*q3*q4**2*q5 + 100*p**3*q1*q2**2*q3*q4*q5**4 + 100*p**3*q1*q2**2*q3*q4*q5**3*q6 - 900*p**3*q1*q2**2*q3*q4*q5**3 + 100*p**3*q1*q2**2*q3*q4*q5**2*q6**2 - 900*p**3*q1*q2**2*q3*q4*q5**2*q6 + 3600*p**3*q1*q2**2*q3*q4*q5**2 + 100*p**3*q1*q2**2*q3*q4*q5*q6**3 - 900*p**3*q1*q2**2*q3*q4*q5*q6**2 + 3600*p**3*q1*q2**2*q3*q4*q5*q6 - 8400*p**3*q1*q2**2*q3*q4*q5 + 100*p**3*q1*q2*q3**5*q4*q5 + 100*p**3*q1*q2*q3**4*q4**2*q5 + 100*p**3*q1*q2*q3**4*q4*q5**2 + 100*p**3*q1*q2*q3**4*q4*q5*q6 - 900*p**3*q1*q2*q3**4*q4*q5 + 100*p**3*q1*q2*q3**3*q4**3*q5 + 100*p**3*q1*q2*q3**3*q4**2*q5**2 + 100*p**3*q1*q2*q3**3*q4**2*q5*q6 - 900*p**3*q1*q2*q3**3*q4**2*q5 + 100*p**3*q1*q2*q3**3*q4*q5**3 + 100*p**3*q1*q2*q3**3*q4*q5**2*q6 - 900*p**3*q1*q2*q3**3*q4*q5**2 + 100*p**3*q1*q2*q3**3*q4*q5*q6**2 - 900*p**3*q1*q2*q3**3*q4*q5*q6 + 3600*p**3*q1*q2*q3**3*q4*q5 + 100*p**3*q1*q2*q3**2*q4**4*q5 + 100*p**3*q1*q2*q3**2*q4**3*q5**2 + 100*p**3*q1*q2*q3**2*q4**3*q5*q6 - 900*p**3*q1*q2*q3**2*q4**3*q5 + 100*p**3*q1*q2*q3**2*q4**2*q5**3 + 100*p**3*q1*q2*q3**2*q4**2*q5**2*q6 - 900*p**3*q1*q2*q3**2*q4**2*q5**2 + 100*p**3*q1*q2*q3**2*q4**2*q5*q6**2 - 900*p**3*q1*q2*q3**2*q4**2*q5*q6 + 3600*p**3*q1*q2*q3**2*q4**2*q5 + 100*p**3*q1*q2*q3**2*q4*q5**4 + 100*p**3*q1*q2*q3**2*q4*q5**3*q6 - 900*p**3*q1*q2*q3**2*q4*q5**3 + 100*p**3*q1*q2*q3**2*q4*q5**2*q6**2 - 900*p**3*q1*q2*q3**2*q4*q5**2*q6 + 3600*p**3*q1*q2*q3**2*q4*q5**2 + 100*p**3*q1*q2*q3**2*q4*q5*q6**3 - 900*p**3*q1*q2*q3**2*q4*q5*q6**2 + 3600*p**3*q1*q2*q3**2*q4*q5*q6 - 8400*p**3*q1*q2*q3**2*q4*q5 + 100*p**3*q1*q2*q3*q4**5*q5 + 100*p**3*q1*q2*q3*q4**4*q5**2 + 100*p**3*q1*q2*q3*q4**4*q5*q6 - 900*p**3*q1*q2*q3*q4**4*q5 + 100*p**3*q1*q2*q3*q4**3*q5**3 + 100*p**3*q1*q2*q3*q4**3*q5**2*q6 - 900*p**3*q1*q2*q3*q4**3*q5**2 + 100*p**3*q1*q2*q3*q4**3*q5*q6**2 - 900*p**3*q1*q2*q3*q4**3*q5*q6 + 3600*p**3*q1*q2*q3*q4**3*q5 + 100*p**3*q1*q2*q3*q4**2*q5**4 + 100*p**3*q1*q2*q3*q4**2*q5**3*q6 - 900*p**3*q1*q2*q3*q4**2*q5**3 + 100*p**3*q1*q2*q3*q4**2*q5**2*q6**2 - 900*p**3*q1*q2*q3*q4**2*q5**2*q6 + 3600*p**3*q1*q2*q3*q4**2*q5**2 + 100*p**3*q1*q2*q3*q4**2*q5*q6**3 - 900*p**3*q1*q2*q3*q4**2*q5*q6**2 + 3600*p**3*q1*q2*q3*q4**2*q5*q6 - 8400*p**3*q1*q2*q3*q4**2*q5 + 100*p**3*q1*q2*q3*q4*q5**5 + 100*p**3*q1*q2*q3*q4*q5**4*q6 - 900*p**3*q1*q2*q3*q4*q5**4 + 100*p**3*q1*q2*q3*q4*q5**3*q6**2 - 900*p**3*q1*q2*q3*q4*q5**3*q6 + 3600*p**3*q1*q2*q3*q4*q5**3 + 100*p**3*q1*q2*q3*q4*q5**2*q6**3 - 900*p**3*q1*q2*q3*q4*q5**2*q6**2 + 3600*p**3*q1*q2*q3*q4*q5**2*q6 - 8400*p**3*q1*q2*q3*q4*q5**2 + 100*p**3*q1*q2*q3*q4*q5*q6**4 - 900*p**3*q1*q2*q3*q4*q5*q6**3 + 3600*p**3*q1*q2*q3*q4*q5*q6**2 - 8400*p**3*q1*q2*q3*q4*q5*q6 + 12600*p**3*q1*q2*q3*q4*q5 - 270*p**3*q2**5*q3*q4*q5 - 270*p**3*q2**4*q3**2*q4*q5 - 270*p**3*q2**4*q3*q4**2*q5 - 270*p**3*q2**4*q3*q4*q5**2 - 270*p**3*q2**4*q3*q4*q5*q6 + 2160*p**3*q2**4*q3*q4*q5 - 270*p**3*q2**3*q3**3*q4*q5 - 270*p**3*q2**3*q3**2*q4**2*q5 - 270*p**3*q2**3*q3**2*q4*q5**2 - 270*p**3*q2**3*q3**2*q4*q5*q6 + 2160*p**3*q2**3*q3**2*q4*q5 - 270*p**3*q2**3*q3*q4**3*q5 - 270*p**3*q2**3*q3*q4**2*q5**2 - 270*p**3*q2**3*q3*q4**2*q5*q6 + 2160*p**3*q2**3*q3*q4**2*q5 - 270*p**3*q2**3*q3*q4*q5**3 - 270*p**3*q2**3*q3*q4*q5**2*q6 + 2160*p**3*q2**3*q3*q4*q5**2 - 270*p**3*q2**3*q3*q4*q5*q6**2 + 2160*p**3*q2**3*q3*q4*q5*q6 - 7560*p**3*q2**3*q3*q4*q5 - 270*p**3*q2**2*q3**4*q4*q5 - 270*p**3*q2**2*q3**3*q4**2*q5 - 270*p**3*q2**2*q3**3*q4*q5**2 - 270*p**3*q2**2*q3**3*q4*q5*q6 + 2160*p**3*q2**2*q3**3*q4*q5 - 270*p**3*q2**2*q3**2*q4**3*q5 - 270*p**3*q2**2*q3**2*q4**2*q5**2 - 270*p**3*q2**2*q3**2*q4**2*q5*q6 + 2160*p**3*q2**2*q3**2*q4**2*q5 - 270*p**3*q2**2*q3**2*q4*q5**3 - 270*p**3*q2**2*q3**2*q4*q5**2*q6 + 2160*p**3*q2**2*q3**2*q4*q5**2 - 270*p**3*q2**2*q3**2*q4*q5*q6**2 + 2160*p**3*q2**2*q3**2*q4*q5*q6 - 7560*p**3*q2**2*q3**2*q4*q5 - 270*p**3*q2**2*q3*q4**4*q5 - 270*p**3*q2**2*q3*q4**3*q5**2 - 270*p**3*q2**2*q3*q4**3*q5*q6 + 2160*p**3*q2**2*q3*q4**3*q5 - 270*p**3*q2**2*q3*q4**2*q5**3 - 270*p**3*q2**2*q3*q4**2*q5**2*q6 + 2160*p**3*q2**2*q3*q4**2*q5**2 - 270*p**3*q2**2*q3*q4**2*q5*q6**2 + 2160*p**3*q2**2*q3*q4**2*q5*q6 - 7560*p**3*q2**2*q3*q4**2*q5 - 270*p**3*q2**2*q3*q4*q5**4 - 270*p**3*q2**2*q3*q4*q5**3*q6 + 2160*p**3*q2**2*q3*q4*q5**3 - 270*p**3*q2**2*q3*q4*q5**2*q6**2 + 2160*p**3*q2**2*q3*q4*q5**2*q6 - 7560*p**3*q2**2*q3*q4*q5**2 - 270*p**3*q2**2*q3*q4*q5*q6**3 + 2160*p**3*q2**2*q3*q4*q5*q6**2 - 7560*p**3*q2**2*q3*q4*q5*q6 + 15120*p**3*q2**2*q3*q4*q5 - 270*p**3*q2*q3**5*q4*q5 - 270*p**3*q2*q3**4*q4**2*q5 - 270*p**3*q2*q3**4*q4*q5**2 - 270*p**3*q2*q3**4*q4*q5*q6 + 2160*p**3*q2*q3**4*q4*q5 - 270*p**3*q2*q3**3*q4**3*q5 - 270*p**3*q2*q3**3*q4**2*q5**2 - 270*p**3*q2*q3**3*q4**2*q5*q6 + 2160*p**3*q2*q3**3*q4**2*q5 - 270*p**3*q2*q3**3*q4*q5**3 - 270*p**3*q2*q3**3*q4*q5**2*q6 + 2160*p**3*q2*q3**3*q4*q5**2 - 270*p**3*q2*q3**3*q4*q5*q6**2 + 2160*p**3*q2*q3**3*q4*q5*q6 - 7560*p**3*q2*q3**3*q4*q5 - 270*p**3*q2*q3**2*q4**4*q5 - 270*p**3*q2*q3**2*q4**3*q5**2 - 270*p**3*q2*q3**2*q4**3*q5*q6 + 2160*p**3*q2*q3**2*q4**3*q5 - 270*p**3*q2*q3**2*q4**2*q5**3 - 270*p**3*q2*q3**2*q4**2*q5**2*q6 + 2160*p**3*q2*q3**2*q4**2*q5**2 - 270*p**3*q2*q3**2*q4**2*q5*q6**2 + 2160*p**3*q2*q3**2*q4**2*q5*q6 - 7560*p**3*q2*q3**2*q4**2*q5 - 270*p**3*q2*q3**2*q4*q5**4 - 270*p**3*q2*q3**2*q4*q5**3*q6 + 2160*p**3*q2*q3**2*q4*q5**3 - 270*p**3*q2*q3**2*q4*q5**2*q6**2 + 2160*p**3*q2*q3**2*q4*q5**2*q6 - 7560*p**3*q2*q3**2*q4*q5**2 - 270*p**3*q2*q3**2*q4*q5*q6**3 + 2160*p**3*q2*q3**2*q4*q5*q6**2 - 7560*p**3*q2*q3**2*q4*q5*q6 + 15120*p**3*q2*q3**2*q4*q5 - 270*p**3*q2*q3*q4**5*q5 - 270*p**3*q2*q3*q4**4*q5**2 - 270*p**3*q2*q3*q4**4*q5*q6 + 2160*p**3*q2*q3*q4**4*q5 - 270*p**3*q2*q3*q4**3*q5**3 - 270*p**3*q2*q3*q4**3*q5**2*q6 + 2160*p**3*q2*q3*q4**3*q5**2 - 270*p**3*q2*q3*q4**3*q5*q6**2 + 2160*p**3*q2*q3*q4**3*q5*q6 - 7560*p**3*q2*q3*q4**3*q5 - 270*p**3*q2*q3*q4**2*q5**4 - 270*p**3*q2*q3*q4**2*q5**3*q6 + 2160*p**3*q2*q3*q4**2*q5**3 - 270*p**3*q2*q3*q4**2*q5**2*q6**2 + 2160*p**3*q2*q3*q4**2*q5**2*q6 - 7560*p**3*q2*q3*q4**2*q5**2 - 270*p**3*q2*q3*q4**2*q5*q6**3 + 2160*p**3*q2*q3*q4**2*q5*q6**2 - 7560*p**3*q2*q3*q4**2*q5*q6 + 15120*p**3*q2*q3*q4**2*q5 - 270*p**3*q2*q3*q4*q5**5 - 270*p**3*q2*q3*q4*q5**4*q6 + 2160*p**3*q2*q3*q4*q5**4 - 270*p**3*q2*q3*q4*q5**3*q6**2 + 2160*p**3*q2*q3*q4*q5**3*q6 - 7560*p**3*q2*q3*q4*q5**3 - 270*p**3*q2*q3*q4*q5**2*q6**3 + 2160*p**3*q2*q3*q4*q5**2*q6**2 - 7560*p**3*q2*q3*q4*q5**2*q6 + 15120*p**3*q2*q3*q4*q5**2 - 270*p**3*q2*q3*q4*q5*q6**4 + 2160*p**3*q2*q3*q4*q5*q6**3 - 7560*p**3*q2*q3*q4*q5*q6**2 + 15120*p**3*q2*q3*q4*q5*q6 - 18900*p**3*q2*q3*q4*q5 + 360*p**3*q3**5*q4*q5 + 360*p**3*q3**4*q4**2*q5 + 360*p**3*q3**4*q4*q5**2 + 360*p**3*q3**4*q4*q5*q6 - 2520*p**3*q3**4*q4*q5 + 360*p**3*q3**3*q4**3*q5 + 360*p**3*q3**3*q4**2*q5**2 + 360*p**3*q3**3*q4**2*q5*q6 - 2520*p**3*q3**3*q4**2*q5 + 360*p**3*q3**3*q4*q5**3 + 360*p**3*q3**3*q4*q5**2*q6 - 2520*p**3*q3**3*q4*q5**2 + 360*p**3*q3**3*q4*q5*q6**2 - 2520*p**3*q3**3*q4*q5*q6 + 7560*p**3*q3**3*q4*q5 + 360*p**3*q3**2*q4**4*q5 + 360*p**3*q3**2*q4**3*q5**2 + 360*p**3*q3**2*q4**3*q5*q6 - 2520*p**3*q3**2*q4**3*q5 + 360*p**3*q3**2*q4**2*q5**3 + 360*p**3*q3**2*q4**2*q5**2*q6 - 2520*p**3*q3**2*q4**2*q5**2 + 360*p**3*q3**2*q4**2*q5*q6**2 - 2520*p**3*q3**2*q4**2*q5*q6 + 7560*p**3*q3**2*q4**2*q5 + 360*p**3*q3**2*q4*q5**4 + 360*p**3*q3**2*q4*q5**3*q6 - 2520*p**3*q3**2*q4*q5**3 + 360*p**3*q3**2*q4*q5**2*q6**2 - 2520*p**3*q3**2*q4*q5**2*q6 + 7560*p**3*q3**2*q4*q5**2 + 360*p**3*q3**2*q4*q5*q6**3 - 2520*p**3*q3**2*q4*q5*q6**2 + 7560*p**3*q3**2*q4*q5*q6 - 12600*p**3*q3**2*q4*q5 + 360*p**3*q3*q4**5*q5 + 360*p**3*q3*q4**4*q5**2 + 360*p**3*q3*q4**4*q5*q6 - 2520*p**3*q3*q4**4*q5 + 360*p**3*q3*q4**3*q5**3 + 360*p**3*q3*q4**3*q5**2*q6 - 2520*p**3*q3*q4**3*q5**2 + 360*p**3*q3*q4**3*q5*q6**2 - 2520*p**3*q3*q4**3*q5*q6 + 7560*p**3*q3*q4**3*q5 + 360*p**3*q3*q4**2*q5**4 + 360*p**3*q3*q4**2*q5**3*q6 - 2520*p**3*q3*q4**2*q5**3 + 360*p**3*q3*q4**2*q5**2*q6**2 - 2520*p**3*q3*q4**2*q5**2*q6 + 7560*p**3*q3*q4**2*q5**2 + 360*p**3*q3*q4**2*q5*q6**3 - 2520*p**3*q3*q4**2*q5*q6**2 + 7560*p**3*q3*q4**2*q5*q6 - 12600*p**3*q3*q4**2*q5 + 360*p**3*q3*q4*q5**5 + 360*p**3*q3*q4*q5**4*q6 - 2520*p**3*q3*q4*q5**4 + 360*p**3*q3*q4*q5**3*q6**2 - 2520*p**3*q3*q4*q5**3*q6 + 7560*p**3*q3*q4*q5**3 + 360*p**3*q3*q4*q5**2*q6**3 - 2520*p**3*q3*q4*q5**2*q6**2 + 7560*p**3*q3*q4*q5**2*q6 - 12600*p**3*q3*q4*q5**2 + 360*p**3*q3*q4*q5*q6**4 - 2520*p**3*q3*q4*q5*q6**3 + 7560*p**3*q3*q4*q5*q6**2 - 12600*p**3*q3*q4*q5*q6 + 12600*p**3*q3*q4*q5 - 210*p**3*q4**5*q5 - 210*p**3*q4**4*q5**2 - 210*p**3*q4**4*q5*q6 + 1260*p**3*q4**4*q5 - 210*p**3*q4**3*q5**3 - 210*p**3*q4**3*q5**2*q6 + 1260*p**3*q4**3*q5**2 - 210*p**3*q4**3*q5*q6**2 + 1260*p**3*q4**3*q5*q6 - 3150*p**3*q4**3*q5 - 210*p**3*q4**2*q5**4 - 210*p**3*q4**2*q5**3*q6 + 1260*p**3*q4**2*q5**3 - 210*p**3*q4**2*q5**2*q6**2 + 1260*p**3*q4**2*q5**2*q6 - 3150*p**3*q4**2*q5**2 - 210*p**3*q4**2*q5*q6**3 + 1260*p**3*q4**2*q5*q6**2 - 3150*p**3*q4**2*q5*q6 + 4200*p**3*q4**2*q5 - 210*p**3*q4*q5**5 - 210*p**3*q4*q5**4*q6 + 1260*p**3*q4*q5**4 - 210*p**3*q4*q5**3*q6**2 + 1260*p**3*q4*q5**3*q6 - 3150*p**3*q4*q5**3 - 210*p**3*q4*q5**2*q6**3 + 1260*p**3*q4*q5**2*q6**2 - 3150*p**3*q4*q5**2*q6 + 4200*p**3*q4*q5**2 - 210*p**3*q4*q5*q6**4 + 1260*p**3*q4*q5*q6**3 - 3150*p**3*q4*q5*q6**2 + 4200*p**3*q4*q5*q6 - 3150*p**3*q4*q5 - 100*p**2*q1**5*q2*q3*q4*q5 - 100*p**2*q1**4*q2**2*q3*q4*q5 - 100*p**2*q1**4*q2*q3**2*q4*q5 - 100*p**2*q1**4*q2*q3*q4**2*q5 - 100*p**2*q1**4*q2*q3*q4*q5**2 - 100*p**2*q1**4*q2*q3*q4*q5*q6 + 900*p**2*q1**4*q2*q3*q4*q5 - 100*p**2*q1**3*q2**3*q3*q4*q5 - 100*p**2*q1**3*q2**2*q3**2*q4*q5 - 100*p**2*q1**3*q2**2*q3*q4**2*q5 - 100*p**2*q1**3*q2**2*q3*q4*q5**2 - 100*p**2*q1**3*q2**2*q3*q4*q5*q6 + 900*p**2*q1**3*q2**2*q3*q4*q5 - 100*p**2*q1**3*q2*q3**3*q4*q5 - 100*p**2*q1**3*q2*q3**2*q4**2*q5 - 100*p**2*q1**3*q2*q3**2*q4*q5**2 - 100*p**2*q1**3*q2*q3**2*q4*q5*q6 + 900*p**2*q1**3*q2*q3**2*q4*q5 - 100*p**2*q1**3*q2*q3*q4**3*q5 - 100*p**2*q1**3*q2*q3*q4**2*q5**2 - 100*p**2*q1**3*q2*q3*q4**2*q5*q6 + 900*p**2*q1**3*q2*q3*q4**2*q5 - 100*p**2*q1**3*q2*q3*q4*q5**3 - 100*p**2*q1**3*q2*q3*q4*q5**2*q6 + 900*p**2*q1**3*q2*q3*q4*q5**2 - 100*p**2*q1**3*q2*q3*q4*q5*q6**2 + 900*p**2*q1**3*q2*q3*q4*q5*q6 - 3600*p**2*q1**3*q2*q3*q4*q5 - 100*p**2*q1**2*q2**4*q3*q4*q5 - 100*p**2*q1**2*q2**3*q3**2*q4*q5 - 100*p**2*q1**2*q2**3*q3*q4**2*q5 - 100*p**2*q1**2*q2**3*q3*q4*q5**2 - 100*p**2*q1**2*q2**3*q3*q4*q5*q6 + 900*p**2*q1**2*q2**3*q3*q4*q5 - 100*p**2*q1**2*q2**2*q3**3*q4*q5 - 100*p**2*q1**2*q2**2*q3**2*q4**2*q5 - 100*p**2*q1**2*q2**2*q3**2*q4*q5**2 - 100*p**2*q1**2*q2**2*q3**2*q4*q5*q6 + 900*p**2*q1**2*q2**2*q3**2*q4*q5 - 100*p**2*q1**2*q2**2*q3*q4**3*q5 - 100*p**2*q1**2*q2**2*q3*q4**2*q5**2 - 100*p**2*q1**2*q2**2*q3*q4**2*q5*q6 + 900*p**2*q1**2*q2**2*q3*q4**2*q5 - 100*p**2*q1**2*q2**2*q3*q4*q5**3 - 100*p**2*q1**2*q2**2*q3*q4*q5**2*q6 + 900*p**2*q1**2*q2**2*q3*q4*q5**2 - 100*p**2*q1**2*q2**2*q3*q4*q5*q6**2 + 900*p**2*q1**2*q2**2*q3*q4*q5*q6 - 3600*p**2*q1**2*q2**2*q3*q4*q5 - 100*p**2*q1**2*q2*q3**4*q4*q5 - 100*p**2*q1**2*q2*q3**3*q4**2*q5 - 100*p**2*q1**2*q2*q3**3*q4*q5**2 - 100*p**2*q1**2*q2*q3**3*q4*q5*q6 + 900*p**2*q1**2*q2*q3**3*q4*q5 - 100*p**2*q1**2*q2*q3**2*q4**3*q5 - 100*p**2*q1**2*q2*q3**2*q4**2*q5**2 - 100*p**2*q1**2*q2*q3**2*q4**2*q5*q6 + 900*p**2*q1**2*q2*q3**2*q4**2*q5 - 100*p**2*q1**2*q2*q3**2*q4*q5**3 - 100*p**2*q1**2*q2*q3**2*q4*q5**2*q6 + 900*p**2*q1**2*q2*q3**2*q4*q5**2 - 100*p**2*q1**2*q2*q3**2*q4*q5*q6**2 + 900*p**2*q1**2*q2*q3**2*q4*q5*q6 - 3600*p**2*q1**2*q2*q3**2*q4*q5 - 100*p**2*q1**2*q2*q3*q4**4*q5 - 100*p**2*q1**2*q2*q3*q4**3*q5**2 - 100*p**2*q1**2*q2*q3*q4**3*q5*q6 + 900*p**2*q1**2*q2*q3*q4**3*q5 - 100*p**2*q1**2*q2*q3*q4**2*q5**3 - 100*p**2*q1**2*q2*q3*q4**2*q5**2*q6 + 900*p**2*q1**2*q2*q3*q4**2*q5**2 - 100*p**2*q1**2*q2*q3*q4**2*q5*q6**2 + 900*p**2*q1**2*q2*q3*q4**2*q5*q6 - 3600*p**2*q1**2*q2*q3*q4**2*q5 - 100*p**2*q1**2*q2*q3*q4*q5**4 - 100*p**2*q1**2*q2*q3*q4*q5**3*q6 + 900*p**2*q1**2*q2*q3*q4*q5**3 - 100*p**2*q1**2*q2*q3*q4*q5**2*q6**2 + 900*p**2*q1**2*q2*q3*q4*q5**2*q6 - 3600*p**2*q1**2*q2*q3*q4*q5**2 - 100*p**2*q1**2*q2*q3*q4*q5*q6**3 + 900*p**2*q1**2*q2*q3*q4*q5*q6**2 - 3600*p**2*q1**2*q2*q3*q4*q5*q6 + 8400*p**2*q1**2*q2*q3*q4*q5 - 100*p**2*q1*q2**5*q3*q4*q5 - 100*p**2*q1*q2**4*q3**2*q4*q5 - 100*p**2*q1*q2**4*q3*q4**2*q5 - 100*p**2*q1*q2**4*q3*q4*q5**2 - 100*p**2*q1*q2**4*q3*q4*q5*q6 + 900*p**2*q1*q2**4*q3*q4*q5 - 100*p**2*q1*q2**3*q3**3*q4*q5 - 100*p**2*q1*q2**3*q3**2*q4**2*q5 - 100*p**2*q1*q2**3*q3**2*q4*q5**2 - 100*p**2*q1*q2**3*q3**2*q4*q5*q6 + 900*p**2*q1*q2**3*q3**2*q4*q5 - 100*p**2*q1*q2**3*q3*q4**3*q5 - 100*p**2*q1*q2**3*q3*q4**2*q5**2 - 100*p**2*q1*q2**3*q3*q4**2*q5*q6 + 900*p**2*q1*q2**3*q3*q4**2*q5 - 100*p**2*q1*q2**3*q3*q4*q5**3 - 100*p**2*q1*q2**3*q3*q4*q5**2*q6 + 900*p**2*q1*q2**3*q3*q4*q5**2 - 100*p**2*q1*q2**3*q3*q4*q5*q6**2 + 900*p**2*q1*q2**3*q3*q4*q5*q6 - 3600*p**2*q1*q2**3*q3*q4*q5 - 100*p**2*q1*q2**2*q3**4*q4*q5 - 100*p**2*q1*q2**2*q3**3*q4**2*q5 - 100*p**2*q1*q2**2*q3**3*q4*q5**2 - 100*p**2*q1*q2**2*q3**3*q4*q5*q6 + 900*p**2*q1*q2**2*q3**3*q4*q5 - 100*p**2*q1*q2**2*q3**2*q4**3*q5 - 100*p**2*q1*q2**2*q3**2*q4**2*q5**2 - 100*p**2*q1*q2**2*q3**2*q4**2*q5*q6 + 900*p**2*q1*q2**2*q3**2*q4**2*q5 - 100*p**2*q1*q2**2*q3**2*q4*q5**3 - 100*p**2*q1*q2**2*q3**2*q4*q5**2*q6 + 900*p**2*q1*q2**2*q3**2*q4*q5**2 - 100*p**2*q1*q2**2*q3**2*q4*q5*q6**2 + 900*p**2*q1*q2**2*q3**2*q4*q5*q6 - 3600*p**2*q1*q2**2*q3**2*q4*q5 - 100*p**2*q1*q2**2*q3*q4**4*q5 - 100*p**2*q1*q2**2*q3*q4**3*q5**2 - 100*p**2*q1*q2**2*q3*q4**3*q5*q6 + 900*p**2*q1*q2**2*q3*q4**3*q5 - 100*p**2*q1*q2**2*q3*q4**2*q5**3 - 100*p**2*q1*q2**2*q3*q4**2*q5**2*q6 + 900*p**2*q1*q2**2*q3*q4**2*q5**2 - 100*p**2*q1*q2**2*q3*q4**2*q5*q6**2 + 900*p**2*q1*q2**2*q3*q4**2*q5*q6 - 3600*p**2*q1*q2**2*q3*q4**2*q5 - 100*p**2*q1*q2**2*q3*q4*q5**4 - 100*p**2*q1*q2**2*q3*q4*q5**3*q6 + 900*p**2*q1*q2**2*q3*q4*q5**3 - 100*p**2*q1*q2**2*q3*q4*q5**2*q6**2 + 900*p**2*q1*q2**2*q3*q4*q5**2*q6 - 3600*p**2*q1*q2**2*q3*q4*q5**2 - 100*p**2*q1*q2**2*q3*q4*q5*q6**3 + 900*p**2*q1*q2**2*q3*q4*q5*q6**2 - 3600*p**2*q1*q2**2*q3*q4*q5*q6 + 8400*p**2*q1*q2**2*q3*q4*q5 - 100*p**2*q1*q2*q3**5*q4*q5 - 100*p**2*q1*q2*q3**4*q4**2*q5 - 100*p**2*q1*q2*q3**4*q4*q5**2 - 100*p**2*q1*q2*q3**4*q4*q5*q6 + 900*p**2*q1*q2*q3**4*q4*q5 - 100*p**2*q1*q2*q3**3*q4**3*q5 - 100*p**2*q1*q2*q3**3*q4**2*q5**2 - 100*p**2*q1*q2*q3**3*q4**2*q5*q6 + 900*p**2*q1*q2*q3**3*q4**2*q5 - 100*p**2*q1*q2*q3**3*q4*q5**3 - 100*p**2*q1*q2*q3**3*q4*q5**2*q6 + 900*p**2*q1*q2*q3**3*q4*q5**2 - 100*p**2*q1*q2*q3**3*q4*q5*q6**2 + 900*p**2*q1*q2*q3**3*q4*q5*q6 - 3600*p**2*q1*q2*q3**3*q4*q5 - 100*p**2*q1*q2*q3**2*q4**4*q5 - 100*p**2*q1*q2*q3**2*q4**3*q5**2 - 100*p**2*q1*q2*q3**2*q4**3*q5*q6 + 900*p**2*q1*q2*q3**2*q4**3*q5 - 100*p**2*q1*q2*q3**2*q4**2*q5**3 - 100*p**2*q1*q2*q3**2*q4**2*q5**2*q6 + 900*p**2*q1*q2*q3**2*q4**2*q5**2 - 100*p**2*q1*q2*q3**2*q4**2*q5*q6**2 + 900*p**2*q1*q2*q3**2*q4**2*q5*q6 - 3600*p**2*q1*q2*q3**2*q4**2*q5 - 100*p**2*q1*q2*q3**2*q4*q5**4 - 100*p**2*q1*q2*q3**2*q4*q5**3*q6 + 900*p**2*q1*q2*q3**2*q4*q5**3 - 100*p**2*q1*q2*q3**2*q4*q5**2*q6**2 + 900*p**2*q1*q2*q3**2*q4*q5**2*q6 - 3600*p**2*q1*q2*q3**2*q4*q5**2 - 100*p**2*q1*q2*q3**2*q4*q5*q6**3 + 900*p**2*q1*q2*q3**2*q4*q5*q6**2 - 3600*p**2*q1*q2*q3**2*q4*q5*q6 + 8400*p**2*q1*q2*q3**2*q4*q5 - 100*p**2*q1*q2*q3*q4**5*q5 - 100*p**2*q1*q2*q3*q4**4*q5**2 - 100*p**2*q1*q2*q3*q4**4*q5*q6 + 900*p**2*q1*q2*q3*q4**4*q5 - 100*p**2*q1*q2*q3*q4**3*q5**3 - 100*p**2*q1*q2*q3*q4**3*q5**2*q6 + 900*p**2*q1*q2*q3*q4**3*q5**2 - 100*p**2*q1*q2*q3*q4**3*q5*q6**2 + 900*p**2*q1*q2*q3*q4**3*q5*q6 - 3600*p**2*q1*q2*q3*q4**3*q5 - 100*p**2*q1*q2*q3*q4**2*q5**4 - 100*p**2*q1*q2*q3*q4**2*q5**3*q6 + 900*p**2*q1*q2*q3*q4**2*q5**3 - 100*p**2*q1*q2*q3*q4**2*q5**2*q6**2 + 900*p**2*q1*q2*q3*q4**2*q5**2*q6 - 3600*p**2*q1*q2*q3*q4**2*q5**2 - 100*p**2*q1*q2*q3*q4**2*q5*q6**3 + 900*p**2*q1*q2*q3*q4**2*q5*q6**2 - 3600*p**2*q1*q2*q3*q4**2*q5*q6 + 8400*p**2*q1*q2*q3*q4**2*q5 - 100*p**2*q1*q2*q3*q4*q5**5 - 100*p**2*q1*q2*q3*q4*q5**4*q6 + 900*p**2*q1*q2*q3*q4*q5**4 - 100*p**2*q1*q2*q3*q4*q5**3*q6**2 + 900*p**2*q1*q2*q3*q4*q5**3*q6 - 3600*p**2*q1*q2*q3*q4*q5**3 - 100*p**2*q1*q2*q3*q4*q5**2*q6**3 + 900*p**2*q1*q2*q3*q4*q5**2*q6**2 - 3600*p**2*q1*q2*q3*q4*q5**2*q6 + 8400*p**2*q1*q2*q3*q4*q5**2 - 100*p**2*q1*q2*q3*q4*q5*q6**4 + 900*p**2*q1*q2*q3*q4*q5*q6**3 - 3600*p**2*q1*q2*q3*q4*q5*q6**2 + 8400*p**2*q1*q2*q3*q4*q5*q6 - 12600*p**2*q1*q2*q3*q4*q5 + 180*p**2*q2**5*q3*q4*q5 + 180*p**2*q2**4*q3**2*q4*q5 + 180*p**2*q2**4*q3*q4**2*q5 + 180*p**2*q2**4*q3*q4*q5**2 + 180*p**2*q2**4*q3*q4*q5*q6 - 1440*p**2*q2**4*q3*q4*q5 + 180*p**2*q2**3*q3**3*q4*q5 + 180*p**2*q2**3*q3**2*q4**2*q5 + 180*p**2*q2**3*q3**2*q4*q5**2 + 180*p**2*q2**3*q3**2*q4*q5*q6 - 1440*p**2*q2**3*q3**2*q4*q5 + 180*p**2*q2**3*q3*q4**3*q5 + 180*p**2*q2**3*q3*q4**2*q5**2 + 180*p**2*q2**3*q3*q4**2*q5*q6 - 1440*p**2*q2**3*q3*q4**2*q5 + 180*p**2*q2**3*q3*q4*q5**3 + 180*p**2*q2**3*q3*q4*q5**2*q6 - 1440*p**2*q2**3*q3*q4*q5**2 + 180*p**2*q2**3*q3*q4*q5*q6**2 - 1440*p**2*q2**3*q3*q4*q5*q6 + 5040*p**2*q2**3*q3*q4*q5 + 180*p**2*q2**2*q3**4*q4*q5 + 180*p**2*q2**2*q3**3*q4**2*q5 + 180*p**2*q2**2*q3**3*q4*q5**2 + 180*p**2*q2**2*q3**3*q4*q5*q6 - 1440*p**2*q2**2*q3**3*q4*q5 + 180*p**2*q2**2*q3**2*q4**3*q5 + 180*p**2*q2**2*q3**2*q4**2*q5**2 + 180*p**2*q2**2*q3**2*q4**2*q5*q6 - 1440*p**2*q2**2*q3**2*q4**2*q5 + 180*p**2*q2**2*q3**2*q4*q5**3 + 180*p**2*q2**2*q3**2*q4*q5**2*q6 - 1440*p**2*q2**2*q3**2*q4*q5**2 + 180*p**2*q2**2*q3**2*q4*q5*q6**2 - 1440*p**2*q2**2*q3**2*q4*q5*q6 + 5040*p**2*q2**2*q3**2*q4*q5 + 180*p**2*q2**2*q3*q4**4*q5 + 180*p**2*q2**2*q3*q4**3*q5**2 + 180*p**2*q2**2*q3*q4**3*q5*q6 - 1440*p**2*q2**2*q3*q4**3*q5 + 180*p**2*q2**2*q3*q4**2*q5**3 + 180*p**2*q2**2*q3*q4**2*q5**2*q6 - 1440*p**2*q2**2*q3*q4**2*q5**2 + 180*p**2*q2**2*q3*q4**2*q5*q6**2 - 1440*p**2*q2**2*q3*q4**2*q5*q6 + 5040*p**2*q2**2*q3*q4**2*q5 + 180*p**2*q2**2*q3*q4*q5**4 + 180*p**2*q2**2*q3*q4*q5**3*q6 - 1440*p**2*q2**2*q3*q4*q5**3 + 180*p**2*q2**2*q3*q4*q5**2*q6**2 - 1440*p**2*q2**2*q3*q4*q5**2*q6 + 5040*p**2*q2**2*q3*q4*q5**2 + 180*p**2*q2**2*q3*q4*q5*q6**3 - 1440*p**2*q2**2*q3*q4*q5*q6**2 + 5040*p**2*q2**2*q3*q4*q5*q6 - 10080*p**2*q2**2*q3*q4*q5 + 180*p**2*q2*q3**5*q4*q5 + 180*p**2*q2*q3**4*q4**2*q5 + 180*p**2*q2*q3**4*q4*q5**2 + 180*p**2*q2*q3**4*q4*q5*q6 - 1440*p**2*q2*q3**4*q4*q5 + 180*p**2*q2*q3**3*q4**3*q5 + 180*p**2*q2*q3**3*q4**2*q5**2 + 180*p**2*q2*q3**3*q4**2*q5*q6 - 1440*p**2*q2*q3**3*q4**2*q5 + 180*p**2*q2*q3**3*q4*q5**3 + 180*p**2*q2*q3**3*q4*q5**2*q6 - 1440*p**2*q2*q3**3*q4*q5**2 + 180*p**2*q2*q3**3*q4*q5*q6**2 - 1440*p**2*q2*q3**3*q4*q5*q6 + 5040*p**2*q2*q3**3*q4*q5 + 180*p**2*q2*q3**2*q4**4*q5 + 180*p**2*q2*q3**2*q4**3*q5**2 + 180*p**2*q2*q3**2*q4**3*q5*q6 - 1440*p**2*q2*q3**2*q4**3*q5 + 180*p**2*q2*q3**2*q4**2*q5**3 + 180*p**2*q2*q3**2*q4**2*q5**2*q6 - 1440*p**2*q2*q3**2*q4**2*q5**2 + 180*p**2*q2*q3**2*q4**2*q5*q6**2 - 1440*p**2*q2*q3**2*q4**2*q5*q6 + 5040*p**2*q2*q3**2*q4**2*q5 + 180*p**2*q2*q3**2*q4*q5**4 + 180*p**2*q2*q3**2*q4*q5**3*q6 - 1440*p**2*q2*q3**2*q4*q5**3 + 180*p**2*q2*q3**2*q4*q5**2*q6**2 - 1440*p**2*q2*q3**2*q4*q5**2*q6 + 5040*p**2*q2*q3**2*q4*q5**2 + 180*p**2*q2*q3**2*q4*q5*q6**3 - 1440*p**2*q2*q3**2*q4*q5*q6**2 + 5040*p**2*q2*q3**2*q4*q5*q6 - 10080*p**2*q2*q3**2*q4*q5 + 180*p**2*q2*q3*q4**5*q5 + 180*p**2*q2*q3*q4**4*q5**2 + 180*p**2*q2*q3*q4**4*q5*q6 - 1440*p**2*q2*q3*q4**4*q5 + 180*p**2*q2*q3*q4**3*q5**3 + 180*p**2*q2*q3*q4**3*q5**2*q6 - 1440*p**2*q2*q3*q4**3*q5**2 + 180*p**2*q2*q3*q4**3*q5*q6**2 - 1440*p**2*q2*q3*q4**3*q5*q6 + 5040*p**2*q2*q3*q4**3*q5 + 180*p**2*q2*q3*q4**2*q5**4 + 180*p**2*q2*q3*q4**2*q5**3*q6 - 1440*p**2*q2*q3*q4**2*q5**3 + 180*p**2*q2*q3*q4**2*q5**2*q6**2 - 1440*p**2*q2*q3*q4**2*q5**2*q6 + 5040*p**2*q2*q3*q4**2*q5**2 + 180*p**2*q2*q3*q4**2*q5*q6**3 - 1440*p**2*q2*q3*q4**2*q5*q6**2 + 5040*p**2*q2*q3*q4**2*q5*q6 - 10080*p**2*q2*q3*q4**2*q5 + 180*p**2*q2*q3*q4*q5**5 + 180*p**2*q2*q3*q4*q5**4*q6 - 1440*p**2*q2*q3*q4*q5**4 + 180*p**2*q2*q3*q4*q5**3*q6**2 - 1440*p**2*q2*q3*q4*q5**3*q6 + 5040*p**2*q2*q3*q4*q5**3 + 180*p**2*q2*q3*q4*q5**2*q6**3 - 1440*p**2*q2*q3*q4*q5**2*q6**2 + 5040*p**2*q2*q3*q4*q5**2*q6 - 10080*p**2*q2*q3*q4*q5**2 + 180*p**2*q2*q3*q4*q5*q6**4 - 1440*p**2*q2*q3*q4*q5*q6**3 + 5040*p**2*q2*q3*q4*q5*q6**2 - 10080*p**2*q2*q3*q4*q5*q6 + 12600*p**2*q2*q3*q4*q5 - 120*p**2*q3**5*q4*q5 - 120*p**2*q3**4*q4**2*q5 - 120*p**2*q3**4*q4*q5**2 - 120*p**2*q3**4*q4*q5*q6 + 840*p**2*q3**4*q4*q5 - 120*p**2*q3**3*q4**3*q5 - 120*p**2*q3**3*q4**2*q5**2 - 120*p**2*q3**3*q4**2*q5*q6 + 840*p**2*q3**3*q4**2*q5 - 120*p**2*q3**3*q4*q5**3 - 120*p**2*q3**3*q4*q5**2*q6 + 840*p**2*q3**3*q4*q5**2 - 120*p**2*q3**3*q4*q5*q6**2 + 840*p**2*q3**3*q4*q5*q6 - 2520*p**2*q3**3*q4*q5 - 120*p**2*q3**2*q4**4*q5 - 120*p**2*q3**2*q4**3*q5**2 - 120*p**2*q3**2*q4**3*q5*q6 + 840*p**2*q3**2*q4**3*q5 - 120*p**2*q3**2*q4**2*q5**3 - 120*p**2*q3**2*q4**2*q5**2*q6 + 840*p**2*q3**2*q4**2*q5**2 - 120*p**2*q3**2*q4**2*q5*q6**2 + 840*p**2*q3**2*q4**2*q5*q6 - 2520*p**2*q3**2*q4**2*q5 - 120*p**2*q3**2*q4*q5**4 - 120*p**2*q3**2*q4*q5**3*q6 + 840*p**2*q3**2*q4*q5**3 - 120*p**2*q3**2*q4*q5**2*q6**2 + 840*p**2*q3**2*q4*q5**2*q6 - 2520*p**2*q3**2*q4*q5**2 - 120*p**2*q3**2*q4*q5*q6**3 + 840*p**2*q3**2*q4*q5*q6**2 - 2520*p**2*q3**2*q4*q5*q6 + 4200*p**2*q3**2*q4*q5 - 120*p**2*q3*q4**5*q5 - 120*p**2*q3*q4**4*q5**2 - 120*p**2*q3*q4**4*q5*q6 + 840*p**2*q3*q4**4*q5 - 120*p**2*q3*q4**3*q5**3 - 120*p**2*q3*q4**3*q5**2*q6 + 840*p**2*q3*q4**3*q5**2 - 120*p**2*q3*q4**3*q5*q6**2 + 840*p**2*q3*q4**3*q5*q6 - 2520*p**2*q3*q4**3*q5 - 120*p**2*q3*q4**2*q5**4 - 120*p**2*q3*q4**2*q5**3*q6 + 840*p**2*q3*q4**2*q5**3 - 120*p**2*q3*q4**2*q5**2*q6**2 + 840*p**2*q3*q4**2*q5**2*q6 - 2520*p**2*q3*q4**2*q5**2 - 120*p**2*q3*q4**2*q5*q6**3 + 840*p**2*q3*q4**2*q5*q6**2 - 2520*p**2*q3*q4**2*q5*q6 + 4200*p**2*q3*q4**2*q5 - 120*p**2*q3*q4*q5**5 - 120*p**2*q3*q4*q5**4*q6 + 840*p**2*q3*q4*q5**4 - 120*p**2*q3*q4*q5**3*q6**2 + 840*p**2*q3*q4*q5**3*q6 - 2520*p**2*q3*q4*q5**3 - 120*p**2*q3*q4*q5**2*q6**3 + 840*p**2*q3*q4*q5**2*q6**2 - 2520*p**2*q3*q4*q5**2*q6 + 4200*p**2*q3*q4*q5**2 - 120*p**2*q3*q4*q5*q6**4 + 840*p**2*q3*q4*q5*q6**3 - 2520*p**2*q3*q4*q5*q6**2 + 4200*p**2*q3*q4*q5*q6 - 4200*p**2*q3*q4*q5 + 50*p*q1**5*q2*q3*q4*q5 + 50*p*q1**4*q2**2*q3*q4*q5 + 50*p*q1**4*q2*q3**2*q4*q5 + 50*p*q1**4*q2*q3*q4**2*q5 + 50*p*q1**4*q2*q3*q4*q5**2 + 50*p*q1**4*q2*q3*q4*q5*q6 - 450*p*q1**4*q2*q3*q4*q5 + 50*p*q1**3*q2**3*q3*q4*q5 + 50*p*q1**3*q2**2*q3**2*q4*q5 + 50*p*q1**3*q2**2*q3*q4**2*q5 + 50*p*q1**3*q2**2*q3*q4*q5**2 + 50*p*q1**3*q2**2*q3*q4*q5*q6 - 450*p*q1**3*q2**2*q3*q4*q5 + 50*p*q1**3*q2*q3**3*q4*q5 + 50*p*q1**3*q2*q3**2*q4**2*q5 + 50*p*q1**3*q2*q3**2*q4*q5**2 + 50*p*q1**3*q2*q3**2*q4*q5*q6 - 450*p*q1**3*q2*q3**2*q4*q5 + 50*p*q1**3*q2*q3*q4**3*q5 + 50*p*q1**3*q2*q3*q4**2*q5**2 + 50*p*q1**3*q2*q3*q4**2*q5*q6 - 450*p*q1**3*q2*q3*q4**2*q5 + 50*p*q1**3*q2*q3*q4*q5**3 + 50*p*q1**3*q2*q3*q4*q5**2*q6 - 450*p*q1**3*q2*q3*q4*q5**2 + 50*p*q1**3*q2*q3*q4*q5*q6**2 - 450*p*q1**3*q2*q3*q4*q5*q6 + 1800*p*q1**3*q2*q3*q4*q5 + 50*p*q1**2*q2**4*q3*q4*q5 + 50*p*q1**2*q2**3*q3**2*q4*q5 + 50*p*q1**2*q2**3*q3*q4**2*q5 + 50*p*q1**2*q2**3*q3*q4*q5**2 + 50*p*q1**2*q2**3*q3*q4*q5*q6 - 450*p*q1**2*q2**3*q3*q4*q5 + 50*p*q1**2*q2**2*q3**3*q4*q5 + 50*p*q1**2*q2**2*q3**2*q4**2*q5 + 50*p*q1**2*q2**2*q3**2*q4*q5**2 + 50*p*q1**2*q2**2*q3**2*q4*q5*q6 - 450*p*q1**2*q2**2*q3**2*q4*q5 + 50*p*q1**2*q2**2*q3*q4**3*q5 + 50*p*q1**2*q2**2*q3*q4**2*q5**2 + 50*p*q1**2*q2**2*q3*q4**2*q5*q6 - 450*p*q1**2*q2**2*q3*q4**2*q5 + 50*p*q1**2*q2**2*q3*q4*q5**3 + 50*p*q1**2*q2**2*q3*q4*q5**2*q6 - 450*p*q1**2*q2**2*q3*q4*q5**2 + 50*p*q1**2*q2**2*q3*q4*q5*q6**2 - 450*p*q1**2*q2**2*q3*q4*q5*q6 + 1800*p*q1**2*q2**2*q3*q4*q5 + 50*p*q1**2*q2*q3**4*q4*q5 + 50*p*q1**2*q2*q3**3*q4**2*q5 + 50*p*q1**2*q2*q3**3*q4*q5**2 + 50*p*q1**2*q2*q3**3*q4*q5*q6 - 450*p*q1**2*q2*q3**3*q4*q5 + 50*p*q1**2*q2*q3**2*q4**3*q5 + 50*p*q1**2*q2*q3**2*q4**2*q5**2 + 50*p*q1**2*q2*q3**2*q4**2*q5*q6 - 450*p*q1**2*q2*q3**2*q4**2*q5 + 50*p*q1**2*q2*q3**2*q4*q5**3 + 50*p*q1**2*q2*q3**2*q4*q5**2*q6 - 450*p*q1**2*q2*q3**2*q4*q5**2 + 50*p*q1**2*q2*q3**2*q4*q5*q6**2 - 450*p*q1**2*q2*q3**2*q4*q5*q6 + 1800*p*q1**2*q2*q3**2*q4*q5 + 50*p*q1**2*q2*q3*q4**4*q5 + 50*p*q1**2*q2*q3*q4**3*q5**2 + 50*p*q1**2*q2*q3*q4**3*q5*q6 - 450*p*q1**2*q2*q3*q4**3*q5 + 50*p*q1**2*q2*q3*q4**2*q5**3 + 50*p*q1**2*q2*q3*q4**2*q5**2*q6 - 450*p*q1**2*q2*q3*q4**2*q5**2 + 50*p*q1**2*q2*q3*q4**2*q5*q6**2 - 450*p*q1**2*q2*q3*q4**2*q5*q6 + 1800*p*q1**2*q2*q3*q4**2*q5 + 50*p*q1**2*q2*q3*q4*q5**4 + 50*p*q1**2*q2*q3*q4*q5**3*q6 - 450*p*q1**2*q2*q3*q4*q5**3 + 50*p*q1**2*q2*q3*q4*q5**2*q6**2 - 450*p*q1**2*q2*q3*q4*q5**2*q6 + 1800*p*q1**2*q2*q3*q4*q5**2 + 50*p*q1**2*q2*q3*q4*q5*q6**3 - 450*p*q1**2*q2*q3*q4*q5*q6**2 + 1800*p*q1**2*q2*q3*q4*q5*q6 - 4200*p*q1**2*q2*q3*q4*q5 + 50*p*q1*q2**5*q3*q4*q5 + 50*p*q1*q2**4*q3**2*q4*q5 + 50*p*q1*q2**4*q3*q4**2*q5 + 50*p*q1*q2**4*q3*q4*q5**2 + 50*p*q1*q2**4*q3*q4*q5*q6 - 450*p*q1*q2**4*q3*q4*q5 + 50*p*q1*q2**3*q3**3*q4*q5 + 50*p*q1*q2**3*q3**2*q4**2*q5 + 50*p*q1*q2**3*q3**2*q4*q5**2 + 50*p*q1*q2**3*q3**2*q4*q5*q6 - 450*p*q1*q2**3*q3**2*q4*q5 + 50*p*q1*q2**3*q3*q4**3*q5 + 50*p*q1*q2**3*q3*q4**2*q5**2 + 50*p*q1*q2**3*q3*q4**2*q5*q6 - 450*p*q1*q2**3*q3*q4**2*q5 + 50*p*q1*q2**3*q3*q4*q5**3 + 50*p*q1*q2**3*q3*q4*q5**2*q6 - 450*p*q1*q2**3*q3*q4*q5**2 + 50*p*q1*q2**3*q3*q4*q5*q6**2 - 450*p*q1*q2**3*q3*q4*q5*q6 + 1800*p*q1*q2**3*q3*q4*q5 + 50*p*q1*q2**2*q3**4*q4*q5 + 50*p*q1*q2**2*q3**3*q4**2*q5 + 50*p*q1*q2**2*q3**3*q4*q5**2 + 50*p*q1*q2**2*q3**3*q4*q5*q6 - 450*p*q1*q2**2*q3**3*q4*q5 + 50*p*q1*q2**2*q3**2*q4**3*q5 + 50*p*q1*q2**2*q3**2*q4**2*q5**2 + 50*p*q1*q2**2*q3**2*q4**2*q5*q6 - 450*p*q1*q2**2*q3**2*q4**2*q5 + 50*p*q1*q2**2*q3**2*q4*q5**3 + 50*p*q1*q2**2*q3**2*q4*q5**2*q6 - 450*p*q1*q2**2*q3**2*q4*q5**2 + 50*p*q1*q2**2*q3**2*q4*q5*q6**2 - 450*p*q1*q2**2*q3**2*q4*q5*q6 + 1800*p*q1*q2**2*q3**2*q4*q5 + 50*p*q1*q2**2*q3*q4**4*q5 + 50*p*q1*q2**2*q3*q4**3*q5**2 + 50*p*q1*q2**2*q3*q4**3*q5*q6 - 450*p*q1*q2**2*q3*q4**3*q5 + 50*p*q1*q2**2*q3*q4**2*q5**3 + 50*p*q1*q2**2*q3*q4**2*q5**2*q6 - 450*p*q1*q2**2*q3*q4**2*q5**2 + 50*p*q1*q2**2*q3*q4**2*q5*q6**2 - 450*p*q1*q2**2*q3*q4**2*q5*q6 + 1800*p*q1*q2**2*q3*q4**2*q5 + 50*p*q1*q2**2*q3*q4*q5**4 + 50*p*q1*q2**2*q3*q4*q5**3*q6 - 450*p*q1*q2**2*q3*q4*q5**3 + 50*p*q1*q2**2*q3*q4*q5**2*q6**2 - 450*p*q1*q2**2*q3*q4*q5**2*q6 + 1800*p*q1*q2**2*q3*q4*q5**2 + 50*p*q1*q2**2*q3*q4*q5*q6**3 - 450*p*q1*q2**2*q3*q4*q5*q6**2 + 1800*p*q1*q2**2*q3*q4*q5*q6 - 4200*p*q1*q2**2*q3*q4*q5 + 50*p*q1*q2*q3**5*q4*q5 + 50*p*q1*q2*q3**4*q4**2*q5 + 50*p*q1*q2*q3**4*q4*q5**2 + 50*p*q1*q2*q3**4*q4*q5*q6 - 450*p*q1*q2*q3**4*q4*q5 + 50*p*q1*q2*q3**3*q4**3*q5 + 50*p*q1*q2*q3**3*q4**2*q5**2 + 50*p*q1*q2*q3**3*q4**2*q5*q6 - 450*p*q1*q2*q3**3*q4**2*q5 + 50*p*q1*q2*q3**3*q4*q5**3 + 50*p*q1*q2*q3**3*q4*q5**2*q6 - 450*p*q1*q2*q3**3*q4*q5**2 + 50*p*q1*q2*q3**3*q4*q5*q6**2 - 450*p*q1*q2*q3**3*q4*q5*q6 + 1800*p*q1*q2*q3**3*q4*q5 + 50*p*q1*q2*q3**2*q4**4*q5 + 50*p*q1*q2*q3**2*q4**3*q5**2 + 50*p*q1*q2*q3**2*q4**3*q5*q6 - 450*p*q1*q2*q3**2*q4**3*q5 + 50*p*q1*q2*q3**2*q4**2*q5**3 + 50*p*q1*q2*q3**2*q4**2*q5**2*q6 - 450*p*q1*q2*q3**2*q4**2*q5**2 + 50*p*q1*q2*q3**2*q4**2*q5*q6**2 - 450*p*q1*q2*q3**2*q4**2*q5*q6 + 1800*p*q1*q2*q3**2*q4**2*q5 + 50*p*q1*q2*q3**2*q4*q5**4 + 50*p*q1*q2*q3**2*q4*q5**3*q6 - 450*p*q1*q2*q3**2*q4*q5**3 + 50*p*q1*q2*q3**2*q4*q5**2*q6**2 - 450*p*q1*q2*q3**2*q4*q5**2*q6 + 1800*p*q1*q2*q3**2*q4*q5**2 + 50*p*q1*q2*q3**2*q4*q5*q6**3 - 450*p*q1*q2*q3**2*q4*q5*q6**2 + 1800*p*q1*q2*q3**2*q4*q5*q6 - 4200*p*q1*q2*q3**2*q4*q5 + 50*p*q1*q2*q3*q4**5*q5 + 50*p*q1*q2*q3*q4**4*q5**2 + 50*p*q1*q2*q3*q4**4*q5*q6 - 450*p*q1*q2*q3*q4**4*q5 + 50*p*q1*q2*q3*q4**3*q5**3 + 50*p*q1*q2*q3*q4**3*q5**2*q6 - 450*p*q1*q2*q3*q4**3*q5**2 + 50*p*q1*q2*q3*q4**3*q5*q6**2 - 450*p*q1*q2*q3*q4**3*q5*q6 + 1800*p*q1*q2*q3*q4**3*q5 + 50*p*q1*q2*q3*q4**2*q5**4 + 50*p*q1*q2*q3*q4**2*q5**3*q6 - 450*p*q1*q2*q3*q4**2*q5**3 + 50*p*q1*q2*q3*q4**2*q5**2*q6**2 - 450*p*q1*q2*q3*q4**2*q5**2*q6 + 1800*p*q1*q2*q3*q4**2*q5**2 + 50*p*q1*q2*q3*q4**2*q5*q6**3 - 450*p*q1*q2*q3*q4**2*q5*q6**2 + 1800*p*q1*q2*q3*q4**2*q5*q6 - 4200*p*q1*q2*q3*q4**2*q5 + 50*p*q1*q2*q3*q4*q5**5 + 50*p*q1*q2*q3*q4*q5**4*q6 - 450*p*q1*q2*q3*q4*q5**4 + 50*p*q1*q2*q3*q4*q5**3*q6**2 - 450*p*q1*q2*q3*q4*q5**3*q6 + 1800*p*q1*q2*q3*q4*q5**3 + 50*p*q1*q2*q3*q4*q5**2*q6**3 - 450*p*q1*q2*q3*q4*q5**2*q6**2 + 1800*p*q1*q2*q3*q4*q5**2*q6 - 4200*p*q1*q2*q3*q4*q5**2 + 50*p*q1*q2*q3*q4*q5*q6**4 - 450*p*q1*q2*q3*q4*q5*q6**3 + 1800*p*q1*q2*q3*q4*q5*q6**2 - 4200*p*q1*q2*q3*q4*q5*q6 + 6300*p*q1*q2*q3*q4*q5 - 45*p*q2**5*q3*q4*q5 - 45*p*q2**4*q3**2*q4*q5 - 45*p*q2**4*q3*q4**2*q5 - 45*p*q2**4*q3*q4*q5**2 - 45*p*q2**4*q3*q4*q5*q6 + 360*p*q2**4*q3*q4*q5 - 45*p*q2**3*q3**3*q4*q5 - 45*p*q2**3*q3**2*q4**2*q5 - 45*p*q2**3*q3**2*q4*q5**2 - 45*p*q2**3*q3**2*q4*q5*q6 + 360*p*q2**3*q3**2*q4*q5 - 45*p*q2**3*q3*q4**3*q5 - 45*p*q2**3*q3*q4**2*q5**2 - 45*p*q2**3*q3*q4**2*q5*q6 + 360*p*q2**3*q3*q4**2*q5 - 45*p*q2**3*q3*q4*q5**3 - 45*p*q2**3*q3*q4*q5**2*q6 + 360*p*q2**3*q3*q4*q5**2 - 45*p*q2**3*q3*q4*q5*q6**2 + 360*p*q2**3*q3*q4*q5*q6 - 1260*p*q2**3*q3*q4*q5 - 45*p*q2**2*q3**4*q4*q5 - 45*p*q2**2*q3**3*q4**2*q5 - 45*p*q2**2*q3**3*q4*q5**2 - 45*p*q2**2*q3**3*q4*q5*q6 + 360*p*q2**2*q3**3*q4*q5 - 45*p*q2**2*q3**2*q4**3*q5 - 45*p*q2**2*q3**2*q4**2*q5**2 - 45*p*q2**2*q3**2*q4**2*q5*q6 + 360*p*q2**2*q3**2*q4**2*q5 - 45*p*q2**2*q3**2*q4*q5**3 - 45*p*q2**2*q3**2*q4*q5**2*q6 + 360*p*q2**2*q3**2*q4*q5**2 - 45*p*q2**2*q3**2*q4*q5*q6**2 + 360*p*q2**2*q3**2*q4*q5*q6 - 1260*p*q2**2*q3**2*q4*q5 - 45*p*q2**2*q3*q4**4*q5 - 45*p*q2**2*q3*q4**3*q5**2 - 45*p*q2**2*q3*q4**3*q5*q6 + 360*p*q2**2*q3*q4**3*q5 - 45*p*q2**2*q3*q4**2*q5**3 - 45*p*q2**2*q3*q4**2*q5**2*q6 + 360*p*q2**2*q3*q4**2*q5**2 - 45*p*q2**2*q3*q4**2*q5*q6**2 + 360*p*q2**2*q3*q4**2*q5*q6 - 1260*p*q2**2*q3*q4**2*q5 - 45*p*q2**2*q3*q4*q5**4 - 45*p*q2**2*q3*q4*q5**3*q6 + 360*p*q2**2*q3*q4*q5**3 - 45*p*q2**2*q3*q4*q5**2*q6**2 + 360*p*q2**2*q3*q4*q5**2*q6 - 1260*p*q2**2*q3*q4*q5**2 - 45*p*q2**2*q3*q4*q5*q6**3 + 360*p*q2**2*q3*q4*q5*q6**2 - 1260*p*q2**2*q3*q4*q5*q6 + 2520*p*q2**2*q3*q4*q5 - 45*p*q2*q3**5*q4*q5 - 45*p*q2*q3**4*q4**2*q5 - 45*p*q2*q3**4*q4*q5**2 - 45*p*q2*q3**4*q4*q5*q6 + 360*p*q2*q3**4*q4*q5 - 45*p*q2*q3**3*q4**3*q5 - 45*p*q2*q3**3*q4**2*q5**2 - 45*p*q2*q3**3*q4**2*q5*q6 + 360*p*q2*q3**3*q4**2*q5 - 45*p*q2*q3**3*q4*q5**3 - 45*p*q2*q3**3*q4*q5**2*q6 + 360*p*q2*q3**3*q4*q5**2 - 45*p*q2*q3**3*q4*q5*q6**2 + 360*p*q2*q3**3*q4*q5*q6 - 1260*p*q2*q3**3*q4*q5 - 45*p*q2*q3**2*q4**4*q5 - 45*p*q2*q3**2*q4**3*q5**2 - 45*p*q2*q3**2*q4**3*q5*q6 + 360*p*q2*q3**2*q4**3*q5 - 45*p*q2*q3**2*q4**2*q5**3 - 45*p*q2*q3**2*q4**2*q5**2*q6 + 360*p*q2*q3**2*q4**2*q5**2 - 45*p*q2*q3**2*q4**2*q5*q6**2 + 360*p*q2*q3**2*q4**2*q5*q6 - 1260*p*q2*q3**2*q4**2*q5 - 45*p*q2*q3**2*q4*q5**4 - 45*p*q2*q3**2*q4*q5**3*q6 + 360*p*q2*q3**2*q4*q5**3 - 45*p*q2*q3**2*q4*q5**2*q6**2 + 360*p*q2*q3**2*q4*q5**2*q6 - 1260*p*q2*q3**2*q4*q5**2 - 45*p*q2*q3**2*q4*q5*q6**3 + 360*p*q2*q3**2*q4*q5*q6**2 - 1260*p*q2*q3**2*q4*q5*q6 + 2520*p*q2*q3**2*q4*q5 - 45*p*q2*q3*q4**5*q5 - 45*p*q2*q3*q4**4*q5**2 - 45*p*q2*q3*q4**4*q5*q6 + 360*p*q2*q3*q4**4*q5 - 45*p*q2*q3*q4**3*q5**3 - 45*p*q2*q3*q4**3*q5**2*q6 + 360*p*q2*q3*q4**3*q5**2 - 45*p*q2*q3*q4**3*q5*q6**2 + 360*p*q2*q3*q4**3*q5*q6 - 1260*p*q2*q3*q4**3*q5 - 45*p*q2*q3*q4**2*q5**4 - 45*p*q2*q3*q4**2*q5**3*q6 + 360*p*q2*q3*q4**2*q5**3 - 45*p*q2*q3*q4**2*q5**2*q6**2 + 360*p*q2*q3*q4**2*q5**2*q6 - 1260*p*q2*q3*q4**2*q5**2 - 45*p*q2*q3*q4**2*q5*q6**3 + 360*p*q2*q3*q4**2*q5*q6**2 - 1260*p*q2*q3*q4**2*q5*q6 + 2520*p*q2*q3*q4**2*q5 - 45*p*q2*q3*q4*q5**5 - 45*p*q2*q3*q4*q5**4*q6 + 360*p*q2*q3*q4*q5**4 - 45*p*q2*q3*q4*q5**3*q6**2 + 360*p*q2*q3*q4*q5**3*q6 - 1260*p*q2*q3*q4*q5**3 - 45*p*q2*q3*q4*q5**2*q6**3 + 360*p*q2*q3*q4*q5**2*q6**2 - 1260*p*q2*q3*q4*q5**2*q6 + 2520*p*q2*q3*q4*q5**2 - 45*p*q2*q3*q4*q5*q6**4 + 360*p*q2*q3*q4*q5*q6**3 - 1260*p*q2*q3*q4*q5*q6**2 + 2520*p*q2*q3*q4*q5*q6 - 3150*p*q2*q3*q4*q5 - 10*q1**5*q2*q3*q4*q5 - 10*q1**4*q2**2*q3*q4*q5 - 10*q1**4*q2*q3**2*q4*q5 - 10*q1**4*q2*q3*q4**2*q5 - 10*q1**4*q2*q3*q4*q5**2 - 10*q1**4*q2*q3*q4*q5*q6 + 90*q1**4*q2*q3*q4*q5 - 10*q1**3*q2**3*q3*q4*q5 - 10*q1**3*q2**2*q3**2*q4*q5 - 10*q1**3*q2**2*q3*q4**2*q5 - 10*q1**3*q2**2*q3*q4*q5**2 - 10*q1**3*q2**2*q3*q4*q5*q6 + 90*q1**3*q2**2*q3*q4*q5 - 10*q1**3*q2*q3**3*q4*q5 - 10*q1**3*q2*q3**2*q4**2*q5 - 10*q1**3*q2*q3**2*q4*q5**2 - 10*q1**3*q2*q3**2*q4*q5*q6 + 90*q1**3*q2*q3**2*q4*q5 - 10*q1**3*q2*q3*q4**3*q5 - 10*q1**3*q2*q3*q4**2*q5**2 - 10*q1**3*q2*q3*q4**2*q5*q6 + 90*q1**3*q2*q3*q4**2*q5 - 10*q1**3*q2*q3*q4*q5**3 - 10*q1**3*q2*q3*q4*q5**2*q6 + 90*q1**3*q2*q3*q4*q5**2 - 10*q1**3*q2*q3*q4*q5*q6**2 + 90*q1**3*q2*q3*q4*q5*q6 - 360*q1**3*q2*q3*q4*q5 - 10*q1**2*q2**4*q3*q4*q5 - 10*q1**2*q2**3*q3**2*q4*q5 - 10*q1**2*q2**3*q3*q4**2*q5 - 10*q1**2*q2**3*q3*q4*q5**2 - 10*q1**2*q2**3*q3*q4*q5*q6 + 90*q1**2*q2**3*q3*q4*q5 - 10*q1**2*q2**2*q3**3*q4*q5 - 10*q1**2*q2**2*q3**2*q4**2*q5 - 10*q1**2*q2**2*q3**2*q4*q5**2 - 10*q1**2*q2**2*q3**2*q4*q5*q6 + 90*q1**2*q2**2*q3**2*q4*q5 - 10*q1**2*q2**2*q3*q4**3*q5 - 10*q1**2*q2**2*q3*q4**2*q5**2 - 10*q1**2*q2**2*q3*q4**2*q5*q6 + 90*q1**2*q2**2*q3*q4**2*q5 - 10*q1**2*q2**2*q3*q4*q5**3 - 10*q1**2*q2**2*q3*q4*q5**2*q6 + 90*q1**2*q2**2*q3*q4*q5**2 - 10*q1**2*q2**2*q3*q4*q5*q6**2 + 90*q1**2*q2**2*q3*q4*q5*q6 - 360*q1**2*q2**2*q3*q4*q5 - 10*q1**2*q2*q3**4*q4*q5 - 10*q1**2*q2*q3**3*q4**2*q5 - 10*q1**2*q2*q3**3*q4*q5**2 - 10*q1**2*q2*q3**3*q4*q5*q6 + 90*q1**2*q2*q3**3*q4*q5 - 10*q1**2*q2*q3**2*q4**3*q5 - 10*q1**2*q2*q3**2*q4**2*q5**2 - 10*q1**2*q2*q3**2*q4**2*q5*q6 + 90*q1**2*q2*q3**2*q4**2*q5 - 10*q1**2*q2*q3**2*q4*q5**3 - 10*q1**2*q2*q3**2*q4*q5**2*q6 + 90*q1**2*q2*q3**2*q4*q5**2 - 10*q1**2*q2*q3**2*q4*q5*q6**2 + 90*q1**2*q2*q3**2*q4*q5*q6 - 360*q1**2*q2*q3**2*q4*q5 - 10*q1**2*q2*q3*q4**4*q5 - 10*q1**2*q2*q3*q4**3*q5**2 - 10*q1**2*q2*q3*q4**3*q5*q6 + 90*q1**2*q2*q3*q4**3*q5 - 10*q1**2*q2*q3*q4**2*q5**3 - 10*q1**2*q2*q3*q4**2*q5**2*q6 + 90*q1**2*q2*q3*q4**2*q5**2 - 10*q1**2*q2*q3*q4**2*q5*q6**2 + 90*q1**2*q2*q3*q4**2*q5*q6 - 360*q1**2*q2*q3*q4**2*q5 - 10*q1**2*q2*q3*q4*q5**4 - 10*q1**2*q2*q3*q4*q5**3*q6 + 90*q1**2*q2*q3*q4*q5**3 - 10*q1**2*q2*q3*q4*q5**2*q6**2 + 90*q1**2*q2*q3*q4*q5**2*q6 - 360*q1**2*q2*q3*q4*q5**2 - 10*q1**2*q2*q3*q4*q5*q6**3 + 90*q1**2*q2*q3*q4*q5*q6**2 - 360*q1**2*q2*q3*q4*q5*q6 + 840*q1**2*q2*q3*q4*q5 - 10*q1*q2**5*q3*q4*q5 - 10*q1*q2**4*q3**2*q4*q5 - 10*q1*q2**4*q3*q4**2*q5 - 10*q1*q2**4*q3*q4*q5**2 - 10*q1*q2**4*q3*q4*q5*q6 + 90*q1*q2**4*q3*q4*q5 - 10*q1*q2**3*q3**3*q4*q5 - 10*q1*q2**3*q3**2*q4**2*q5 - 10*q1*q2**3*q3**2*q4*q5**2 - 10*q1*q2**3*q3**2*q4*q5*q6 + 90*q1*q2**3*q3**2*q4*q5 - 10*q1*q2**3*q3*q4**3*q5 - 10*q1*q2**3*q3*q4**2*q5**2 - 10*q1*q2**3*q3*q4**2*q5*q6 + 90*q1*q2**3*q3*q4**2*q5 - 10*q1*q2**3*q3*q4*q5**3 - 10*q1*q2**3*q3*q4*q5**2*q6 + 90*q1*q2**3*q3*q4*q5**2 - 10*q1*q2**3*q3*q4*q5*q6**2 + 90*q1*q2**3*q3*q4*q5*q6 - 360*q1*q2**3*q3*q4*q5 - 10*q1*q2**2*q3**4*q4*q5 - 10*q1*q2**2*q3**3*q4**2*q5 - 10*q1*q2**2*q3**3*q4*q5**2 - 10*q1*q2**2*q3**3*q4*q5*q6 + 90*q1*q2**2*q3**3*q4*q5 - 10*q1*q2**2*q3**2*q4**3*q5 - 10*q1*q2**2*q3**2*q4**2*q5**2 - 10*q1*q2**2*q3**2*q4**2*q5*q6 + 90*q1*q2**2*q3**2*q4**2*q5 - 10*q1*q2**2*q3**2*q4*q5**3 - 10*q1*q2**2*q3**2*q4*q5**2*q6 + 90*q1*q2**2*q3**2*q4*q5**2 - 10*q1*q2**2*q3**2*q4*q5*q6**2 + 90*q1*q2**2*q3**2*q4*q5*q6 - 360*q1*q2**2*q3**2*q4*q5 - 10*q1*q2**2*q3*q4**4*q5 - 10*q1*q2**2*q3*q4**3*q5**2 - 10*q1*q2**2*q3*q4**3*q5*q6 + 90*q1*q2**2*q3*q4**3*q5 - 10*q1*q2**2*q3*q4**2*q5**3 - 10*q1*q2**2*q3*q4**2*q5**2*q6 + 90*q1*q2**2*q3*q4**2*q5**2 - 10*q1*q2**2*q3*q4**2*q5*q6**2 + 90*q1*q2**2*q3*q4**2*q5*q6 - 360*q1*q2**2*q3*q4**2*q5 - 10*q1*q2**2*q3*q4*q5**4 - 10*q1*q2**2*q3*q4*q5**3*q6 + 90*q1*q2**2*q3*q4*q5**3 - 10*q1*q2**2*q3*q4*q5**2*q6**2 + 90*q1*q2**2*q3*q4*q5**2*q6 - 360*q1*q2**2*q3*q4*q5**2 - 10*q1*q2**2*q3*q4*q5*q6**3 + 90*q1*q2**2*q3*q4*q5*q6**2 - 360*q1*q2**2*q3*q4*q5*q6 + 840*q1*q2**2*q3*q4*q5 - 10*q1*q2*q3**5*q4*q5 - 10*q1*q2*q3**4*q4**2*q5 - 10*q1*q2*q3**4*q4*q5**2 - 10*q1*q2*q3**4*q4*q5*q6 + 90*q1*q2*q3**4*q4*q5 - 10*q1*q2*q3**3*q4**3*q5 - 10*q1*q2*q3**3*q4**2*q5**2 - 10*q1*q2*q3**3*q4**2*q5*q6 + 90*q1*q2*q3**3*q4**2*q5 - 10*q1*q2*q3**3*q4*q5**3 - 10*q1*q2*q3**3*q4*q5**2*q6 + 90*q1*q2*q3**3*q4*q5**2 - 10*q1*q2*q3**3*q4*q5*q6**2 + 90*q1*q2*q3**3*q4*q5*q6 - 360*q1*q2*q3**3*q4*q5 - 10*q1*q2*q3**2*q4**4*q5 - 10*q1*q2*q3**2*q4**3*q5**2 - 10*q1*q2*q3**2*q4**3*q5*q6 + 90*q1*q2*q3**2*q4**3*q5 - 10*q1*q2*q3**2*q4**2*q5**3 - 10*q1*q2*q3**2*q4**2*q5**2*q6 + 90*q1*q2*q3**2*q4**2*q5**2 - 10*q1*q2*q3**2*q4**2*q5*q6**2 + 90*q1*q2*q3**2*q4**2*q5*q6 - 360*q1*q2*q3**2*q4**2*q5 - 10*q1*q2*q3**2*q4*q5**4 - 10*q1*q2*q3**2*q4*q5**3*q6 + 90*q1*q2*q3**2*q4*q5**3 - 10*q1*q2*q3**2*q4*q5**2*q6**2 + 90*q1*q2*q3**2*q4*q5**2*q6 - 360*q1*q2*q3**2*q4*q5**2 - 10*q1*q2*q3**2*q4*q5*q6**3 + 90*q1*q2*q3**2*q4*q5*q6**2 - 360*q1*q2*q3**2*q4*q5*q6 + 840*q1*q2*q3**2*q4*q5 - 10*q1*q2*q3*q4**5*q5 - 10*q1*q2*q3*q4**4*q5**2 - 10*q1*q2*q3*q4**4*q5*q6 + 90*q1*q2*q3*q4**4*q5 - 10*q1*q2*q3*q4**3*q5**3 - 10*q1*q2*q3*q4**3*q5**2*q6 + 90*q1*q2*q3*q4**3*q5**2 - 10*q1*q2*q3*q4**3*q5*q6**2 + 90*q1*q2*q3*q4**3*q5*q6 - 360*q1*q2*q3*q4**3*q5 - 10*q1*q2*q3*q4**2*q5**4 - 10*q1*q2*q3*q4**2*q5**3*q6 + 90*q1*q2*q3*q4**2*q5**3 - 10*q1*q2*q3*q4**2*q5**2*q6**2 + 90*q1*q2*q3*q4**2*q5**2*q6 - 360*q1*q2*q3*q4**2*q5**2 - 10*q1*q2*q3*q4**2*q5*q6**3 + 90*q1*q2*q3*q4**2*q5*q6**2 - 360*q1*q2*q3*q4**2*q5*q6 + 840*q1*q2*q3*q4**2*q5 - 10*q1*q2*q3*q4*q5**5 - 10*q1*q2*q3*q4*q5**4*q6 + 90*q1*q2*q3*q4*q5**4 - 10*q1*q2*q3*q4*q5**3*q6**2 + 90*q1*q2*q3*q4*q5**3*q6 - 360*q1*q2*q3*q4*q5**3 - 10*q1*q2*q3*q4*q5**2*q6**3 + 90*q1*q2*q3*q4*q5**2*q6**2 - 360*q1*q2*q3*q4*q5**2*q6 + 840*q1*q2*q3*q4*q5**2 - 10*q1*q2*q3*q4*q5*q6**4 + 90*q1*q2*q3*q4*q5*q6**3 - 360*q1*q2*q3*q4*q5*q6**2 + 840*q1*q2*q3*q4*q5*q6 - 1260*q1*q2*q3*q4*q5)'
f_multiparam[10][7] = 'p*(p - 1)**3*(10*p**6*q1**4*q2*q3*q4*q5*q6 + 10*p**6*q1**3*q2**2*q3*q4*q5*q6 + 10*p**6*q1**3*q2*q3**2*q4*q5*q6 + 10*p**6*q1**3*q2*q3*q4**2*q5*q6 + 10*p**6*q1**3*q2*q3*q4*q5**2*q6 + 10*p**6*q1**3*q2*q3*q4*q5*q6**2 + 10*p**6*q1**3*q2*q3*q4*q5*q6*q7 - 90*p**6*q1**3*q2*q3*q4*q5*q6 + 10*p**6*q1**2*q2**3*q3*q4*q5*q6 + 10*p**6*q1**2*q2**2*q3**2*q4*q5*q6 + 10*p**6*q1**2*q2**2*q3*q4**2*q5*q6 + 10*p**6*q1**2*q2**2*q3*q4*q5**2*q6 + 10*p**6*q1**2*q2**2*q3*q4*q5*q6**2 + 10*p**6*q1**2*q2**2*q3*q4*q5*q6*q7 - 90*p**6*q1**2*q2**2*q3*q4*q5*q6 + 10*p**6*q1**2*q2*q3**3*q4*q5*q6 + 10*p**6*q1**2*q2*q3**2*q4**2*q5*q6 + 10*p**6*q1**2*q2*q3**2*q4*q5**2*q6 + 10*p**6*q1**2*q2*q3**2*q4*q5*q6**2 + 10*p**6*q1**2*q2*q3**2*q4*q5*q6*q7 - 90*p**6*q1**2*q2*q3**2*q4*q5*q6 + 10*p**6*q1**2*q2*q3*q4**3*q5*q6 + 10*p**6*q1**2*q2*q3*q4**2*q5**2*q6 + 10*p**6*q1**2*q2*q3*q4**2*q5*q6**2 + 10*p**6*q1**2*q2*q3*q4**2*q5*q6*q7 - 90*p**6*q1**2*q2*q3*q4**2*q5*q6 + 10*p**6*q1**2*q2*q3*q4*q5**3*q6 + 10*p**6*q1**2*q2*q3*q4*q5**2*q6**2 + 10*p**6*q1**2*q2*q3*q4*q5**2*q6*q7 - 90*p**6*q1**2*q2*q3*q4*q5**2*q6 + 10*p**6*q1**2*q2*q3*q4*q5*q6**3 + 10*p**6*q1**2*q2*q3*q4*q5*q6**2*q7 - 90*p**6*q1**2*q2*q3*q4*q5*q6**2 + 10*p**6*q1**2*q2*q3*q4*q5*q6*q7**2 - 90*p**6*q1**2*q2*q3*q4*q5*q6*q7 + 360*p**6*q1**2*q2*q3*q4*q5*q6 + 10*p**6*q1*q2**4*q3*q4*q5*q6 + 10*p**6*q1*q2**3*q3**2*q4*q5*q6 + 10*p**6*q1*q2**3*q3*q4**2*q5*q6 + 10*p**6*q1*q2**3*q3*q4*q5**2*q6 + 10*p**6*q1*q2**3*q3*q4*q5*q6**2 + 10*p**6*q1*q2**3*q3*q4*q5*q6*q7 - 90*p**6*q1*q2**3*q3*q4*q5*q6 + 10*p**6*q1*q2**2*q3**3*q4*q5*q6 + 10*p**6*q1*q2**2*q3**2*q4**2*q5*q6 + 10*p**6*q1*q2**2*q3**2*q4*q5**2*q6 + 10*p**6*q1*q2**2*q3**2*q4*q5*q6**2 + 10*p**6*q1*q2**2*q3**2*q4*q5*q6*q7 - 90*p**6*q1*q2**2*q3**2*q4*q5*q6 + 10*p**6*q1*q2**2*q3*q4**3*q5*q6 + 10*p**6*q1*q2**2*q3*q4**2*q5**2*q6 + 10*p**6*q1*q2**2*q3*q4**2*q5*q6**2 + 10*p**6*q1*q2**2*q3*q4**2*q5*q6*q7 - 90*p**6*q1*q2**2*q3*q4**2*q5*q6 + 10*p**6*q1*q2**2*q3*q4*q5**3*q6 + 10*p**6*q1*q2**2*q3*q4*q5**2*q6**2 + 10*p**6*q1*q2**2*q3*q4*q5**2*q6*q7 - 90*p**6*q1*q2**2*q3*q4*q5**2*q6 + 10*p**6*q1*q2**2*q3*q4*q5*q6**3 + 10*p**6*q1*q2**2*q3*q4*q5*q6**2*q7 - 90*p**6*q1*q2**2*q3*q4*q5*q6**2 + 10*p**6*q1*q2**2*q3*q4*q5*q6*q7**2 - 90*p**6*q1*q2**2*q3*q4*q5*q6*q7 + 360*p**6*q1*q2**2*q3*q4*q5*q6 + 10*p**6*q1*q2*q3**4*q4*q5*q6 + 10*p**6*q1*q2*q3**3*q4**2*q5*q6 + 10*p**6*q1*q2*q3**3*q4*q5**2*q6 + 10*p**6*q1*q2*q3**3*q4*q5*q6**2 + 10*p**6*q1*q2*q3**3*q4*q5*q6*q7 - 90*p**6*q1*q2*q3**3*q4*q5*q6 + 10*p**6*q1*q2*q3**2*q4**3*q5*q6 + 10*p**6*q1*q2*q3**2*q4**2*q5**2*q6 + 10*p**6*q1*q2*q3**2*q4**2*q5*q6**2 + 10*p**6*q1*q2*q3**2*q4**2*q5*q6*q7 - 90*p**6*q1*q2*q3**2*q4**2*q5*q6 + 10*p**6*q1*q2*q3**2*q4*q5**3*q6 + 10*p**6*q1*q2*q3**2*q4*q5**2*q6**2 + 10*p**6*q1*q2*q3**2*q4*q5**2*q6*q7 - 90*p**6*q1*q2*q3**2*q4*q5**2*q6 + 10*p**6*q1*q2*q3**2*q4*q5*q6**3 + 10*p**6*q1*q2*q3**2*q4*q5*q6**2*q7 - 90*p**6*q1*q2*q3**2*q4*q5*q6**2 + 10*p**6*q1*q2*q3**2*q4*q5*q6*q7**2 - 90*p**6*q1*q2*q3**2*q4*q5*q6*q7 + 360*p**6*q1*q2*q3**2*q4*q5*q6 + 10*p**6*q1*q2*q3*q4**4*q5*q6 + 10*p**6*q1*q2*q3*q4**3*q5**2*q6 + 10*p**6*q1*q2*q3*q4**3*q5*q6**2 + 10*p**6*q1*q2*q3*q4**3*q5*q6*q7 - 90*p**6*q1*q2*q3*q4**3*q5*q6 + 10*p**6*q1*q2*q3*q4**2*q5**3*q6 + 10*p**6*q1*q2*q3*q4**2*q5**2*q6**2 + 10*p**6*q1*q2*q3*q4**2*q5**2*q6*q7 - 90*p**6*q1*q2*q3*q4**2*q5**2*q6 + 10*p**6*q1*q2*q3*q4**2*q5*q6**3 + 10*p**6*q1*q2*q3*q4**2*q5*q6**2*q7 - 90*p**6*q1*q2*q3*q4**2*q5*q6**2 + 10*p**6*q1*q2*q3*q4**2*q5*q6*q7**2 - 90*p**6*q1*q2*q3*q4**2*q5*q6*q7 + 360*p**6*q1*q2*q3*q4**2*q5*q6 + 10*p**6*q1*q2*q3*q4*q5**4*q6 + 10*p**6*q1*q2*q3*q4*q5**3*q6**2 + 10*p**6*q1*q2*q3*q4*q5**3*q6*q7 - 90*p**6*q1*q2*q3*q4*q5**3*q6 + 10*p**6*q1*q2*q3*q4*q5**2*q6**3 + 10*p**6*q1*q2*q3*q4*q5**2*q6**2*q7 - 90*p**6*q1*q2*q3*q4*q5**2*q6**2 + 10*p**6*q1*q2*q3*q4*q5**2*q6*q7**2 - 90*p**6*q1*q2*q3*q4*q5**2*q6*q7 + 360*p**6*q1*q2*q3*q4*q5**2*q6 + 10*p**6*q1*q2*q3*q4*q5*q6**4 + 10*p**6*q1*q2*q3*q4*q5*q6**3*q7 - 90*p**6*q1*q2*q3*q4*q5*q6**3 + 10*p**6*q1*q2*q3*q4*q5*q6**2*q7**2 - 90*p**6*q1*q2*q3*q4*q5*q6**2*q7 + 360*p**6*q1*q2*q3*q4*q5*q6**2 + 10*p**6*q1*q2*q3*q4*q5*q6*q7**3 - 90*p**6*q1*q2*q3*q4*q5*q6*q7**2 + 360*p**6*q1*q2*q3*q4*q5*q6*q7 - 840*p**6*q1*q2*q3*q4*q5*q6 - 45*p**6*q2**4*q3*q4*q5*q6 - 45*p**6*q2**3*q3**2*q4*q5*q6 - 45*p**6*q2**3*q3*q4**2*q5*q6 - 45*p**6*q2**3*q3*q4*q5**2*q6 - 45*p**6*q2**3*q3*q4*q5*q6**2 - 45*p**6*q2**3*q3*q4*q5*q6*q7 + 360*p**6*q2**3*q3*q4*q5*q6 - 45*p**6*q2**2*q3**3*q4*q5*q6 - 45*p**6*q2**2*q3**2*q4**2*q5*q6 - 45*p**6*q2**2*q3**2*q4*q5**2*q6 - 45*p**6*q2**2*q3**2*q4*q5*q6**2 - 45*p**6*q2**2*q3**2*q4*q5*q6*q7 + 360*p**6*q2**2*q3**2*q4*q5*q6 - 45*p**6*q2**2*q3*q4**3*q5*q6 - 45*p**6*q2**2*q3*q4**2*q5**2*q6 - 45*p**6*q2**2*q3*q4**2*q5*q6**2 - 45*p**6*q2**2*q3*q4**2*q5*q6*q7 + 360*p**6*q2**2*q3*q4**2*q5*q6 - 45*p**6*q2**2*q3*q4*q5**3*q6 - 45*p**6*q2**2*q3*q4*q5**2*q6**2 - 45*p**6*q2**2*q3*q4*q5**2*q6*q7 + 360*p**6*q2**2*q3*q4*q5**2*q6 - 45*p**6*q2**2*q3*q4*q5*q6**3 - 45*p**6*q2**2*q3*q4*q5*q6**2*q7 + 360*p**6*q2**2*q3*q4*q5*q6**2 - 45*p**6*q2**2*q3*q4*q5*q6*q7**2 + 360*p**6*q2**2*q3*q4*q5*q6*q7 - 1260*p**6*q2**2*q3*q4*q5*q6 - 45*p**6*q2*q3**4*q4*q5*q6 - 45*p**6*q2*q3**3*q4**2*q5*q6 - 45*p**6*q2*q3**3*q4*q5**2*q6 - 45*p**6*q2*q3**3*q4*q5*q6**2 - 45*p**6*q2*q3**3*q4*q5*q6*q7 + 360*p**6*q2*q3**3*q4*q5*q6 - 45*p**6*q2*q3**2*q4**3*q5*q6 - 45*p**6*q2*q3**2*q4**2*q5**2*q6 - 45*p**6*q2*q3**2*q4**2*q5*q6**2 - 45*p**6*q2*q3**2*q4**2*q5*q6*q7 + 360*p**6*q2*q3**2*q4**2*q5*q6 - 45*p**6*q2*q3**2*q4*q5**3*q6 - 45*p**6*q2*q3**2*q4*q5**2*q6**2 - 45*p**6*q2*q3**2*q4*q5**2*q6*q7 + 360*p**6*q2*q3**2*q4*q5**2*q6 - 45*p**6*q2*q3**2*q4*q5*q6**3 - 45*p**6*q2*q3**2*q4*q5*q6**2*q7 + 360*p**6*q2*q3**2*q4*q5*q6**2 - 45*p**6*q2*q3**2*q4*q5*q6*q7**2 + 360*p**6*q2*q3**2*q4*q5*q6*q7 - 1260*p**6*q2*q3**2*q4*q5*q6 - 45*p**6*q2*q3*q4**4*q5*q6 - 45*p**6*q2*q3*q4**3*q5**2*q6 - 45*p**6*q2*q3*q4**3*q5*q6**2 - 45*p**6*q2*q3*q4**3*q5*q6*q7 + 360*p**6*q2*q3*q4**3*q5*q6 - 45*p**6*q2*q3*q4**2*q5**3*q6 - 45*p**6*q2*q3*q4**2*q5**2*q6**2 - 45*p**6*q2*q3*q4**2*q5**2*q6*q7 + 360*p**6*q2*q3*q4**2*q5**2*q6 - 45*p**6*q2*q3*q4**2*q5*q6**3 - 45*p**6*q2*q3*q4**2*q5*q6**2*q7 + 360*p**6*q2*q3*q4**2*q5*q6**2 - 45*p**6*q2*q3*q4**2*q5*q6*q7**2 + 360*p**6*q2*q3*q4**2*q5*q6*q7 - 1260*p**6*q2*q3*q4**2*q5*q6 - 45*p**6*q2*q3*q4*q5**4*q6 - 45*p**6*q2*q3*q4*q5**3*q6**2 - 45*p**6*q2*q3*q4*q5**3*q6*q7 + 360*p**6*q2*q3*q4*q5**3*q6 - 45*p**6*q2*q3*q4*q5**2*q6**3 - 45*p**6*q2*q3*q4*q5**2*q6**2*q7 + 360*p**6*q2*q3*q4*q5**2*q6**2 - 45*p**6*q2*q3*q4*q5**2*q6*q7**2 + 360*p**6*q2*q3*q4*q5**2*q6*q7 - 1260*p**6*q2*q3*q4*q5**2*q6 - 45*p**6*q2*q3*q4*q5*q6**4 - 45*p**6*q2*q3*q4*q5*q6**3*q7 + 360*p**6*q2*q3*q4*q5*q6**3 - 45*p**6*q2*q3*q4*q5*q6**2*q7**2 + 360*p**6*q2*q3*q4*q5*q6**2*q7 - 1260*p**6*q2*q3*q4*q5*q6**2 - 45*p**6*q2*q3*q4*q5*q6*q7**3 + 360*p**6*q2*q3*q4*q5*q6*q7**2 - 1260*p**6*q2*q3*q4*q5*q6*q7 + 2520*p**6*q2*q3*q4*q5*q6 + 120*p**6*q3**4*q4*q5*q6 + 120*p**6*q3**3*q4**2*q5*q6 + 120*p**6*q3**3*q4*q5**2*q6 + 120*p**6*q3**3*q4*q5*q6**2 + 120*p**6*q3**3*q4*q5*q6*q7 - 840*p**6*q3**3*q4*q5*q6 + 120*p**6*q3**2*q4**3*q5*q6 + 120*p**6*q3**2*q4**2*q5**2*q6 + 120*p**6*q3**2*q4**2*q5*q6**2 + 120*p**6*q3**2*q4**2*q5*q6*q7 - 840*p**6*q3**2*q4**2*q5*q6 + 120*p**6*q3**2*q4*q5**3*q6 + 120*p**6*q3**2*q4*q5**2*q6**2 + 120*p**6*q3**2*q4*q5**2*q6*q7 - 840*p**6*q3**2*q4*q5**2*q6 + 120*p**6*q3**2*q4*q5*q6**3 + 120*p**6*q3**2*q4*q5*q6**2*q7 - 840*p**6*q3**2*q4*q5*q6**2 + 120*p**6*q3**2*q4*q5*q6*q7**2 - 840*p**6*q3**2*q4*q5*q6*q7 + 2520*p**6*q3**2*q4*q5*q6 + 120*p**6*q3*q4**4*q5*q6 + 120*p**6*q3*q4**3*q5**2*q6 + 120*p**6*q3*q4**3*q5*q6**2 + 120*p**6*q3*q4**3*q5*q6*q7 - 840*p**6*q3*q4**3*q5*q6 + 120*p**6*q3*q4**2*q5**3*q6 + 120*p**6*q3*q4**2*q5**2*q6**2 + 120*p**6*q3*q4**2*q5**2*q6*q7 - 840*p**6*q3*q4**2*q5**2*q6 + 120*p**6*q3*q4**2*q5*q6**3 + 120*p**6*q3*q4**2*q5*q6**2*q7 - 840*p**6*q3*q4**2*q5*q6**2 + 120*p**6*q3*q4**2*q5*q6*q7**2 - 840*p**6*q3*q4**2*q5*q6*q7 + 2520*p**6*q3*q4**2*q5*q6 + 120*p**6*q3*q4*q5**4*q6 + 120*p**6*q3*q4*q5**3*q6**2 + 120*p**6*q3*q4*q5**3*q6*q7 - 840*p**6*q3*q4*q5**3*q6 + 120*p**6*q3*q4*q5**2*q6**3 + 120*p**6*q3*q4*q5**2*q6**2*q7 - 840*p**6*q3*q4*q5**2*q6**2 + 120*p**6*q3*q4*q5**2*q6*q7**2 - 840*p**6*q3*q4*q5**2*q6*q7 + 2520*p**6*q3*q4*q5**2*q6 + 120*p**6*q3*q4*q5*q6**4 + 120*p**6*q3*q4*q5*q6**3*q7 - 840*p**6*q3*q4*q5*q6**3 + 120*p**6*q3*q4*q5*q6**2*q7**2 - 840*p**6*q3*q4*q5*q6**2*q7 + 2520*p**6*q3*q4*q5*q6**2 + 120*p**6*q3*q4*q5*q6*q7**3 - 840*p**6*q3*q4*q5*q6*q7**2 + 2520*p**6*q3*q4*q5*q6*q7 - 4200*p**6*q3*q4*q5*q6 - 210*p**6*q4**4*q5*q6 - 210*p**6*q4**3*q5**2*q6 - 210*p**6*q4**3*q5*q6**2 - 210*p**6*q4**3*q5*q6*q7 + 1260*p**6*q4**3*q5*q6 - 210*p**6*q4**2*q5**3*q6 - 210*p**6*q4**2*q5**2*q6**2 - 210*p**6*q4**2*q5**2*q6*q7 + 1260*p**6*q4**2*q5**2*q6 - 210*p**6*q4**2*q5*q6**3 - 210*p**6*q4**2*q5*q6**2*q7 + 1260*p**6*q4**2*q5*q6**2 - 210*p**6*q4**2*q5*q6*q7**2 + 1260*p**6*q4**2*q5*q6*q7 - 3150*p**6*q4**2*q5*q6 - 210*p**6*q4*q5**4*q6 - 210*p**6*q4*q5**3*q6**2 - 210*p**6*q4*q5**3*q6*q7 + 1260*p**6*q4*q5**3*q6 - 210*p**6*q4*q5**2*q6**3 - 210*p**6*q4*q5**2*q6**2*q7 + 1260*p**6*q4*q5**2*q6**2 - 210*p**6*q4*q5**2*q6*q7**2 + 1260*p**6*q4*q5**2*q6*q7 - 3150*p**6*q4*q5**2*q6 - 210*p**6*q4*q5*q6**4 - 210*p**6*q4*q5*q6**3*q7 + 1260*p**6*q4*q5*q6**3 - 210*p**6*q4*q5*q6**2*q7**2 + 1260*p**6*q4*q5*q6**2*q7 - 3150*p**6*q4*q5*q6**2 - 210*p**6*q4*q5*q6*q7**3 + 1260*p**6*q4*q5*q6*q7**2 - 3150*p**6*q4*q5*q6*q7 + 4200*p**6*q4*q5*q6 + 252*p**6*q5**4*q6 + 252*p**6*q5**3*q6**2 + 252*p**6*q5**3*q6*q7 - 1260*p**6*q5**3*q6 + 252*p**6*q5**2*q6**3 + 252*p**6*q5**2*q6**2*q7 - 1260*p**6*q5**2*q6**2 + 252*p**6*q5**2*q6*q7**2 - 1260*p**6*q5**2*q6*q7 + 2520*p**6*q5**2*q6 + 252*p**6*q5*q6**4 + 252*p**6*q5*q6**3*q7 - 1260*p**6*q5*q6**3 + 252*p**6*q5*q6**2*q7**2 - 1260*p**6*q5*q6**2*q7 + 2520*p**6*q5*q6**2 + 252*p**6*q5*q6*q7**3 - 1260*p**6*q5*q6*q7**2 + 2520*p**6*q5*q6*q7 - 2520*p**6*q5*q6 - 210*p**6*q6**4 - 210*p**6*q6**3*q7 + 840*p**6*q6**3 - 210*p**6*q6**2*q7**2 + 840*p**6*q6**2*q7 - 1260*p**6*q6**2 - 210*p**6*q6*q7**3 + 840*p**6*q6*q7**2 - 1260*p**6*q6*q7 + 840*p**6*q6 + 120*p**6*q7**3 - 360*p**6*q7**2 + 360*p**6*q7 - 120*p**6 - 60*p**5*q1**4*q2*q3*q4*q5*q6 - 60*p**5*q1**3*q2**2*q3*q4*q5*q6 - 60*p**5*q1**3*q2*q3**2*q4*q5*q6 - 60*p**5*q1**3*q2*q3*q4**2*q5*q6 - 60*p**5*q1**3*q2*q3*q4*q5**2*q6 - 60*p**5*q1**3*q2*q3*q4*q5*q6**2 - 60*p**5*q1**3*q2*q3*q4*q5*q6*q7 + 540*p**5*q1**3*q2*q3*q4*q5*q6 - 60*p**5*q1**2*q2**3*q3*q4*q5*q6 - 60*p**5*q1**2*q2**2*q3**2*q4*q5*q6 - 60*p**5*q1**2*q2**2*q3*q4**2*q5*q6 - 60*p**5*q1**2*q2**2*q3*q4*q5**2*q6 - 60*p**5*q1**2*q2**2*q3*q4*q5*q6**2 - 60*p**5*q1**2*q2**2*q3*q4*q5*q6*q7 + 540*p**5*q1**2*q2**2*q3*q4*q5*q6 - 60*p**5*q1**2*q2*q3**3*q4*q5*q6 - 60*p**5*q1**2*q2*q3**2*q4**2*q5*q6 - 60*p**5*q1**2*q2*q3**2*q4*q5**2*q6 - 60*p**5*q1**2*q2*q3**2*q4*q5*q6**2 - 60*p**5*q1**2*q2*q3**2*q4*q5*q6*q7 + 540*p**5*q1**2*q2*q3**2*q4*q5*q6 - 60*p**5*q1**2*q2*q3*q4**3*q5*q6 - 60*p**5*q1**2*q2*q3*q4**2*q5**2*q6 - 60*p**5*q1**2*q2*q3*q4**2*q5*q6**2 - 60*p**5*q1**2*q2*q3*q4**2*q5*q6*q7 + 540*p**5*q1**2*q2*q3*q4**2*q5*q6 - 60*p**5*q1**2*q2*q3*q4*q5**3*q6 - 60*p**5*q1**2*q2*q3*q4*q5**2*q6**2 - 60*p**5*q1**2*q2*q3*q4*q5**2*q6*q7 + 540*p**5*q1**2*q2*q3*q4*q5**2*q6 - 60*p**5*q1**2*q2*q3*q4*q5*q6**3 - 60*p**5*q1**2*q2*q3*q4*q5*q6**2*q7 + 540*p**5*q1**2*q2*q3*q4*q5*q6**2 - 60*p**5*q1**2*q2*q3*q4*q5*q6*q7**2 + 540*p**5*q1**2*q2*q3*q4*q5*q6*q7 - 2160*p**5*q1**2*q2*q3*q4*q5*q6 - 60*p**5*q1*q2**4*q3*q4*q5*q6 - 60*p**5*q1*q2**3*q3**2*q4*q5*q6 - 60*p**5*q1*q2**3*q3*q4**2*q5*q6 - 60*p**5*q1*q2**3*q3*q4*q5**2*q6 - 60*p**5*q1*q2**3*q3*q4*q5*q6**2 - 60*p**5*q1*q2**3*q3*q4*q5*q6*q7 + 540*p**5*q1*q2**3*q3*q4*q5*q6 - 60*p**5*q1*q2**2*q3**3*q4*q5*q6 - 60*p**5*q1*q2**2*q3**2*q4**2*q5*q6 - 60*p**5*q1*q2**2*q3**2*q4*q5**2*q6 - 60*p**5*q1*q2**2*q3**2*q4*q5*q6**2 - 60*p**5*q1*q2**2*q3**2*q4*q5*q6*q7 + 540*p**5*q1*q2**2*q3**2*q4*q5*q6 - 60*p**5*q1*q2**2*q3*q4**3*q5*q6 - 60*p**5*q1*q2**2*q3*q4**2*q5**2*q6 - 60*p**5*q1*q2**2*q3*q4**2*q5*q6**2 - 60*p**5*q1*q2**2*q3*q4**2*q5*q6*q7 + 540*p**5*q1*q2**2*q3*q4**2*q5*q6 - 60*p**5*q1*q2**2*q3*q4*q5**3*q6 - 60*p**5*q1*q2**2*q3*q4*q5**2*q6**2 - 60*p**5*q1*q2**2*q3*q4*q5**2*q6*q7 + 540*p**5*q1*q2**2*q3*q4*q5**2*q6 - 60*p**5*q1*q2**2*q3*q4*q5*q6**3 - 60*p**5*q1*q2**2*q3*q4*q5*q6**2*q7 + 540*p**5*q1*q2**2*q3*q4*q5*q6**2 - 60*p**5*q1*q2**2*q3*q4*q5*q6*q7**2 + 540*p**5*q1*q2**2*q3*q4*q5*q6*q7 - 2160*p**5*q1*q2**2*q3*q4*q5*q6 - 60*p**5*q1*q2*q3**4*q4*q5*q6 - 60*p**5*q1*q2*q3**3*q4**2*q5*q6 - 60*p**5*q1*q2*q3**3*q4*q5**2*q6 - 60*p**5*q1*q2*q3**3*q4*q5*q6**2 - 60*p**5*q1*q2*q3**3*q4*q5*q6*q7 + 540*p**5*q1*q2*q3**3*q4*q5*q6 - 60*p**5*q1*q2*q3**2*q4**3*q5*q6 - 60*p**5*q1*q2*q3**2*q4**2*q5**2*q6 - 60*p**5*q1*q2*q3**2*q4**2*q5*q6**2 - 60*p**5*q1*q2*q3**2*q4**2*q5*q6*q7 + 540*p**5*q1*q2*q3**2*q4**2*q5*q6 - 60*p**5*q1*q2*q3**2*q4*q5**3*q6 - 60*p**5*q1*q2*q3**2*q4*q5**2*q6**2 - 60*p**5*q1*q2*q3**2*q4*q5**2*q6*q7 + 540*p**5*q1*q2*q3**2*q4*q5**2*q6 - 60*p**5*q1*q2*q3**2*q4*q5*q6**3 - 60*p**5*q1*q2*q3**2*q4*q5*q6**2*q7 + 540*p**5*q1*q2*q3**2*q4*q5*q6**2 - 60*p**5*q1*q2*q3**2*q4*q5*q6*q7**2 + 540*p**5*q1*q2*q3**2*q4*q5*q6*q7 - 2160*p**5*q1*q2*q3**2*q4*q5*q6 - 60*p**5*q1*q2*q3*q4**4*q5*q6 - 60*p**5*q1*q2*q3*q4**3*q5**2*q6 - 60*p**5*q1*q2*q3*q4**3*q5*q6**2 - 60*p**5*q1*q2*q3*q4**3*q5*q6*q7 + 540*p**5*q1*q2*q3*q4**3*q5*q6 - 60*p**5*q1*q2*q3*q4**2*q5**3*q6 - 60*p**5*q1*q2*q3*q4**2*q5**2*q6**2 - 60*p**5*q1*q2*q3*q4**2*q5**2*q6*q7 + 540*p**5*q1*q2*q3*q4**2*q5**2*q6 - 60*p**5*q1*q2*q3*q4**2*q5*q6**3 - 60*p**5*q1*q2*q3*q4**2*q5*q6**2*q7 + 540*p**5*q1*q2*q3*q4**2*q5*q6**2 - 60*p**5*q1*q2*q3*q4**2*q5*q6*q7**2 + 540*p**5*q1*q2*q3*q4**2*q5*q6*q7 - 2160*p**5*q1*q2*q3*q4**2*q5*q6 - 60*p**5*q1*q2*q3*q4*q5**4*q6 - 60*p**5*q1*q2*q3*q4*q5**3*q6**2 - 60*p**5*q1*q2*q3*q4*q5**3*q6*q7 + 540*p**5*q1*q2*q3*q4*q5**3*q6 - 60*p**5*q1*q2*q3*q4*q5**2*q6**3 - 60*p**5*q1*q2*q3*q4*q5**2*q6**2*q7 + 540*p**5*q1*q2*q3*q4*q5**2*q6**2 - 60*p**5*q1*q2*q3*q4*q5**2*q6*q7**2 + 540*p**5*q1*q2*q3*q4*q5**2*q6*q7 - 2160*p**5*q1*q2*q3*q4*q5**2*q6 - 60*p**5*q1*q2*q3*q4*q5*q6**4 - 60*p**5*q1*q2*q3*q4*q5*q6**3*q7 + 540*p**5*q1*q2*q3*q4*q5*q6**3 - 60*p**5*q1*q2*q3*q4*q5*q6**2*q7**2 + 540*p**5*q1*q2*q3*q4*q5*q6**2*q7 - 2160*p**5*q1*q2*q3*q4*q5*q6**2 - 60*p**5*q1*q2*q3*q4*q5*q6*q7**3 + 540*p**5*q1*q2*q3*q4*q5*q6*q7**2 - 2160*p**5*q1*q2*q3*q4*q5*q6*q7 + 5040*p**5*q1*q2*q3*q4*q5*q6 + 225*p**5*q2**4*q3*q4*q5*q6 + 225*p**5*q2**3*q3**2*q4*q5*q6 + 225*p**5*q2**3*q3*q4**2*q5*q6 + 225*p**5*q2**3*q3*q4*q5**2*q6 + 225*p**5*q2**3*q3*q4*q5*q6**2 + 225*p**5*q2**3*q3*q4*q5*q6*q7 - 1800*p**5*q2**3*q3*q4*q5*q6 + 225*p**5*q2**2*q3**3*q4*q5*q6 + 225*p**5*q2**2*q3**2*q4**2*q5*q6 + 225*p**5*q2**2*q3**2*q4*q5**2*q6 + 225*p**5*q2**2*q3**2*q4*q5*q6**2 + 225*p**5*q2**2*q3**2*q4*q5*q6*q7 - 1800*p**5*q2**2*q3**2*q4*q5*q6 + 225*p**5*q2**2*q3*q4**3*q5*q6 + 225*p**5*q2**2*q3*q4**2*q5**2*q6 + 225*p**5*q2**2*q3*q4**2*q5*q6**2 + 225*p**5*q2**2*q3*q4**2*q5*q6*q7 - 1800*p**5*q2**2*q3*q4**2*q5*q6 + 225*p**5*q2**2*q3*q4*q5**3*q6 + 225*p**5*q2**2*q3*q4*q5**2*q6**2 + 225*p**5*q2**2*q3*q4*q5**2*q6*q7 - 1800*p**5*q2**2*q3*q4*q5**2*q6 + 225*p**5*q2**2*q3*q4*q5*q6**3 + 225*p**5*q2**2*q3*q4*q5*q6**2*q7 - 1800*p**5*q2**2*q3*q4*q5*q6**2 + 225*p**5*q2**2*q3*q4*q5*q6*q7**2 - 1800*p**5*q2**2*q3*q4*q5*q6*q7 + 6300*p**5*q2**2*q3*q4*q5*q6 + 225*p**5*q2*q3**4*q4*q5*q6 + 225*p**5*q2*q3**3*q4**2*q5*q6 + 225*p**5*q2*q3**3*q4*q5**2*q6 + 225*p**5*q2*q3**3*q4*q5*q6**2 + 225*p**5*q2*q3**3*q4*q5*q6*q7 - 1800*p**5*q2*q3**3*q4*q5*q6 + 225*p**5*q2*q3**2*q4**3*q5*q6 + 225*p**5*q2*q3**2*q4**2*q5**2*q6 + 225*p**5*q2*q3**2*q4**2*q5*q6**2 + 225*p**5*q2*q3**2*q4**2*q5*q6*q7 - 1800*p**5*q2*q3**2*q4**2*q5*q6 + 225*p**5*q2*q3**2*q4*q5**3*q6 + 225*p**5*q2*q3**2*q4*q5**2*q6**2 + 225*p**5*q2*q3**2*q4*q5**2*q6*q7 - 1800*p**5*q2*q3**2*q4*q5**2*q6 + 225*p**5*q2*q3**2*q4*q5*q6**3 + 225*p**5*q2*q3**2*q4*q5*q6**2*q7 - 1800*p**5*q2*q3**2*q4*q5*q6**2 + 225*p**5*q2*q3**2*q4*q5*q6*q7**2 - 1800*p**5*q2*q3**2*q4*q5*q6*q7 + 6300*p**5*q2*q3**2*q4*q5*q6 + 225*p**5*q2*q3*q4**4*q5*q6 + 225*p**5*q2*q3*q4**3*q5**2*q6 + 225*p**5*q2*q3*q4**3*q5*q6**2 + 225*p**5*q2*q3*q4**3*q5*q6*q7 - 1800*p**5*q2*q3*q4**3*q5*q6 + 225*p**5*q2*q3*q4**2*q5**3*q6 + 225*p**5*q2*q3*q4**2*q5**2*q6**2 + 225*p**5*q2*q3*q4**2*q5**2*q6*q7 - 1800*p**5*q2*q3*q4**2*q5**2*q6 + 225*p**5*q2*q3*q4**2*q5*q6**3 + 225*p**5*q2*q3*q4**2*q5*q6**2*q7 - 1800*p**5*q2*q3*q4**2*q5*q6**2 + 225*p**5*q2*q3*q4**2*q5*q6*q7**2 - 1800*p**5*q2*q3*q4**2*q5*q6*q7 + 6300*p**5*q2*q3*q4**2*q5*q6 + 225*p**5*q2*q3*q4*q5**4*q6 + 225*p**5*q2*q3*q4*q5**3*q6**2 + 225*p**5*q2*q3*q4*q5**3*q6*q7 - 1800*p**5*q2*q3*q4*q5**3*q6 + 225*p**5*q2*q3*q4*q5**2*q6**3 + 225*p**5*q2*q3*q4*q5**2*q6**2*q7 - 1800*p**5*q2*q3*q4*q5**2*q6**2 + 225*p**5*q2*q3*q4*q5**2*q6*q7**2 - 1800*p**5*q2*q3*q4*q5**2*q6*q7 + 6300*p**5*q2*q3*q4*q5**2*q6 + 225*p**5*q2*q3*q4*q5*q6**4 + 225*p**5*q2*q3*q4*q5*q6**3*q7 - 1800*p**5*q2*q3*q4*q5*q6**3 + 225*p**5*q2*q3*q4*q5*q6**2*q7**2 - 1800*p**5*q2*q3*q4*q5*q6**2*q7 + 6300*p**5*q2*q3*q4*q5*q6**2 + 225*p**5*q2*q3*q4*q5*q6*q7**3 - 1800*p**5*q2*q3*q4*q5*q6*q7**2 + 6300*p**5*q2*q3*q4*q5*q6*q7 - 12600*p**5*q2*q3*q4*q5*q6 - 480*p**5*q3**4*q4*q5*q6 - 480*p**5*q3**3*q4**2*q5*q6 - 480*p**5*q3**3*q4*q5**2*q6 - 480*p**5*q3**3*q4*q5*q6**2 - 480*p**5*q3**3*q4*q5*q6*q7 + 3360*p**5*q3**3*q4*q5*q6 - 480*p**5*q3**2*q4**3*q5*q6 - 480*p**5*q3**2*q4**2*q5**2*q6 - 480*p**5*q3**2*q4**2*q5*q6**2 - 480*p**5*q3**2*q4**2*q5*q6*q7 + 3360*p**5*q3**2*q4**2*q5*q6 - 480*p**5*q3**2*q4*q5**3*q6 - 480*p**5*q3**2*q4*q5**2*q6**2 - 480*p**5*q3**2*q4*q5**2*q6*q7 + 3360*p**5*q3**2*q4*q5**2*q6 - 480*p**5*q3**2*q4*q5*q6**3 - 480*p**5*q3**2*q4*q5*q6**2*q7 + 3360*p**5*q3**2*q4*q5*q6**2 - 480*p**5*q3**2*q4*q5*q6*q7**2 + 3360*p**5*q3**2*q4*q5*q6*q7 - 10080*p**5*q3**2*q4*q5*q6 - 480*p**5*q3*q4**4*q5*q6 - 480*p**5*q3*q4**3*q5**2*q6 - 480*p**5*q3*q4**3*q5*q6**2 - 480*p**5*q3*q4**3*q5*q6*q7 + 3360*p**5*q3*q4**3*q5*q6 - 480*p**5*q3*q4**2*q5**3*q6 - 480*p**5*q3*q4**2*q5**2*q6**2 - 480*p**5*q3*q4**2*q5**2*q6*q7 + 3360*p**5*q3*q4**2*q5**2*q6 - 480*p**5*q3*q4**2*q5*q6**3 - 480*p**5*q3*q4**2*q5*q6**2*q7 + 3360*p**5*q3*q4**2*q5*q6**2 - 480*p**5*q3*q4**2*q5*q6*q7**2 + 3360*p**5*q3*q4**2*q5*q6*q7 - 10080*p**5*q3*q4**2*q5*q6 - 480*p**5*q3*q4*q5**4*q6 - 480*p**5*q3*q4*q5**3*q6**2 - 480*p**5*q3*q4*q5**3*q6*q7 + 3360*p**5*q3*q4*q5**3*q6 - 480*p**5*q3*q4*q5**2*q6**3 - 480*p**5*q3*q4*q5**2*q6**2*q7 + 3360*p**5*q3*q4*q5**2*q6**2 - 480*p**5*q3*q4*q5**2*q6*q7**2 + 3360*p**5*q3*q4*q5**2*q6*q7 - 10080*p**5*q3*q4*q5**2*q6 - 480*p**5*q3*q4*q5*q6**4 - 480*p**5*q3*q4*q5*q6**3*q7 + 3360*p**5*q3*q4*q5*q6**3 - 480*p**5*q3*q4*q5*q6**2*q7**2 + 3360*p**5*q3*q4*q5*q6**2*q7 - 10080*p**5*q3*q4*q5*q6**2 - 480*p**5*q3*q4*q5*q6*q7**3 + 3360*p**5*q3*q4*q5*q6*q7**2 - 10080*p**5*q3*q4*q5*q6*q7 + 16800*p**5*q3*q4*q5*q6 + 630*p**5*q4**4*q5*q6 + 630*p**5*q4**3*q5**2*q6 + 630*p**5*q4**3*q5*q6**2 + 630*p**5*q4**3*q5*q6*q7 - 3780*p**5*q4**3*q5*q6 + 630*p**5*q4**2*q5**3*q6 + 630*p**5*q4**2*q5**2*q6**2 + 630*p**5*q4**2*q5**2*q6*q7 - 3780*p**5*q4**2*q5**2*q6 + 630*p**5*q4**2*q5*q6**3 + 630*p**5*q4**2*q5*q6**2*q7 - 3780*p**5*q4**2*q5*q6**2 + 630*p**5*q4**2*q5*q6*q7**2 - 3780*p**5*q4**2*q5*q6*q7 + 9450*p**5*q4**2*q5*q6 + 630*p**5*q4*q5**4*q6 + 630*p**5*q4*q5**3*q6**2 + 630*p**5*q4*q5**3*q6*q7 - 3780*p**5*q4*q5**3*q6 + 630*p**5*q4*q5**2*q6**3 + 630*p**5*q4*q5**2*q6**2*q7 - 3780*p**5*q4*q5**2*q6**2 + 630*p**5*q4*q5**2*q6*q7**2 - 3780*p**5*q4*q5**2*q6*q7 + 9450*p**5*q4*q5**2*q6 + 630*p**5*q4*q5*q6**4 + 630*p**5*q4*q5*q6**3*q7 - 3780*p**5*q4*q5*q6**3 + 630*p**5*q4*q5*q6**2*q7**2 - 3780*p**5*q4*q5*q6**2*q7 + 9450*p**5*q4*q5*q6**2 + 630*p**5*q4*q5*q6*q7**3 - 3780*p**5*q4*q5*q6*q7**2 + 9450*p**5*q4*q5*q6*q7 - 12600*p**5*q4*q5*q6 - 504*p**5*q5**4*q6 - 504*p**5*q5**3*q6**2 - 504*p**5*q5**3*q6*q7 + 2520*p**5*q5**3*q6 - 504*p**5*q5**2*q6**3 - 504*p**5*q5**2*q6**2*q7 + 2520*p**5*q5**2*q6**2 - 504*p**5*q5**2*q6*q7**2 + 2520*p**5*q5**2*q6*q7 - 5040*p**5*q5**2*q6 - 504*p**5*q5*q6**4 - 504*p**5*q5*q6**3*q7 + 2520*p**5*q5*q6**3 - 504*p**5*q5*q6**2*q7**2 + 2520*p**5*q5*q6**2*q7 - 5040*p**5*q5*q6**2 - 504*p**5*q5*q6*q7**3 + 2520*p**5*q5*q6*q7**2 - 5040*p**5*q5*q6*q7 + 5040*p**5*q5*q6 + 210*p**5*q6**4 + 210*p**5*q6**3*q7 - 840*p**5*q6**3 + 210*p**5*q6**2*q7**2 - 840*p**5*q6**2*q7 + 1260*p**5*q6**2 + 210*p**5*q6*q7**3 - 840*p**5*q6*q7**2 + 1260*p**5*q6*q7 - 840*p**5*q6 + 150*p**4*q1**4*q2*q3*q4*q5*q6 + 150*p**4*q1**3*q2**2*q3*q4*q5*q6 + 150*p**4*q1**3*q2*q3**2*q4*q5*q6 + 150*p**4*q1**3*q2*q3*q4**2*q5*q6 + 150*p**4*q1**3*q2*q3*q4*q5**2*q6 + 150*p**4*q1**3*q2*q3*q4*q5*q6**2 + 150*p**4*q1**3*q2*q3*q4*q5*q6*q7 - 1350*p**4*q1**3*q2*q3*q4*q5*q6 + 150*p**4*q1**2*q2**3*q3*q4*q5*q6 + 150*p**4*q1**2*q2**2*q3**2*q4*q5*q6 + 150*p**4*q1**2*q2**2*q3*q4**2*q5*q6 + 150*p**4*q1**2*q2**2*q3*q4*q5**2*q6 + 150*p**4*q1**2*q2**2*q3*q4*q5*q6**2 + 150*p**4*q1**2*q2**2*q3*q4*q5*q6*q7 - 1350*p**4*q1**2*q2**2*q3*q4*q5*q6 + 150*p**4*q1**2*q2*q3**3*q4*q5*q6 + 150*p**4*q1**2*q2*q3**2*q4**2*q5*q6 + 150*p**4*q1**2*q2*q3**2*q4*q5**2*q6 + 150*p**4*q1**2*q2*q3**2*q4*q5*q6**2 + 150*p**4*q1**2*q2*q3**2*q4*q5*q6*q7 - 1350*p**4*q1**2*q2*q3**2*q4*q5*q6 + 150*p**4*q1**2*q2*q3*q4**3*q5*q6 + 150*p**4*q1**2*q2*q3*q4**2*q5**2*q6 + 150*p**4*q1**2*q2*q3*q4**2*q5*q6**2 + 150*p**4*q1**2*q2*q3*q4**2*q5*q6*q7 - 1350*p**4*q1**2*q2*q3*q4**2*q5*q6 + 150*p**4*q1**2*q2*q3*q4*q5**3*q6 + 150*p**4*q1**2*q2*q3*q4*q5**2*q6**2 + 150*p**4*q1**2*q2*q3*q4*q5**2*q6*q7 - 1350*p**4*q1**2*q2*q3*q4*q5**2*q6 + 150*p**4*q1**2*q2*q3*q4*q5*q6**3 + 150*p**4*q1**2*q2*q3*q4*q5*q6**2*q7 - 1350*p**4*q1**2*q2*q3*q4*q5*q6**2 + 150*p**4*q1**2*q2*q3*q4*q5*q6*q7**2 - 1350*p**4*q1**2*q2*q3*q4*q5*q6*q7 + 5400*p**4*q1**2*q2*q3*q4*q5*q6 + 150*p**4*q1*q2**4*q3*q4*q5*q6 + 150*p**4*q1*q2**3*q3**2*q4*q5*q6 + 150*p**4*q1*q2**3*q3*q4**2*q5*q6 + 150*p**4*q1*q2**3*q3*q4*q5**2*q6 + 150*p**4*q1*q2**3*q3*q4*q5*q6**2 + 150*p**4*q1*q2**3*q3*q4*q5*q6*q7 - 1350*p**4*q1*q2**3*q3*q4*q5*q6 + 150*p**4*q1*q2**2*q3**3*q4*q5*q6 + 150*p**4*q1*q2**2*q3**2*q4**2*q5*q6 + 150*p**4*q1*q2**2*q3**2*q4*q5**2*q6 + 150*p**4*q1*q2**2*q3**2*q4*q5*q6**2 + 150*p**4*q1*q2**2*q3**2*q4*q5*q6*q7 - 1350*p**4*q1*q2**2*q3**2*q4*q5*q6 + 150*p**4*q1*q2**2*q3*q4**3*q5*q6 + 150*p**4*q1*q2**2*q3*q4**2*q5**2*q6 + 150*p**4*q1*q2**2*q3*q4**2*q5*q6**2 + 150*p**4*q1*q2**2*q3*q4**2*q5*q6*q7 - 1350*p**4*q1*q2**2*q3*q4**2*q5*q6 + 150*p**4*q1*q2**2*q3*q4*q5**3*q6 + 150*p**4*q1*q2**2*q3*q4*q5**2*q6**2 + 150*p**4*q1*q2**2*q3*q4*q5**2*q6*q7 - 1350*p**4*q1*q2**2*q3*q4*q5**2*q6 + 150*p**4*q1*q2**2*q3*q4*q5*q6**3 + 150*p**4*q1*q2**2*q3*q4*q5*q6**2*q7 - 1350*p**4*q1*q2**2*q3*q4*q5*q6**2 + 150*p**4*q1*q2**2*q3*q4*q5*q6*q7**2 - 1350*p**4*q1*q2**2*q3*q4*q5*q6*q7 + 5400*p**4*q1*q2**2*q3*q4*q5*q6 + 150*p**4*q1*q2*q3**4*q4*q5*q6 + 150*p**4*q1*q2*q3**3*q4**2*q5*q6 + 150*p**4*q1*q2*q3**3*q4*q5**2*q6 + 150*p**4*q1*q2*q3**3*q4*q5*q6**2 + 150*p**4*q1*q2*q3**3*q4*q5*q6*q7 - 1350*p**4*q1*q2*q3**3*q4*q5*q6 + 150*p**4*q1*q2*q3**2*q4**3*q5*q6 + 150*p**4*q1*q2*q3**2*q4**2*q5**2*q6 + 150*p**4*q1*q2*q3**2*q4**2*q5*q6**2 + 150*p**4*q1*q2*q3**2*q4**2*q5*q6*q7 - 1350*p**4*q1*q2*q3**2*q4**2*q5*q6 + 150*p**4*q1*q2*q3**2*q4*q5**3*q6 + 150*p**4*q1*q2*q3**2*q4*q5**2*q6**2 + 150*p**4*q1*q2*q3**2*q4*q5**2*q6*q7 - 1350*p**4*q1*q2*q3**2*q4*q5**2*q6 + 150*p**4*q1*q2*q3**2*q4*q5*q6**3 + 150*p**4*q1*q2*q3**2*q4*q5*q6**2*q7 - 1350*p**4*q1*q2*q3**2*q4*q5*q6**2 + 150*p**4*q1*q2*q3**2*q4*q5*q6*q7**2 - 1350*p**4*q1*q2*q3**2*q4*q5*q6*q7 + 5400*p**4*q1*q2*q3**2*q4*q5*q6 + 150*p**4*q1*q2*q3*q4**4*q5*q6 + 150*p**4*q1*q2*q3*q4**3*q5**2*q6 + 150*p**4*q1*q2*q3*q4**3*q5*q6**2 + 150*p**4*q1*q2*q3*q4**3*q5*q6*q7 - 1350*p**4*q1*q2*q3*q4**3*q5*q6 + 150*p**4*q1*q2*q3*q4**2*q5**3*q6 + 150*p**4*q1*q2*q3*q4**2*q5**2*q6**2 + 150*p**4*q1*q2*q3*q4**2*q5**2*q6*q7 - 1350*p**4*q1*q2*q3*q4**2*q5**2*q6 + 150*p**4*q1*q2*q3*q4**2*q5*q6**3 + 150*p**4*q1*q2*q3*q4**2*q5*q6**2*q7 - 1350*p**4*q1*q2*q3*q4**2*q5*q6**2 + 150*p**4*q1*q2*q3*q4**2*q5*q6*q7**2 - 1350*p**4*q1*q2*q3*q4**2*q5*q6*q7 + 5400*p**4*q1*q2*q3*q4**2*q5*q6 + 150*p**4*q1*q2*q3*q4*q5**4*q6 + 150*p**4*q1*q2*q3*q4*q5**3*q6**2 + 150*p**4*q1*q2*q3*q4*q5**3*q6*q7 - 1350*p**4*q1*q2*q3*q4*q5**3*q6 + 150*p**4*q1*q2*q3*q4*q5**2*q6**3 + 150*p**4*q1*q2*q3*q4*q5**2*q6**2*q7 - 1350*p**4*q1*q2*q3*q4*q5**2*q6**2 + 150*p**4*q1*q2*q3*q4*q5**2*q6*q7**2 - 1350*p**4*q1*q2*q3*q4*q5**2*q6*q7 + 5400*p**4*q1*q2*q3*q4*q5**2*q6 + 150*p**4*q1*q2*q3*q4*q5*q6**4 + 150*p**4*q1*q2*q3*q4*q5*q6**3*q7 - 1350*p**4*q1*q2*q3*q4*q5*q6**3 + 150*p**4*q1*q2*q3*q4*q5*q6**2*q7**2 - 1350*p**4*q1*q2*q3*q4*q5*q6**2*q7 + 5400*p**4*q1*q2*q3*q4*q5*q6**2 + 150*p**4*q1*q2*q3*q4*q5*q6*q7**3 - 1350*p**4*q1*q2*q3*q4*q5*q6*q7**2 + 5400*p**4*q1*q2*q3*q4*q5*q6*q7 - 12600*p**4*q1*q2*q3*q4*q5*q6 - 450*p**4*q2**4*q3*q4*q5*q6 - 450*p**4*q2**3*q3**2*q4*q5*q6 - 450*p**4*q2**3*q3*q4**2*q5*q6 - 450*p**4*q2**3*q3*q4*q5**2*q6 - 450*p**4*q2**3*q3*q4*q5*q6**2 - 450*p**4*q2**3*q3*q4*q5*q6*q7 + 3600*p**4*q2**3*q3*q4*q5*q6 - 450*p**4*q2**2*q3**3*q4*q5*q6 - 450*p**4*q2**2*q3**2*q4**2*q5*q6 - 450*p**4*q2**2*q3**2*q4*q5**2*q6 - 450*p**4*q2**2*q3**2*q4*q5*q6**2 - 450*p**4*q2**2*q3**2*q4*q5*q6*q7 + 3600*p**4*q2**2*q3**2*q4*q5*q6 - 450*p**4*q2**2*q3*q4**3*q5*q6 - 450*p**4*q2**2*q3*q4**2*q5**2*q6 - 450*p**4*q2**2*q3*q4**2*q5*q6**2 - 450*p**4*q2**2*q3*q4**2*q5*q6*q7 + 3600*p**4*q2**2*q3*q4**2*q5*q6 - 450*p**4*q2**2*q3*q4*q5**3*q6 - 450*p**4*q2**2*q3*q4*q5**2*q6**2 - 450*p**4*q2**2*q3*q4*q5**2*q6*q7 + 3600*p**4*q2**2*q3*q4*q5**2*q6 - 450*p**4*q2**2*q3*q4*q5*q6**3 - 450*p**4*q2**2*q3*q4*q5*q6**2*q7 + 3600*p**4*q2**2*q3*q4*q5*q6**2 - 450*p**4*q2**2*q3*q4*q5*q6*q7**2 + 3600*p**4*q2**2*q3*q4*q5*q6*q7 - 12600*p**4*q2**2*q3*q4*q5*q6 - 450*p**4*q2*q3**4*q4*q5*q6 - 450*p**4*q2*q3**3*q4**2*q5*q6 - 450*p**4*q2*q3**3*q4*q5**2*q6 - 450*p**4*q2*q3**3*q4*q5*q6**2 - 450*p**4*q2*q3**3*q4*q5*q6*q7 + 3600*p**4*q2*q3**3*q4*q5*q6 - 450*p**4*q2*q3**2*q4**3*q5*q6 - 450*p**4*q2*q3**2*q4**2*q5**2*q6 - 450*p**4*q2*q3**2*q4**2*q5*q6**2 - 450*p**4*q2*q3**2*q4**2*q5*q6*q7 + 3600*p**4*q2*q3**2*q4**2*q5*q6 - 450*p**4*q2*q3**2*q4*q5**3*q6 - 450*p**4*q2*q3**2*q4*q5**2*q6**2 - 450*p**4*q2*q3**2*q4*q5**2*q6*q7 + 3600*p**4*q2*q3**2*q4*q5**2*q6 - 450*p**4*q2*q3**2*q4*q5*q6**3 - 450*p**4*q2*q3**2*q4*q5*q6**2*q7 + 3600*p**4*q2*q3**2*q4*q5*q6**2 - 450*p**4*q2*q3**2*q4*q5*q6*q7**2 + 3600*p**4*q2*q3**2*q4*q5*q6*q7 - 12600*p**4*q2*q3**2*q4*q5*q6 - 450*p**4*q2*q3*q4**4*q5*q6 - 450*p**4*q2*q3*q4**3*q5**2*q6 - 450*p**4*q2*q3*q4**3*q5*q6**2 - 450*p**4*q2*q3*q4**3*q5*q6*q7 + 3600*p**4*q2*q3*q4**3*q5*q6 - 450*p**4*q2*q3*q4**2*q5**3*q6 - 450*p**4*q2*q3*q4**2*q5**2*q6**2 - 450*p**4*q2*q3*q4**2*q5**2*q6*q7 + 3600*p**4*q2*q3*q4**2*q5**2*q6 - 450*p**4*q2*q3*q4**2*q5*q6**3 - 450*p**4*q2*q3*q4**2*q5*q6**2*q7 + 3600*p**4*q2*q3*q4**2*q5*q6**2 - 450*p**4*q2*q3*q4**2*q5*q6*q7**2 + 3600*p**4*q2*q3*q4**2*q5*q6*q7 - 12600*p**4*q2*q3*q4**2*q5*q6 - 450*p**4*q2*q3*q4*q5**4*q6 - 450*p**4*q2*q3*q4*q5**3*q6**2 - 450*p**4*q2*q3*q4*q5**3*q6*q7 + 3600*p**4*q2*q3*q4*q5**3*q6 - 450*p**4*q2*q3*q4*q5**2*q6**3 - 450*p**4*q2*q3*q4*q5**2*q6**2*q7 + 3600*p**4*q2*q3*q4*q5**2*q6**2 - 450*p**4*q2*q3*q4*q5**2*q6*q7**2 + 3600*p**4*q2*q3*q4*q5**2*q6*q7 - 12600*p**4*q2*q3*q4*q5**2*q6 - 450*p**4*q2*q3*q4*q5*q6**4 - 450*p**4*q2*q3*q4*q5*q6**3*q7 + 3600*p**4*q2*q3*q4*q5*q6**3 - 450*p**4*q2*q3*q4*q5*q6**2*q7**2 + 3600*p**4*q2*q3*q4*q5*q6**2*q7 - 12600*p**4*q2*q3*q4*q5*q6**2 - 450*p**4*q2*q3*q4*q5*q6*q7**3 + 3600*p**4*q2*q3*q4*q5*q6*q7**2 - 12600*p**4*q2*q3*q4*q5*q6*q7 + 25200*p**4*q2*q3*q4*q5*q6 + 720*p**4*q3**4*q4*q5*q6 + 720*p**4*q3**3*q4**2*q5*q6 + 720*p**4*q3**3*q4*q5**2*q6 + 720*p**4*q3**3*q4*q5*q6**2 + 720*p**4*q3**3*q4*q5*q6*q7 - 5040*p**4*q3**3*q4*q5*q6 + 720*p**4*q3**2*q4**3*q5*q6 + 720*p**4*q3**2*q4**2*q5**2*q6 + 720*p**4*q3**2*q4**2*q5*q6**2 + 720*p**4*q3**2*q4**2*q5*q6*q7 - 5040*p**4*q3**2*q4**2*q5*q6 + 720*p**4*q3**2*q4*q5**3*q6 + 720*p**4*q3**2*q4*q5**2*q6**2 + 720*p**4*q3**2*q4*q5**2*q6*q7 - 5040*p**4*q3**2*q4*q5**2*q6 + 720*p**4*q3**2*q4*q5*q6**3 + 720*p**4*q3**2*q4*q5*q6**2*q7 - 5040*p**4*q3**2*q4*q5*q6**2 + 720*p**4*q3**2*q4*q5*q6*q7**2 - 5040*p**4*q3**2*q4*q5*q6*q7 + 15120*p**4*q3**2*q4*q5*q6 + 720*p**4*q3*q4**4*q5*q6 + 720*p**4*q3*q4**3*q5**2*q6 + 720*p**4*q3*q4**3*q5*q6**2 + 720*p**4*q3*q4**3*q5*q6*q7 - 5040*p**4*q3*q4**3*q5*q6 + 720*p**4*q3*q4**2*q5**3*q6 + 720*p**4*q3*q4**2*q5**2*q6**2 + 720*p**4*q3*q4**2*q5**2*q6*q7 - 5040*p**4*q3*q4**2*q5**2*q6 + 720*p**4*q3*q4**2*q5*q6**3 + 720*p**4*q3*q4**2*q5*q6**2*q7 - 5040*p**4*q3*q4**2*q5*q6**2 + 720*p**4*q3*q4**2*q5*q6*q7**2 - 5040*p**4*q3*q4**2*q5*q6*q7 + 15120*p**4*q3*q4**2*q5*q6 + 720*p**4*q3*q4*q5**4*q6 + 720*p**4*q3*q4*q5**3*q6**2 + 720*p**4*q3*q4*q5**3*q6*q7 - 5040*p**4*q3*q4*q5**3*q6 + 720*p**4*q3*q4*q5**2*q6**3 + 720*p**4*q3*q4*q5**2*q6**2*q7 - 5040*p**4*q3*q4*q5**2*q6**2 + 720*p**4*q3*q4*q5**2*q6*q7**2 - 5040*p**4*q3*q4*q5**2*q6*q7 + 15120*p**4*q3*q4*q5**2*q6 + 720*p**4*q3*q4*q5*q6**4 + 720*p**4*q3*q4*q5*q6**3*q7 - 5040*p**4*q3*q4*q5*q6**3 + 720*p**4*q3*q4*q5*q6**2*q7**2 - 5040*p**4*q3*q4*q5*q6**2*q7 + 15120*p**4*q3*q4*q5*q6**2 + 720*p**4*q3*q4*q5*q6*q7**3 - 5040*p**4*q3*q4*q5*q6*q7**2 + 15120*p**4*q3*q4*q5*q6*q7 - 25200*p**4*q3*q4*q5*q6 - 630*p**4*q4**4*q5*q6 - 630*p**4*q4**3*q5**2*q6 - 630*p**4*q4**3*q5*q6**2 - 630*p**4*q4**3*q5*q6*q7 + 3780*p**4*q4**3*q5*q6 - 630*p**4*q4**2*q5**3*q6 - 630*p**4*q4**2*q5**2*q6**2 - 630*p**4*q4**2*q5**2*q6*q7 + 3780*p**4*q4**2*q5**2*q6 - 630*p**4*q4**2*q5*q6**3 - 630*p**4*q4**2*q5*q6**2*q7 + 3780*p**4*q4**2*q5*q6**2 - 630*p**4*q4**2*q5*q6*q7**2 + 3780*p**4*q4**2*q5*q6*q7 - 9450*p**4*q4**2*q5*q6 - 630*p**4*q4*q5**4*q6 - 630*p**4*q4*q5**3*q6**2 - 630*p**4*q4*q5**3*q6*q7 + 3780*p**4*q4*q5**3*q6 - 630*p**4*q4*q5**2*q6**3 - 630*p**4*q4*q5**2*q6**2*q7 + 3780*p**4*q4*q5**2*q6**2 - 630*p**4*q4*q5**2*q6*q7**2 + 3780*p**4*q4*q5**2*q6*q7 - 9450*p**4*q4*q5**2*q6 - 630*p**4*q4*q5*q6**4 - 630*p**4*q4*q5*q6**3*q7 + 3780*p**4*q4*q5*q6**3 - 630*p**4*q4*q5*q6**2*q7**2 + 3780*p**4*q4*q5*q6**2*q7 - 9450*p**4*q4*q5*q6**2 - 630*p**4*q4*q5*q6*q7**3 + 3780*p**4*q4*q5*q6*q7**2 - 9450*p**4*q4*q5*q6*q7 + 12600*p**4*q4*q5*q6 + 252*p**4*q5**4*q6 + 252*p**4*q5**3*q6**2 + 252*p**4*q5**3*q6*q7 - 1260*p**4*q5**3*q6 + 252*p**4*q5**2*q6**3 + 252*p**4*q5**2*q6**2*q7 - 1260*p**4*q5**2*q6**2 + 252*p**4*q5**2*q6*q7**2 - 1260*p**4*q5**2*q6*q7 + 2520*p**4*q5**2*q6 + 252*p**4*q5*q6**4 + 252*p**4*q5*q6**3*q7 - 1260*p**4*q5*q6**3 + 252*p**4*q5*q6**2*q7**2 - 1260*p**4*q5*q6**2*q7 + 2520*p**4*q5*q6**2 + 252*p**4*q5*q6*q7**3 - 1260*p**4*q5*q6*q7**2 + 2520*p**4*q5*q6*q7 - 2520*p**4*q5*q6 - 200*p**3*q1**4*q2*q3*q4*q5*q6 - 200*p**3*q1**3*q2**2*q3*q4*q5*q6 - 200*p**3*q1**3*q2*q3**2*q4*q5*q6 - 200*p**3*q1**3*q2*q3*q4**2*q5*q6 - 200*p**3*q1**3*q2*q3*q4*q5**2*q6 - 200*p**3*q1**3*q2*q3*q4*q5*q6**2 - 200*p**3*q1**3*q2*q3*q4*q5*q6*q7 + 1800*p**3*q1**3*q2*q3*q4*q5*q6 - 200*p**3*q1**2*q2**3*q3*q4*q5*q6 - 200*p**3*q1**2*q2**2*q3**2*q4*q5*q6 - 200*p**3*q1**2*q2**2*q3*q4**2*q5*q6 - 200*p**3*q1**2*q2**2*q3*q4*q5**2*q6 - 200*p**3*q1**2*q2**2*q3*q4*q5*q6**2 - 200*p**3*q1**2*q2**2*q3*q4*q5*q6*q7 + 1800*p**3*q1**2*q2**2*q3*q4*q5*q6 - 200*p**3*q1**2*q2*q3**3*q4*q5*q6 - 200*p**3*q1**2*q2*q3**2*q4**2*q5*q6 - 200*p**3*q1**2*q2*q3**2*q4*q5**2*q6 - 200*p**3*q1**2*q2*q3**2*q4*q5*q6**2 - 200*p**3*q1**2*q2*q3**2*q4*q5*q6*q7 + 1800*p**3*q1**2*q2*q3**2*q4*q5*q6 - 200*p**3*q1**2*q2*q3*q4**3*q5*q6 - 200*p**3*q1**2*q2*q3*q4**2*q5**2*q6 - 200*p**3*q1**2*q2*q3*q4**2*q5*q6**2 - 200*p**3*q1**2*q2*q3*q4**2*q5*q6*q7 + 1800*p**3*q1**2*q2*q3*q4**2*q5*q6 - 200*p**3*q1**2*q2*q3*q4*q5**3*q6 - 200*p**3*q1**2*q2*q3*q4*q5**2*q6**2 - 200*p**3*q1**2*q2*q3*q4*q5**2*q6*q7 + 1800*p**3*q1**2*q2*q3*q4*q5**2*q6 - 200*p**3*q1**2*q2*q3*q4*q5*q6**3 - 200*p**3*q1**2*q2*q3*q4*q5*q6**2*q7 + 1800*p**3*q1**2*q2*q3*q4*q5*q6**2 - 200*p**3*q1**2*q2*q3*q4*q5*q6*q7**2 + 1800*p**3*q1**2*q2*q3*q4*q5*q6*q7 - 7200*p**3*q1**2*q2*q3*q4*q5*q6 - 200*p**3*q1*q2**4*q3*q4*q5*q6 - 200*p**3*q1*q2**3*q3**2*q4*q5*q6 - 200*p**3*q1*q2**3*q3*q4**2*q5*q6 - 200*p**3*q1*q2**3*q3*q4*q5**2*q6 - 200*p**3*q1*q2**3*q3*q4*q5*q6**2 - 200*p**3*q1*q2**3*q3*q4*q5*q6*q7 + 1800*p**3*q1*q2**3*q3*q4*q5*q6 - 200*p**3*q1*q2**2*q3**3*q4*q5*q6 - 200*p**3*q1*q2**2*q3**2*q4**2*q5*q6 - 200*p**3*q1*q2**2*q3**2*q4*q5**2*q6 - 200*p**3*q1*q2**2*q3**2*q4*q5*q6**2 - 200*p**3*q1*q2**2*q3**2*q4*q5*q6*q7 + 1800*p**3*q1*q2**2*q3**2*q4*q5*q6 - 200*p**3*q1*q2**2*q3*q4**3*q5*q6 - 200*p**3*q1*q2**2*q3*q4**2*q5**2*q6 - 200*p**3*q1*q2**2*q3*q4**2*q5*q6**2 - 200*p**3*q1*q2**2*q3*q4**2*q5*q6*q7 + 1800*p**3*q1*q2**2*q3*q4**2*q5*q6 - 200*p**3*q1*q2**2*q3*q4*q5**3*q6 - 200*p**3*q1*q2**2*q3*q4*q5**2*q6**2 - 200*p**3*q1*q2**2*q3*q4*q5**2*q6*q7 + 1800*p**3*q1*q2**2*q3*q4*q5**2*q6 - 200*p**3*q1*q2**2*q3*q4*q5*q6**3 - 200*p**3*q1*q2**2*q3*q4*q5*q6**2*q7 + 1800*p**3*q1*q2**2*q3*q4*q5*q6**2 - 200*p**3*q1*q2**2*q3*q4*q5*q6*q7**2 + 1800*p**3*q1*q2**2*q3*q4*q5*q6*q7 - 7200*p**3*q1*q2**2*q3*q4*q5*q6 - 200*p**3*q1*q2*q3**4*q4*q5*q6 - 200*p**3*q1*q2*q3**3*q4**2*q5*q6 - 200*p**3*q1*q2*q3**3*q4*q5**2*q6 - 200*p**3*q1*q2*q3**3*q4*q5*q6**2 - 200*p**3*q1*q2*q3**3*q4*q5*q6*q7 + 1800*p**3*q1*q2*q3**3*q4*q5*q6 - 200*p**3*q1*q2*q3**2*q4**3*q5*q6 - 200*p**3*q1*q2*q3**2*q4**2*q5**2*q6 - 200*p**3*q1*q2*q3**2*q4**2*q5*q6**2 - 200*p**3*q1*q2*q3**2*q4**2*q5*q6*q7 + 1800*p**3*q1*q2*q3**2*q4**2*q5*q6 - 200*p**3*q1*q2*q3**2*q4*q5**3*q6 - 200*p**3*q1*q2*q3**2*q4*q5**2*q6**2 - 200*p**3*q1*q2*q3**2*q4*q5**2*q6*q7 + 1800*p**3*q1*q2*q3**2*q4*q5**2*q6 - 200*p**3*q1*q2*q3**2*q4*q5*q6**3 - 200*p**3*q1*q2*q3**2*q4*q5*q6**2*q7 + 1800*p**3*q1*q2*q3**2*q4*q5*q6**2 - 200*p**3*q1*q2*q3**2*q4*q5*q6*q7**2 + 1800*p**3*q1*q2*q3**2*q4*q5*q6*q7 - 7200*p**3*q1*q2*q3**2*q4*q5*q6 - 200*p**3*q1*q2*q3*q4**4*q5*q6 - 200*p**3*q1*q2*q3*q4**3*q5**2*q6 - 200*p**3*q1*q2*q3*q4**3*q5*q6**2 - 200*p**3*q1*q2*q3*q4**3*q5*q6*q7 + 1800*p**3*q1*q2*q3*q4**3*q5*q6 - 200*p**3*q1*q2*q3*q4**2*q5**3*q6 - 200*p**3*q1*q2*q3*q4**2*q5**2*q6**2 - 200*p**3*q1*q2*q3*q4**2*q5**2*q6*q7 + 1800*p**3*q1*q2*q3*q4**2*q5**2*q6 - 200*p**3*q1*q2*q3*q4**2*q5*q6**3 - 200*p**3*q1*q2*q3*q4**2*q5*q6**2*q7 + 1800*p**3*q1*q2*q3*q4**2*q5*q6**2 - 200*p**3*q1*q2*q3*q4**2*q5*q6*q7**2 + 1800*p**3*q1*q2*q3*q4**2*q5*q6*q7 - 7200*p**3*q1*q2*q3*q4**2*q5*q6 - 200*p**3*q1*q2*q3*q4*q5**4*q6 - 200*p**3*q1*q2*q3*q4*q5**3*q6**2 - 200*p**3*q1*q2*q3*q4*q5**3*q6*q7 + 1800*p**3*q1*q2*q3*q4*q5**3*q6 - 200*p**3*q1*q2*q3*q4*q5**2*q6**3 - 200*p**3*q1*q2*q3*q4*q5**2*q6**2*q7 + 1800*p**3*q1*q2*q3*q4*q5**2*q6**2 - 200*p**3*q1*q2*q3*q4*q5**2*q6*q7**2 + 1800*p**3*q1*q2*q3*q4*q5**2*q6*q7 - 7200*p**3*q1*q2*q3*q4*q5**2*q6 - 200*p**3*q1*q2*q3*q4*q5*q6**4 - 200*p**3*q1*q2*q3*q4*q5*q6**3*q7 + 1800*p**3*q1*q2*q3*q4*q5*q6**3 - 200*p**3*q1*q2*q3*q4*q5*q6**2*q7**2 + 1800*p**3*q1*q2*q3*q4*q5*q6**2*q7 - 7200*p**3*q1*q2*q3*q4*q5*q6**2 - 200*p**3*q1*q2*q3*q4*q5*q6*q7**3 + 1800*p**3*q1*q2*q3*q4*q5*q6*q7**2 - 7200*p**3*q1*q2*q3*q4*q5*q6*q7 + 16800*p**3*q1*q2*q3*q4*q5*q6 + 450*p**3*q2**4*q3*q4*q5*q6 + 450*p**3*q2**3*q3**2*q4*q5*q6 + 450*p**3*q2**3*q3*q4**2*q5*q6 + 450*p**3*q2**3*q3*q4*q5**2*q6 + 450*p**3*q2**3*q3*q4*q5*q6**2 + 450*p**3*q2**3*q3*q4*q5*q6*q7 - 3600*p**3*q2**3*q3*q4*q5*q6 + 450*p**3*q2**2*q3**3*q4*q5*q6 + 450*p**3*q2**2*q3**2*q4**2*q5*q6 + 450*p**3*q2**2*q3**2*q4*q5**2*q6 + 450*p**3*q2**2*q3**2*q4*q5*q6**2 + 450*p**3*q2**2*q3**2*q4*q5*q6*q7 - 3600*p**3*q2**2*q3**2*q4*q5*q6 + 450*p**3*q2**2*q3*q4**3*q5*q6 + 450*p**3*q2**2*q3*q4**2*q5**2*q6 + 450*p**3*q2**2*q3*q4**2*q5*q6**2 + 450*p**3*q2**2*q3*q4**2*q5*q6*q7 - 3600*p**3*q2**2*q3*q4**2*q5*q6 + 450*p**3*q2**2*q3*q4*q5**3*q6 + 450*p**3*q2**2*q3*q4*q5**2*q6**2 + 450*p**3*q2**2*q3*q4*q5**2*q6*q7 - 3600*p**3*q2**2*q3*q4*q5**2*q6 + 450*p**3*q2**2*q3*q4*q5*q6**3 + 450*p**3*q2**2*q3*q4*q5*q6**2*q7 - 3600*p**3*q2**2*q3*q4*q5*q6**2 + 450*p**3*q2**2*q3*q4*q5*q6*q7**2 - 3600*p**3*q2**2*q3*q4*q5*q6*q7 + 12600*p**3*q2**2*q3*q4*q5*q6 + 450*p**3*q2*q3**4*q4*q5*q6 + 450*p**3*q2*q3**3*q4**2*q5*q6 + 450*p**3*q2*q3**3*q4*q5**2*q6 + 450*p**3*q2*q3**3*q4*q5*q6**2 + 450*p**3*q2*q3**3*q4*q5*q6*q7 - 3600*p**3*q2*q3**3*q4*q5*q6 + 450*p**3*q2*q3**2*q4**3*q5*q6 + 450*p**3*q2*q3**2*q4**2*q5**2*q6 + 450*p**3*q2*q3**2*q4**2*q5*q6**2 + 450*p**3*q2*q3**2*q4**2*q5*q6*q7 - 3600*p**3*q2*q3**2*q4**2*q5*q6 + 450*p**3*q2*q3**2*q4*q5**3*q6 + 450*p**3*q2*q3**2*q4*q5**2*q6**2 + 450*p**3*q2*q3**2*q4*q5**2*q6*q7 - 3600*p**3*q2*q3**2*q4*q5**2*q6 + 450*p**3*q2*q3**2*q4*q5*q6**3 + 450*p**3*q2*q3**2*q4*q5*q6**2*q7 - 3600*p**3*q2*q3**2*q4*q5*q6**2 + 450*p**3*q2*q3**2*q4*q5*q6*q7**2 - 3600*p**3*q2*q3**2*q4*q5*q6*q7 + 12600*p**3*q2*q3**2*q4*q5*q6 + 450*p**3*q2*q3*q4**4*q5*q6 + 450*p**3*q2*q3*q4**3*q5**2*q6 + 450*p**3*q2*q3*q4**3*q5*q6**2 + 450*p**3*q2*q3*q4**3*q5*q6*q7 - 3600*p**3*q2*q3*q4**3*q5*q6 + 450*p**3*q2*q3*q4**2*q5**3*q6 + 450*p**3*q2*q3*q4**2*q5**2*q6**2 + 450*p**3*q2*q3*q4**2*q5**2*q6*q7 - 3600*p**3*q2*q3*q4**2*q5**2*q6 + 450*p**3*q2*q3*q4**2*q5*q6**3 + 450*p**3*q2*q3*q4**2*q5*q6**2*q7 - 3600*p**3*q2*q3*q4**2*q5*q6**2 + 450*p**3*q2*q3*q4**2*q5*q6*q7**2 - 3600*p**3*q2*q3*q4**2*q5*q6*q7 + 12600*p**3*q2*q3*q4**2*q5*q6 + 450*p**3*q2*q3*q4*q5**4*q6 + 450*p**3*q2*q3*q4*q5**3*q6**2 + 450*p**3*q2*q3*q4*q5**3*q6*q7 - 3600*p**3*q2*q3*q4*q5**3*q6 + 450*p**3*q2*q3*q4*q5**2*q6**3 + 450*p**3*q2*q3*q4*q5**2*q6**2*q7 - 3600*p**3*q2*q3*q4*q5**2*q6**2 + 450*p**3*q2*q3*q4*q5**2*q6*q7**2 - 3600*p**3*q2*q3*q4*q5**2*q6*q7 + 12600*p**3*q2*q3*q4*q5**2*q6 + 450*p**3*q2*q3*q4*q5*q6**4 + 450*p**3*q2*q3*q4*q5*q6**3*q7 - 3600*p**3*q2*q3*q4*q5*q6**3 + 450*p**3*q2*q3*q4*q5*q6**2*q7**2 - 3600*p**3*q2*q3*q4*q5*q6**2*q7 + 12600*p**3*q2*q3*q4*q5*q6**2 + 450*p**3*q2*q3*q4*q5*q6*q7**3 - 3600*p**3*q2*q3*q4*q5*q6*q7**2 + 12600*p**3*q2*q3*q4*q5*q6*q7 - 25200*p**3*q2*q3*q4*q5*q6 - 480*p**3*q3**4*q4*q5*q6 - 480*p**3*q3**3*q4**2*q5*q6 - 480*p**3*q3**3*q4*q5**2*q6 - 480*p**3*q3**3*q4*q5*q6**2 - 480*p**3*q3**3*q4*q5*q6*q7 + 3360*p**3*q3**3*q4*q5*q6 - 480*p**3*q3**2*q4**3*q5*q6 - 480*p**3*q3**2*q4**2*q5**2*q6 - 480*p**3*q3**2*q4**2*q5*q6**2 - 480*p**3*q3**2*q4**2*q5*q6*q7 + 3360*p**3*q3**2*q4**2*q5*q6 - 480*p**3*q3**2*q4*q5**3*q6 - 480*p**3*q3**2*q4*q5**2*q6**2 - 480*p**3*q3**2*q4*q5**2*q6*q7 + 3360*p**3*q3**2*q4*q5**2*q6 - 480*p**3*q3**2*q4*q5*q6**3 - 480*p**3*q3**2*q4*q5*q6**2*q7 + 3360*p**3*q3**2*q4*q5*q6**2 - 480*p**3*q3**2*q4*q5*q6*q7**2 + 3360*p**3*q3**2*q4*q5*q6*q7 - 10080*p**3*q3**2*q4*q5*q6 - 480*p**3*q3*q4**4*q5*q6 - 480*p**3*q3*q4**3*q5**2*q6 - 480*p**3*q3*q4**3*q5*q6**2 - 480*p**3*q3*q4**3*q5*q6*q7 + 3360*p**3*q3*q4**3*q5*q6 - 480*p**3*q3*q4**2*q5**3*q6 - 480*p**3*q3*q4**2*q5**2*q6**2 - 480*p**3*q3*q4**2*q5**2*q6*q7 + 3360*p**3*q3*q4**2*q5**2*q6 - 480*p**3*q3*q4**2*q5*q6**3 - 480*p**3*q3*q4**2*q5*q6**2*q7 + 3360*p**3*q3*q4**2*q5*q6**2 - 480*p**3*q3*q4**2*q5*q6*q7**2 + 3360*p**3*q3*q4**2*q5*q6*q7 - 10080*p**3*q3*q4**2*q5*q6 - 480*p**3*q3*q4*q5**4*q6 - 480*p**3*q3*q4*q5**3*q6**2 - 480*p**3*q3*q4*q5**3*q6*q7 + 3360*p**3*q3*q4*q5**3*q6 - 480*p**3*q3*q4*q5**2*q6**3 - 480*p**3*q3*q4*q5**2*q6**2*q7 + 3360*p**3*q3*q4*q5**2*q6**2 - 480*p**3*q3*q4*q5**2*q6*q7**2 + 3360*p**3*q3*q4*q5**2*q6*q7 - 10080*p**3*q3*q4*q5**2*q6 - 480*p**3*q3*q4*q5*q6**4 - 480*p**3*q3*q4*q5*q6**3*q7 + 3360*p**3*q3*q4*q5*q6**3 - 480*p**3*q3*q4*q5*q6**2*q7**2 + 3360*p**3*q3*q4*q5*q6**2*q7 - 10080*p**3*q3*q4*q5*q6**2 - 480*p**3*q3*q4*q5*q6*q7**3 + 3360*p**3*q3*q4*q5*q6*q7**2 - 10080*p**3*q3*q4*q5*q6*q7 + 16800*p**3*q3*q4*q5*q6 + 210*p**3*q4**4*q5*q6 + 210*p**3*q4**3*q5**2*q6 + 210*p**3*q4**3*q5*q6**2 + 210*p**3*q4**3*q5*q6*q7 - 1260*p**3*q4**3*q5*q6 + 210*p**3*q4**2*q5**3*q6 + 210*p**3*q4**2*q5**2*q6**2 + 210*p**3*q4**2*q5**2*q6*q7 - 1260*p**3*q4**2*q5**2*q6 + 210*p**3*q4**2*q5*q6**3 + 210*p**3*q4**2*q5*q6**2*q7 - 1260*p**3*q4**2*q5*q6**2 + 210*p**3*q4**2*q5*q6*q7**2 - 1260*p**3*q4**2*q5*q6*q7 + 3150*p**3*q4**2*q5*q6 + 210*p**3*q4*q5**4*q6 + 210*p**3*q4*q5**3*q6**2 + 210*p**3*q4*q5**3*q6*q7 - 1260*p**3*q4*q5**3*q6 + 210*p**3*q4*q5**2*q6**3 + 210*p**3*q4*q5**2*q6**2*q7 - 1260*p**3*q4*q5**2*q6**2 + 210*p**3*q4*q5**2*q6*q7**2 - 1260*p**3*q4*q5**2*q6*q7 + 3150*p**3*q4*q5**2*q6 + 210*p**3*q4*q5*q6**4 + 210*p**3*q4*q5*q6**3*q7 - 1260*p**3*q4*q5*q6**3 + 210*p**3*q4*q5*q6**2*q7**2 - 1260*p**3*q4*q5*q6**2*q7 + 3150*p**3*q4*q5*q6**2 + 210*p**3*q4*q5*q6*q7**3 - 1260*p**3*q4*q5*q6*q7**2 + 3150*p**3*q4*q5*q6*q7 - 4200*p**3*q4*q5*q6 + 150*p**2*q1**4*q2*q3*q4*q5*q6 + 150*p**2*q1**3*q2**2*q3*q4*q5*q6 + 150*p**2*q1**3*q2*q3**2*q4*q5*q6 + 150*p**2*q1**3*q2*q3*q4**2*q5*q6 + 150*p**2*q1**3*q2*q3*q4*q5**2*q6 + 150*p**2*q1**3*q2*q3*q4*q5*q6**2 + 150*p**2*q1**3*q2*q3*q4*q5*q6*q7 - 1350*p**2*q1**3*q2*q3*q4*q5*q6 + 150*p**2*q1**2*q2**3*q3*q4*q5*q6 + 150*p**2*q1**2*q2**2*q3**2*q4*q5*q6 + 150*p**2*q1**2*q2**2*q3*q4**2*q5*q6 + 150*p**2*q1**2*q2**2*q3*q4*q5**2*q6 + 150*p**2*q1**2*q2**2*q3*q4*q5*q6**2 + 150*p**2*q1**2*q2**2*q3*q4*q5*q6*q7 - 1350*p**2*q1**2*q2**2*q3*q4*q5*q6 + 150*p**2*q1**2*q2*q3**3*q4*q5*q6 + 150*p**2*q1**2*q2*q3**2*q4**2*q5*q6 + 150*p**2*q1**2*q2*q3**2*q4*q5**2*q6 + 150*p**2*q1**2*q2*q3**2*q4*q5*q6**2 + 150*p**2*q1**2*q2*q3**2*q4*q5*q6*q7 - 1350*p**2*q1**2*q2*q3**2*q4*q5*q6 + 150*p**2*q1**2*q2*q3*q4**3*q5*q6 + 150*p**2*q1**2*q2*q3*q4**2*q5**2*q6 + 150*p**2*q1**2*q2*q3*q4**2*q5*q6**2 + 150*p**2*q1**2*q2*q3*q4**2*q5*q6*q7 - 1350*p**2*q1**2*q2*q3*q4**2*q5*q6 + 150*p**2*q1**2*q2*q3*q4*q5**3*q6 + 150*p**2*q1**2*q2*q3*q4*q5**2*q6**2 + 150*p**2*q1**2*q2*q3*q4*q5**2*q6*q7 - 1350*p**2*q1**2*q2*q3*q4*q5**2*q6 + 150*p**2*q1**2*q2*q3*q4*q5*q6**3 + 150*p**2*q1**2*q2*q3*q4*q5*q6**2*q7 - 1350*p**2*q1**2*q2*q3*q4*q5*q6**2 + 150*p**2*q1**2*q2*q3*q4*q5*q6*q7**2 - 1350*p**2*q1**2*q2*q3*q4*q5*q6*q7 + 5400*p**2*q1**2*q2*q3*q4*q5*q6 + 150*p**2*q1*q2**4*q3*q4*q5*q6 + 150*p**2*q1*q2**3*q3**2*q4*q5*q6 + 150*p**2*q1*q2**3*q3*q4**2*q5*q6 + 150*p**2*q1*q2**3*q3*q4*q5**2*q6 + 150*p**2*q1*q2**3*q3*q4*q5*q6**2 + 150*p**2*q1*q2**3*q3*q4*q5*q6*q7 - 1350*p**2*q1*q2**3*q3*q4*q5*q6 + 150*p**2*q1*q2**2*q3**3*q4*q5*q6 + 150*p**2*q1*q2**2*q3**2*q4**2*q5*q6 + 150*p**2*q1*q2**2*q3**2*q4*q5**2*q6 + 150*p**2*q1*q2**2*q3**2*q4*q5*q6**2 + 150*p**2*q1*q2**2*q3**2*q4*q5*q6*q7 - 1350*p**2*q1*q2**2*q3**2*q4*q5*q6 + 150*p**2*q1*q2**2*q3*q4**3*q5*q6 + 150*p**2*q1*q2**2*q3*q4**2*q5**2*q6 + 150*p**2*q1*q2**2*q3*q4**2*q5*q6**2 + 150*p**2*q1*q2**2*q3*q4**2*q5*q6*q7 - 1350*p**2*q1*q2**2*q3*q4**2*q5*q6 + 150*p**2*q1*q2**2*q3*q4*q5**3*q6 + 150*p**2*q1*q2**2*q3*q4*q5**2*q6**2 + 150*p**2*q1*q2**2*q3*q4*q5**2*q6*q7 - 1350*p**2*q1*q2**2*q3*q4*q5**2*q6 + 150*p**2*q1*q2**2*q3*q4*q5*q6**3 + 150*p**2*q1*q2**2*q3*q4*q5*q6**2*q7 - 1350*p**2*q1*q2**2*q3*q4*q5*q6**2 + 150*p**2*q1*q2**2*q3*q4*q5*q6*q7**2 - 1350*p**2*q1*q2**2*q3*q4*q5*q6*q7 + 5400*p**2*q1*q2**2*q3*q4*q5*q6 + 150*p**2*q1*q2*q3**4*q4*q5*q6 + 150*p**2*q1*q2*q3**3*q4**2*q5*q6 + 150*p**2*q1*q2*q3**3*q4*q5**2*q6 + 150*p**2*q1*q2*q3**3*q4*q5*q6**2 + 150*p**2*q1*q2*q3**3*q4*q5*q6*q7 - 1350*p**2*q1*q2*q3**3*q4*q5*q6 + 150*p**2*q1*q2*q3**2*q4**3*q5*q6 + 150*p**2*q1*q2*q3**2*q4**2*q5**2*q6 + 150*p**2*q1*q2*q3**2*q4**2*q5*q6**2 + 150*p**2*q1*q2*q3**2*q4**2*q5*q6*q7 - 1350*p**2*q1*q2*q3**2*q4**2*q5*q6 + 150*p**2*q1*q2*q3**2*q4*q5**3*q6 + 150*p**2*q1*q2*q3**2*q4*q5**2*q6**2 + 150*p**2*q1*q2*q3**2*q4*q5**2*q6*q7 - 1350*p**2*q1*q2*q3**2*q4*q5**2*q6 + 150*p**2*q1*q2*q3**2*q4*q5*q6**3 + 150*p**2*q1*q2*q3**2*q4*q5*q6**2*q7 - 1350*p**2*q1*q2*q3**2*q4*q5*q6**2 + 150*p**2*q1*q2*q3**2*q4*q5*q6*q7**2 - 1350*p**2*q1*q2*q3**2*q4*q5*q6*q7 + 5400*p**2*q1*q2*q3**2*q4*q5*q6 + 150*p**2*q1*q2*q3*q4**4*q5*q6 + 150*p**2*q1*q2*q3*q4**3*q5**2*q6 + 150*p**2*q1*q2*q3*q4**3*q5*q6**2 + 150*p**2*q1*q2*q3*q4**3*q5*q6*q7 - 1350*p**2*q1*q2*q3*q4**3*q5*q6 + 150*p**2*q1*q2*q3*q4**2*q5**3*q6 + 150*p**2*q1*q2*q3*q4**2*q5**2*q6**2 + 150*p**2*q1*q2*q3*q4**2*q5**2*q6*q7 - 1350*p**2*q1*q2*q3*q4**2*q5**2*q6 + 150*p**2*q1*q2*q3*q4**2*q5*q6**3 + 150*p**2*q1*q2*q3*q4**2*q5*q6**2*q7 - 1350*p**2*q1*q2*q3*q4**2*q5*q6**2 + 150*p**2*q1*q2*q3*q4**2*q5*q6*q7**2 - 1350*p**2*q1*q2*q3*q4**2*q5*q6*q7 + 5400*p**2*q1*q2*q3*q4**2*q5*q6 + 150*p**2*q1*q2*q3*q4*q5**4*q6 + 150*p**2*q1*q2*q3*q4*q5**3*q6**2 + 150*p**2*q1*q2*q3*q4*q5**3*q6*q7 - 1350*p**2*q1*q2*q3*q4*q5**3*q6 + 150*p**2*q1*q2*q3*q4*q5**2*q6**3 + 150*p**2*q1*q2*q3*q4*q5**2*q6**2*q7 - 1350*p**2*q1*q2*q3*q4*q5**2*q6**2 + 150*p**2*q1*q2*q3*q4*q5**2*q6*q7**2 - 1350*p**2*q1*q2*q3*q4*q5**2*q6*q7 + 5400*p**2*q1*q2*q3*q4*q5**2*q6 + 150*p**2*q1*q2*q3*q4*q5*q6**4 + 150*p**2*q1*q2*q3*q4*q5*q6**3*q7 - 1350*p**2*q1*q2*q3*q4*q5*q6**3 + 150*p**2*q1*q2*q3*q4*q5*q6**2*q7**2 - 1350*p**2*q1*q2*q3*q4*q5*q6**2*q7 + 5400*p**2*q1*q2*q3*q4*q5*q6**2 + 150*p**2*q1*q2*q3*q4*q5*q6*q7**3 - 1350*p**2*q1*q2*q3*q4*q5*q6*q7**2 + 5400*p**2*q1*q2*q3*q4*q5*q6*q7 - 12600*p**2*q1*q2*q3*q4*q5*q6 - 225*p**2*q2**4*q3*q4*q5*q6 - 225*p**2*q2**3*q3**2*q4*q5*q6 - 225*p**2*q2**3*q3*q4**2*q5*q6 - 225*p**2*q2**3*q3*q4*q5**2*q6 - 225*p**2*q2**3*q3*q4*q5*q6**2 - 225*p**2*q2**3*q3*q4*q5*q6*q7 + 1800*p**2*q2**3*q3*q4*q5*q6 - 225*p**2*q2**2*q3**3*q4*q5*q6 - 225*p**2*q2**2*q3**2*q4**2*q5*q6 - 225*p**2*q2**2*q3**2*q4*q5**2*q6 - 225*p**2*q2**2*q3**2*q4*q5*q6**2 - 225*p**2*q2**2*q3**2*q4*q5*q6*q7 + 1800*p**2*q2**2*q3**2*q4*q5*q6 - 225*p**2*q2**2*q3*q4**3*q5*q6 - 225*p**2*q2**2*q3*q4**2*q5**2*q6 - 225*p**2*q2**2*q3*q4**2*q5*q6**2 - 225*p**2*q2**2*q3*q4**2*q5*q6*q7 + 1800*p**2*q2**2*q3*q4**2*q5*q6 - 225*p**2*q2**2*q3*q4*q5**3*q6 - 225*p**2*q2**2*q3*q4*q5**2*q6**2 - 225*p**2*q2**2*q3*q4*q5**2*q6*q7 + 1800*p**2*q2**2*q3*q4*q5**2*q6 - 225*p**2*q2**2*q3*q4*q5*q6**3 - 225*p**2*q2**2*q3*q4*q5*q6**2*q7 + 1800*p**2*q2**2*q3*q4*q5*q6**2 - 225*p**2*q2**2*q3*q4*q5*q6*q7**2 + 1800*p**2*q2**2*q3*q4*q5*q6*q7 - 6300*p**2*q2**2*q3*q4*q5*q6 - 225*p**2*q2*q3**4*q4*q5*q6 - 225*p**2*q2*q3**3*q4**2*q5*q6 - 225*p**2*q2*q3**3*q4*q5**2*q6 - 225*p**2*q2*q3**3*q4*q5*q6**2 - 225*p**2*q2*q3**3*q4*q5*q6*q7 + 1800*p**2*q2*q3**3*q4*q5*q6 - 225*p**2*q2*q3**2*q4**3*q5*q6 - 225*p**2*q2*q3**2*q4**2*q5**2*q6 - 225*p**2*q2*q3**2*q4**2*q5*q6**2 - 225*p**2*q2*q3**2*q4**2*q5*q6*q7 + 1800*p**2*q2*q3**2*q4**2*q5*q6 - 225*p**2*q2*q3**2*q4*q5**3*q6 - 225*p**2*q2*q3**2*q4*q5**2*q6**2 - 225*p**2*q2*q3**2*q4*q5**2*q6*q7 + 1800*p**2*q2*q3**2*q4*q5**2*q6 - 225*p**2*q2*q3**2*q4*q5*q6**3 - 225*p**2*q2*q3**2*q4*q5*q6**2*q7 + 1800*p**2*q2*q3**2*q4*q5*q6**2 - 225*p**2*q2*q3**2*q4*q5*q6*q7**2 + 1800*p**2*q2*q3**2*q4*q5*q6*q7 - 6300*p**2*q2*q3**2*q4*q5*q6 - 225*p**2*q2*q3*q4**4*q5*q6 - 225*p**2*q2*q3*q4**3*q5**2*q6 - 225*p**2*q2*q3*q4**3*q5*q6**2 - 225*p**2*q2*q3*q4**3*q5*q6*q7 + 1800*p**2*q2*q3*q4**3*q5*q6 - 225*p**2*q2*q3*q4**2*q5**3*q6 - 225*p**2*q2*q3*q4**2*q5**2*q6**2 - 225*p**2*q2*q3*q4**2*q5**2*q6*q7 + 1800*p**2*q2*q3*q4**2*q5**2*q6 - 225*p**2*q2*q3*q4**2*q5*q6**3 - 225*p**2*q2*q3*q4**2*q5*q6**2*q7 + 1800*p**2*q2*q3*q4**2*q5*q6**2 - 225*p**2*q2*q3*q4**2*q5*q6*q7**2 + 1800*p**2*q2*q3*q4**2*q5*q6*q7 - 6300*p**2*q2*q3*q4**2*q5*q6 - 225*p**2*q2*q3*q4*q5**4*q6 - 225*p**2*q2*q3*q4*q5**3*q6**2 - 225*p**2*q2*q3*q4*q5**3*q6*q7 + 1800*p**2*q2*q3*q4*q5**3*q6 - 225*p**2*q2*q3*q4*q5**2*q6**3 - 225*p**2*q2*q3*q4*q5**2*q6**2*q7 + 1800*p**2*q2*q3*q4*q5**2*q6**2 - 225*p**2*q2*q3*q4*q5**2*q6*q7**2 + 1800*p**2*q2*q3*q4*q5**2*q6*q7 - 6300*p**2*q2*q3*q4*q5**2*q6 - 225*p**2*q2*q3*q4*q5*q6**4 - 225*p**2*q2*q3*q4*q5*q6**3*q7 + 1800*p**2*q2*q3*q4*q5*q6**3 - 225*p**2*q2*q3*q4*q5*q6**2*q7**2 + 1800*p**2*q2*q3*q4*q5*q6**2*q7 - 6300*p**2*q2*q3*q4*q5*q6**2 - 225*p**2*q2*q3*q4*q5*q6*q7**3 + 1800*p**2*q2*q3*q4*q5*q6*q7**2 - 6300*p**2*q2*q3*q4*q5*q6*q7 + 12600*p**2*q2*q3*q4*q5*q6 + 120*p**2*q3**4*q4*q5*q6 + 120*p**2*q3**3*q4**2*q5*q6 + 120*p**2*q3**3*q4*q5**2*q6 + 120*p**2*q3**3*q4*q5*q6**2 + 120*p**2*q3**3*q4*q5*q6*q7 - 840*p**2*q3**3*q4*q5*q6 + 120*p**2*q3**2*q4**3*q5*q6 + 120*p**2*q3**2*q4**2*q5**2*q6 + 120*p**2*q3**2*q4**2*q5*q6**2 + 120*p**2*q3**2*q4**2*q5*q6*q7 - 840*p**2*q3**2*q4**2*q5*q6 + 120*p**2*q3**2*q4*q5**3*q6 + 120*p**2*q3**2*q4*q5**2*q6**2 + 120*p**2*q3**2*q4*q5**2*q6*q7 - 840*p**2*q3**2*q4*q5**2*q6 + 120*p**2*q3**2*q4*q5*q6**3 + 120*p**2*q3**2*q4*q5*q6**2*q7 - 840*p**2*q3**2*q4*q5*q6**2 + 120*p**2*q3**2*q4*q5*q6*q7**2 - 840*p**2*q3**2*q4*q5*q6*q7 + 2520*p**2*q3**2*q4*q5*q6 + 120*p**2*q3*q4**4*q5*q6 + 120*p**2*q3*q4**3*q5**2*q6 + 120*p**2*q3*q4**3*q5*q6**2 + 120*p**2*q3*q4**3*q5*q6*q7 - 840*p**2*q3*q4**3*q5*q6 + 120*p**2*q3*q4**2*q5**3*q6 + 120*p**2*q3*q4**2*q5**2*q6**2 + 120*p**2*q3*q4**2*q5**2*q6*q7 - 840*p**2*q3*q4**2*q5**2*q6 + 120*p**2*q3*q4**2*q5*q6**3 + 120*p**2*q3*q4**2*q5*q6**2*q7 - 840*p**2*q3*q4**2*q5*q6**2 + 120*p**2*q3*q4**2*q5*q6*q7**2 - 840*p**2*q3*q4**2*q5*q6*q7 + 2520*p**2*q3*q4**2*q5*q6 + 120*p**2*q3*q4*q5**4*q6 + 120*p**2*q3*q4*q5**3*q6**2 + 120*p**2*q3*q4*q5**3*q6*q7 - 840*p**2*q3*q4*q5**3*q6 + 120*p**2*q3*q4*q5**2*q6**3 + 120*p**2*q3*q4*q5**2*q6**2*q7 - 840*p**2*q3*q4*q5**2*q6**2 + 120*p**2*q3*q4*q5**2*q6*q7**2 - 840*p**2*q3*q4*q5**2*q6*q7 + 2520*p**2*q3*q4*q5**2*q6 + 120*p**2*q3*q4*q5*q6**4 + 120*p**2*q3*q4*q5*q6**3*q7 - 840*p**2*q3*q4*q5*q6**3 + 120*p**2*q3*q4*q5*q6**2*q7**2 - 840*p**2*q3*q4*q5*q6**2*q7 + 2520*p**2*q3*q4*q5*q6**2 + 120*p**2*q3*q4*q5*q6*q7**3 - 840*p**2*q3*q4*q5*q6*q7**2 + 2520*p**2*q3*q4*q5*q6*q7 - 4200*p**2*q3*q4*q5*q6 - 60*p*q1**4*q2*q3*q4*q5*q6 - 60*p*q1**3*q2**2*q3*q4*q5*q6 - 60*p*q1**3*q2*q3**2*q4*q5*q6 - 60*p*q1**3*q2*q3*q4**2*q5*q6 - 60*p*q1**3*q2*q3*q4*q5**2*q6 - 60*p*q1**3*q2*q3*q4*q5*q6**2 - 60*p*q1**3*q2*q3*q4*q5*q6*q7 + 540*p*q1**3*q2*q3*q4*q5*q6 - 60*p*q1**2*q2**3*q3*q4*q5*q6 - 60*p*q1**2*q2**2*q3**2*q4*q5*q6 - 60*p*q1**2*q2**2*q3*q4**2*q5*q6 - 60*p*q1**2*q2**2*q3*q4*q5**2*q6 - 60*p*q1**2*q2**2*q3*q4*q5*q6**2 - 60*p*q1**2*q2**2*q3*q4*q5*q6*q7 + 540*p*q1**2*q2**2*q3*q4*q5*q6 - 60*p*q1**2*q2*q3**3*q4*q5*q6 - 60*p*q1**2*q2*q3**2*q4**2*q5*q6 - 60*p*q1**2*q2*q3**2*q4*q5**2*q6 - 60*p*q1**2*q2*q3**2*q4*q5*q6**2 - 60*p*q1**2*q2*q3**2*q4*q5*q6*q7 + 540*p*q1**2*q2*q3**2*q4*q5*q6 - 60*p*q1**2*q2*q3*q4**3*q5*q6 - 60*p*q1**2*q2*q3*q4**2*q5**2*q6 - 60*p*q1**2*q2*q3*q4**2*q5*q6**2 - 60*p*q1**2*q2*q3*q4**2*q5*q6*q7 + 540*p*q1**2*q2*q3*q4**2*q5*q6 - 60*p*q1**2*q2*q3*q4*q5**3*q6 - 60*p*q1**2*q2*q3*q4*q5**2*q6**2 - 60*p*q1**2*q2*q3*q4*q5**2*q6*q7 + 540*p*q1**2*q2*q3*q4*q5**2*q6 - 60*p*q1**2*q2*q3*q4*q5*q6**3 - 60*p*q1**2*q2*q3*q4*q5*q6**2*q7 + 540*p*q1**2*q2*q3*q4*q5*q6**2 - 60*p*q1**2*q2*q3*q4*q5*q6*q7**2 + 540*p*q1**2*q2*q3*q4*q5*q6*q7 - 2160*p*q1**2*q2*q3*q4*q5*q6 - 60*p*q1*q2**4*q3*q4*q5*q6 - 60*p*q1*q2**3*q3**2*q4*q5*q6 - 60*p*q1*q2**3*q3*q4**2*q5*q6 - 60*p*q1*q2**3*q3*q4*q5**2*q6 - 60*p*q1*q2**3*q3*q4*q5*q6**2 - 60*p*q1*q2**3*q3*q4*q5*q6*q7 + 540*p*q1*q2**3*q3*q4*q5*q6 - 60*p*q1*q2**2*q3**3*q4*q5*q6 - 60*p*q1*q2**2*q3**2*q4**2*q5*q6 - 60*p*q1*q2**2*q3**2*q4*q5**2*q6 - 60*p*q1*q2**2*q3**2*q4*q5*q6**2 - 60*p*q1*q2**2*q3**2*q4*q5*q6*q7 + 540*p*q1*q2**2*q3**2*q4*q5*q6 - 60*p*q1*q2**2*q3*q4**3*q5*q6 - 60*p*q1*q2**2*q3*q4**2*q5**2*q6 - 60*p*q1*q2**2*q3*q4**2*q5*q6**2 - 60*p*q1*q2**2*q3*q4**2*q5*q6*q7 + 540*p*q1*q2**2*q3*q4**2*q5*q6 - 60*p*q1*q2**2*q3*q4*q5**3*q6 - 60*p*q1*q2**2*q3*q4*q5**2*q6**2 - 60*p*q1*q2**2*q3*q4*q5**2*q6*q7 + 540*p*q1*q2**2*q3*q4*q5**2*q6 - 60*p*q1*q2**2*q3*q4*q5*q6**3 - 60*p*q1*q2**2*q3*q4*q5*q6**2*q7 + 540*p*q1*q2**2*q3*q4*q5*q6**2 - 60*p*q1*q2**2*q3*q4*q5*q6*q7**2 + 540*p*q1*q2**2*q3*q4*q5*q6*q7 - 2160*p*q1*q2**2*q3*q4*q5*q6 - 60*p*q1*q2*q3**4*q4*q5*q6 - 60*p*q1*q2*q3**3*q4**2*q5*q6 - 60*p*q1*q2*q3**3*q4*q5**2*q6 - 60*p*q1*q2*q3**3*q4*q5*q6**2 - 60*p*q1*q2*q3**3*q4*q5*q6*q7 + 540*p*q1*q2*q3**3*q4*q5*q6 - 60*p*q1*q2*q3**2*q4**3*q5*q6 - 60*p*q1*q2*q3**2*q4**2*q5**2*q6 - 60*p*q1*q2*q3**2*q4**2*q5*q6**2 - 60*p*q1*q2*q3**2*q4**2*q5*q6*q7 + 540*p*q1*q2*q3**2*q4**2*q5*q6 - 60*p*q1*q2*q3**2*q4*q5**3*q6 - 60*p*q1*q2*q3**2*q4*q5**2*q6**2 - 60*p*q1*q2*q3**2*q4*q5**2*q6*q7 + 540*p*q1*q2*q3**2*q4*q5**2*q6 - 60*p*q1*q2*q3**2*q4*q5*q6**3 - 60*p*q1*q2*q3**2*q4*q5*q6**2*q7 + 540*p*q1*q2*q3**2*q4*q5*q6**2 - 60*p*q1*q2*q3**2*q4*q5*q6*q7**2 + 540*p*q1*q2*q3**2*q4*q5*q6*q7 - 2160*p*q1*q2*q3**2*q4*q5*q6 - 60*p*q1*q2*q3*q4**4*q5*q6 - 60*p*q1*q2*q3*q4**3*q5**2*q6 - 60*p*q1*q2*q3*q4**3*q5*q6**2 - 60*p*q1*q2*q3*q4**3*q5*q6*q7 + 540*p*q1*q2*q3*q4**3*q5*q6 - 60*p*q1*q2*q3*q4**2*q5**3*q6 - 60*p*q1*q2*q3*q4**2*q5**2*q6**2 - 60*p*q1*q2*q3*q4**2*q5**2*q6*q7 + 540*p*q1*q2*q3*q4**2*q5**2*q6 - 60*p*q1*q2*q3*q4**2*q5*q6**3 - 60*p*q1*q2*q3*q4**2*q5*q6**2*q7 + 540*p*q1*q2*q3*q4**2*q5*q6**2 - 60*p*q1*q2*q3*q4**2*q5*q6*q7**2 + 540*p*q1*q2*q3*q4**2*q5*q6*q7 - 2160*p*q1*q2*q3*q4**2*q5*q6 - 60*p*q1*q2*q3*q4*q5**4*q6 - 60*p*q1*q2*q3*q4*q5**3*q6**2 - 60*p*q1*q2*q3*q4*q5**3*q6*q7 + 540*p*q1*q2*q3*q4*q5**3*q6 - 60*p*q1*q2*q3*q4*q5**2*q6**3 - 60*p*q1*q2*q3*q4*q5**2*q6**2*q7 + 540*p*q1*q2*q3*q4*q5**2*q6**2 - 60*p*q1*q2*q3*q4*q5**2*q6*q7**2 + 540*p*q1*q2*q3*q4*q5**2*q6*q7 - 2160*p*q1*q2*q3*q4*q5**2*q6 - 60*p*q1*q2*q3*q4*q5*q6**4 - 60*p*q1*q2*q3*q4*q5*q6**3*q7 + 540*p*q1*q2*q3*q4*q5*q6**3 - 60*p*q1*q2*q3*q4*q5*q6**2*q7**2 + 540*p*q1*q2*q3*q4*q5*q6**2*q7 - 2160*p*q1*q2*q3*q4*q5*q6**2 - 60*p*q1*q2*q3*q4*q5*q6*q7**3 + 540*p*q1*q2*q3*q4*q5*q6*q7**2 - 2160*p*q1*q2*q3*q4*q5*q6*q7 + 5040*p*q1*q2*q3*q4*q5*q6 + 45*p*q2**4*q3*q4*q5*q6 + 45*p*q2**3*q3**2*q4*q5*q6 + 45*p*q2**3*q3*q4**2*q5*q6 + 45*p*q2**3*q3*q4*q5**2*q6 + 45*p*q2**3*q3*q4*q5*q6**2 + 45*p*q2**3*q3*q4*q5*q6*q7 - 360*p*q2**3*q3*q4*q5*q6 + 45*p*q2**2*q3**3*q4*q5*q6 + 45*p*q2**2*q3**2*q4**2*q5*q6 + 45*p*q2**2*q3**2*q4*q5**2*q6 + 45*p*q2**2*q3**2*q4*q5*q6**2 + 45*p*q2**2*q3**2*q4*q5*q6*q7 - 360*p*q2**2*q3**2*q4*q5*q6 + 45*p*q2**2*q3*q4**3*q5*q6 + 45*p*q2**2*q3*q4**2*q5**2*q6 + 45*p*q2**2*q3*q4**2*q5*q6**2 + 45*p*q2**2*q3*q4**2*q5*q6*q7 - 360*p*q2**2*q3*q4**2*q5*q6 + 45*p*q2**2*q3*q4*q5**3*q6 + 45*p*q2**2*q3*q4*q5**2*q6**2 + 45*p*q2**2*q3*q4*q5**2*q6*q7 - 360*p*q2**2*q3*q4*q5**2*q6 + 45*p*q2**2*q3*q4*q5*q6**3 + 45*p*q2**2*q3*q4*q5*q6**2*q7 - 360*p*q2**2*q3*q4*q5*q6**2 + 45*p*q2**2*q3*q4*q5*q6*q7**2 - 360*p*q2**2*q3*q4*q5*q6*q7 + 1260*p*q2**2*q3*q4*q5*q6 + 45*p*q2*q3**4*q4*q5*q6 + 45*p*q2*q3**3*q4**2*q5*q6 + 45*p*q2*q3**3*q4*q5**2*q6 + 45*p*q2*q3**3*q4*q5*q6**2 + 45*p*q2*q3**3*q4*q5*q6*q7 - 360*p*q2*q3**3*q4*q5*q6 + 45*p*q2*q3**2*q4**3*q5*q6 + 45*p*q2*q3**2*q4**2*q5**2*q6 + 45*p*q2*q3**2*q4**2*q5*q6**2 + 45*p*q2*q3**2*q4**2*q5*q6*q7 - 360*p*q2*q3**2*q4**2*q5*q6 + 45*p*q2*q3**2*q4*q5**3*q6 + 45*p*q2*q3**2*q4*q5**2*q6**2 + 45*p*q2*q3**2*q4*q5**2*q6*q7 - 360*p*q2*q3**2*q4*q5**2*q6 + 45*p*q2*q3**2*q4*q5*q6**3 + 45*p*q2*q3**2*q4*q5*q6**2*q7 - 360*p*q2*q3**2*q4*q5*q6**2 + 45*p*q2*q3**2*q4*q5*q6*q7**2 - 360*p*q2*q3**2*q4*q5*q6*q7 + 1260*p*q2*q3**2*q4*q5*q6 + 45*p*q2*q3*q4**4*q5*q6 + 45*p*q2*q3*q4**3*q5**2*q6 + 45*p*q2*q3*q4**3*q5*q6**2 + 45*p*q2*q3*q4**3*q5*q6*q7 - 360*p*q2*q3*q4**3*q5*q6 + 45*p*q2*q3*q4**2*q5**3*q6 + 45*p*q2*q3*q4**2*q5**2*q6**2 + 45*p*q2*q3*q4**2*q5**2*q6*q7 - 360*p*q2*q3*q4**2*q5**2*q6 + 45*p*q2*q3*q4**2*q5*q6**3 + 45*p*q2*q3*q4**2*q5*q6**2*q7 - 360*p*q2*q3*q4**2*q5*q6**2 + 45*p*q2*q3*q4**2*q5*q6*q7**2 - 360*p*q2*q3*q4**2*q5*q6*q7 + 1260*p*q2*q3*q4**2*q5*q6 + 45*p*q2*q3*q4*q5**4*q6 + 45*p*q2*q3*q4*q5**3*q6**2 + 45*p*q2*q3*q4*q5**3*q6*q7 - 360*p*q2*q3*q4*q5**3*q6 + 45*p*q2*q3*q4*q5**2*q6**3 + 45*p*q2*q3*q4*q5**2*q6**2*q7 - 360*p*q2*q3*q4*q5**2*q6**2 + 45*p*q2*q3*q4*q5**2*q6*q7**2 - 360*p*q2*q3*q4*q5**2*q6*q7 + 1260*p*q2*q3*q4*q5**2*q6 + 45*p*q2*q3*q4*q5*q6**4 + 45*p*q2*q3*q4*q5*q6**3*q7 - 360*p*q2*q3*q4*q5*q6**3 + 45*p*q2*q3*q4*q5*q6**2*q7**2 - 360*p*q2*q3*q4*q5*q6**2*q7 + 1260*p*q2*q3*q4*q5*q6**2 + 45*p*q2*q3*q4*q5*q6*q7**3 - 360*p*q2*q3*q4*q5*q6*q7**2 + 1260*p*q2*q3*q4*q5*q6*q7 - 2520*p*q2*q3*q4*q5*q6 + 10*q1**4*q2*q3*q4*q5*q6 + 10*q1**3*q2**2*q3*q4*q5*q6 + 10*q1**3*q2*q3**2*q4*q5*q6 + 10*q1**3*q2*q3*q4**2*q5*q6 + 10*q1**3*q2*q3*q4*q5**2*q6 + 10*q1**3*q2*q3*q4*q5*q6**2 + 10*q1**3*q2*q3*q4*q5*q6*q7 - 90*q1**3*q2*q3*q4*q5*q6 + 10*q1**2*q2**3*q3*q4*q5*q6 + 10*q1**2*q2**2*q3**2*q4*q5*q6 + 10*q1**2*q2**2*q3*q4**2*q5*q6 + 10*q1**2*q2**2*q3*q4*q5**2*q6 + 10*q1**2*q2**2*q3*q4*q5*q6**2 + 10*q1**2*q2**2*q3*q4*q5*q6*q7 - 90*q1**2*q2**2*q3*q4*q5*q6 + 10*q1**2*q2*q3**3*q4*q5*q6 + 10*q1**2*q2*q3**2*q4**2*q5*q6 + 10*q1**2*q2*q3**2*q4*q5**2*q6 + 10*q1**2*q2*q3**2*q4*q5*q6**2 + 10*q1**2*q2*q3**2*q4*q5*q6*q7 - 90*q1**2*q2*q3**2*q4*q5*q6 + 10*q1**2*q2*q3*q4**3*q5*q6 + 10*q1**2*q2*q3*q4**2*q5**2*q6 + 10*q1**2*q2*q3*q4**2*q5*q6**2 + 10*q1**2*q2*q3*q4**2*q5*q6*q7 - 90*q1**2*q2*q3*q4**2*q5*q6 + 10*q1**2*q2*q3*q4*q5**3*q6 + 10*q1**2*q2*q3*q4*q5**2*q6**2 + 10*q1**2*q2*q3*q4*q5**2*q6*q7 - 90*q1**2*q2*q3*q4*q5**2*q6 + 10*q1**2*q2*q3*q4*q5*q6**3 + 10*q1**2*q2*q3*q4*q5*q6**2*q7 - 90*q1**2*q2*q3*q4*q5*q6**2 + 10*q1**2*q2*q3*q4*q5*q6*q7**2 - 90*q1**2*q2*q3*q4*q5*q6*q7 + 360*q1**2*q2*q3*q4*q5*q6 + 10*q1*q2**4*q3*q4*q5*q6 + 10*q1*q2**3*q3**2*q4*q5*q6 + 10*q1*q2**3*q3*q4**2*q5*q6 + 10*q1*q2**3*q3*q4*q5**2*q6 + 10*q1*q2**3*q3*q4*q5*q6**2 + 10*q1*q2**3*q3*q4*q5*q6*q7 - 90*q1*q2**3*q3*q4*q5*q6 + 10*q1*q2**2*q3**3*q4*q5*q6 + 10*q1*q2**2*q3**2*q4**2*q5*q6 + 10*q1*q2**2*q3**2*q4*q5**2*q6 + 10*q1*q2**2*q3**2*q4*q5*q6**2 + 10*q1*q2**2*q3**2*q4*q5*q6*q7 - 90*q1*q2**2*q3**2*q4*q5*q6 + 10*q1*q2**2*q3*q4**3*q5*q6 + 10*q1*q2**2*q3*q4**2*q5**2*q6 + 10*q1*q2**2*q3*q4**2*q5*q6**2 + 10*q1*q2**2*q3*q4**2*q5*q6*q7 - 90*q1*q2**2*q3*q4**2*q5*q6 + 10*q1*q2**2*q3*q4*q5**3*q6 + 10*q1*q2**2*q3*q4*q5**2*q6**2 + 10*q1*q2**2*q3*q4*q5**2*q6*q7 - 90*q1*q2**2*q3*q4*q5**2*q6 + 10*q1*q2**2*q3*q4*q5*q6**3 + 10*q1*q2**2*q3*q4*q5*q6**2*q7 - 90*q1*q2**2*q3*q4*q5*q6**2 + 10*q1*q2**2*q3*q4*q5*q6*q7**2 - 90*q1*q2**2*q3*q4*q5*q6*q7 + 360*q1*q2**2*q3*q4*q5*q6 + 10*q1*q2*q3**4*q4*q5*q6 + 10*q1*q2*q3**3*q4**2*q5*q6 + 10*q1*q2*q3**3*q4*q5**2*q6 + 10*q1*q2*q3**3*q4*q5*q6**2 + 10*q1*q2*q3**3*q4*q5*q6*q7 - 90*q1*q2*q3**3*q4*q5*q6 + 10*q1*q2*q3**2*q4**3*q5*q6 + 10*q1*q2*q3**2*q4**2*q5**2*q6 + 10*q1*q2*q3**2*q4**2*q5*q6**2 + 10*q1*q2*q3**2*q4**2*q5*q6*q7 - 90*q1*q2*q3**2*q4**2*q5*q6 + 10*q1*q2*q3**2*q4*q5**3*q6 + 10*q1*q2*q3**2*q4*q5**2*q6**2 + 10*q1*q2*q3**2*q4*q5**2*q6*q7 - 90*q1*q2*q3**2*q4*q5**2*q6 + 10*q1*q2*q3**2*q4*q5*q6**3 + 10*q1*q2*q3**2*q4*q5*q6**2*q7 - 90*q1*q2*q3**2*q4*q5*q6**2 + 10*q1*q2*q3**2*q4*q5*q6*q7**2 - 90*q1*q2*q3**2*q4*q5*q6*q7 + 360*q1*q2*q3**2*q4*q5*q6 + 10*q1*q2*q3*q4**4*q5*q6 + 10*q1*q2*q3*q4**3*q5**2*q6 + 10*q1*q2*q3*q4**3*q5*q6**2 + 10*q1*q2*q3*q4**3*q5*q6*q7 - 90*q1*q2*q3*q4**3*q5*q6 + 10*q1*q2*q3*q4**2*q5**3*q6 + 10*q1*q2*q3*q4**2*q5**2*q6**2 + 10*q1*q2*q3*q4**2*q5**2*q6*q7 - 90*q1*q2*q3*q4**2*q5**2*q6 + 10*q1*q2*q3*q4**2*q5*q6**3 + 10*q1*q2*q3*q4**2*q5*q6**2*q7 - 90*q1*q2*q3*q4**2*q5*q6**2 + 10*q1*q2*q3*q4**2*q5*q6*q7**2 - 90*q1*q2*q3*q4**2*q5*q6*q7 + 360*q1*q2*q3*q4**2*q5*q6 + 10*q1*q2*q3*q4*q5**4*q6 + 10*q1*q2*q3*q4*q5**3*q6**2 + 10*q1*q2*q3*q4*q5**3*q6*q7 - 90*q1*q2*q3*q4*q5**3*q6 + 10*q1*q2*q3*q4*q5**2*q6**3 + 10*q1*q2*q3*q4*q5**2*q6**2*q7 - 90*q1*q2*q3*q4*q5**2*q6**2 + 10*q1*q2*q3*q4*q5**2*q6*q7**2 - 90*q1*q2*q3*q4*q5**2*q6*q7 + 360*q1*q2*q3*q4*q5**2*q6 + 10*q1*q2*q3*q4*q5*q6**4 + 10*q1*q2*q3*q4*q5*q6**3*q7 - 90*q1*q2*q3*q4*q5*q6**3 + 10*q1*q2*q3*q4*q5*q6**2*q7**2 - 90*q1*q2*q3*q4*q5*q6**2*q7 + 360*q1*q2*q3*q4*q5*q6**2 + 10*q1*q2*q3*q4*q5*q6*q7**3 - 90*q1*q2*q3*q4*q5*q6*q7**2 + 360*q1*q2*q3*q4*q5*q6*q7 - 840*q1*q2*q3*q4*q5*q6)'
f_multiparam[10][8] = '-p*(p - 1)**2*(10*p**7*q1**3*q2*q3*q4*q5*q6*q7 + 10*p**7*q1**2*q2**2*q3*q4*q5*q6*q7 + 10*p**7*q1**2*q2*q3**2*q4*q5*q6*q7 + 10*p**7*q1**2*q2*q3*q4**2*q5*q6*q7 + 10*p**7*q1**2*q2*q3*q4*q5**2*q6*q7 + 10*p**7*q1**2*q2*q3*q4*q5*q6**2*q7 + 10*p**7*q1**2*q2*q3*q4*q5*q6*q7**2 + 10*p**7*q1**2*q2*q3*q4*q5*q6*q7*q8 - 90*p**7*q1**2*q2*q3*q4*q5*q6*q7 + 10*p**7*q1*q2**3*q3*q4*q5*q6*q7 + 10*p**7*q1*q2**2*q3**2*q4*q5*q6*q7 + 10*p**7*q1*q2**2*q3*q4**2*q5*q6*q7 + 10*p**7*q1*q2**2*q3*q4*q5**2*q6*q7 + 10*p**7*q1*q2**2*q3*q4*q5*q6**2*q7 + 10*p**7*q1*q2**2*q3*q4*q5*q6*q7**2 + 10*p**7*q1*q2**2*q3*q4*q5*q6*q7*q8 - 90*p**7*q1*q2**2*q3*q4*q5*q6*q7 + 10*p**7*q1*q2*q3**3*q4*q5*q6*q7 + 10*p**7*q1*q2*q3**2*q4**2*q5*q6*q7 + 10*p**7*q1*q2*q3**2*q4*q5**2*q6*q7 + 10*p**7*q1*q2*q3**2*q4*q5*q6**2*q7 + 10*p**7*q1*q2*q3**2*q4*q5*q6*q7**2 + 10*p**7*q1*q2*q3**2*q4*q5*q6*q7*q8 - 90*p**7*q1*q2*q3**2*q4*q5*q6*q7 + 10*p**7*q1*q2*q3*q4**3*q5*q6*q7 + 10*p**7*q1*q2*q3*q4**2*q5**2*q6*q7 + 10*p**7*q1*q2*q3*q4**2*q5*q6**2*q7 + 10*p**7*q1*q2*q3*q4**2*q5*q6*q7**2 + 10*p**7*q1*q2*q3*q4**2*q5*q6*q7*q8 - 90*p**7*q1*q2*q3*q4**2*q5*q6*q7 + 10*p**7*q1*q2*q3*q4*q5**3*q6*q7 + 10*p**7*q1*q2*q3*q4*q5**2*q6**2*q7 + 10*p**7*q1*q2*q3*q4*q5**2*q6*q7**2 + 10*p**7*q1*q2*q3*q4*q5**2*q6*q7*q8 - 90*p**7*q1*q2*q3*q4*q5**2*q6*q7 + 10*p**7*q1*q2*q3*q4*q5*q6**3*q7 + 10*p**7*q1*q2*q3*q4*q5*q6**2*q7**2 + 10*p**7*q1*q2*q3*q4*q5*q6**2*q7*q8 - 90*p**7*q1*q2*q3*q4*q5*q6**2*q7 + 10*p**7*q1*q2*q3*q4*q5*q6*q7**3 + 10*p**7*q1*q2*q3*q4*q5*q6*q7**2*q8 - 90*p**7*q1*q2*q3*q4*q5*q6*q7**2 + 10*p**7*q1*q2*q3*q4*q5*q6*q7*q8**2 - 90*p**7*q1*q2*q3*q4*q5*q6*q7*q8 + 360*p**7*q1*q2*q3*q4*q5*q6*q7 - 45*p**7*q2**3*q3*q4*q5*q6*q7 - 45*p**7*q2**2*q3**2*q4*q5*q6*q7 - 45*p**7*q2**2*q3*q4**2*q5*q6*q7 - 45*p**7*q2**2*q3*q4*q5**2*q6*q7 - 45*p**7*q2**2*q3*q4*q5*q6**2*q7 - 45*p**7*q2**2*q3*q4*q5*q6*q7**2 - 45*p**7*q2**2*q3*q4*q5*q6*q7*q8 + 360*p**7*q2**2*q3*q4*q5*q6*q7 - 45*p**7*q2*q3**3*q4*q5*q6*q7 - 45*p**7*q2*q3**2*q4**2*q5*q6*q7 - 45*p**7*q2*q3**2*q4*q5**2*q6*q7 - 45*p**7*q2*q3**2*q4*q5*q6**2*q7 - 45*p**7*q2*q3**2*q4*q5*q6*q7**2 - 45*p**7*q2*q3**2*q4*q5*q6*q7*q8 + 360*p**7*q2*q3**2*q4*q5*q6*q7 - 45*p**7*q2*q3*q4**3*q5*q6*q7 - 45*p**7*q2*q3*q4**2*q5**2*q6*q7 - 45*p**7*q2*q3*q4**2*q5*q6**2*q7 - 45*p**7*q2*q3*q4**2*q5*q6*q7**2 - 45*p**7*q2*q3*q4**2*q5*q6*q7*q8 + 360*p**7*q2*q3*q4**2*q5*q6*q7 - 45*p**7*q2*q3*q4*q5**3*q6*q7 - 45*p**7*q2*q3*q4*q5**2*q6**2*q7 - 45*p**7*q2*q3*q4*q5**2*q6*q7**2 - 45*p**7*q2*q3*q4*q5**2*q6*q7*q8 + 360*p**7*q2*q3*q4*q5**2*q6*q7 - 45*p**7*q2*q3*q4*q5*q6**3*q7 - 45*p**7*q2*q3*q4*q5*q6**2*q7**2 - 45*p**7*q2*q3*q4*q5*q6**2*q7*q8 + 360*p**7*q2*q3*q4*q5*q6**2*q7 - 45*p**7*q2*q3*q4*q5*q6*q7**3 - 45*p**7*q2*q3*q4*q5*q6*q7**2*q8 + 360*p**7*q2*q3*q4*q5*q6*q7**2 - 45*p**7*q2*q3*q4*q5*q6*q7*q8**2 + 360*p**7*q2*q3*q4*q5*q6*q7*q8 - 1260*p**7*q2*q3*q4*q5*q6*q7 + 120*p**7*q3**3*q4*q5*q6*q7 + 120*p**7*q3**2*q4**2*q5*q6*q7 + 120*p**7*q3**2*q4*q5**2*q6*q7 + 120*p**7*q3**2*q4*q5*q6**2*q7 + 120*p**7*q3**2*q4*q5*q6*q7**2 + 120*p**7*q3**2*q4*q5*q6*q7*q8 - 840*p**7*q3**2*q4*q5*q6*q7 + 120*p**7*q3*q4**3*q5*q6*q7 + 120*p**7*q3*q4**2*q5**2*q6*q7 + 120*p**7*q3*q4**2*q5*q6**2*q7 + 120*p**7*q3*q4**2*q5*q6*q7**2 + 120*p**7*q3*q4**2*q5*q6*q7*q8 - 840*p**7*q3*q4**2*q5*q6*q7 + 120*p**7*q3*q4*q5**3*q6*q7 + 120*p**7*q3*q4*q5**2*q6**2*q7 + 120*p**7*q3*q4*q5**2*q6*q7**2 + 120*p**7*q3*q4*q5**2*q6*q7*q8 - 840*p**7*q3*q4*q5**2*q6*q7 + 120*p**7*q3*q4*q5*q6**3*q7 + 120*p**7*q3*q4*q5*q6**2*q7**2 + 120*p**7*q3*q4*q5*q6**2*q7*q8 - 840*p**7*q3*q4*q5*q6**2*q7 + 120*p**7*q3*q4*q5*q6*q7**3 + 120*p**7*q3*q4*q5*q6*q7**2*q8 - 840*p**7*q3*q4*q5*q6*q7**2 + 120*p**7*q3*q4*q5*q6*q7*q8**2 - 840*p**7*q3*q4*q5*q6*q7*q8 + 2520*p**7*q3*q4*q5*q6*q7 - 210*p**7*q4**3*q5*q6*q7 - 210*p**7*q4**2*q5**2*q6*q7 - 210*p**7*q4**2*q5*q6**2*q7 - 210*p**7*q4**2*q5*q6*q7**2 - 210*p**7*q4**2*q5*q6*q7*q8 + 1260*p**7*q4**2*q5*q6*q7 - 210*p**7*q4*q5**3*q6*q7 - 210*p**7*q4*q5**2*q6**2*q7 - 210*p**7*q4*q5**2*q6*q7**2 - 210*p**7*q4*q5**2*q6*q7*q8 + 1260*p**7*q4*q5**2*q6*q7 - 210*p**7*q4*q5*q6**3*q7 - 210*p**7*q4*q5*q6**2*q7**2 - 210*p**7*q4*q5*q6**2*q7*q8 + 1260*p**7*q4*q5*q6**2*q7 - 210*p**7*q4*q5*q6*q7**3 - 210*p**7*q4*q5*q6*q7**2*q8 + 1260*p**7*q4*q5*q6*q7**2 - 210*p**7*q4*q5*q6*q7*q8**2 + 1260*p**7*q4*q5*q6*q7*q8 - 3150*p**7*q4*q5*q6*q7 + 252*p**7*q5**3*q6*q7 + 252*p**7*q5**2*q6**2*q7 + 252*p**7*q5**2*q6*q7**2 + 252*p**7*q5**2*q6*q7*q8 - 1260*p**7*q5**2*q6*q7 + 252*p**7*q5*q6**3*q7 + 252*p**7*q5*q6**2*q7**2 + 252*p**7*q5*q6**2*q7*q8 - 1260*p**7*q5*q6**2*q7 + 252*p**7*q5*q6*q7**3 + 252*p**7*q5*q6*q7**2*q8 - 1260*p**7*q5*q6*q7**2 + 252*p**7*q5*q6*q7*q8**2 - 1260*p**7*q5*q6*q7*q8 + 2520*p**7*q5*q6*q7 - 210*p**7*q6**3*q7 - 210*p**7*q6**2*q7**2 - 210*p**7*q6**2*q7*q8 + 840*p**7*q6**2*q7 - 210*p**7*q6*q7**3 - 210*p**7*q6*q7**2*q8 + 840*p**7*q6*q7**2 - 210*p**7*q6*q7*q8**2 + 840*p**7*q6*q7*q8 - 1260*p**7*q6*q7 + 120*p**7*q7**3 + 120*p**7*q7**2*q8 - 360*p**7*q7**2 + 120*p**7*q7*q8**2 - 360*p**7*q7*q8 + 360*p**7*q7 - 45*p**7*q8**2 + 90*p**7*q8 - 45*p**7 - 70*p**6*q1**3*q2*q3*q4*q5*q6*q7 - 70*p**6*q1**2*q2**2*q3*q4*q5*q6*q7 - 70*p**6*q1**2*q2*q3**2*q4*q5*q6*q7 - 70*p**6*q1**2*q2*q3*q4**2*q5*q6*q7 - 70*p**6*q1**2*q2*q3*q4*q5**2*q6*q7 - 70*p**6*q1**2*q2*q3*q4*q5*q6**2*q7 - 70*p**6*q1**2*q2*q3*q4*q5*q6*q7**2 - 70*p**6*q1**2*q2*q3*q4*q5*q6*q7*q8 + 630*p**6*q1**2*q2*q3*q4*q5*q6*q7 - 70*p**6*q1*q2**3*q3*q4*q5*q6*q7 - 70*p**6*q1*q2**2*q3**2*q4*q5*q6*q7 - 70*p**6*q1*q2**2*q3*q4**2*q5*q6*q7 - 70*p**6*q1*q2**2*q3*q4*q5**2*q6*q7 - 70*p**6*q1*q2**2*q3*q4*q5*q6**2*q7 - 70*p**6*q1*q2**2*q3*q4*q5*q6*q7**2 - 70*p**6*q1*q2**2*q3*q4*q5*q6*q7*q8 + 630*p**6*q1*q2**2*q3*q4*q5*q6*q7 - 70*p**6*q1*q2*q3**3*q4*q5*q6*q7 - 70*p**6*q1*q2*q3**2*q4**2*q5*q6*q7 - 70*p**6*q1*q2*q3**2*q4*q5**2*q6*q7 - 70*p**6*q1*q2*q3**2*q4*q5*q6**2*q7 - 70*p**6*q1*q2*q3**2*q4*q5*q6*q7**2 - 70*p**6*q1*q2*q3**2*q4*q5*q6*q7*q8 + 630*p**6*q1*q2*q3**2*q4*q5*q6*q7 - 70*p**6*q1*q2*q3*q4**3*q5*q6*q7 - 70*p**6*q1*q2*q3*q4**2*q5**2*q6*q7 - 70*p**6*q1*q2*q3*q4**2*q5*q6**2*q7 - 70*p**6*q1*q2*q3*q4**2*q5*q6*q7**2 - 70*p**6*q1*q2*q3*q4**2*q5*q6*q7*q8 + 630*p**6*q1*q2*q3*q4**2*q5*q6*q7 - 70*p**6*q1*q2*q3*q4*q5**3*q6*q7 - 70*p**6*q1*q2*q3*q4*q5**2*q6**2*q7 - 70*p**6*q1*q2*q3*q4*q5**2*q6*q7**2 - 70*p**6*q1*q2*q3*q4*q5**2*q6*q7*q8 + 630*p**6*q1*q2*q3*q4*q5**2*q6*q7 - 70*p**6*q1*q2*q3*q4*q5*q6**3*q7 - 70*p**6*q1*q2*q3*q4*q5*q6**2*q7**2 - 70*p**6*q1*q2*q3*q4*q5*q6**2*q7*q8 + 630*p**6*q1*q2*q3*q4*q5*q6**2*q7 - 70*p**6*q1*q2*q3*q4*q5*q6*q7**3 - 70*p**6*q1*q2*q3*q4*q5*q6*q7**2*q8 + 630*p**6*q1*q2*q3*q4*q5*q6*q7**2 - 70*p**6*q1*q2*q3*q4*q5*q6*q7*q8**2 + 630*p**6*q1*q2*q3*q4*q5*q6*q7*q8 - 2520*p**6*q1*q2*q3*q4*q5*q6*q7 + 270*p**6*q2**3*q3*q4*q5*q6*q7 + 270*p**6*q2**2*q3**2*q4*q5*q6*q7 + 270*p**6*q2**2*q3*q4**2*q5*q6*q7 + 270*p**6*q2**2*q3*q4*q5**2*q6*q7 + 270*p**6*q2**2*q3*q4*q5*q6**2*q7 + 270*p**6*q2**2*q3*q4*q5*q6*q7**2 + 270*p**6*q2**2*q3*q4*q5*q6*q7*q8 - 2160*p**6*q2**2*q3*q4*q5*q6*q7 + 270*p**6*q2*q3**3*q4*q5*q6*q7 + 270*p**6*q2*q3**2*q4**2*q5*q6*q7 + 270*p**6*q2*q3**2*q4*q5**2*q6*q7 + 270*p**6*q2*q3**2*q4*q5*q6**2*q7 + 270*p**6*q2*q3**2*q4*q5*q6*q7**2 + 270*p**6*q2*q3**2*q4*q5*q6*q7*q8 - 2160*p**6*q2*q3**2*q4*q5*q6*q7 + 270*p**6*q2*q3*q4**3*q5*q6*q7 + 270*p**6*q2*q3*q4**2*q5**2*q6*q7 + 270*p**6*q2*q3*q4**2*q5*q6**2*q7 + 270*p**6*q2*q3*q4**2*q5*q6*q7**2 + 270*p**6*q2*q3*q4**2*q5*q6*q7*q8 - 2160*p**6*q2*q3*q4**2*q5*q6*q7 + 270*p**6*q2*q3*q4*q5**3*q6*q7 + 270*p**6*q2*q3*q4*q5**2*q6**2*q7 + 270*p**6*q2*q3*q4*q5**2*q6*q7**2 + 270*p**6*q2*q3*q4*q5**2*q6*q7*q8 - 2160*p**6*q2*q3*q4*q5**2*q6*q7 + 270*p**6*q2*q3*q4*q5*q6**3*q7 + 270*p**6*q2*q3*q4*q5*q6**2*q7**2 + 270*p**6*q2*q3*q4*q5*q6**2*q7*q8 - 2160*p**6*q2*q3*q4*q5*q6**2*q7 + 270*p**6*q2*q3*q4*q5*q6*q7**3 + 270*p**6*q2*q3*q4*q5*q6*q7**2*q8 - 2160*p**6*q2*q3*q4*q5*q6*q7**2 + 270*p**6*q2*q3*q4*q5*q6*q7*q8**2 - 2160*p**6*q2*q3*q4*q5*q6*q7*q8 + 7560*p**6*q2*q3*q4*q5*q6*q7 - 600*p**6*q3**3*q4*q5*q6*q7 - 600*p**6*q3**2*q4**2*q5*q6*q7 - 600*p**6*q3**2*q4*q5**2*q6*q7 - 600*p**6*q3**2*q4*q5*q6**2*q7 - 600*p**6*q3**2*q4*q5*q6*q7**2 - 600*p**6*q3**2*q4*q5*q6*q7*q8 + 4200*p**6*q3**2*q4*q5*q6*q7 - 600*p**6*q3*q4**3*q5*q6*q7 - 600*p**6*q3*q4**2*q5**2*q6*q7 - 600*p**6*q3*q4**2*q5*q6**2*q7 - 600*p**6*q3*q4**2*q5*q6*q7**2 - 600*p**6*q3*q4**2*q5*q6*q7*q8 + 4200*p**6*q3*q4**2*q5*q6*q7 - 600*p**6*q3*q4*q5**3*q6*q7 - 600*p**6*q3*q4*q5**2*q6**2*q7 - 600*p**6*q3*q4*q5**2*q6*q7**2 - 600*p**6*q3*q4*q5**2*q6*q7*q8 + 4200*p**6*q3*q4*q5**2*q6*q7 - 600*p**6*q3*q4*q5*q6**3*q7 - 600*p**6*q3*q4*q5*q6**2*q7**2 - 600*p**6*q3*q4*q5*q6**2*q7*q8 + 4200*p**6*q3*q4*q5*q6**2*q7 - 600*p**6*q3*q4*q5*q6*q7**3 - 600*p**6*q3*q4*q5*q6*q7**2*q8 + 4200*p**6*q3*q4*q5*q6*q7**2 - 600*p**6*q3*q4*q5*q6*q7*q8**2 + 4200*p**6*q3*q4*q5*q6*q7*q8 - 12600*p**6*q3*q4*q5*q6*q7 + 840*p**6*q4**3*q5*q6*q7 + 840*p**6*q4**2*q5**2*q6*q7 + 840*p**6*q4**2*q5*q6**2*q7 + 840*p**6*q4**2*q5*q6*q7**2 + 840*p**6*q4**2*q5*q6*q7*q8 - 5040*p**6*q4**2*q5*q6*q7 + 840*p**6*q4*q5**3*q6*q7 + 840*p**6*q4*q5**2*q6**2*q7 + 840*p**6*q4*q5**2*q6*q7**2 + 840*p**6*q4*q5**2*q6*q7*q8 - 5040*p**6*q4*q5**2*q6*q7 + 840*p**6*q4*q5*q6**3*q7 + 840*p**6*q4*q5*q6**2*q7**2 + 840*p**6*q4*q5*q6**2*q7*q8 - 5040*p**6*q4*q5*q6**2*q7 + 840*p**6*q4*q5*q6*q7**3 + 840*p**6*q4*q5*q6*q7**2*q8 - 5040*p**6*q4*q5*q6*q7**2 + 840*p**6*q4*q5*q6*q7*q8**2 - 5040*p**6*q4*q5*q6*q7*q8 + 12600*p**6*q4*q5*q6*q7 - 756*p**6*q5**3*q6*q7 - 756*p**6*q5**2*q6**2*q7 - 756*p**6*q5**2*q6*q7**2 - 756*p**6*q5**2*q6*q7*q8 + 3780*p**6*q5**2*q6*q7 - 756*p**6*q5*q6**3*q7 - 756*p**6*q5*q6**2*q7**2 - 756*p**6*q5*q6**2*q7*q8 + 3780*p**6*q5*q6**2*q7 - 756*p**6*q5*q6*q7**3 - 756*p**6*q5*q6*q7**2*q8 + 3780*p**6*q5*q6*q7**2 - 756*p**6*q5*q6*q7*q8**2 + 3780*p**6*q5*q6*q7*q8 - 7560*p**6*q5*q6*q7 + 420*p**6*q6**3*q7 + 420*p**6*q6**2*q7**2 + 420*p**6*q6**2*q7*q8 - 1680*p**6*q6**2*q7 + 420*p**6*q6*q7**3 + 420*p**6*q6*q7**2*q8 - 1680*p**6*q6*q7**2 + 420*p**6*q6*q7*q8**2 - 1680*p**6*q6*q7*q8 + 2520*p**6*q6*q7 - 120*p**6*q7**3 - 120*p**6*q7**2*q8 + 360*p**6*q7**2 - 120*p**6*q7*q8**2 + 360*p**6*q7*q8 - 360*p**6*q7 + 210*p**5*q1**3*q2*q3*q4*q5*q6*q7 + 210*p**5*q1**2*q2**2*q3*q4*q5*q6*q7 + 210*p**5*q1**2*q2*q3**2*q4*q5*q6*q7 + 210*p**5*q1**2*q2*q3*q4**2*q5*q6*q7 + 210*p**5*q1**2*q2*q3*q4*q5**2*q6*q7 + 210*p**5*q1**2*q2*q3*q4*q5*q6**2*q7 + 210*p**5*q1**2*q2*q3*q4*q5*q6*q7**2 + 210*p**5*q1**2*q2*q3*q4*q5*q6*q7*q8 - 1890*p**5*q1**2*q2*q3*q4*q5*q6*q7 + 210*p**5*q1*q2**3*q3*q4*q5*q6*q7 + 210*p**5*q1*q2**2*q3**2*q4*q5*q6*q7 + 210*p**5*q1*q2**2*q3*q4**2*q5*q6*q7 + 210*p**5*q1*q2**2*q3*q4*q5**2*q6*q7 + 210*p**5*q1*q2**2*q3*q4*q5*q6**2*q7 + 210*p**5*q1*q2**2*q3*q4*q5*q6*q7**2 + 210*p**5*q1*q2**2*q3*q4*q5*q6*q7*q8 - 1890*p**5*q1*q2**2*q3*q4*q5*q6*q7 + 210*p**5*q1*q2*q3**3*q4*q5*q6*q7 + 210*p**5*q1*q2*q3**2*q4**2*q5*q6*q7 + 210*p**5*q1*q2*q3**2*q4*q5**2*q6*q7 + 210*p**5*q1*q2*q3**2*q4*q5*q6**2*q7 + 210*p**5*q1*q2*q3**2*q4*q5*q6*q7**2 + 210*p**5*q1*q2*q3**2*q4*q5*q6*q7*q8 - 1890*p**5*q1*q2*q3**2*q4*q5*q6*q7 + 210*p**5*q1*q2*q3*q4**3*q5*q6*q7 + 210*p**5*q1*q2*q3*q4**2*q5**2*q6*q7 + 210*p**5*q1*q2*q3*q4**2*q5*q6**2*q7 + 210*p**5*q1*q2*q3*q4**2*q5*q6*q7**2 + 210*p**5*q1*q2*q3*q4**2*q5*q6*q7*q8 - 1890*p**5*q1*q2*q3*q4**2*q5*q6*q7 + 210*p**5*q1*q2*q3*q4*q5**3*q6*q7 + 210*p**5*q1*q2*q3*q4*q5**2*q6**2*q7 + 210*p**5*q1*q2*q3*q4*q5**2*q6*q7**2 + 210*p**5*q1*q2*q3*q4*q5**2*q6*q7*q8 - 1890*p**5*q1*q2*q3*q4*q5**2*q6*q7 + 210*p**5*q1*q2*q3*q4*q5*q6**3*q7 + 210*p**5*q1*q2*q3*q4*q5*q6**2*q7**2 + 210*p**5*q1*q2*q3*q4*q5*q6**2*q7*q8 - 1890*p**5*q1*q2*q3*q4*q5*q6**2*q7 + 210*p**5*q1*q2*q3*q4*q5*q6*q7**3 + 210*p**5*q1*q2*q3*q4*q5*q6*q7**2*q8 - 1890*p**5*q1*q2*q3*q4*q5*q6*q7**2 + 210*p**5*q1*q2*q3*q4*q5*q6*q7*q8**2 - 1890*p**5*q1*q2*q3*q4*q5*q6*q7*q8 + 7560*p**5*q1*q2*q3*q4*q5*q6*q7 - 675*p**5*q2**3*q3*q4*q5*q6*q7 - 675*p**5*q2**2*q3**2*q4*q5*q6*q7 - 675*p**5*q2**2*q3*q4**2*q5*q6*q7 - 675*p**5*q2**2*q3*q4*q5**2*q6*q7 - 675*p**5*q2**2*q3*q4*q5*q6**2*q7 - 675*p**5*q2**2*q3*q4*q5*q6*q7**2 - 675*p**5*q2**2*q3*q4*q5*q6*q7*q8 + 5400*p**5*q2**2*q3*q4*q5*q6*q7 - 675*p**5*q2*q3**3*q4*q5*q6*q7 - 675*p**5*q2*q3**2*q4**2*q5*q6*q7 - 675*p**5*q2*q3**2*q4*q5**2*q6*q7 - 675*p**5*q2*q3**2*q4*q5*q6**2*q7 - 675*p**5*q2*q3**2*q4*q5*q6*q7**2 - 675*p**5*q2*q3**2*q4*q5*q6*q7*q8 + 5400*p**5*q2*q3**2*q4*q5*q6*q7 - 675*p**5*q2*q3*q4**3*q5*q6*q7 - 675*p**5*q2*q3*q4**2*q5**2*q6*q7 - 675*p**5*q2*q3*q4**2*q5*q6**2*q7 - 675*p**5*q2*q3*q4**2*q5*q6*q7**2 - 675*p**5*q2*q3*q4**2*q5*q6*q7*q8 + 5400*p**5*q2*q3*q4**2*q5*q6*q7 - 675*p**5*q2*q3*q4*q5**3*q6*q7 - 675*p**5*q2*q3*q4*q5**2*q6**2*q7 - 675*p**5*q2*q3*q4*q5**2*q6*q7**2 - 675*p**5*q2*q3*q4*q5**2*q6*q7*q8 + 5400*p**5*q2*q3*q4*q5**2*q6*q7 - 675*p**5*q2*q3*q4*q5*q6**3*q7 - 675*p**5*q2*q3*q4*q5*q6**2*q7**2 - 675*p**5*q2*q3*q4*q5*q6**2*q7*q8 + 5400*p**5*q2*q3*q4*q5*q6**2*q7 - 675*p**5*q2*q3*q4*q5*q6*q7**3 - 675*p**5*q2*q3*q4*q5*q6*q7**2*q8 + 5400*p**5*q2*q3*q4*q5*q6*q7**2 - 675*p**5*q2*q3*q4*q5*q6*q7*q8**2 + 5400*p**5*q2*q3*q4*q5*q6*q7*q8 - 18900*p**5*q2*q3*q4*q5*q6*q7 + 1200*p**5*q3**3*q4*q5*q6*q7 + 1200*p**5*q3**2*q4**2*q5*q6*q7 + 1200*p**5*q3**2*q4*q5**2*q6*q7 + 1200*p**5*q3**2*q4*q5*q6**2*q7 + 1200*p**5*q3**2*q4*q5*q6*q7**2 + 1200*p**5*q3**2*q4*q5*q6*q7*q8 - 8400*p**5*q3**2*q4*q5*q6*q7 + 1200*p**5*q3*q4**3*q5*q6*q7 + 1200*p**5*q3*q4**2*q5**2*q6*q7 + 1200*p**5*q3*q4**2*q5*q6**2*q7 + 1200*p**5*q3*q4**2*q5*q6*q7**2 + 1200*p**5*q3*q4**2*q5*q6*q7*q8 - 8400*p**5*q3*q4**2*q5*q6*q7 + 1200*p**5*q3*q4*q5**3*q6*q7 + 1200*p**5*q3*q4*q5**2*q6**2*q7 + 1200*p**5*q3*q4*q5**2*q6*q7**2 + 1200*p**5*q3*q4*q5**2*q6*q7*q8 - 8400*p**5*q3*q4*q5**2*q6*q7 + 1200*p**5*q3*q4*q5*q6**3*q7 + 1200*p**5*q3*q4*q5*q6**2*q7**2 + 1200*p**5*q3*q4*q5*q6**2*q7*q8 - 8400*p**5*q3*q4*q5*q6**2*q7 + 1200*p**5*q3*q4*q5*q6*q7**3 + 1200*p**5*q3*q4*q5*q6*q7**2*q8 - 8400*p**5*q3*q4*q5*q6*q7**2 + 1200*p**5*q3*q4*q5*q6*q7*q8**2 - 8400*p**5*q3*q4*q5*q6*q7*q8 + 25200*p**5*q3*q4*q5*q6*q7 - 1260*p**5*q4**3*q5*q6*q7 - 1260*p**5*q4**2*q5**2*q6*q7 - 1260*p**5*q4**2*q5*q6**2*q7 - 1260*p**5*q4**2*q5*q6*q7**2 - 1260*p**5*q4**2*q5*q6*q7*q8 + 7560*p**5*q4**2*q5*q6*q7 - 1260*p**5*q4*q5**3*q6*q7 - 1260*p**5*q4*q5**2*q6**2*q7 - 1260*p**5*q4*q5**2*q6*q7**2 - 1260*p**5*q4*q5**2*q6*q7*q8 + 7560*p**5*q4*q5**2*q6*q7 - 1260*p**5*q4*q5*q6**3*q7 - 1260*p**5*q4*q5*q6**2*q7**2 - 1260*p**5*q4*q5*q6**2*q7*q8 + 7560*p**5*q4*q5*q6**2*q7 - 1260*p**5*q4*q5*q6*q7**3 - 1260*p**5*q4*q5*q6*q7**2*q8 + 7560*p**5*q4*q5*q6*q7**2 - 1260*p**5*q4*q5*q6*q7*q8**2 + 7560*p**5*q4*q5*q6*q7*q8 - 18900*p**5*q4*q5*q6*q7 + 756*p**5*q5**3*q6*q7 + 756*p**5*q5**2*q6**2*q7 + 756*p**5*q5**2*q6*q7**2 + 756*p**5*q5**2*q6*q7*q8 - 3780*p**5*q5**2*q6*q7 + 756*p**5*q5*q6**3*q7 + 756*p**5*q5*q6**2*q7**2 + 756*p**5*q5*q6**2*q7*q8 - 3780*p**5*q5*q6**2*q7 + 756*p**5*q5*q6*q7**3 + 756*p**5*q5*q6*q7**2*q8 - 3780*p**5*q5*q6*q7**2 + 756*p**5*q5*q6*q7*q8**2 - 3780*p**5*q5*q6*q7*q8 + 7560*p**5*q5*q6*q7 - 210*p**5*q6**3*q7 - 210*p**5*q6**2*q7**2 - 210*p**5*q6**2*q7*q8 + 840*p**5*q6**2*q7 - 210*p**5*q6*q7**3 - 210*p**5*q6*q7**2*q8 + 840*p**5*q6*q7**2 - 210*p**5*q6*q7*q8**2 + 840*p**5*q6*q7*q8 - 1260*p**5*q6*q7 - 350*p**4*q1**3*q2*q3*q4*q5*q6*q7 - 350*p**4*q1**2*q2**2*q3*q4*q5*q6*q7 - 350*p**4*q1**2*q2*q3**2*q4*q5*q6*q7 - 350*p**4*q1**2*q2*q3*q4**2*q5*q6*q7 - 350*p**4*q1**2*q2*q3*q4*q5**2*q6*q7 - 350*p**4*q1**2*q2*q3*q4*q5*q6**2*q7 - 350*p**4*q1**2*q2*q3*q4*q5*q6*q7**2 - 350*p**4*q1**2*q2*q3*q4*q5*q6*q7*q8 + 3150*p**4*q1**2*q2*q3*q4*q5*q6*q7 - 350*p**4*q1*q2**3*q3*q4*q5*q6*q7 - 350*p**4*q1*q2**2*q3**2*q4*q5*q6*q7 - 350*p**4*q1*q2**2*q3*q4**2*q5*q6*q7 - 350*p**4*q1*q2**2*q3*q4*q5**2*q6*q7 - 350*p**4*q1*q2**2*q3*q4*q5*q6**2*q7 - 350*p**4*q1*q2**2*q3*q4*q5*q6*q7**2 - 350*p**4*q1*q2**2*q3*q4*q5*q6*q7*q8 + 3150*p**4*q1*q2**2*q3*q4*q5*q6*q7 - 350*p**4*q1*q2*q3**3*q4*q5*q6*q7 - 350*p**4*q1*q2*q3**2*q4**2*q5*q6*q7 - 350*p**4*q1*q2*q3**2*q4*q5**2*q6*q7 - 350*p**4*q1*q2*q3**2*q4*q5*q6**2*q7 - 350*p**4*q1*q2*q3**2*q4*q5*q6*q7**2 - 350*p**4*q1*q2*q3**2*q4*q5*q6*q7*q8 + 3150*p**4*q1*q2*q3**2*q4*q5*q6*q7 - 350*p**4*q1*q2*q3*q4**3*q5*q6*q7 - 350*p**4*q1*q2*q3*q4**2*q5**2*q6*q7 - 350*p**4*q1*q2*q3*q4**2*q5*q6**2*q7 - 350*p**4*q1*q2*q3*q4**2*q5*q6*q7**2 - 350*p**4*q1*q2*q3*q4**2*q5*q6*q7*q8 + 3150*p**4*q1*q2*q3*q4**2*q5*q6*q7 - 350*p**4*q1*q2*q3*q4*q5**3*q6*q7 - 350*p**4*q1*q2*q3*q4*q5**2*q6**2*q7 - 350*p**4*q1*q2*q3*q4*q5**2*q6*q7**2 - 350*p**4*q1*q2*q3*q4*q5**2*q6*q7*q8 + 3150*p**4*q1*q2*q3*q4*q5**2*q6*q7 - 350*p**4*q1*q2*q3*q4*q5*q6**3*q7 - 350*p**4*q1*q2*q3*q4*q5*q6**2*q7**2 - 350*p**4*q1*q2*q3*q4*q5*q6**2*q7*q8 + 3150*p**4*q1*q2*q3*q4*q5*q6**2*q7 - 350*p**4*q1*q2*q3*q4*q5*q6*q7**3 - 350*p**4*q1*q2*q3*q4*q5*q6*q7**2*q8 + 3150*p**4*q1*q2*q3*q4*q5*q6*q7**2 - 350*p**4*q1*q2*q3*q4*q5*q6*q7*q8**2 + 3150*p**4*q1*q2*q3*q4*q5*q6*q7*q8 - 12600*p**4*q1*q2*q3*q4*q5*q6*q7 + 900*p**4*q2**3*q3*q4*q5*q6*q7 + 900*p**4*q2**2*q3**2*q4*q5*q6*q7 + 900*p**4*q2**2*q3*q4**2*q5*q6*q7 + 900*p**4*q2**2*q3*q4*q5**2*q6*q7 + 900*p**4*q2**2*q3*q4*q5*q6**2*q7 + 900*p**4*q2**2*q3*q4*q5*q6*q7**2 + 900*p**4*q2**2*q3*q4*q5*q6*q7*q8 - 7200*p**4*q2**2*q3*q4*q5*q6*q7 + 900*p**4*q2*q3**3*q4*q5*q6*q7 + 900*p**4*q2*q3**2*q4**2*q5*q6*q7 + 900*p**4*q2*q3**2*q4*q5**2*q6*q7 + 900*p**4*q2*q3**2*q4*q5*q6**2*q7 + 900*p**4*q2*q3**2*q4*q5*q6*q7**2 + 900*p**4*q2*q3**2*q4*q5*q6*q7*q8 - 7200*p**4*q2*q3**2*q4*q5*q6*q7 + 900*p**4*q2*q3*q4**3*q5*q6*q7 + 900*p**4*q2*q3*q4**2*q5**2*q6*q7 + 900*p**4*q2*q3*q4**2*q5*q6**2*q7 + 900*p**4*q2*q3*q4**2*q5*q6*q7**2 + 900*p**4*q2*q3*q4**2*q5*q6*q7*q8 - 7200*p**4*q2*q3*q4**2*q5*q6*q7 + 900*p**4*q2*q3*q4*q5**3*q6*q7 + 900*p**4*q2*q3*q4*q5**2*q6**2*q7 + 900*p**4*q2*q3*q4*q5**2*q6*q7**2 + 900*p**4*q2*q3*q4*q5**2*q6*q7*q8 - 7200*p**4*q2*q3*q4*q5**2*q6*q7 + 900*p**4*q2*q3*q4*q5*q6**3*q7 + 900*p**4*q2*q3*q4*q5*q6**2*q7**2 + 900*p**4*q2*q3*q4*q5*q6**2*q7*q8 - 7200*p**4*q2*q3*q4*q5*q6**2*q7 + 900*p**4*q2*q3*q4*q5*q6*q7**3 + 900*p**4*q2*q3*q4*q5*q6*q7**2*q8 - 7200*p**4*q2*q3*q4*q5*q6*q7**2 + 900*p**4*q2*q3*q4*q5*q6*q7*q8**2 - 7200*p**4*q2*q3*q4*q5*q6*q7*q8 + 25200*p**4*q2*q3*q4*q5*q6*q7 - 1200*p**4*q3**3*q4*q5*q6*q7 - 1200*p**4*q3**2*q4**2*q5*q6*q7 - 1200*p**4*q3**2*q4*q5**2*q6*q7 - 1200*p**4*q3**2*q4*q5*q6**2*q7 - 1200*p**4*q3**2*q4*q5*q6*q7**2 - 1200*p**4*q3**2*q4*q5*q6*q7*q8 + 8400*p**4*q3**2*q4*q5*q6*q7 - 1200*p**4*q3*q4**3*q5*q6*q7 - 1200*p**4*q3*q4**2*q5**2*q6*q7 - 1200*p**4*q3*q4**2*q5*q6**2*q7 - 1200*p**4*q3*q4**2*q5*q6*q7**2 - 1200*p**4*q3*q4**2*q5*q6*q7*q8 + 8400*p**4*q3*q4**2*q5*q6*q7 - 1200*p**4*q3*q4*q5**3*q6*q7 - 1200*p**4*q3*q4*q5**2*q6**2*q7 - 1200*p**4*q3*q4*q5**2*q6*q7**2 - 1200*p**4*q3*q4*q5**2*q6*q7*q8 + 8400*p**4*q3*q4*q5**2*q6*q7 - 1200*p**4*q3*q4*q5*q6**3*q7 - 1200*p**4*q3*q4*q5*q6**2*q7**2 - 1200*p**4*q3*q4*q5*q6**2*q7*q8 + 8400*p**4*q3*q4*q5*q6**2*q7 - 1200*p**4*q3*q4*q5*q6*q7**3 - 1200*p**4*q3*q4*q5*q6*q7**2*q8 + 8400*p**4*q3*q4*q5*q6*q7**2 - 1200*p**4*q3*q4*q5*q6*q7*q8**2 + 8400*p**4*q3*q4*q5*q6*q7*q8 - 25200*p**4*q3*q4*q5*q6*q7 + 840*p**4*q4**3*q5*q6*q7 + 840*p**4*q4**2*q5**2*q6*q7 + 840*p**4*q4**2*q5*q6**2*q7 + 840*p**4*q4**2*q5*q6*q7**2 + 840*p**4*q4**2*q5*q6*q7*q8 - 5040*p**4*q4**2*q5*q6*q7 + 840*p**4*q4*q5**3*q6*q7 + 840*p**4*q4*q5**2*q6**2*q7 + 840*p**4*q4*q5**2*q6*q7**2 + 840*p**4*q4*q5**2*q6*q7*q8 - 5040*p**4*q4*q5**2*q6*q7 + 840*p**4*q4*q5*q6**3*q7 + 840*p**4*q4*q5*q6**2*q7**2 + 840*p**4*q4*q5*q6**2*q7*q8 - 5040*p**4*q4*q5*q6**2*q7 + 840*p**4*q4*q5*q6*q7**3 + 840*p**4*q4*q5*q6*q7**2*q8 - 5040*p**4*q4*q5*q6*q7**2 + 840*p**4*q4*q5*q6*q7*q8**2 - 5040*p**4*q4*q5*q6*q7*q8 + 12600*p**4*q4*q5*q6*q7 - 252*p**4*q5**3*q6*q7 - 252*p**4*q5**2*q6**2*q7 - 252*p**4*q5**2*q6*q7**2 - 252*p**4*q5**2*q6*q7*q8 + 1260*p**4*q5**2*q6*q7 - 252*p**4*q5*q6**3*q7 - 252*p**4*q5*q6**2*q7**2 - 252*p**4*q5*q6**2*q7*q8 + 1260*p**4*q5*q6**2*q7 - 252*p**4*q5*q6*q7**3 - 252*p**4*q5*q6*q7**2*q8 + 1260*p**4*q5*q6*q7**2 - 252*p**4*q5*q6*q7*q8**2 + 1260*p**4*q5*q6*q7*q8 - 2520*p**4*q5*q6*q7 + 350*p**3*q1**3*q2*q3*q4*q5*q6*q7 + 350*p**3*q1**2*q2**2*q3*q4*q5*q6*q7 + 350*p**3*q1**2*q2*q3**2*q4*q5*q6*q7 + 350*p**3*q1**2*q2*q3*q4**2*q5*q6*q7 + 350*p**3*q1**2*q2*q3*q4*q5**2*q6*q7 + 350*p**3*q1**2*q2*q3*q4*q5*q6**2*q7 + 350*p**3*q1**2*q2*q3*q4*q5*q6*q7**2 + 350*p**3*q1**2*q2*q3*q4*q5*q6*q7*q8 - 3150*p**3*q1**2*q2*q3*q4*q5*q6*q7 + 350*p**3*q1*q2**3*q3*q4*q5*q6*q7 + 350*p**3*q1*q2**2*q3**2*q4*q5*q6*q7 + 350*p**3*q1*q2**2*q3*q4**2*q5*q6*q7 + 350*p**3*q1*q2**2*q3*q4*q5**2*q6*q7 + 350*p**3*q1*q2**2*q3*q4*q5*q6**2*q7 + 350*p**3*q1*q2**2*q3*q4*q5*q6*q7**2 + 350*p**3*q1*q2**2*q3*q4*q5*q6*q7*q8 - 3150*p**3*q1*q2**2*q3*q4*q5*q6*q7 + 350*p**3*q1*q2*q3**3*q4*q5*q6*q7 + 350*p**3*q1*q2*q3**2*q4**2*q5*q6*q7 + 350*p**3*q1*q2*q3**2*q4*q5**2*q6*q7 + 350*p**3*q1*q2*q3**2*q4*q5*q6**2*q7 + 350*p**3*q1*q2*q3**2*q4*q5*q6*q7**2 + 350*p**3*q1*q2*q3**2*q4*q5*q6*q7*q8 - 3150*p**3*q1*q2*q3**2*q4*q5*q6*q7 + 350*p**3*q1*q2*q3*q4**3*q5*q6*q7 + 350*p**3*q1*q2*q3*q4**2*q5**2*q6*q7 + 350*p**3*q1*q2*q3*q4**2*q5*q6**2*q7 + 350*p**3*q1*q2*q3*q4**2*q5*q6*q7**2 + 350*p**3*q1*q2*q3*q4**2*q5*q6*q7*q8 - 3150*p**3*q1*q2*q3*q4**2*q5*q6*q7 + 350*p**3*q1*q2*q3*q4*q5**3*q6*q7 + 350*p**3*q1*q2*q3*q4*q5**2*q6**2*q7 + 350*p**3*q1*q2*q3*q4*q5**2*q6*q7**2 + 350*p**3*q1*q2*q3*q4*q5**2*q6*q7*q8 - 3150*p**3*q1*q2*q3*q4*q5**2*q6*q7 + 350*p**3*q1*q2*q3*q4*q5*q6**3*q7 + 350*p**3*q1*q2*q3*q4*q5*q6**2*q7**2 + 350*p**3*q1*q2*q3*q4*q5*q6**2*q7*q8 - 3150*p**3*q1*q2*q3*q4*q5*q6**2*q7 + 350*p**3*q1*q2*q3*q4*q5*q6*q7**3 + 350*p**3*q1*q2*q3*q4*q5*q6*q7**2*q8 - 3150*p**3*q1*q2*q3*q4*q5*q6*q7**2 + 350*p**3*q1*q2*q3*q4*q5*q6*q7*q8**2 - 3150*p**3*q1*q2*q3*q4*q5*q6*q7*q8 + 12600*p**3*q1*q2*q3*q4*q5*q6*q7 - 675*p**3*q2**3*q3*q4*q5*q6*q7 - 675*p**3*q2**2*q3**2*q4*q5*q6*q7 - 675*p**3*q2**2*q3*q4**2*q5*q6*q7 - 675*p**3*q2**2*q3*q4*q5**2*q6*q7 - 675*p**3*q2**2*q3*q4*q5*q6**2*q7 - 675*p**3*q2**2*q3*q4*q5*q6*q7**2 - 675*p**3*q2**2*q3*q4*q5*q6*q7*q8 + 5400*p**3*q2**2*q3*q4*q5*q6*q7 - 675*p**3*q2*q3**3*q4*q5*q6*q7 - 675*p**3*q2*q3**2*q4**2*q5*q6*q7 - 675*p**3*q2*q3**2*q4*q5**2*q6*q7 - 675*p**3*q2*q3**2*q4*q5*q6**2*q7 - 675*p**3*q2*q3**2*q4*q5*q6*q7**2 - 675*p**3*q2*q3**2*q4*q5*q6*q7*q8 + 5400*p**3*q2*q3**2*q4*q5*q6*q7 - 675*p**3*q2*q3*q4**3*q5*q6*q7 - 675*p**3*q2*q3*q4**2*q5**2*q6*q7 - 675*p**3*q2*q3*q4**2*q5*q6**2*q7 - 675*p**3*q2*q3*q4**2*q5*q6*q7**2 - 675*p**3*q2*q3*q4**2*q5*q6*q7*q8 + 5400*p**3*q2*q3*q4**2*q5*q6*q7 - 675*p**3*q2*q3*q4*q5**3*q6*q7 - 675*p**3*q2*q3*q4*q5**2*q6**2*q7 - 675*p**3*q2*q3*q4*q5**2*q6*q7**2 - 675*p**3*q2*q3*q4*q5**2*q6*q7*q8 + 5400*p**3*q2*q3*q4*q5**2*q6*q7 - 675*p**3*q2*q3*q4*q5*q6**3*q7 - 675*p**3*q2*q3*q4*q5*q6**2*q7**2 - 675*p**3*q2*q3*q4*q5*q6**2*q7*q8 + 5400*p**3*q2*q3*q4*q5*q6**2*q7 - 675*p**3*q2*q3*q4*q5*q6*q7**3 - 675*p**3*q2*q3*q4*q5*q6*q7**2*q8 + 5400*p**3*q2*q3*q4*q5*q6*q7**2 - 675*p**3*q2*q3*q4*q5*q6*q7*q8**2 + 5400*p**3*q2*q3*q4*q5*q6*q7*q8 - 18900*p**3*q2*q3*q4*q5*q6*q7 + 600*p**3*q3**3*q4*q5*q6*q7 + 600*p**3*q3**2*q4**2*q5*q6*q7 + 600*p**3*q3**2*q4*q5**2*q6*q7 + 600*p**3*q3**2*q4*q5*q6**2*q7 + 600*p**3*q3**2*q4*q5*q6*q7**2 + 600*p**3*q3**2*q4*q5*q6*q7*q8 - 4200*p**3*q3**2*q4*q5*q6*q7 + 600*p**3*q3*q4**3*q5*q6*q7 + 600*p**3*q3*q4**2*q5**2*q6*q7 + 600*p**3*q3*q4**2*q5*q6**2*q7 + 600*p**3*q3*q4**2*q5*q6*q7**2 + 600*p**3*q3*q4**2*q5*q6*q7*q8 - 4200*p**3*q3*q4**2*q5*q6*q7 + 600*p**3*q3*q4*q5**3*q6*q7 + 600*p**3*q3*q4*q5**2*q6**2*q7 + 600*p**3*q3*q4*q5**2*q6*q7**2 + 600*p**3*q3*q4*q5**2*q6*q7*q8 - 4200*p**3*q3*q4*q5**2*q6*q7 + 600*p**3*q3*q4*q5*q6**3*q7 + 600*p**3*q3*q4*q5*q6**2*q7**2 + 600*p**3*q3*q4*q5*q6**2*q7*q8 - 4200*p**3*q3*q4*q5*q6**2*q7 + 600*p**3*q3*q4*q5*q6*q7**3 + 600*p**3*q3*q4*q5*q6*q7**2*q8 - 4200*p**3*q3*q4*q5*q6*q7**2 + 600*p**3*q3*q4*q5*q6*q7*q8**2 - 4200*p**3*q3*q4*q5*q6*q7*q8 + 12600*p**3*q3*q4*q5*q6*q7 - 210*p**3*q4**3*q5*q6*q7 - 210*p**3*q4**2*q5**2*q6*q7 - 210*p**3*q4**2*q5*q6**2*q7 - 210*p**3*q4**2*q5*q6*q7**2 - 210*p**3*q4**2*q5*q6*q7*q8 + 1260*p**3*q4**2*q5*q6*q7 - 210*p**3*q4*q5**3*q6*q7 - 210*p**3*q4*q5**2*q6**2*q7 - 210*p**3*q4*q5**2*q6*q7**2 - 210*p**3*q4*q5**2*q6*q7*q8 + 1260*p**3*q4*q5**2*q6*q7 - 210*p**3*q4*q5*q6**3*q7 - 210*p**3*q4*q5*q6**2*q7**2 - 210*p**3*q4*q5*q6**2*q7*q8 + 1260*p**3*q4*q5*q6**2*q7 - 210*p**3*q4*q5*q6*q7**3 - 210*p**3*q4*q5*q6*q7**2*q8 + 1260*p**3*q4*q5*q6*q7**2 - 210*p**3*q4*q5*q6*q7*q8**2 + 1260*p**3*q4*q5*q6*q7*q8 - 3150*p**3*q4*q5*q6*q7 - 210*p**2*q1**3*q2*q3*q4*q5*q6*q7 - 210*p**2*q1**2*q2**2*q3*q4*q5*q6*q7 - 210*p**2*q1**2*q2*q3**2*q4*q5*q6*q7 - 210*p**2*q1**2*q2*q3*q4**2*q5*q6*q7 - 210*p**2*q1**2*q2*q3*q4*q5**2*q6*q7 - 210*p**2*q1**2*q2*q3*q4*q5*q6**2*q7 - 210*p**2*q1**2*q2*q3*q4*q5*q6*q7**2 - 210*p**2*q1**2*q2*q3*q4*q5*q6*q7*q8 + 1890*p**2*q1**2*q2*q3*q4*q5*q6*q7 - 210*p**2*q1*q2**3*q3*q4*q5*q6*q7 - 210*p**2*q1*q2**2*q3**2*q4*q5*q6*q7 - 210*p**2*q1*q2**2*q3*q4**2*q5*q6*q7 - 210*p**2*q1*q2**2*q3*q4*q5**2*q6*q7 - 210*p**2*q1*q2**2*q3*q4*q5*q6**2*q7 - 210*p**2*q1*q2**2*q3*q4*q5*q6*q7**2 - 210*p**2*q1*q2**2*q3*q4*q5*q6*q7*q8 + 1890*p**2*q1*q2**2*q3*q4*q5*q6*q7 - 210*p**2*q1*q2*q3**3*q4*q5*q6*q7 - 210*p**2*q1*q2*q3**2*q4**2*q5*q6*q7 - 210*p**2*q1*q2*q3**2*q4*q5**2*q6*q7 - 210*p**2*q1*q2*q3**2*q4*q5*q6**2*q7 - 210*p**2*q1*q2*q3**2*q4*q5*q6*q7**2 - 210*p**2*q1*q2*q3**2*q4*q5*q6*q7*q8 + 1890*p**2*q1*q2*q3**2*q4*q5*q6*q7 - 210*p**2*q1*q2*q3*q4**3*q5*q6*q7 - 210*p**2*q1*q2*q3*q4**2*q5**2*q6*q7 - 210*p**2*q1*q2*q3*q4**2*q5*q6**2*q7 - 210*p**2*q1*q2*q3*q4**2*q5*q6*q7**2 - 210*p**2*q1*q2*q3*q4**2*q5*q6*q7*q8 + 1890*p**2*q1*q2*q3*q4**2*q5*q6*q7 - 210*p**2*q1*q2*q3*q4*q5**3*q6*q7 - 210*p**2*q1*q2*q3*q4*q5**2*q6**2*q7 - 210*p**2*q1*q2*q3*q4*q5**2*q6*q7**2 - 210*p**2*q1*q2*q3*q4*q5**2*q6*q7*q8 + 1890*p**2*q1*q2*q3*q4*q5**2*q6*q7 - 210*p**2*q1*q2*q3*q4*q5*q6**3*q7 - 210*p**2*q1*q2*q3*q4*q5*q6**2*q7**2 - 210*p**2*q1*q2*q3*q4*q5*q6**2*q7*q8 + 1890*p**2*q1*q2*q3*q4*q5*q6**2*q7 - 210*p**2*q1*q2*q3*q4*q5*q6*q7**3 - 210*p**2*q1*q2*q3*q4*q5*q6*q7**2*q8 + 1890*p**2*q1*q2*q3*q4*q5*q6*q7**2 - 210*p**2*q1*q2*q3*q4*q5*q6*q7*q8**2 + 1890*p**2*q1*q2*q3*q4*q5*q6*q7*q8 - 7560*p**2*q1*q2*q3*q4*q5*q6*q7 + 270*p**2*q2**3*q3*q4*q5*q6*q7 + 270*p**2*q2**2*q3**2*q4*q5*q6*q7 + 270*p**2*q2**2*q3*q4**2*q5*q6*q7 + 270*p**2*q2**2*q3*q4*q5**2*q6*q7 + 270*p**2*q2**2*q3*q4*q5*q6**2*q7 + 270*p**2*q2**2*q3*q4*q5*q6*q7**2 + 270*p**2*q2**2*q3*q4*q5*q6*q7*q8 - 2160*p**2*q2**2*q3*q4*q5*q6*q7 + 270*p**2*q2*q3**3*q4*q5*q6*q7 + 270*p**2*q2*q3**2*q4**2*q5*q6*q7 + 270*p**2*q2*q3**2*q4*q5**2*q6*q7 + 270*p**2*q2*q3**2*q4*q5*q6**2*q7 + 270*p**2*q2*q3**2*q4*q5*q6*q7**2 + 270*p**2*q2*q3**2*q4*q5*q6*q7*q8 - 2160*p**2*q2*q3**2*q4*q5*q6*q7 + 270*p**2*q2*q3*q4**3*q5*q6*q7 + 270*p**2*q2*q3*q4**2*q5**2*q6*q7 + 270*p**2*q2*q3*q4**2*q5*q6**2*q7 + 270*p**2*q2*q3*q4**2*q5*q6*q7**2 + 270*p**2*q2*q3*q4**2*q5*q6*q7*q8 - 2160*p**2*q2*q3*q4**2*q5*q6*q7 + 270*p**2*q2*q3*q4*q5**3*q6*q7 + 270*p**2*q2*q3*q4*q5**2*q6**2*q7 + 270*p**2*q2*q3*q4*q5**2*q6*q7**2 + 270*p**2*q2*q3*q4*q5**2*q6*q7*q8 - 2160*p**2*q2*q3*q4*q5**2*q6*q7 + 270*p**2*q2*q3*q4*q5*q6**3*q7 + 270*p**2*q2*q3*q4*q5*q6**2*q7**2 + 270*p**2*q2*q3*q4*q5*q6**2*q7*q8 - 2160*p**2*q2*q3*q4*q5*q6**2*q7 + 270*p**2*q2*q3*q4*q5*q6*q7**3 + 270*p**2*q2*q3*q4*q5*q6*q7**2*q8 - 2160*p**2*q2*q3*q4*q5*q6*q7**2 + 270*p**2*q2*q3*q4*q5*q6*q7*q8**2 - 2160*p**2*q2*q3*q4*q5*q6*q7*q8 + 7560*p**2*q2*q3*q4*q5*q6*q7 - 120*p**2*q3**3*q4*q5*q6*q7 - 120*p**2*q3**2*q4**2*q5*q6*q7 - 120*p**2*q3**2*q4*q5**2*q6*q7 - 120*p**2*q3**2*q4*q5*q6**2*q7 - 120*p**2*q3**2*q4*q5*q6*q7**2 - 120*p**2*q3**2*q4*q5*q6*q7*q8 + 840*p**2*q3**2*q4*q5*q6*q7 - 120*p**2*q3*q4**3*q5*q6*q7 - 120*p**2*q3*q4**2*q5**2*q6*q7 - 120*p**2*q3*q4**2*q5*q6**2*q7 - 120*p**2*q3*q4**2*q5*q6*q7**2 - 120*p**2*q3*q4**2*q5*q6*q7*q8 + 840*p**2*q3*q4**2*q5*q6*q7 - 120*p**2*q3*q4*q5**3*q6*q7 - 120*p**2*q3*q4*q5**2*q6**2*q7 - 120*p**2*q3*q4*q5**2*q6*q7**2 - 120*p**2*q3*q4*q5**2*q6*q7*q8 + 840*p**2*q3*q4*q5**2*q6*q7 - 120*p**2*q3*q4*q5*q6**3*q7 - 120*p**2*q3*q4*q5*q6**2*q7**2 - 120*p**2*q3*q4*q5*q6**2*q7*q8 + 840*p**2*q3*q4*q5*q6**2*q7 - 120*p**2*q3*q4*q5*q6*q7**3 - 120*p**2*q3*q4*q5*q6*q7**2*q8 + 840*p**2*q3*q4*q5*q6*q7**2 - 120*p**2*q3*q4*q5*q6*q7*q8**2 + 840*p**2*q3*q4*q5*q6*q7*q8 - 2520*p**2*q3*q4*q5*q6*q7 + 70*p*q1**3*q2*q3*q4*q5*q6*q7 + 70*p*q1**2*q2**2*q3*q4*q5*q6*q7 + 70*p*q1**2*q2*q3**2*q4*q5*q6*q7 + 70*p*q1**2*q2*q3*q4**2*q5*q6*q7 + 70*p*q1**2*q2*q3*q4*q5**2*q6*q7 + 70*p*q1**2*q2*q3*q4*q5*q6**2*q7 + 70*p*q1**2*q2*q3*q4*q5*q6*q7**2 + 70*p*q1**2*q2*q3*q4*q5*q6*q7*q8 - 630*p*q1**2*q2*q3*q4*q5*q6*q7 + 70*p*q1*q2**3*q3*q4*q5*q6*q7 + 70*p*q1*q2**2*q3**2*q4*q5*q6*q7 + 70*p*q1*q2**2*q3*q4**2*q5*q6*q7 + 70*p*q1*q2**2*q3*q4*q5**2*q6*q7 + 70*p*q1*q2**2*q3*q4*q5*q6**2*q7 + 70*p*q1*q2**2*q3*q4*q5*q6*q7**2 + 70*p*q1*q2**2*q3*q4*q5*q6*q7*q8 - 630*p*q1*q2**2*q3*q4*q5*q6*q7 + 70*p*q1*q2*q3**3*q4*q5*q6*q7 + 70*p*q1*q2*q3**2*q4**2*q5*q6*q7 + 70*p*q1*q2*q3**2*q4*q5**2*q6*q7 + 70*p*q1*q2*q3**2*q4*q5*q6**2*q7 + 70*p*q1*q2*q3**2*q4*q5*q6*q7**2 + 70*p*q1*q2*q3**2*q4*q5*q6*q7*q8 - 630*p*q1*q2*q3**2*q4*q5*q6*q7 + 70*p*q1*q2*q3*q4**3*q5*q6*q7 + 70*p*q1*q2*q3*q4**2*q5**2*q6*q7 + 70*p*q1*q2*q3*q4**2*q5*q6**2*q7 + 70*p*q1*q2*q3*q4**2*q5*q6*q7**2 + 70*p*q1*q2*q3*q4**2*q5*q6*q7*q8 - 630*p*q1*q2*q3*q4**2*q5*q6*q7 + 70*p*q1*q2*q3*q4*q5**3*q6*q7 + 70*p*q1*q2*q3*q4*q5**2*q6**2*q7 + 70*p*q1*q2*q3*q4*q5**2*q6*q7**2 + 70*p*q1*q2*q3*q4*q5**2*q6*q7*q8 - 630*p*q1*q2*q3*q4*q5**2*q6*q7 + 70*p*q1*q2*q3*q4*q5*q6**3*q7 + 70*p*q1*q2*q3*q4*q5*q6**2*q7**2 + 70*p*q1*q2*q3*q4*q5*q6**2*q7*q8 - 630*p*q1*q2*q3*q4*q5*q6**2*q7 + 70*p*q1*q2*q3*q4*q5*q6*q7**3 + 70*p*q1*q2*q3*q4*q5*q6*q7**2*q8 - 630*p*q1*q2*q3*q4*q5*q6*q7**2 + 70*p*q1*q2*q3*q4*q5*q6*q7*q8**2 - 630*p*q1*q2*q3*q4*q5*q6*q7*q8 + 2520*p*q1*q2*q3*q4*q5*q6*q7 - 45*p*q2**3*q3*q4*q5*q6*q7 - 45*p*q2**2*q3**2*q4*q5*q6*q7 - 45*p*q2**2*q3*q4**2*q5*q6*q7 - 45*p*q2**2*q3*q4*q5**2*q6*q7 - 45*p*q2**2*q3*q4*q5*q6**2*q7 - 45*p*q2**2*q3*q4*q5*q6*q7**2 - 45*p*q2**2*q3*q4*q5*q6*q7*q8 + 360*p*q2**2*q3*q4*q5*q6*q7 - 45*p*q2*q3**3*q4*q5*q6*q7 - 45*p*q2*q3**2*q4**2*q5*q6*q7 - 45*p*q2*q3**2*q4*q5**2*q6*q7 - 45*p*q2*q3**2*q4*q5*q6**2*q7 - 45*p*q2*q3**2*q4*q5*q6*q7**2 - 45*p*q2*q3**2*q4*q5*q6*q7*q8 + 360*p*q2*q3**2*q4*q5*q6*q7 - 45*p*q2*q3*q4**3*q5*q6*q7 - 45*p*q2*q3*q4**2*q5**2*q6*q7 - 45*p*q2*q3*q4**2*q5*q6**2*q7 - 45*p*q2*q3*q4**2*q5*q6*q7**2 - 45*p*q2*q3*q4**2*q5*q6*q7*q8 + 360*p*q2*q3*q4**2*q5*q6*q7 - 45*p*q2*q3*q4*q5**3*q6*q7 - 45*p*q2*q3*q4*q5**2*q6**2*q7 - 45*p*q2*q3*q4*q5**2*q6*q7**2 - 45*p*q2*q3*q4*q5**2*q6*q7*q8 + 360*p*q2*q3*q4*q5**2*q6*q7 - 45*p*q2*q3*q4*q5*q6**3*q7 - 45*p*q2*q3*q4*q5*q6**2*q7**2 - 45*p*q2*q3*q4*q5*q6**2*q7*q8 + 360*p*q2*q3*q4*q5*q6**2*q7 - 45*p*q2*q3*q4*q5*q6*q7**3 - 45*p*q2*q3*q4*q5*q6*q7**2*q8 + 360*p*q2*q3*q4*q5*q6*q7**2 - 45*p*q2*q3*q4*q5*q6*q7*q8**2 + 360*p*q2*q3*q4*q5*q6*q7*q8 - 1260*p*q2*q3*q4*q5*q6*q7 - 10*q1**3*q2*q3*q4*q5*q6*q7 - 10*q1**2*q2**2*q3*q4*q5*q6*q7 - 10*q1**2*q2*q3**2*q4*q5*q6*q7 - 10*q1**2*q2*q3*q4**2*q5*q6*q7 - 10*q1**2*q2*q3*q4*q5**2*q6*q7 - 10*q1**2*q2*q3*q4*q5*q6**2*q7 - 10*q1**2*q2*q3*q4*q5*q6*q7**2 - 10*q1**2*q2*q3*q4*q5*q6*q7*q8 + 90*q1**2*q2*q3*q4*q5*q6*q7 - 10*q1*q2**3*q3*q4*q5*q6*q7 - 10*q1*q2**2*q3**2*q4*q5*q6*q7 - 10*q1*q2**2*q3*q4**2*q5*q6*q7 - 10*q1*q2**2*q3*q4*q5**2*q6*q7 - 10*q1*q2**2*q3*q4*q5*q6**2*q7 - 10*q1*q2**2*q3*q4*q5*q6*q7**2 - 10*q1*q2**2*q3*q4*q5*q6*q7*q8 + 90*q1*q2**2*q3*q4*q5*q6*q7 - 10*q1*q2*q3**3*q4*q5*q6*q7 - 10*q1*q2*q3**2*q4**2*q5*q6*q7 - 10*q1*q2*q3**2*q4*q5**2*q6*q7 - 10*q1*q2*q3**2*q4*q5*q6**2*q7 - 10*q1*q2*q3**2*q4*q5*q6*q7**2 - 10*q1*q2*q3**2*q4*q5*q6*q7*q8 + 90*q1*q2*q3**2*q4*q5*q6*q7 - 10*q1*q2*q3*q4**3*q5*q6*q7 - 10*q1*q2*q3*q4**2*q5**2*q6*q7 - 10*q1*q2*q3*q4**2*q5*q6**2*q7 - 10*q1*q2*q3*q4**2*q5*q6*q7**2 - 10*q1*q2*q3*q4**2*q5*q6*q7*q8 + 90*q1*q2*q3*q4**2*q5*q6*q7 - 10*q1*q2*q3*q4*q5**3*q6*q7 - 10*q1*q2*q3*q4*q5**2*q6**2*q7 - 10*q1*q2*q3*q4*q5**2*q6*q7**2 - 10*q1*q2*q3*q4*q5**2*q6*q7*q8 + 90*q1*q2*q3*q4*q5**2*q6*q7 - 10*q1*q2*q3*q4*q5*q6**3*q7 - 10*q1*q2*q3*q4*q5*q6**2*q7**2 - 10*q1*q2*q3*q4*q5*q6**2*q7*q8 + 90*q1*q2*q3*q4*q5*q6**2*q7 - 10*q1*q2*q3*q4*q5*q6*q7**3 - 10*q1*q2*q3*q4*q5*q6*q7**2*q8 + 90*q1*q2*q3*q4*q5*q6*q7**2 - 10*q1*q2*q3*q4*q5*q6*q7*q8**2 + 90*q1*q2*q3*q4*q5*q6*q7*q8 - 360*q1*q2*q3*q4*q5*q6*q7)'
f_multiparam[10][9] = 'p*(p - 1)*(10*p**8*q1**2*q2*q3*q4*q5*q6*q7*q8 + 10*p**8*q1*q2**2*q3*q4*q5*q6*q7*q8 + 10*p**8*q1*q2*q3**2*q4*q5*q6*q7*q8 + 10*p**8*q1*q2*q3*q4**2*q5*q6*q7*q8 + 10*p**8*q1*q2*q3*q4*q5**2*q6*q7*q8 + 10*p**8*q1*q2*q3*q4*q5*q6**2*q7*q8 + 10*p**8*q1*q2*q3*q4*q5*q6*q7**2*q8 + 10*p**8*q1*q2*q3*q4*q5*q6*q7*q8**2 + 10*p**8*q1*q2*q3*q4*q5*q6*q7*q8*q9 - 90*p**8*q1*q2*q3*q4*q5*q6*q7*q8 - 45*p**8*q2**2*q3*q4*q5*q6*q7*q8 - 45*p**8*q2*q3**2*q4*q5*q6*q7*q8 - 45*p**8*q2*q3*q4**2*q5*q6*q7*q8 - 45*p**8*q2*q3*q4*q5**2*q6*q7*q8 - 45*p**8*q2*q3*q4*q5*q6**2*q7*q8 - 45*p**8*q2*q3*q4*q5*q6*q7**2*q8 - 45*p**8*q2*q3*q4*q5*q6*q7*q8**2 - 45*p**8*q2*q3*q4*q5*q6*q7*q8*q9 + 360*p**8*q2*q3*q4*q5*q6*q7*q8 + 120*p**8*q3**2*q4*q5*q6*q7*q8 + 120*p**8*q3*q4**2*q5*q6*q7*q8 + 120*p**8*q3*q4*q5**2*q6*q7*q8 + 120*p**8*q3*q4*q5*q6**2*q7*q8 + 120*p**8*q3*q4*q5*q6*q7**2*q8 + 120*p**8*q3*q4*q5*q6*q7*q8**2 + 120*p**8*q3*q4*q5*q6*q7*q8*q9 - 840*p**8*q3*q4*q5*q6*q7*q8 - 210*p**8*q4**2*q5*q6*q7*q8 - 210*p**8*q4*q5**2*q6*q7*q8 - 210*p**8*q4*q5*q6**2*q7*q8 - 210*p**8*q4*q5*q6*q7**2*q8 - 210*p**8*q4*q5*q6*q7*q8**2 - 210*p**8*q4*q5*q6*q7*q8*q9 + 1260*p**8*q4*q5*q6*q7*q8 + 252*p**8*q5**2*q6*q7*q8 + 252*p**8*q5*q6**2*q7*q8 + 252*p**8*q5*q6*q7**2*q8 + 252*p**8*q5*q6*q7*q8**2 + 252*p**8*q5*q6*q7*q8*q9 - 1260*p**8*q5*q6*q7*q8 - 210*p**8*q6**2*q7*q8 - 210*p**8*q6*q7**2*q8 - 210*p**8*q6*q7*q8**2 - 210*p**8*q6*q7*q8*q9 + 840*p**8*q6*q7*q8 + 120*p**8*q7**2*q8 + 120*p**8*q7*q8**2 + 120*p**8*q7*q8*q9 - 360*p**8*q7*q8 - 45*p**8*q8**2 - 45*p**8*q8*q9 + 90*p**8*q8 + 10*p**8*q9 - 10*p**8 - 80*p**7*q1**2*q2*q3*q4*q5*q6*q7*q8 - 80*p**7*q1*q2**2*q3*q4*q5*q6*q7*q8 - 80*p**7*q1*q2*q3**2*q4*q5*q6*q7*q8 - 80*p**7*q1*q2*q3*q4**2*q5*q6*q7*q8 - 80*p**7*q1*q2*q3*q4*q5**2*q6*q7*q8 - 80*p**7*q1*q2*q3*q4*q5*q6**2*q7*q8 - 80*p**7*q1*q2*q3*q4*q5*q6*q7**2*q8 - 80*p**7*q1*q2*q3*q4*q5*q6*q7*q8**2 - 80*p**7*q1*q2*q3*q4*q5*q6*q7*q8*q9 + 720*p**7*q1*q2*q3*q4*q5*q6*q7*q8 + 315*p**7*q2**2*q3*q4*q5*q6*q7*q8 + 315*p**7*q2*q3**2*q4*q5*q6*q7*q8 + 315*p**7*q2*q3*q4**2*q5*q6*q7*q8 + 315*p**7*q2*q3*q4*q5**2*q6*q7*q8 + 315*p**7*q2*q3*q4*q5*q6**2*q7*q8 + 315*p**7*q2*q3*q4*q5*q6*q7**2*q8 + 315*p**7*q2*q3*q4*q5*q6*q7*q8**2 + 315*p**7*q2*q3*q4*q5*q6*q7*q8*q9 - 2520*p**7*q2*q3*q4*q5*q6*q7*q8 - 720*p**7*q3**2*q4*q5*q6*q7*q8 - 720*p**7*q3*q4**2*q5*q6*q7*q8 - 720*p**7*q3*q4*q5**2*q6*q7*q8 - 720*p**7*q3*q4*q5*q6**2*q7*q8 - 720*p**7*q3*q4*q5*q6*q7**2*q8 - 720*p**7*q3*q4*q5*q6*q7*q8**2 - 720*p**7*q3*q4*q5*q6*q7*q8*q9 + 5040*p**7*q3*q4*q5*q6*q7*q8 + 1050*p**7*q4**2*q5*q6*q7*q8 + 1050*p**7*q4*q5**2*q6*q7*q8 + 1050*p**7*q4*q5*q6**2*q7*q8 + 1050*p**7*q4*q5*q6*q7**2*q8 + 1050*p**7*q4*q5*q6*q7*q8**2 + 1050*p**7*q4*q5*q6*q7*q8*q9 - 6300*p**7*q4*q5*q6*q7*q8 - 1008*p**7*q5**2*q6*q7*q8 - 1008*p**7*q5*q6**2*q7*q8 - 1008*p**7*q5*q6*q7**2*q8 - 1008*p**7*q5*q6*q7*q8**2 - 1008*p**7*q5*q6*q7*q8*q9 + 5040*p**7*q5*q6*q7*q8 + 630*p**7*q6**2*q7*q8 + 630*p**7*q6*q7**2*q8 + 630*p**7*q6*q7*q8**2 + 630*p**7*q6*q7*q8*q9 - 2520*p**7*q6*q7*q8 - 240*p**7*q7**2*q8 - 240*p**7*q7*q8**2 - 240*p**7*q7*q8*q9 + 720*p**7*q7*q8 + 45*p**7*q8**2 + 45*p**7*q8*q9 - 90*p**7*q8 + 280*p**6*q1**2*q2*q3*q4*q5*q6*q7*q8 + 280*p**6*q1*q2**2*q3*q4*q5*q6*q7*q8 + 280*p**6*q1*q2*q3**2*q4*q5*q6*q7*q8 + 280*p**6*q1*q2*q3*q4**2*q5*q6*q7*q8 + 280*p**6*q1*q2*q3*q4*q5**2*q6*q7*q8 + 280*p**6*q1*q2*q3*q4*q5*q6**2*q7*q8 + 280*p**6*q1*q2*q3*q4*q5*q6*q7**2*q8 + 280*p**6*q1*q2*q3*q4*q5*q6*q7*q8**2 + 280*p**6*q1*q2*q3*q4*q5*q6*q7*q8*q9 - 2520*p**6*q1*q2*q3*q4*q5*q6*q7*q8 - 945*p**6*q2**2*q3*q4*q5*q6*q7*q8 - 945*p**6*q2*q3**2*q4*q5*q6*q7*q8 - 945*p**6*q2*q3*q4**2*q5*q6*q7*q8 - 945*p**6*q2*q3*q4*q5**2*q6*q7*q8 - 945*p**6*q2*q3*q4*q5*q6**2*q7*q8 - 945*p**6*q2*q3*q4*q5*q6*q7**2*q8 - 945*p**6*q2*q3*q4*q5*q6*q7*q8**2 - 945*p**6*q2*q3*q4*q5*q6*q7*q8*q9 + 7560*p**6*q2*q3*q4*q5*q6*q7*q8 + 1800*p**6*q3**2*q4*q5*q6*q7*q8 + 1800*p**6*q3*q4**2*q5*q6*q7*q8 + 1800*p**6*q3*q4*q5**2*q6*q7*q8 + 1800*p**6*q3*q4*q5*q6**2*q7*q8 + 1800*p**6*q3*q4*q5*q6*q7**2*q8 + 1800*p**6*q3*q4*q5*q6*q7*q8**2 + 1800*p**6*q3*q4*q5*q6*q7*q8*q9 - 12600*p**6*q3*q4*q5*q6*q7*q8 - 2100*p**6*q4**2*q5*q6*q7*q8 - 2100*p**6*q4*q5**2*q6*q7*q8 - 2100*p**6*q4*q5*q6**2*q7*q8 - 2100*p**6*q4*q5*q6*q7**2*q8 - 2100*p**6*q4*q5*q6*q7*q8**2 - 2100*p**6*q4*q5*q6*q7*q8*q9 + 12600*p**6*q4*q5*q6*q7*q8 + 1512*p**6*q5**2*q6*q7*q8 + 1512*p**6*q5*q6**2*q7*q8 + 1512*p**6*q5*q6*q7**2*q8 + 1512*p**6*q5*q6*q7*q8**2 + 1512*p**6*q5*q6*q7*q8*q9 - 7560*p**6*q5*q6*q7*q8 - 630*p**6*q6**2*q7*q8 - 630*p**6*q6*q7**2*q8 - 630*p**6*q6*q7*q8**2 - 630*p**6*q6*q7*q8*q9 + 2520*p**6*q6*q7*q8 + 120*p**6*q7**2*q8 + 120*p**6*q7*q8**2 + 120*p**6*q7*q8*q9 - 360*p**6*q7*q8 - 560*p**5*q1**2*q2*q3*q4*q5*q6*q7*q8 - 560*p**5*q1*q2**2*q3*q4*q5*q6*q7*q8 - 560*p**5*q1*q2*q3**2*q4*q5*q6*q7*q8 - 560*p**5*q1*q2*q3*q4**2*q5*q6*q7*q8 - 560*p**5*q1*q2*q3*q4*q5**2*q6*q7*q8 - 560*p**5*q1*q2*q3*q4*q5*q6**2*q7*q8 - 560*p**5*q1*q2*q3*q4*q5*q6*q7**2*q8 - 560*p**5*q1*q2*q3*q4*q5*q6*q7*q8**2 - 560*p**5*q1*q2*q3*q4*q5*q6*q7*q8*q9 + 5040*p**5*q1*q2*q3*q4*q5*q6*q7*q8 + 1575*p**5*q2**2*q3*q4*q5*q6*q7*q8 + 1575*p**5*q2*q3**2*q4*q5*q6*q7*q8 + 1575*p**5*q2*q3*q4**2*q5*q6*q7*q8 + 1575*p**5*q2*q3*q4*q5**2*q6*q7*q8 + 1575*p**5*q2*q3*q4*q5*q6**2*q7*q8 + 1575*p**5*q2*q3*q4*q5*q6*q7**2*q8 + 1575*p**5*q2*q3*q4*q5*q6*q7*q8**2 + 1575*p**5*q2*q3*q4*q5*q6*q7*q8*q9 - 12600*p**5*q2*q3*q4*q5*q6*q7*q8 - 2400*p**5*q3**2*q4*q5*q6*q7*q8 - 2400*p**5*q3*q4**2*q5*q6*q7*q8 - 2400*p**5*q3*q4*q5**2*q6*q7*q8 - 2400*p**5*q3*q4*q5*q6**2*q7*q8 - 2400*p**5*q3*q4*q5*q6*q7**2*q8 - 2400*p**5*q3*q4*q5*q6*q7*q8**2 - 2400*p**5*q3*q4*q5*q6*q7*q8*q9 + 16800*p**5*q3*q4*q5*q6*q7*q8 + 2100*p**5*q4**2*q5*q6*q7*q8 + 2100*p**5*q4*q5**2*q6*q7*q8 + 2100*p**5*q4*q5*q6**2*q7*q8 + 2100*p**5*q4*q5*q6*q7**2*q8 + 2100*p**5*q4*q5*q6*q7*q8**2 + 2100*p**5*q4*q5*q6*q7*q8*q9 - 12600*p**5*q4*q5*q6*q7*q8 - 1008*p**5*q5**2*q6*q7*q8 - 1008*p**5*q5*q6**2*q7*q8 - 1008*p**5*q5*q6*q7**2*q8 - 1008*p**5*q5*q6*q7*q8**2 - 1008*p**5*q5*q6*q7*q8*q9 + 5040*p**5*q5*q6*q7*q8 + 210*p**5*q6**2*q7*q8 + 210*p**5*q6*q7**2*q8 + 210*p**5*q6*q7*q8**2 + 210*p**5*q6*q7*q8*q9 - 840*p**5*q6*q7*q8 + 700*p**4*q1**2*q2*q3*q4*q5*q6*q7*q8 + 700*p**4*q1*q2**2*q3*q4*q5*q6*q7*q8 + 700*p**4*q1*q2*q3**2*q4*q5*q6*q7*q8 + 700*p**4*q1*q2*q3*q4**2*q5*q6*q7*q8 + 700*p**4*q1*q2*q3*q4*q5**2*q6*q7*q8 + 700*p**4*q1*q2*q3*q4*q5*q6**2*q7*q8 + 700*p**4*q1*q2*q3*q4*q5*q6*q7**2*q8 + 700*p**4*q1*q2*q3*q4*q5*q6*q7*q8**2 + 700*p**4*q1*q2*q3*q4*q5*q6*q7*q8*q9 - 6300*p**4*q1*q2*q3*q4*q5*q6*q7*q8 - 1575*p**4*q2**2*q3*q4*q5*q6*q7*q8 - 1575*p**4*q2*q3**2*q4*q5*q6*q7*q8 - 1575*p**4*q2*q3*q4**2*q5*q6*q7*q8 - 1575*p**4*q2*q3*q4*q5**2*q6*q7*q8 - 1575*p**4*q2*q3*q4*q5*q6**2*q7*q8 - 1575*p**4*q2*q3*q4*q5*q6*q7**2*q8 - 1575*p**4*q2*q3*q4*q5*q6*q7*q8**2 - 1575*p**4*q2*q3*q4*q5*q6*q7*q8*q9 + 12600*p**4*q2*q3*q4*q5*q6*q7*q8 + 1800*p**4*q3**2*q4*q5*q6*q7*q8 + 1800*p**4*q3*q4**2*q5*q6*q7*q8 + 1800*p**4*q3*q4*q5**2*q6*q7*q8 + 1800*p**4*q3*q4*q5*q6**2*q7*q8 + 1800*p**4*q3*q4*q5*q6*q7**2*q8 + 1800*p**4*q3*q4*q5*q6*q7*q8**2 + 1800*p**4*q3*q4*q5*q6*q7*q8*q9 - 12600*p**4*q3*q4*q5*q6*q7*q8 - 1050*p**4*q4**2*q5*q6*q7*q8 - 1050*p**4*q4*q5**2*q6*q7*q8 - 1050*p**4*q4*q5*q6**2*q7*q8 - 1050*p**4*q4*q5*q6*q7**2*q8 - 1050*p**4*q4*q5*q6*q7*q8**2 - 1050*p**4*q4*q5*q6*q7*q8*q9 + 6300*p**4*q4*q5*q6*q7*q8 + 252*p**4*q5**2*q6*q7*q8 + 252*p**4*q5*q6**2*q7*q8 + 252*p**4*q5*q6*q7**2*q8 + 252*p**4*q5*q6*q7*q8**2 + 252*p**4*q5*q6*q7*q8*q9 - 1260*p**4*q5*q6*q7*q8 - 560*p**3*q1**2*q2*q3*q4*q5*q6*q7*q8 - 560*p**3*q1*q2**2*q3*q4*q5*q6*q7*q8 - 560*p**3*q1*q2*q3**2*q4*q5*q6*q7*q8 - 560*p**3*q1*q2*q3*q4**2*q5*q6*q7*q8 - 560*p**3*q1*q2*q3*q4*q5**2*q6*q7*q8 - 560*p**3*q1*q2*q3*q4*q5*q6**2*q7*q8 - 560*p**3*q1*q2*q3*q4*q5*q6*q7**2*q8 - 560*p**3*q1*q2*q3*q4*q5*q6*q7*q8**2 - 560*p**3*q1*q2*q3*q4*q5*q6*q7*q8*q9 + 5040*p**3*q1*q2*q3*q4*q5*q6*q7*q8 + 945*p**3*q2**2*q3*q4*q5*q6*q7*q8 + 945*p**3*q2*q3**2*q4*q5*q6*q7*q8 + 945*p**3*q2*q3*q4**2*q5*q6*q7*q8 + 945*p**3*q2*q3*q4*q5**2*q6*q7*q8 + 945*p**3*q2*q3*q4*q5*q6**2*q7*q8 + 945*p**3*q2*q3*q4*q5*q6*q7**2*q8 + 945*p**3*q2*q3*q4*q5*q6*q7*q8**2 + 945*p**3*q2*q3*q4*q5*q6*q7*q8*q9 - 7560*p**3*q2*q3*q4*q5*q6*q7*q8 - 720*p**3*q3**2*q4*q5*q6*q7*q8 - 720*p**3*q3*q4**2*q5*q6*q7*q8 - 720*p**3*q3*q4*q5**2*q6*q7*q8 - 720*p**3*q3*q4*q5*q6**2*q7*q8 - 720*p**3*q3*q4*q5*q6*q7**2*q8 - 720*p**3*q3*q4*q5*q6*q7*q8**2 - 720*p**3*q3*q4*q5*q6*q7*q8*q9 + 5040*p**3*q3*q4*q5*q6*q7*q8 + 210*p**3*q4**2*q5*q6*q7*q8 + 210*p**3*q4*q5**2*q6*q7*q8 + 210*p**3*q4*q5*q6**2*q7*q8 + 210*p**3*q4*q5*q6*q7**2*q8 + 210*p**3*q4*q5*q6*q7*q8**2 + 210*p**3*q4*q5*q6*q7*q8*q9 - 1260*p**3*q4*q5*q6*q7*q8 + 280*p**2*q1**2*q2*q3*q4*q5*q6*q7*q8 + 280*p**2*q1*q2**2*q3*q4*q5*q6*q7*q8 + 280*p**2*q1*q2*q3**2*q4*q5*q6*q7*q8 + 280*p**2*q1*q2*q3*q4**2*q5*q6*q7*q8 + 280*p**2*q1*q2*q3*q4*q5**2*q6*q7*q8 + 280*p**2*q1*q2*q3*q4*q5*q6**2*q7*q8 + 280*p**2*q1*q2*q3*q4*q5*q6*q7**2*q8 + 280*p**2*q1*q2*q3*q4*q5*q6*q7*q8**2 + 280*p**2*q1*q2*q3*q4*q5*q6*q7*q8*q9 - 2520*p**2*q1*q2*q3*q4*q5*q6*q7*q8 - 315*p**2*q2**2*q3*q4*q5*q6*q7*q8 - 315*p**2*q2*q3**2*q4*q5*q6*q7*q8 - 315*p**2*q2*q3*q4**2*q5*q6*q7*q8 - 315*p**2*q2*q3*q4*q5**2*q6*q7*q8 - 315*p**2*q2*q3*q4*q5*q6**2*q7*q8 - 315*p**2*q2*q3*q4*q5*q6*q7**2*q8 - 315*p**2*q2*q3*q4*q5*q6*q7*q8**2 - 315*p**2*q2*q3*q4*q5*q6*q7*q8*q9 + 2520*p**2*q2*q3*q4*q5*q6*q7*q8 + 120*p**2*q3**2*q4*q5*q6*q7*q8 + 120*p**2*q3*q4**2*q5*q6*q7*q8 + 120*p**2*q3*q4*q5**2*q6*q7*q8 + 120*p**2*q3*q4*q5*q6**2*q7*q8 + 120*p**2*q3*q4*q5*q6*q7**2*q8 + 120*p**2*q3*q4*q5*q6*q7*q8**2 + 120*p**2*q3*q4*q5*q6*q7*q8*q9 - 840*p**2*q3*q4*q5*q6*q7*q8 - 80*p*q1**2*q2*q3*q4*q5*q6*q7*q8 - 80*p*q1*q2**2*q3*q4*q5*q6*q7*q8 - 80*p*q1*q2*q3**2*q4*q5*q6*q7*q8 - 80*p*q1*q2*q3*q4**2*q5*q6*q7*q8 - 80*p*q1*q2*q3*q4*q5**2*q6*q7*q8 - 80*p*q1*q2*q3*q4*q5*q6**2*q7*q8 - 80*p*q1*q2*q3*q4*q5*q6*q7**2*q8 - 80*p*q1*q2*q3*q4*q5*q6*q7*q8**2 - 80*p*q1*q2*q3*q4*q5*q6*q7*q8*q9 + 720*p*q1*q2*q3*q4*q5*q6*q7*q8 + 45*p*q2**2*q3*q4*q5*q6*q7*q8 + 45*p*q2*q3**2*q4*q5*q6*q7*q8 + 45*p*q2*q3*q4**2*q5*q6*q7*q8 + 45*p*q2*q3*q4*q5**2*q6*q7*q8 + 45*p*q2*q3*q4*q5*q6**2*q7*q8 + 45*p*q2*q3*q4*q5*q6*q7**2*q8 + 45*p*q2*q3*q4*q5*q6*q7*q8**2 + 45*p*q2*q3*q4*q5*q6*q7*q8*q9 - 360*p*q2*q3*q4*q5*q6*q7*q8 + 10*q1**2*q2*q3*q4*q5*q6*q7*q8 + 10*q1*q2**2*q3*q4*q5*q6*q7*q8 + 10*q1*q2*q3**2*q4*q5*q6*q7*q8 + 10*q1*q2*q3*q4**2*q5*q6*q7*q8 + 10*q1*q2*q3*q4*q5**2*q6*q7*q8 + 10*q1*q2*q3*q4*q5*q6**2*q7*q8 + 10*q1*q2*q3*q4*q5*q6*q7**2*q8 + 10*q1*q2*q3*q4*q5*q6*q7*q8**2 + 10*q1*q2*q3*q4*q5*q6*q7*q8*q9 - 90*q1*q2*q3*q4*q5*q6*q7*q8)'
f_multiparam[10][10] = '-p*(10*p**9*q1*q2*q3*q4*q5*q6*q7*q8*q9 - 45*p**9*q2*q3*q4*q5*q6*q7*q8*q9 + 120*p**9*q3*q4*q5*q6*q7*q8*q9 - 210*p**9*q4*q5*q6*q7*q8*q9 + 252*p**9*q5*q6*q7*q8*q9 - 210*p**9*q6*q7*q8*q9 + 120*p**9*q7*q8*q9 - 45*p**9*q8*q9 + 10*p**9*q9 - p**9 - 90*p**8*q1*q2*q3*q4*q5*q6*q7*q8*q9 + 360*p**8*q2*q3*q4*q5*q6*q7*q8*q9 - 840*p**8*q3*q4*q5*q6*q7*q8*q9 + 1260*p**8*q4*q5*q6*q7*q8*q9 - 1260*p**8*q5*q6*q7*q8*q9 + 840*p**8*q6*q7*q8*q9 - 360*p**8*q7*q8*q9 + 90*p**8*q8*q9 - 10*p**8*q9 + 360*p**7*q1*q2*q3*q4*q5*q6*q7*q8*q9 - 1260*p**7*q2*q3*q4*q5*q6*q7*q8*q9 + 2520*p**7*q3*q4*q5*q6*q7*q8*q9 - 3150*p**7*q4*q5*q6*q7*q8*q9 + 2520*p**7*q5*q6*q7*q8*q9 - 1260*p**7*q6*q7*q8*q9 + 360*p**7*q7*q8*q9 - 45*p**7*q8*q9 - 840*p**6*q1*q2*q3*q4*q5*q6*q7*q8*q9 + 2520*p**6*q2*q3*q4*q5*q6*q7*q8*q9 - 4200*p**6*q3*q4*q5*q6*q7*q8*q9 + 4200*p**6*q4*q5*q6*q7*q8*q9 - 2520*p**6*q5*q6*q7*q8*q9 + 840*p**6*q6*q7*q8*q9 - 120*p**6*q7*q8*q9 + 1260*p**5*q1*q2*q3*q4*q5*q6*q7*q8*q9 - 3150*p**5*q2*q3*q4*q5*q6*q7*q8*q9 + 4200*p**5*q3*q4*q5*q6*q7*q8*q9 - 3150*p**5*q4*q5*q6*q7*q8*q9 + 1260*p**5*q5*q6*q7*q8*q9 - 210*p**5*q6*q7*q8*q9 - 1260*p**4*q1*q2*q3*q4*q5*q6*q7*q8*q9 + 2520*p**4*q2*q3*q4*q5*q6*q7*q8*q9 - 2520*p**4*q3*q4*q5*q6*q7*q8*q9 + 1260*p**4*q4*q5*q6*q7*q8*q9 - 252*p**4*q5*q6*q7*q8*q9 + 840*p**3*q1*q2*q3*q4*q5*q6*q7*q8*q9 - 1260*p**3*q2*q3*q4*q5*q6*q7*q8*q9 + 840*p**3*q3*q4*q5*q6*q7*q8*q9 - 210*p**3*q4*q5*q6*q7*q8*q9 - 360*p**2*q1*q2*q3*q4*q5*q6*q7*q8*q9 + 360*p**2*q2*q3*q4*q5*q6*q7*q8*q9 - 120*p**2*q3*q4*q5*q6*q7*q8*q9 + 90*p*q1*q2*q3*q4*q5*q6*q7*q8*q9 - 45*p*q2*q3*q4*q5*q6*q7*q8*q9 - 10*q1*q2*q3*q4*q5*q6*q7*q8*q9)'

These agents quantities are loaded

In [ ]:
print(sorted(f_multiparam.keys()))

This may take a while to factorise for f_multiparam_syn[10], so I will load precomputed version for you

In [ ]:
#sys.setrecursionlimit(15000)
#start_time = time.time()
#f_multiparam_syn = get_f("./multiparam_syn*[0-9].txt",True)
#print("  It took", socket.gethostname(), time.time() - start_time, "seconds to run")
#sys.setrecursionlimit(1000)

In [ ]:
f_multiparam_syn = get_f("./multiparam_se*_[0-9].txt",True)
f_multiparam_syn[10]=[0,0,0,0,0,0,0,0,0,0,0]
f_multiparam_syn[10][0] = '(p - 1)**10'
f_multiparam_syn[10][1] = '10*p*(p - 1)**9*(q1 - 1)**9'
f_multiparam_syn[10][2] = '-45*p*(p - 1)**8*(2*p*q1**9 - 16*p*q1**8 + 56*p*q1**7 - 112*p*q1**6 + 140*p*q1**5 - 112*p*q1**4 + 56*p*q1**3 - 16*p*q1**2 + 2*p*q1 - p*q2**8 + 8*p*q2**7 - 28*p*q2**6 + 56*p*q2**5 - 70*p*q2**4 + 56*p*q2**3 - 28*p*q2**2 + 8*p*q2 - p - 2*q1**9 + 16*q1**8 - 56*q1**7 + 112*q1**6 - 140*q1**5 + 112*q1**4 - 56*q1**3 + 16*q1**2 - 2*q1)'
f_multiparam_syn[10][3] = '120*p*(p - 1)**7*(3*p**2*q1**9 - 21*p**2*q1**8 + 63*p**2*q1**7 - 105*p**2*q1**6 + 105*p**2*q1**5 - 63*p**2*q1**4 + 21*p**2*q1**3 - 3*p**2*q1**2 - 3*p**2*q2**8 + 21*p**2*q2**7 - 63*p**2*q2**6 + 105*p**2*q2**5 - 105*p**2*q2**4 + 63*p**2*q2**3 - 21*p**2*q2**2 + 3*p**2*q2 + p**2*q3**7 - 7*p**2*q3**6 + 21*p**2*q3**5 - 35*p**2*q3**4 + 35*p**2*q3**3 - 21*p**2*q3**2 + 7*p**2*q3 - p**2 - 6*p*q1**9 + 42*p*q1**8 - 126*p*q1**7 + 210*p*q1**6 - 210*p*q1**5 + 126*p*q1**4 - 42*p*q1**3 + 6*p*q1**2 + 3*p*q2**8 - 21*p*q2**7 + 63*p*q2**6 - 105*p*q2**5 + 105*p*q2**4 - 63*p*q2**3 + 21*p*q2**2 - 3*p*q2 + 3*q1**9 - 21*q1**8 + 63*q1**7 - 105*q1**6 + 105*q1**5 - 63*q1**4 + 21*q1**3 - 3*q1**2)'
f_multiparam_syn[10][4] = '-210*p*(p - 1)**6*(4*p**3*q1**9 - 24*p**3*q1**8 + 60*p**3*q1**7 - 80*p**3*q1**6 + 60*p**3*q1**5 - 24*p**3*q1**4 + 4*p**3*q1**3 - 6*p**3*q2**8 + 36*p**3*q2**7 - 90*p**3*q2**6 + 120*p**3*q2**5 - 90*p**3*q2**4 + 36*p**3*q2**3 - 6*p**3*q2**2 + 4*p**3*q3**7 - 24*p**3*q3**6 + 60*p**3*q3**5 - 80*p**3*q3**4 + 60*p**3*q3**3 - 24*p**3*q3**2 + 4*p**3*q3 - p**3*q4**6 + 6*p**3*q4**5 - 15*p**3*q4**4 + 20*p**3*q4**3 - 15*p**3*q4**2 + 6*p**3*q4 - p**3 - 12*p**2*q1**9 + 72*p**2*q1**8 - 180*p**2*q1**7 + 240*p**2*q1**6 - 180*p**2*q1**5 + 72*p**2*q1**4 - 12*p**2*q1**3 + 12*p**2*q2**8 - 72*p**2*q2**7 + 180*p**2*q2**6 - 240*p**2*q2**5 + 180*p**2*q2**4 - 72*p**2*q2**3 + 12*p**2*q2**2 - 4*p**2*q3**7 + 24*p**2*q3**6 - 60*p**2*q3**5 + 80*p**2*q3**4 - 60*p**2*q3**3 + 24*p**2*q3**2 - 4*p**2*q3 + 12*p*q1**9 - 72*p*q1**8 + 180*p*q1**7 - 240*p*q1**6 + 180*p*q1**5 - 72*p*q1**4 + 12*p*q1**3 - 6*p*q2**8 + 36*p*q2**7 - 90*p*q2**6 + 120*p*q2**5 - 90*p*q2**4 + 36*p*q2**3 - 6*p*q2**2 - 4*q1**9 + 24*q1**8 - 60*q1**7 + 80*q1**6 - 60*q1**5 + 24*q1**4 - 4*q1**3)'
f_multiparam_syn[10][5] = '252*p*(p - 1)**5*(5*p**4*q1**9 - 25*p**4*q1**8 + 50*p**4*q1**7 - 50*p**4*q1**6 + 25*p**4*q1**5 - 5*p**4*q1**4 - 10*p**4*q2**8 + 50*p**4*q2**7 - 100*p**4*q2**6 + 100*p**4*q2**5 - 50*p**4*q2**4 + 10*p**4*q2**3 + 10*p**4*q3**7 - 50*p**4*q3**6 + 100*p**4*q3**5 - 100*p**4*q3**4 + 50*p**4*q3**3 - 10*p**4*q3**2 - 5*p**4*q4**6 + 25*p**4*q4**5 - 50*p**4*q4**4 + 50*p**4*q4**3 - 25*p**4*q4**2 + 5*p**4*q4 + p**4*q5**5 - 5*p**4*q5**4 + 10*p**4*q5**3 - 10*p**4*q5**2 + 5*p**4*q5 - p**4 - 20*p**3*q1**9 + 100*p**3*q1**8 - 200*p**3*q1**7 + 200*p**3*q1**6 - 100*p**3*q1**5 + 20*p**3*q1**4 + 30*p**3*q2**8 - 150*p**3*q2**7 + 300*p**3*q2**6 - 300*p**3*q2**5 + 150*p**3*q2**4 - 30*p**3*q2**3 - 20*p**3*q3**7 + 100*p**3*q3**6 - 200*p**3*q3**5 + 200*p**3*q3**4 - 100*p**3*q3**3 + 20*p**3*q3**2 + 5*p**3*q4**6 - 25*p**3*q4**5 + 50*p**3*q4**4 - 50*p**3*q4**3 + 25*p**3*q4**2 - 5*p**3*q4 + 30*p**2*q1**9 - 150*p**2*q1**8 + 300*p**2*q1**7 - 300*p**2*q1**6 + 150*p**2*q1**5 - 30*p**2*q1**4 - 30*p**2*q2**8 + 150*p**2*q2**7 - 300*p**2*q2**6 + 300*p**2*q2**5 - 150*p**2*q2**4 + 30*p**2*q2**3 + 10*p**2*q3**7 - 50*p**2*q3**6 + 100*p**2*q3**5 - 100*p**2*q3**4 + 50*p**2*q3**3 - 10*p**2*q3**2 - 20*p*q1**9 + 100*p*q1**8 - 200*p*q1**7 + 200*p*q1**6 - 100*p*q1**5 + 20*p*q1**4 + 10*p*q2**8 - 50*p*q2**7 + 100*p*q2**6 - 100*p*q2**5 + 50*p*q2**4 - 10*p*q2**3 + 5*q1**9 - 25*q1**8 + 50*q1**7 - 50*q1**6 + 25*q1**5 - 5*q1**4)'
f_multiparam_syn[10][6] = '-210*p*(p - 1)**4*(6*p**5*q1**9 - 24*p**5*q1**8 + 36*p**5*q1**7 - 24*p**5*q1**6 + 6*p**5*q1**5 - 15*p**5*q2**8 + 60*p**5*q2**7 - 90*p**5*q2**6 + 60*p**5*q2**5 - 15*p**5*q2**4 + 20*p**5*q3**7 - 80*p**5*q3**6 + 120*p**5*q3**5 - 80*p**5*q3**4 + 20*p**5*q3**3 - 15*p**5*q4**6 + 60*p**5*q4**5 - 90*p**5*q4**4 + 60*p**5*q4**3 - 15*p**5*q4**2 + 6*p**5*q5**5 - 24*p**5*q5**4 + 36*p**5*q5**3 - 24*p**5*q5**2 + 6*p**5*q5 - p**5*q6**4 + 4*p**5*q6**3 - 6*p**5*q6**2 + 4*p**5*q6 - p**5 - 30*p**4*q1**9 + 120*p**4*q1**8 - 180*p**4*q1**7 + 120*p**4*q1**6 - 30*p**4*q1**5 + 60*p**4*q2**8 - 240*p**4*q2**7 + 360*p**4*q2**6 - 240*p**4*q2**5 + 60*p**4*q2**4 - 60*p**4*q3**7 + 240*p**4*q3**6 - 360*p**4*q3**5 + 240*p**4*q3**4 - 60*p**4*q3**3 + 30*p**4*q4**6 - 120*p**4*q4**5 + 180*p**4*q4**4 - 120*p**4*q4**3 + 30*p**4*q4**2 - 6*p**4*q5**5 + 24*p**4*q5**4 - 36*p**4*q5**3 + 24*p**4*q5**2 - 6*p**4*q5 + 60*p**3*q1**9 - 240*p**3*q1**8 + 360*p**3*q1**7 - 240*p**3*q1**6 + 60*p**3*q1**5 - 90*p**3*q2**8 + 360*p**3*q2**7 - 540*p**3*q2**6 + 360*p**3*q2**5 - 90*p**3*q2**4 + 60*p**3*q3**7 - 240*p**3*q3**6 + 360*p**3*q3**5 - 240*p**3*q3**4 + 60*p**3*q3**3 - 15*p**3*q4**6 + 60*p**3*q4**5 - 90*p**3*q4**4 + 60*p**3*q4**3 - 15*p**3*q4**2 - 60*p**2*q1**9 + 240*p**2*q1**8 - 360*p**2*q1**7 + 240*p**2*q1**6 - 60*p**2*q1**5 + 60*p**2*q2**8 - 240*p**2*q2**7 + 360*p**2*q2**6 - 240*p**2*q2**5 + 60*p**2*q2**4 - 20*p**2*q3**7 + 80*p**2*q3**6 - 120*p**2*q3**5 + 80*p**2*q3**4 - 20*p**2*q3**3 + 30*p*q1**9 - 120*p*q1**8 + 180*p*q1**7 - 120*p*q1**6 + 30*p*q1**5 - 15*p*q2**8 + 60*p*q2**7 - 90*p*q2**6 + 60*p*q2**5 - 15*p*q2**4 - 6*q1**9 + 24*q1**8 - 36*q1**7 + 24*q1**6 - 6*q1**5)'
f_multiparam_syn[10][7] = '120*p*(p - 1)**3*(7*p**6*q1**9 - 21*p**6*q1**8 + 21*p**6*q1**7 - 7*p**6*q1**6 - 21*p**6*q2**8 + 63*p**6*q2**7 - 63*p**6*q2**6 + 21*p**6*q2**5 + 35*p**6*q3**7 - 105*p**6*q3**6 + 105*p**6*q3**5 - 35*p**6*q3**4 - 35*p**6*q4**6 + 105*p**6*q4**5 - 105*p**6*q4**4 + 35*p**6*q4**3 + 21*p**6*q5**5 - 63*p**6*q5**4 + 63*p**6*q5**3 - 21*p**6*q5**2 - 7*p**6*q6**4 + 21*p**6*q6**3 - 21*p**6*q6**2 + 7*p**6*q6 + p**6*q7**3 - 3*p**6*q7**2 + 3*p**6*q7 - p**6 - 42*p**5*q1**9 + 126*p**5*q1**8 - 126*p**5*q1**7 + 42*p**5*q1**6 + 105*p**5*q2**8 - 315*p**5*q2**7 + 315*p**5*q2**6 - 105*p**5*q2**5 - 140*p**5*q3**7 + 420*p**5*q3**6 - 420*p**5*q3**5 + 140*p**5*q3**4 + 105*p**5*q4**6 - 315*p**5*q4**5 + 315*p**5*q4**4 - 105*p**5*q4**3 - 42*p**5*q5**5 + 126*p**5*q5**4 - 126*p**5*q5**3 + 42*p**5*q5**2 + 7*p**5*q6**4 - 21*p**5*q6**3 + 21*p**5*q6**2 - 7*p**5*q6 + 105*p**4*q1**9 - 315*p**4*q1**8 + 315*p**4*q1**7 - 105*p**4*q1**6 - 210*p**4*q2**8 + 630*p**4*q2**7 - 630*p**4*q2**6 + 210*p**4*q2**5 + 210*p**4*q3**7 - 630*p**4*q3**6 + 630*p**4*q3**5 - 210*p**4*q3**4 - 105*p**4*q4**6 + 315*p**4*q4**5 - 315*p**4*q4**4 + 105*p**4*q4**3 + 21*p**4*q5**5 - 63*p**4*q5**4 + 63*p**4*q5**3 - 21*p**4*q5**2 - 140*p**3*q1**9 + 420*p**3*q1**8 - 420*p**3*q1**7 + 140*p**3*q1**6 + 210*p**3*q2**8 - 630*p**3*q2**7 + 630*p**3*q2**6 - 210*p**3*q2**5 - 140*p**3*q3**7 + 420*p**3*q3**6 - 420*p**3*q3**5 + 140*p**3*q3**4 + 35*p**3*q4**6 - 105*p**3*q4**5 + 105*p**3*q4**4 - 35*p**3*q4**3 + 105*p**2*q1**9 - 315*p**2*q1**8 + 315*p**2*q1**7 - 105*p**2*q1**6 - 105*p**2*q2**8 + 315*p**2*q2**7 - 315*p**2*q2**6 + 105*p**2*q2**5 + 35*p**2*q3**7 - 105*p**2*q3**6 + 105*p**2*q3**5 - 35*p**2*q3**4 - 42*p*q1**9 + 126*p*q1**8 - 126*p*q1**7 + 42*p*q1**6 + 21*p*q2**8 - 63*p*q2**7 + 63*p*q2**6 - 21*p*q2**5 + 7*q1**9 - 21*q1**8 + 21*q1**7 - 7*q1**6)'
f_multiparam_syn[10][8] = '-45*p*(p - 1)**2*(8*p**7*q1**9 - 16*p**7*q1**8 + 8*p**7*q1**7 - 28*p**7*q2**8 + 56*p**7*q2**7 - 28*p**7*q2**6 + 56*p**7*q3**7 - 112*p**7*q3**6 + 56*p**7*q3**5 - 70*p**7*q4**6 + 140*p**7*q4**5 - 70*p**7*q4**4 + 56*p**7*q5**5 - 112*p**7*q5**4 + 56*p**7*q5**3 - 28*p**7*q6**4 + 56*p**7*q6**3 - 28*p**7*q6**2 + 8*p**7*q7**3 - 16*p**7*q7**2 + 8*p**7*q7 - p**7*q8**2 + 2*p**7*q8 - p**7 - 56*p**6*q1**9 + 112*p**6*q1**8 - 56*p**6*q1**7 + 168*p**6*q2**8 - 336*p**6*q2**7 + 168*p**6*q2**6 - 280*p**6*q3**7 + 560*p**6*q3**6 - 280*p**6*q3**5 + 280*p**6*q4**6 - 560*p**6*q4**5 + 280*p**6*q4**4 - 168*p**6*q5**5 + 336*p**6*q5**4 - 168*p**6*q5**3 + 56*p**6*q6**4 - 112*p**6*q6**3 + 56*p**6*q6**2 - 8*p**6*q7**3 + 16*p**6*q7**2 - 8*p**6*q7 + 168*p**5*q1**9 - 336*p**5*q1**8 + 168*p**5*q1**7 - 420*p**5*q2**8 + 840*p**5*q2**7 - 420*p**5*q2**6 + 560*p**5*q3**7 - 1120*p**5*q3**6 + 560*p**5*q3**5 - 420*p**5*q4**6 + 840*p**5*q4**5 - 420*p**5*q4**4 + 168*p**5*q5**5 - 336*p**5*q5**4 + 168*p**5*q5**3 - 28*p**5*q6**4 + 56*p**5*q6**3 - 28*p**5*q6**2 - 280*p**4*q1**9 + 560*p**4*q1**8 - 280*p**4*q1**7 + 560*p**4*q2**8 - 1120*p**4*q2**7 + 560*p**4*q2**6 - 560*p**4*q3**7 + 1120*p**4*q3**6 - 560*p**4*q3**5 + 280*p**4*q4**6 - 560*p**4*q4**5 + 280*p**4*q4**4 - 56*p**4*q5**5 + 112*p**4*q5**4 - 56*p**4*q5**3 + 280*p**3*q1**9 - 560*p**3*q1**8 + 280*p**3*q1**7 - 420*p**3*q2**8 + 840*p**3*q2**7 - 420*p**3*q2**6 + 280*p**3*q3**7 - 560*p**3*q3**6 + 280*p**3*q3**5 - 70*p**3*q4**6 + 140*p**3*q4**5 - 70*p**3*q4**4 - 168*p**2*q1**9 + 336*p**2*q1**8 - 168*p**2*q1**7 + 168*p**2*q2**8 - 336*p**2*q2**7 + 168*p**2*q2**6 - 56*p**2*q3**7 + 112*p**2*q3**6 - 56*p**2*q3**5 + 56*p*q1**9 - 112*p*q1**8 + 56*p*q1**7 - 28*p*q2**8 + 56*p*q2**7 - 28*p*q2**6 - 8*q1**9 + 16*q1**8 - 8*q1**7)'
f_multiparam_syn[10][9] = '10*p*(p - 1)*(9*p**8*q1**9 - 9*p**8*q1**8 - 36*p**8*q2**8 + 36*p**8*q2**7 + 84*p**8*q3**7 - 84*p**8*q3**6 - 126*p**8*q4**6 + 126*p**8*q4**5 + 126*p**8*q5**5 - 126*p**8*q5**4 - 84*p**8*q6**4 + 84*p**8*q6**3 + 36*p**8*q7**3 - 36*p**8*q7**2 - 9*p**8*q8**2 + 9*p**8*q8 + p**8*q9 - p**8 - 72*p**7*q1**9 + 72*p**7*q1**8 + 252*p**7*q2**8 - 252*p**7*q2**7 - 504*p**7*q3**7 + 504*p**7*q3**6 + 630*p**7*q4**6 - 630*p**7*q4**5 - 504*p**7*q5**5 + 504*p**7*q5**4 + 252*p**7*q6**4 - 252*p**7*q6**3 - 72*p**7*q7**3 + 72*p**7*q7**2 + 9*p**7*q8**2 - 9*p**7*q8 + 252*p**6*q1**9 - 252*p**6*q1**8 - 756*p**6*q2**8 + 756*p**6*q2**7 + 1260*p**6*q3**7 - 1260*p**6*q3**6 - 1260*p**6*q4**6 + 1260*p**6*q4**5 + 756*p**6*q5**5 - 756*p**6*q5**4 - 252*p**6*q6**4 + 252*p**6*q6**3 + 36*p**6*q7**3 - 36*p**6*q7**2 - 504*p**5*q1**9 + 504*p**5*q1**8 + 1260*p**5*q2**8 - 1260*p**5*q2**7 - 1680*p**5*q3**7 + 1680*p**5*q3**6 + 1260*p**5*q4**6 - 1260*p**5*q4**5 - 504*p**5*q5**5 + 504*p**5*q5**4 + 84*p**5*q6**4 - 84*p**5*q6**3 + 630*p**4*q1**9 - 630*p**4*q1**8 - 1260*p**4*q2**8 + 1260*p**4*q2**7 + 1260*p**4*q3**7 - 1260*p**4*q3**6 - 630*p**4*q4**6 + 630*p**4*q4**5 + 126*p**4*q5**5 - 126*p**4*q5**4 - 504*p**3*q1**9 + 504*p**3*q1**8 + 756*p**3*q2**8 - 756*p**3*q2**7 - 504*p**3*q3**7 + 504*p**3*q3**6 + 126*p**3*q4**6 - 126*p**3*q4**5 + 252*p**2*q1**9 - 252*p**2*q1**8 - 252*p**2*q2**8 + 252*p**2*q2**7 + 84*p**2*q3**7 - 84*p**2*q3**6 - 72*p*q1**9 + 72*p*q1**8 + 36*p*q2**8 - 36*p*q2**7 + 9*q1**9 - 9*q1**8)'
f_multiparam_syn[10][10] = '-p*(10*p**9*q1**9 - 45*p**9*q2**8 + 120*p**9*q3**7 - 210*p**9*q4**6 + 252*p**9*q5**5 - 210*p**9*q6**4 + 120*p**9*q7**3 - 45*p**9*q8**2 + 10*p**9*q9 - p**9 - 90*p**8*q1**9 + 360*p**8*q2**8 - 840*p**8*q3**7 + 1260*p**8*q4**6 - 1260*p**8*q5**5 + 840*p**8*q6**4 - 360*p**8*q7**3 + 90*p**8*q8**2 - 10*p**8*q9 + 360*p**7*q1**9 - 1260*p**7*q2**8 + 2520*p**7*q3**7 - 3150*p**7*q4**6 + 2520*p**7*q5**5 - 1260*p**7*q6**4 + 360*p**7*q7**3 - 45*p**7*q8**2 - 840*p**6*q1**9 + 2520*p**6*q2**8 - 4200*p**6*q3**7 + 4200*p**6*q4**6 - 2520*p**6*q5**5 + 840*p**6*q6**4 - 120*p**6*q7**3 + 1260*p**5*q1**9 - 3150*p**5*q2**8 + 4200*p**5*q3**7 - 3150*p**5*q4**6 + 1260*p**5*q5**5 - 210*p**5*q6**4 - 1260*p**4*q1**9 + 2520*p**4*q2**8 - 2520*p**4*q3**7 + 1260*p**4*q4**6 - 252*p**4*q5**5 + 840*p**3*q1**9 - 1260*p**3*q2**8 + 840*p**3*q3**7 - 210*p**3*q4**6 - 360*p**2*q1**9 + 360*p**2*q2**8 - 120*p**2*q3**7 + 90*p*q1**9 - 45*p*q2**8 - 10*q1**9)'

These agents quantities are loaded

In [ ]:
print(sorted(f_multiparam_syn.keys()))

### DATA GENERATING

We will need a function to parse parameter from polynome

In [ ]:
def find_param(polynome):
    """ Finds parameters of a polynomes

    Parameters
    ----------
    polynome : polynome as string
    
    Returns set of strings - parameters
    """
    parameters = polynome.replace('(', '').replace(')', '').replace('**', '*').replace(' ', '')
    parameters = re.split('\+|\*|\-|/',parameters)
    parameters = [i for i in parameters if not i.isnumeric()]
    parameters = set(parameters)
    parameters.add("")
    parameters.remove("")
    #print("hello",set(parameters))
    return set(parameters)

Example - Finding all params of *f_multiparam*

In [ ]:
parameters=set()
for N in agents_quantities:
    for polynome in f_multiparam[N]:
        if len(parameters)<N:
            parameters.update(find_param(polynome))
print(sorted(parameters))

Now we generate the data

In [ ]:
# create distribution data, by Tanja, 14.1.2019
# edited, by xhajnal, 18.1.2019, still name 'a' is not defined
# edited, by xhajnal, 26.1.2019, a set as [] but does not work
# totally rewritten to not paste empty line, doc added, by xhajnal, 03.02.2019

def generate_all_data_twoparam(p_v=None,q_v=None):
    """ Generates data for all agents_quantities in the current as .csv files    
    """
    import random

    dic_fun=f
    if p_v==None:
        p_v=random.uniform(0.,1.)
        p_v= round(p_v,2)
    if q_v==None:
        q_v=random.uniform(0.,1.)
        q_v= round(q_v,2)
    a=[]

    # create the files called "data_n=N.csv" where the first line is 
    # "n=5, p_v--q1_v--q2_v--q3_v--q4_v in the multiparam. case, and, e.g.
    # 0.03 -- 0.45 -- 0.002 -- 0.3 (for N=5, there are 4 entries)


    for N in agents_quantities:
        file = open('data_n='+str(N)+".csv","w")
        file.write('n='+str(N)+', p_v='+str(p_v)+', q_v='+str(q_v)+"\n")
        secondline=""

        for polynome in dic_fun[N]:
            parameters=set() 
            if len(parameters)<N:
                parameters.update(find_param(polynome))
            parameters= sorted(list(parameters))
            parameter_value = [p_v,q_v]

            for param in range(len(parameters)):
                a.append(parameter_value[param])
                globals()[parameters[param]]= parameter_value[param]   

            x = eval(polynome)
            x = round(x,2)
            secondline = secondline+str(x)+","

        file.write(secondline[:-1])
        file.close()

In [ ]:
#os.mkdir("data")
#os.chdir("./data")
#generate_all_data_twoparam(0.81,0.92)
#os.chdir("..")

#os.mkdir("data_1")
#os.chdir("./data_1")
#generate_all_data_twoparam(0.53,0.13)
#os.chdir("..")

### DATA LOADING

The data we are about to load are in a form data points for given N and number of successes, hence they match already loaded f dictionaries. 

They can be seen as a points of the polynomials.

In [ ]:
def load_all_data(path):
    """ loads all experimental data for respective BSCC, returns as dictionary D
    
    Parameters
    ----------
    path: string - file name regex
    
    Returns:
        D: dictionary N -> list of propbabilities for respective BSCC
    """
    D = {}
    for file in glob.glob(path):
        file=open(file,"r")
        N = 0
        for line in file:
            #print("line: ",line)
            if re.search("n", line) is not None:
                N = int(line.split(",")[0].split("=")[1])
                #print("N, ",N)
                D[N] = []
                continue
            D[N] = line[:-1].split(",")
            #print(D[N])
            for value in range(len(D[N])):
                #print(D[N][value])
                try:
                    D[N][value]=float(D[N][value])
                except:
                    print("error while parsing N=",N," i=",value," of value=",D[N][value])
                    D[N][value]=0
                #print(type(D[N][value]))
            #D[N].append(1-sum(D[N]))
            break
            #print(D[N])
    if D:
        return D
    else:
        print("Error, No data loaded, please check path")

Load data

In [ ]:
D = load_all_data("../data/data*.csv")

In [ ]:
D_multiparam = load_all_data("../data/multiparametric/data*.csv")

In [ ]:
D2 = load_all_data("../data_1/data*.csv")

In [ ]:
D2_multiparam = load_all_data("../data_1/multiparametric/data*.csv")

### showing loaded f and data

An example of function and data for number of agents N=2 and number of successes i=1

In [ ]:
f[2][1] 

In [ ]:
f_multiparam[2][1]

In [ ]:
f[5]

In [ ]:
D[2][1] 

Catch data errors (subzero values)

In [ ]:
def catch_data_error(data,minimum,maximum):
    """ Corrects all data value to be in range min,max
    
    Parameters
    ----------
    data: map structure of data
    minimum: minimal value in data to be set to
    maximum: maximal value in data to be set to
    
    """
    for n in data.keys():
        for i in range(len(data[n])):
            if data[n][i]<minimum:
                data[n][i]=minimum
            if data[n][i]>maximum:
                data[n][i]=maximum

In [ ]:
catch_data_error(D,0,1) 
catch_data_error(D2,0,1) 
catch_data_error(D_multiparam,0,1) 
catch_data_error(D2_multiparam,0,1) 

Now all data and f are loaded

### NOISE

Noise to infer intervals in which we wan to search

In [ ]:
def noise(alpha, n_samples, data):
    """ Estimates expected interval with respect to parameters

    Parameters
    ----------
    alpha : confidence interval to compute noise
    n_samples : number of samples to compute noise 
    data: data point
    """
    #return 0.36
    return st.norm.ppf(1-(1-alpha)/2)*math.sqrt(data*(1-data)/n_samples)+0.5/n_samples
    
    ##EXPERIMENTAL NOISE
    #return st.norm.ppf(1-(1-alpha)/2)*math.sqrt(data*(1-data)/n_samples)+0.5/n_samples+0.005

In [ ]:
print(noise(0.95, 100, 0.01))
print(noise(0.99, 100, 0.01))
print(D[2][0]-noise(0.95, 1500, D[2][0]),",",D[2][0]+noise(0.95, 1500, D[2][0]))

EXPERIMENTAL NOISE

`0.029501395417987873
0.0356291779716225
0.024749946149793683 , 0.055250053850206315`

In [ ]:
os.chdir("..")
print(os.getcwd())

<a name="two"></a>

## 2. SAMPLING AND VISUALISATION

In [ ]:
def cartesian_product(*arrays):
    la = len(arrays)
    dtype = np.result_type(*arrays)
    arr = np.empty([len(a) for a in arrays] + [la], dtype=dtype)
    for i, a in enumerate(np.ix_(*arrays)):
        arr[...,i] = a
    return arr.reshape(-1, la)

### SAMPLING

In this subsection we sample paramter space to evaluate synthetised polynomials in the given sampled point. These are further visualised useing bar plots.

In [ ]:
def sample(dic_fun,size_q):
    """ Samples probabilities of i successes for sampled parametrisation

    Parameters
    ----------
    dic_fun : dictionary N -> list of polynomes
    size_q : sample size in each parameter
    
    Returns array of [N,i,parameters,value]
    """
    arr = []
    for N in agents_quantities:
        for polynome in dic_fun[N]:
            parameters=set()
            if len(parameters)<N:
                parameters.update(find_param(polynome))
            #print(parameters)
            parameters= sorted(list(parameters))
            #print(parameters)
            parameter_values = []
            for param in range(len(parameters)):
                parameter_values.append(np.linspace(0, 1, size_q, endpoint=True)) 
            parameter_values=cartesian_product(*parameter_values)
            if (len(parameters)-1)==0:
                parameter_values=np.linspace(0, 1, size_q, endpoint=True)[np.newaxis, :].T
            #print(parameter_values)
            
            for parameter_value in parameter_values:
                #print(parameter_value)
                a=[N,dic_fun[N].index(polynome)]
                for param in range(len(parameters)):
                    a.append(parameter_value[param])
                    #print(parameters[param])
                    #print(parameter_value[param])
                    globals()[parameters[param]]= parameter_value[param]
                #print("eval ", polynome, eval(polynome))
                a.append(eval(polynome))
                arr.append(a)
    return arr

In [ ]:
agents_quantities = [2,5]

Sample f of *p,q* models

In [ ]:
print("  N,   i, params, f(i)")
sample(f,5)

Sample f of multiparam models

In [ ]:
print("  N,   i, params, f(i)")
sample(f_multiparam,5)

### VISUALISATION

Visualisation of polynomials as barplots.

Single polynome:

In [ ]:
def eval_and_show(fun_list,parameter_value):
    """ Creates bar plot of probabilities of i successes for given parametrisation

    Parameters
    ----------
    fun_list : list of polynomes
    N: dictionary key
    parameter_value: array of param values 
    """
    for polynome in fun_list:
        parameters=set()
        parameters.update(find_param(polynome))
    parameters= sorted(list(parameters))
    #print(parameters)

    title=""
    a=[]
    for param in range(len(parameters)):
        #print(parameters[param])
        #print(parameter_value[param])
        globals()[parameters[param]]= parameter_value[param]
        title = "{}{}={} ".format(title,parameters[param],parameter_value[param])
    #print("eval ", polynome, eval(polynome))
    for polynome in fun_list:
        a.append(eval(polynome))
    #print(a)
    fig, ax = plt.subplots()
    width = 0.2
    ax.set_ylabel('Probability')
    ax.set_xlabel('i')
    ax.set_title('{}'.format(title))
    rects1 = ax.bar(range(len(fun_list)), a, width, color='b')
    plt.show()
    return a

In [ ]:
eval_and_show(f[2],[0.1,0.2])

In [ ]:
eval_and_show(f[10],[0.81,0.92])

Sampled polynomials

In [ ]:
def visualise(dic_fun,size_q):
    """ Creates bar plot of probabilities of i successes for sampled parametrisation

    Parameters
    ----------
    dic_fun : dictionary N -> list of polynomes
    size_q : sample size in each parameter
    """
        
    for N in agents_quantities:
        parameters=set()
        for polynome in dic_fun[N]:
            if len(parameters)<N:
                parameters.update(find_param(polynome))
        #print(parameters)
        parameters= sorted(list(parameters))
        #print(parameters)
        parameter_values = []
        for param in range(len(parameters)):
            parameter_values.append(np.linspace(0, 1, size_q, endpoint=True)) 
        parameter_values=cartesian_product(*parameter_values)
        if (len(parameters)-1)==0:
            parameter_values=np.linspace(0, 1, size_q, endpoint=True)[np.newaxis, :].T
        #print(parameter_values)
            
        for parameter_value in parameter_values:
            #print(parameter_value)
            a=[N,dic_fun[N].index(polynome)]
            title=""
            for param in range(len(parameters)):
                a.append(parameter_value[param])
                #print(parameters[param])
                #print(parameter_value[param])
                globals()[parameters[param]]= parameter_value[param]
                title = "{}{}={} ".format(title,parameters[param],parameter_value[param])
            #print("eval ", polynome, eval(polynome))
            for polynome in dic_fun[N]:
                a.append(eval(polynome))
            
            print(a)
            fig, ax = plt.subplots()
            width = 0.2
            ax.set_ylabel('Probability')
            ax.set_xlabel('i')
            ax.set_title('N={} {}'.format(N,title))
            rects1 = ax.bar(range(N+1), a[len(parameters)+2:], width, color='b')
            plt.show()

Visualise samples for *p,q* models

In [ ]:
visualise(f,5)

In [ ]:
visualise(f_multiparam,2)

MORE VISUALISATIONS TO BE ADDED

To see the results of paramter synthesis see notebooks `direct_param_synth` and `analysis`

visualisation of the results of parameter synthesis using SMT solver is within the notebook -- `analysis` 